In [1]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd 
import sys
#import pickle
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random
from sklearn.metrics import pairwise_distances
from sklearn.kernel_ridge import KernelRidge
import math
import dill as pickle

workingdirectory = os.popen('git rev-parse --show-toplevel').read()[:-1]
sys.path.append(workingdirectory)
os.chdir(workingdirectory)

import allensdk.core.json_utilities as ju
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache

from mcmodels.utils import get_aligned_ids
from mcmodels.core import VoxelModelCache
from mcmodels.connectivity.creation import get_connectivity_matrices3
from mcmodels.core.connectivity_data import get_connectivity_data, ConnectivityData
from mcmodels.core.structure_data import StructureData
from mcmodels.core.model_data import ModelData
#from mcmodels.models.voxel.crossvalidation import get_nwloocv_predictions_multimodel_merge_dists
from mcmodels.utils import nonzero_unique
from mcmodels.core.utils import get_structure_id,get_ordered_summary_structures, get_leaves_ontologicalorder, get_indices_2ormore, get_indices, get_indices2,get_eval_indices,screen_index_matrices,screen_index_matrices2,screen_index_matrices3
from mcmodels.regressors import NadarayaWatson
from mcmodels.core.plotting import plot_loss_surface,plot_loss_scatter
from mcmodels.models.expectedloss.crossvalidation import get_loss_surface_cv_spline
from mcmodels.models.expectedloss.crossvalidation import get_embedding_cv
from mcmodels.models.crossvalidation import get_best_hyperparameters,get_loss_best_hyp,get_loss, Crossval
from mcmodels.models.voxel.crossvalidation import get_nwloocv_predictions_multimodel_merge_dists

def get_row_col_names(connectivity_data, target_ordering):
    
    rnames = np.asarray([ia_map[target_ordering[i]] for i in range(len(target_ordering))])
    ipsi_target_regions = connectivity_data.ipsi_target_regions
    contra_target_regions = connectivity_data.contra_target_regions                               
    ipsi_indices= np.asarray([])
    contra_indices = np.asarray([])
    for iy in target_ordering: 
        ipsi_indices = np.concatenate([ipsi_indices, np.where(ipsi_target_regions==iy)[0]] )
        contra_indices = np.concatenate([contra_indices, np.where(contra_target_regions==iy)[0]] )
    ipsi_indices = np.asarray(ipsi_indices, dtype = int)   
    contra_indices = np.asarray(contra_indices, dtype = int)    
    reorder = np.concatenate([ipsi_indices, len(ipsi_indices) + contra_indices])  
    colids = np.concatenate([ipsi_target_regions, contra_target_regions])[reorder]
    cnames = np.asarray([ia_map[colids[i]] for i in range(len(colids))])
    ccomb = np.vstack([np.concatenate([np.repeat('ipsi',connectivity_data.ipsi_target_regions.shape[0]),
                                       np.repeat('contra',connectivity_data.contra_target_regions.shape)]), cnames])
    ccomb = np.asarray(ccomb)
    tuples2 = list(zip(*ccomb))
    cnam_multi = pd.MultiIndex.from_tuples(tuples2, names=['first', 'second'])
    return(cnam_multi, rnames)

/Users/samsonkoelle/anaconda3/envs/allen_010719_5/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/samsonkoelle/anaconda3/envs/allen_010719_5/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
#read data
TOP_DIR = '/Users/samsonkoelle/alleninstitute/mcm_2020/mcm_updated/'
INPUT_JSON = os.path.join(TOP_DIR, 'input_011520.json')
EXPERIMENTS_EXCLUDE_JSON = os.path.join(TOP_DIR, 'experiments_exclude.json')
input_data = ju.read(INPUT_JSON)
experiments_exclude = ju.read(EXPERIMENTS_EXCLUDE_JSON)
manifest_file = input_data.get('manifest_file')
manifest_file = os.path.join(TOP_DIR, manifest_file)
cache = VoxelModelCache(manifest_file=manifest_file)
st = cache.get_structure_tree()
ai_map = st.get_id_acronym_map()
ia_map = {value: key for key, value in ai_map.items()}
major_structures = np.load('/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/paper/info/major_structures.npy')
major_structure_ids = np.load('/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/paper/info/major_structure_ids.npy')
data_info = pd.read_excel('/Users/samsonkoelle/alleninstitute/Whole Brain Cre Image Series_curation only.xlsx', 'all datasets curated_070919pull')
data_info.set_index("id", inplace=True)
with open('data/info/leafs.pickle', 'rb') as handle:
    leafs = pickle.load(handle)

annotation/ccf_2017


In [3]:
connectivity_data = ConnectivityData(cache)

In [4]:
sid = 512
sid_data = StructureData(sid)
structure_set_id=167587189

In [5]:
model_data = ModelData(cache, sid, structure_set_id)

In [6]:
np.asarray(list(model_data.get_experiment_ids(experiments_exclude=experiments_exclude, cre=None))).shape

(36,)

In [7]:
experiment_ids = [e['id'] for e in cache.get_experiments(
            dataframe=False, cre=None, injection_structure_ids=[512])]

In [8]:
cache.get_experiments(
            dataframe=False, cre=None, injection_structure_ids=[512])

[{'transgenic_line': 'Pcp2-Cre_GN135',
  'gender': 'M',
  'injection_structures': [1017],
  'injection_volume': 0.1692286344,
  'injection_x': 12170,
  'injection_y': 3140,
  'injection_z': 8540,
  'product_id': 5,
  'specimen_name': 'Pcp2-Cre (NH)-3624',
  'strain': 'C57BL/6J',
  'structure_abbrev': 'AN',
  'structure_id': 1017,
  'structure_name': 'Ansiform lobule',
  'primary_injection_structure': 1017,
  'id': 168230532},
 {'transgenic_line': 'Htr2a-Cre_KM207',
  'gender': 'M',
  'injection_structures': [96, 846, 1007, 1017, 1033, 1041],
  'injection_volume': 0.00645919225,
  'injection_x': 11160,
  'injection_y': 3220,
  'injection_z': 8380,
  'product_id': 5,
  'specimen_name': 'Htr2a-Cre_KM207-130963',
  'strain': 'unknown',
  'structure_abbrev': 'DN',
  'structure_id': 846,
  'structure_name': 'Dentate nucleus',
  'primary_injection_structure': 96,
  'id': 552283801},
 {'transgenic_line': 'Gabra6-IRES-Cre',
  'gender': 'M',
  'injection_structures': [1007, 1017],
  'injection_v

In [12]:
cache.manifest_file

'/Users/samsonkoelle/alleninstitute/mcm_2020/mcm_updated/new_manifest.json'

In [13]:
len(experiment_ids)

36

In [14]:
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache

In [15]:
MCC = MouseConnectivityCache(manifest_file=manifest_file)

In [16]:
manifest_file

'/Users/samsonkoelle/alleninstitute/mcm_2020/mcm_updated/new_manifest.json'

In [19]:
len(list(MCC.get_experiments(injection_structure_ids=[512])))

36

In [21]:
help(MCC)

Help on MouseConnectivityCache in module allensdk.core.mouse_connectivity_cache object:

class MouseConnectivityCache(allensdk.core.reference_space_cache.ReferenceSpaceCache)
 |  MouseConnectivityCache(resolution=None, cache=True, manifest_file=None, ccf_version=None, base_uri=None, version=None)
 |  
 |  Cache class for storing and accessing data related to the adult mouse
 |  Connectivity Atlas.  By default, this class will cache any downloaded
 |  metadata or files in well known locations defined in a manifest file.
 |  This behavior can be disabled.
 |  
 |  Attributes
 |  ----------
 |  
 |  resolution: int
 |      Resolution of grid data to be downloaded when accessing projection volume,
 |      the annotation volume, and the annotation volume.  Must be one of (10, 25,
 |      50, 100).  Default is 25.
 |  
 |  api: MouseConnectivityApi instance
 |      Used internally to make API queries.
 |  
 |  Parameters
 |  ----------
 |  
 |  resolution: int
 |      Resolution of grid data

In [ ]:
get_experiments(self, dataframe=False, file_name=None, cre=None, injection_structure_ids=None)

In [38]:
from allensdk.api.cache import Cache

In [32]:
MCC.EXPERIMENTS_KEY

'EXPERIMENTS'

In [34]:
from allensdk.api.cache import Cache

In [35]:

file_name = MCC.get_cache_path(None, MCC.EXPERIMENTS_KEY)
experiments = MCC.api.get_experiments_api(path=file_name,
                                           strategy='lazy',
                                           **Cache.cache_json())

In [ ]:
MCC.api.

In [38]:
help(MCC.api)

Help on MouseConnectivityApi in module allensdk.api.queries.mouse_connectivity_api object:

class MouseConnectivityApi(allensdk.api.queries.reference_space_api.ReferenceSpaceApi, allensdk.api.queries.grid_data_api.GridDataApi)
 |  MouseConnectivityApi(base_uri=None)
 |  
 |  HTTP Client for the Allen Mouse Brain Connectivity Atlas.
 |  
 |  See: `Mouse Connectivity API <http://help.brain-map.org/display/mouseconnectivity/API>`_
 |  
 |  Method resolution order:
 |      MouseConnectivityApi
 |      allensdk.api.queries.reference_space_api.ReferenceSpaceApi
 |      allensdk.api.queries.grid_data_api.GridDataApi
 |      allensdk.api.queries.rma_api.RmaApi
 |      allensdk.api.api.Api
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, base_uri=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  build_reference_aligned_image_channel_volumes_url(self, data_set_id)
 |      Construct url to download the red, green, and blue channels

In [37]:
len(experiments)

1751

In [39]:
help(Cache)

Help on class Cache in module allensdk.api.cache:

class Cache(builtins.object)
 |  Cache(manifest=None, cache=True, version=None, **kwargs)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, manifest=None, cache=True, version=None, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  add_manifest_paths(self, manifest_builder)
 |      Add cache-class specific paths to the manifest. In derived classes,
 |      should call super.
 |  
 |  build_manifest(self, file_name)
 |      Creation of default path specifications.
 |      
 |      Parameters
 |      ----------
 |      file_name : string
 |          where to save it
 |  
 |  get_cache_path(self, file_name, manifest_key, *args)
 |      Helper method for accessing path specs from manifest keys.
 |      
 |      Parameters
 |      ----------
 |      file_name : string
 |      manifest_key : string
 |      args : ordered parameters
 |      
 |      Returns
 |      -------
 |      string or None
 | 

In [25]:
import allensdk

In [27]:
from allensdk.api.cache import cacheable, Cache

In [32]:
help(allensdk.api.cache)

Help on module allensdk.api.cache in allensdk.api:

NAME
    allensdk.api.cache

DESCRIPTION
    # Allen Institute Software License - This software license is the 2-clause BSD
    # license plus a third clause that prohibits redistribution for commercial
    # purposes without further permission.
    #
    # Copyright 2015-2017. Allen Institute. All rights reserved.
    #
    # Redistribution and use in source and binary forms, with or without
    # modification, are permitted provided that the following conditions are met:
    #
    # 1. Redistributions of source code must retain the above copyright notice,
    # this list of conditions and the following disclaimer.
    #
    # 2. Redistributions in binary form must reproduce the above copyright notice,
    # this list of conditions and the following disclaimer in the documentation
    # and/or other materials provided with the distribution.
    #
    # 3. Redistributions for commercial purposes are not permitted without the
    # All

In [ ]:

def get_connectivity_data(cache, structure_ids, experiments_exclude, remove_injection=False, structure_set_id = 687527945):
    '''

    :param cache: VoxelModelCache for communicating with AllenSDK
    :param structure_ids: Ids for which to pull data
    :param experiments_exclude: Experiments to exclude (not tested)
    :param remove_injection: Remove injection signal from projection (not tested)
    :param structure_set_id: Structure set specifying list of structures (not same as location of structures)
    :return:
    '''
    connectivity_data = ConnectivityData(cache)

    for sid in structure_ids:
        print(sid)
        sid_data = StructureData(sid)
        # deprecated language
        model_data = ModelData(cache, sid, structure_set_id)
        sid_data.eids = model_data.get_experiment_ids(experiments_exclude=experiments_exclude, cre=None)
 

In [ ]:
kahuna = [
 {
  "transgenic_line": {
   "transgenic_line_id": 177838877,
   "name": "Gabrr3-Cre_KC112"
  },
  "gender": "M",
  "injection_structures": "574/931",
  "injection_volume": 0.105746442921875,
  "injection_x": 9170,
  "injection_y": 6850,
  "injection_z": 6200,
  "product_id": 5,
  "specimen_name": "Gabrr3-Cre_KC112-3467",
  "strain": "C57BL/6J",
  "structure_abbrev": "PG",
  "structure_id": 931,
  "structure_name": "Pontine gray",
  "primary_injection_structure": 574,
  "data_set_id": 301875966,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839494,
   "name": "Hdc-Cre_IM1"
  },
  "gender": "M",
  "injection_structures": "1/210/491/525/1004",
  "injection_volume": 0.02576159075,
  "injection_x": 7810,
  "injection_y": 6550,
  "injection_z": 6450,
  "product_id": 5,
  "specimen_name": "Hdc-Cre_IM1-204103",
  "strain": null,
  "structure_abbrev": "TMv",
  "structure_id": 1,
  "structure_name": "Tuberomammillary nucleus, ventral part",
  "primary_injection_structure": 1,
  "data_set_id": 520336173,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837481,
   "name": "Scnn1a-Tg2-Cre"
  },
  "gender": "F",
  "injection_structures": "136/177/395/773/1098/1107",
  "injection_volume": 0.065650549125,
  "injection_x": 12720,
  "injection_y": 5900,
  "injection_z": 5690,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg2-Cre-3456",
  "strain": "C57BL/6J",
  "structure_abbrev": "XII",
  "structure_id": 773,
  "structure_name": "Hypoglossal nucleus",
  "primary_injection_structure": 136,
  "data_set_id": 177904363,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 298725927,
   "name": "Penk-IRES2-Cre-neo"
  },
  "gender": "F",
  "injection_structures": "993",
  "injection_volume": 0.014112638225,
  "injection_x": 2510,
  "injection_y": 2440,
  "injection_z": 3850,
  "product_id": 45,
  "specimen_name": "Penk-IRES2-Cre-neo-321021",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 602828622,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838494,
   "name": "Prkcd-GluCla-CFP-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "475/726/1044/560581563",
  "injection_volume": 0.04078884705,
  "injection_x": 8470,
  "injection_y": 4160,
  "injection_z": 8090,
  "product_id": 5,
  "specimen_name": "Prkcd-GluCla-CFP-IRES-Cre-73",
  "strain": null,
  "structure_abbrev": "MG",
  "structure_id": 475,
  "structure_name": "Medial geniculate complex",
  "primary_injection_structure": 475,
  "data_set_id": 178489574,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "F",
  "injection_structures": "993",
  "injection_volume": 0.0580139616,
  "injection_x": 3220,
  "injection_y": 2240,
  "injection_z": 4260,
  "product_id": 45,
  "specimen_name": "Tlx3-Cre_PL56;Sst-IRES-FlpO;Ai65F-239016",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 517314004,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182693192,
   "name": "Chrna2-Cre_OE25"
  },
  "gender": "M",
  "injection_structures": "39/48",
  "injection_volume": 0.0524996976,
  "injection_x": 5160,
  "injection_y": 1820,
  "injection_z": 6040,
  "product_id": 5,
  "specimen_name": "Chrna2-Cre_OE25-3873",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 278069982,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837788,
   "name": "Ins2-Cre_25"
  },
  "gender": "M",
  "injection_structures": "118/223/693/830/10671",
  "injection_volume": 0.07434136338125,
  "injection_x": 6910,
  "injection_y": 6830,
  "injection_z": 5830,
  "product_id": 5,
  "specimen_name": "Ins2-Cre-138",
  "strain": "C57BL/6J",
  "structure_abbrev": "ARH",
  "structure_id": 223,
  "structure_name": "Arcuate hypothalamic nucleus",
  "primary_injection_structure": 118,
  "data_set_id": 286318327,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "M",
  "injection_structures": "409/425/312782628",
  "injection_volume": 0.0588408464,
  "injection_x": 9520,
  "injection_y": 2440,
  "injection_z": 9210,
  "product_id": 5,
  "specimen_name": "Syt6-Cre_KI148-131918",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISpl",
  "structure_id": 425,
  "structure_name": "Posterolateral visual area",
  "primary_injection_structure": 409,
  "data_set_id": 299828473,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837281,
   "name": "Calb2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "329/353/182305689",
  "injection_volume": 0.018667759075,
  "injection_x": 6290,
  "injection_y": 1630,
  "injection_z": 8920,
  "product_id": 5,
  "specimen_name": "Calb2-IRES-Cre-172",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 168513750,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182693192,
   "name": "Chrna2-Cre_OE25"
  },
  "gender": "M",
  "injection_structures": "111/353/378/583/672/677/1057",
  "injection_volume": 0.0244798953,
  "injection_x": 5590,
  "injection_y": 4870,
  "injection_z": 9500,
  "product_id": 5,
  "specimen_name": "Chrna2-Cre_OE25-124947",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "AIp",
  "structure_id": 111,
  "structure_name": "Agranular insular area, posterior part",
  "primary_injection_structure": 111,
  "data_set_id": 297858011,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "104/345/378/1057",
  "injection_volume": 0.32121215,
  "injection_x": 4410,
  "injection_y": 4110,
  "injection_z": 9390,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre_KL100-217",
  "strain": "C57BL/6J",
  "structure_abbrev": "GU",
  "structure_id": 1057,
  "structure_name": "Gustatory areas",
  "primary_injection_structure": 104,
  "data_set_id": 249402048,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "402/409/1027/312782574",
  "injection_volume": 0.078424206,
  "injection_x": 8490,
  "injection_y": 2140,
  "injection_z": 2180,
  "product_id": 35,
  "specimen_name": "Cux2-IRES-Cre-243008",
  "strain": null,
  "structure_abbrev": "VISal",
  "structure_id": 402,
  "structure_name": "Anterolateral visual area",
  "primary_injection_structure": 402,
  "data_set_id": 520996382,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838361,
   "name": "Etv1-CreERT2"
  },
  "gender": "M",
  "injection_structures": "329/353/182305689",
  "injection_volume": 0.1899245096,
  "injection_x": 5840,
  "injection_y": 2180,
  "injection_z": 8600,
  "product_id": 5,
  "specimen_name": "Etv1-CreERT2-2883",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 167570021,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.3752175784,
  "injection_x": 8850,
  "injection_y": 1440,
  "injection_z": 3550,
  "product_id": 36,
  "specimen_name": "Emx1-IRES-Cre-212542",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 500836840,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "39/48/993",
  "injection_volume": 0.1561037492,
  "injection_x": 4380,
  "injection_y": 1840,
  "injection_z": 5480,
  "product_id": 35,
  "specimen_name": "Emx1-IRES-Cre-305959",
  "strain": null,
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 584194481,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "M",
  "injection_structures": "409/541/312782574/312782628",
  "injection_volume": 0.0236243112,
  "injection_x": 9210,
  "injection_y": 2600,
  "injection_z": 9470,
  "product_id": 5,
  "specimen_name": "Syt6-Cre_KI148-3838",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISli",
  "structure_id": 312782574,
  "structure_name": "Laterointermediate area",
  "primary_injection_structure": 409,
  "data_set_id": 292791310,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "385/409",
  "injection_volume": 0.6636360864,
  "injection_x": 9230,
  "injection_y": 1910,
  "injection_z": 8430,
  "product_id": 5,
  "specimen_name": "C57BL/6-152922",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 307297141,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.1254239476,
  "injection_x": 8690,
  "injection_y": 900,
  "injection_z": 8110,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-288",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 272821309,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838781,
   "name": "Slc6a4-CreERT2_EZ13"
  },
  "gender": "F",
  "injection_structures": "83/222/230/1048",
  "injection_volume": 0.0486814902,
  "injection_x": 12100,
  "injection_y": 6540,
  "injection_z": 5640,
  "product_id": 5,
  "specimen_name": "Slc6a4-CreERT2-Homo-356",
  "strain": "C57BL/6J",
  "structure_abbrev": "RO",
  "structure_id": 222,
  "structure_name": "Nucleus raphe obscurus",
  "primary_injection_structure": 83,
  "data_set_id": 167212217,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "111/378/677/1057",
  "injection_volume": 1.2988013696,
  "injection_x": 6370,
  "injection_y": 4320,
  "injection_z": 10280,
  "product_id": 5,
  "specimen_name": "378-1826",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISC",
  "structure_id": 677,
  "structure_name": "Visceral area",
  "primary_injection_structure": 111,
  "data_set_id": 180917660,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "345/353/378/677/1057",
  "injection_volume": 0.2231006848,
  "injection_x": 4920,
  "injection_y": 4350,
  "injection_z": 9770,
  "product_id": 5,
  "specimen_name": "378-1766",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSs",
  "structure_id": 378,
  "structure_name": "Supplemental somatosensory area",
  "primary_injection_structure": 345,
  "data_set_id": 174360333,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "385/533/879/886/894",
  "injection_volume": 0.075229259,
  "injection_x": 8680,
  "injection_y": 1050,
  "injection_z": 4050,
  "product_id": 36,
  "specimen_name": "Rbp4-Cre_KL100-245457",
  "strain": null,
  "structure_abbrev": "VISpm",
  "structure_id": 533,
  "structure_name": "posteromedial visual area",
  "primary_injection_structure": 385,
  "data_set_id": 523718823,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "39/44/731/746/972",
  "injection_volume": 0.4770648624,
  "injection_x": 3050,
  "injection_y": 3130,
  "injection_z": 5060,
  "product_id": 36,
  "specimen_name": "Emx1-IRES-Cre-187655",
  "strain": "B6.129",
  "structure_abbrev": "PL",
  "structure_id": 972,
  "structure_name": "Prelimbic area",
  "primary_injection_structure": 39,
  "data_set_id": 478376197,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "879/886",
  "injection_volume": 0.1845043648,
  "injection_x": 7450,
  "injection_y": 810,
  "injection_z": 6090,
  "product_id": 5,
  "specimen_name": "378-601",
  "strain": "C57BL/6J",
  "structure_abbrev": "RSPv",
  "structure_id": 886,
  "structure_name": "Retrosplenial area, ventral part",
  "primary_injection_structure": 879,
  "data_set_id": 100140949,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "128/271/294/580",
  "injection_volume": 0.2196725272,
  "injection_x": 9060,
  "injection_y": 3500,
  "injection_z": 7780,
  "product_id": 5,
  "specimen_name": "378-1596",
  "strain": "C57BL/6J",
  "structure_abbrev": "MRN",
  "structure_id": 128,
  "structure_name": "Midbrain reticular nucleus",
  "primary_injection_structure": 128,
  "data_set_id": 156978574,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838496,
   "name": "Grp-Cre_KH288"
  },
  "gender": "M",
  "injection_structures": "39/993",
  "injection_volume": 0.1342011118,
  "injection_x": 3960,
  "injection_y": 2140,
  "injection_z": 6140,
  "product_id": 5,
  "specimen_name": "Grp-Cre-71",
  "strain": "B6.FVB(ICR)",
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 183472596,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839406,
   "name": "Nos1-CreERT2"
  },
  "gender": "M",
  "injection_structures": "403",
  "injection_volume": 0.0963382238,
  "injection_x": 6710,
  "injection_y": 6380,
  "injection_z": 7670,
  "product_id": 5,
  "specimen_name": "Nos1-CreERT2-139494",
  "strain": "B6.129",
  "structure_abbrev": "MEA",
  "structure_id": 403,
  "structure_name": "Medial amygdalar nucleus",
  "primary_injection_structure": 403,
  "data_set_id": 304694870,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "402/409/312782574",
  "injection_volume": 0.1910349868,
  "injection_x": 8640,
  "injection_y": 1990,
  "injection_z": 2420,
  "product_id": 36,
  "specimen_name": "Emx1-IRES-Cre-212771",
  "strain": null,
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 402,
  "data_set_id": 500836105,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "F",
  "injection_structures": "361/312782546",
  "injection_volume": 0.0648063514,
  "injection_x": 7290,
  "injection_y": 890,
  "injection_z": 3650,
  "product_id": 35,
  "specimen_name": "Tlx3-Cre_PL56-356310",
  "strain": null,
  "structure_abbrev": "VISa",
  "structure_id": 312782546,
  "structure_name": "Anterior area",
  "primary_injection_structure": 361,
  "data_set_id": 651041703,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838302,
   "name": "Gng7-Cre_KH71"
  },
  "gender": "M",
  "injection_structures": "39/48",
  "injection_volume": 0.238910527804102,
  "injection_x": 5170,
  "injection_y": 1550,
  "injection_z": 5990,
  "product_id": 5,
  "specimen_name": "Gng7-Cre-971",
  "strain": "C57BL/6J",
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 158018630,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "533/894",
  "injection_volume": 0.019900177675,
  "injection_x": 8500,
  "injection_y": 490,
  "injection_z": 4320,
  "product_id": 36,
  "specimen_name": "Cux2-IRES-Cre-203186",
  "strain": "C57BL/6J",
  "structure_abbrev": "RSPagl",
  "structure_id": 894,
  "structure_name": "Retrosplenial area, lateral agranular part",
  "primary_injection_structure": 533,
  "data_set_id": 495345959,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "385/533",
  "injection_volume": 0.129882193,
  "injection_x": 8650,
  "injection_y": 710,
  "injection_z": 7560,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre_KL100-125732",
  "strain": null,
  "structure_abbrev": "VISpm",
  "structure_id": 533,
  "structure_name": "posteromedial visual area",
  "primary_injection_structure": 385,
  "data_set_id": 294481346,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "F",
  "injection_structures": "385/409/425",
  "injection_volume": 0.019773720925,
  "injection_x": 9510,
  "injection_y": 2340,
  "injection_z": 2480,
  "product_id": 36,
  "specimen_name": "Ntsr1-Cre_GN220-271815",
  "strain": "unknown",
  "structure_abbrev": "VISpl",
  "structure_id": 425,
  "structure_name": "Posterolateral visual area",
  "primary_injection_structure": 385,
  "data_set_id": 553743594,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "F",
  "injection_structures": "985/993",
  "injection_volume": 0.0367061499,
  "injection_x": 3210,
  "injection_y": 2270,
  "injection_z": 7620,
  "product_id": 5,
  "specimen_name": "Tlx3-Cre_PL56-336161",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 614735393,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "409/425/312782574/312782628",
  "injection_volume": 0.1437289462,
  "injection_x": 9540,
  "injection_y": 2520,
  "injection_z": 9310,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-3806",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISpor",
  "structure_id": 312782628,
  "structure_name": "Postrhinal area",
  "primary_injection_structure": 409,
  "data_set_id": 287953929,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "385/409/425/894",
  "injection_volume": 1.614223072,
  "injection_x": 8460,
  "injection_y": 1630,
  "injection_z": 2340,
  "product_id": 36,
  "specimen_name": "Emx1-IRES-Cre-234272",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 512315551,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "F",
  "injection_structures": "879/894",
  "injection_volume": 0.0097300482125,
  "injection_x": 9820,
  "injection_y": 1360,
  "injection_z": 3940,
  "product_id": 36,
  "specimen_name": "Tlx3-Cre_PL56-313540",
  "strain": null,
  "structure_abbrev": "RSPd",
  "structure_id": 879,
  "structure_name": "Retrosplenial area, dorsal part",
  "primary_injection_structure": 879,
  "data_set_id": 591233395,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 310459452,
   "name": "Slc17a8-IRES2-Cre"
  },
  "gender": "M",
  "injection_structures": "128/238/604/679/872/898/599626927",
  "injection_volume": 0.0260696464,
  "injection_x": 10420,
  "injection_y": 4470,
  "injection_z": 5650,
  "product_id": 5,
  "specimen_name": "Slc17a8-IRES2-Cre-257839",
  "strain": null,
  "structure_abbrev": "NI",
  "structure_id": 604,
  "structure_name": "Nucleus incertus",
  "primary_injection_structure": 128,
  "data_set_id": 547509190,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 553502711,
   "name": "Chat-IRES-Cre-neo"
  },
  "gender": "F",
  "injection_structures": "672/1022",
  "injection_volume": 0.012584404175,
  "injection_x": 5890,
  "injection_y": 3470,
  "injection_z": 8200,
  "product_id": 5,
  "specimen_name": "Chat-IRES-Cre-123456",
  "strain": "B6.129",
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 672,
  "data_set_id": 307909888,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "F",
  "injection_structures": "329/353/182305689",
  "injection_volume": 0.0799520995,
  "injection_x": 5930,
  "injection_y": 2010,
  "injection_z": 8560,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg3-Cre-4769",
  "strain": "B6.C3H",
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 159888336,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "39/993",
  "injection_volume": 0.0990279514,
  "injection_x": 3310,
  "injection_y": 2180,
  "injection_z": 6270,
  "product_id": 5,
  "specimen_name": "378-695",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 39,
  "data_set_id": 100141454,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 301467016,
   "name": "Npr3-IRES2-Cre"
  },
  "gender": "F",
  "injection_structures": "184/993",
  "injection_volume": 0.1595919906,
  "injection_x": 2950,
  "injection_y": 2840,
  "injection_z": 6970,
  "product_id": 5,
  "specimen_name": "Npr3-IRES2-Cre-296845",
  "strain": "unknown",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 184,
  "data_set_id": 575216368,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839285,
   "name": "Gpr26-Cre_KO250"
  },
  "gender": "M",
  "injection_structures": "895/918/922/312782628",
  "injection_volume": 0.0154829024,
  "injection_x": 9530,
  "injection_y": 3400,
  "injection_z": 10100,
  "product_id": 5,
  "specimen_name": "Gpr26-Cre_KO250-131188",
  "strain": null,
  "structure_abbrev": "ENTl",
  "structure_id": 918,
  "structure_name": "Entorhinal area, lateral part",
  "primary_injection_structure": 895,
  "data_set_id": 301265683,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837777,
   "name": "Sim1-Cre_KJ18"
  },
  "gender": "F",
  "injection_structures": "345/353/369/182305689",
  "injection_volume": 0.1809590972,
  "injection_x": 5000,
  "injection_y": 2410,
  "injection_z": 8970,
  "product_id": 5,
  "specimen_name": "Sim1-Cre_KJ18-128569",
  "strain": null,
  "structure_abbrev": "SSp-m",
  "structure_id": 345,
  "structure_name": "Primary somatosensory area, mouth",
  "primary_injection_structure": 345,
  "data_set_id": 297948420,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "M",
  "injection_structures": "394/533",
  "injection_volume": 0.0543355022,
  "injection_x": 8190,
  "injection_y": 1180,
  "injection_z": 7200,
  "product_id": 5,
  "specimen_name": "Syt6-Cre_KI148-131919",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISpm",
  "structure_id": 533,
  "structure_name": "posteromedial visual area",
  "primary_injection_structure": 394,
  "data_set_id": 299733445,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "39/48",
  "injection_volume": 0.1231576486,
  "injection_x": 4760,
  "injection_y": 2110,
  "injection_z": 6110,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-195",
  "strain": null,
  "structure_abbrev": "ACAv",
  "structure_id": 48,
  "structure_name": "Anterior cingulate area, ventral part",
  "primary_injection_structure": 39,
  "data_set_id": 183461297,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "385/409/425",
  "injection_volume": 0.03133389035,
  "injection_x": 9840,
  "injection_y": 1970,
  "injection_z": 9160,
  "product_id": 5,
  "specimen_name": "378-1888",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISpl",
  "structure_id": 425,
  "structure_name": "Posterolateral visual area",
  "primary_injection_structure": 385,
  "data_set_id": 272916202,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182693192,
   "name": "Chrna2-Cre_OE25"
  },
  "gender": "M",
  "injection_structures": "502/918",
  "injection_volume": 0.1346297246,
  "injection_x": 9280,
  "injection_y": 5290,
  "injection_z": 9650,
  "product_id": 5,
  "specimen_name": "Chrna2-Cre_OE25-133244",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "ENTl",
  "structure_id": 918,
  "structure_name": "Entorhinal area, lateral part",
  "primary_injection_structure": 502,
  "data_set_id": 299857096,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "F",
  "injection_structures": "394/533/312782546",
  "injection_volume": 0.0756005467,
  "injection_x": 7880,
  "injection_y": 1160,
  "injection_z": 3870,
  "product_id": 36,
  "specimen_name": "Ntsr1-Cre_GN220-245426",
  "strain": "B6.FVB",
  "structure_abbrev": "VISam",
  "structure_id": 394,
  "structure_name": "Anteromedial visual area",
  "primary_injection_structure": 394,
  "data_set_id": 523714940,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837858,
   "name": "Pvalb-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "378/1002/1011",
  "injection_volume": 0.1342141262,
  "injection_x": 7300,
  "injection_y": 2560,
  "injection_z": 10130,
  "product_id": 5,
  "specimen_name": "Pvalb-IRES-Cre-5017",
  "strain": "B6.129",
  "structure_abbrev": "AUDd",
  "structure_id": 1011,
  "structure_name": "Dorsal auditory area",
  "primary_injection_structure": 378,
  "data_set_id": 160080068,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.03165346345,
  "injection_x": 9230,
  "injection_y": 1390,
  "injection_z": 3070,
  "product_id": 36,
  "specimen_name": "Tlx3-Cre_PL56-232871",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 510581751,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838259,
   "name": "Cort-T2A-Cre"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.1492781864,
  "injection_x": 9480,
  "injection_y": 1360,
  "injection_z": 8590,
  "product_id": 5,
  "specimen_name": "Cort-T2A-Cre-126",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 167375352,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "541/1002/1018/1027",
  "injection_volume": 0.1129929612,
  "injection_x": 8560,
  "injection_y": 2790,
  "injection_z": 1570,
  "product_id": 35,
  "specimen_name": "Cux2-IRES-Cre-292890",
  "strain": "unknown",
  "structure_abbrev": "AUDp",
  "structure_id": 1002,
  "structure_name": "Primary auditory area",
  "primary_injection_structure": 541,
  "data_set_id": 569994739,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "4",
  "injection_volume": 0.0508223737,
  "injection_x": 10280,
  "injection_y": 2000,
  "injection_z": 7290,
  "product_id": 5,
  "specimen_name": "378-1538",
  "strain": "C57BL/6J",
  "structure_abbrev": "IC",
  "structure_id": 4,
  "structure_name": "Inferior colliculus",
  "primary_injection_structure": 4,
  "data_set_id": 147162027,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "F",
  "injection_structures": "294/795/872/975",
  "injection_volume": 0.0189096812625,
  "injection_x": 9330,
  "injection_y": 3510,
  "injection_z": 5840,
  "product_id": 5,
  "specimen_name": "Syt6-Cre_KI148-138161",
  "strain": "C57BL/6J",
  "structure_abbrev": "PAG",
  "structure_id": 795,
  "structure_name": "Periaqueductal gray",
  "primary_injection_structure": 294,
  "data_set_id": 304949216,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "337/985",
  "injection_volume": 0.031938592,
  "injection_x": 5670,
  "injection_y": 1380,
  "injection_z": 7490,
  "product_id": 5,
  "specimen_name": "Tlx3-Cre_PL56-313536",
  "strain": null,
  "structure_abbrev": "SSp-ll",
  "structure_id": 337,
  "structure_name": "Primary somatosensory area, lower limb",
  "primary_injection_structure": 337,
  "data_set_id": 591622344,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "385/879/894",
  "injection_volume": 0.0207786656,
  "injection_x": 9600,
  "injection_y": 1260,
  "injection_z": 3800,
  "product_id": 35,
  "specimen_name": "Rbp4-Cre_KL100-363600",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 657046319,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "672/961/966",
  "injection_volume": 0.2478796712,
  "injection_x": 5890,
  "injection_y": 6220,
  "injection_z": 8840,
  "product_id": 5,
  "specimen_name": "378-1332",
  "strain": "C57BL/6J",
  "structure_abbrev": "PIR",
  "structure_id": 961,
  "structure_name": "Piriform area",
  "primary_injection_structure": 672,
  "data_set_id": 131068390,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837777,
   "name": "Sim1-Cre_KJ18"
  },
  "gender": "M",
  "injection_structures": "403/647",
  "injection_volume": 0.0242617179,
  "injection_x": 7250,
  "injection_y": 6690,
  "injection_z": 7780,
  "product_id": 5,
  "specimen_name": "Sim1-Cre_KJ18-132878",
  "strain": null,
  "structure_abbrev": "MEA",
  "structure_id": 403,
  "structure_name": "Medial amygdalar nucleus",
  "primary_injection_structure": 403,
  "data_set_id": 299896150,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 301467016,
   "name": "Npr3-IRES2-Cre"
  },
  "gender": "M",
  "injection_structures": "329/353",
  "injection_volume": 0.0366856238,
  "injection_x": 6120,
  "injection_y": 2160,
  "injection_z": 8800,
  "product_id": 5,
  "specimen_name": "Npr3-IRES2-Cre-213449",
  "strain": null,
  "structure_abbrev": "SSp-n",
  "structure_id": 353,
  "structure_name": "Primary somatosensory area, nose",
  "primary_injection_structure": 329,
  "data_set_id": 512130913,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.02099990795,
  "injection_x": 8150,
  "injection_y": 1040,
  "injection_z": 8020,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-123231",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 298351630,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837779,
   "name": "Pcp2-Cre_GN135"
  },
  "gender": "M",
  "injection_structures": "1017",
  "injection_volume": 0.1692286344,
  "injection_x": 12170,
  "injection_y": 3140,
  "injection_z": 8540,
  "product_id": 5,
  "specimen_name": "Pcp2-Cre (NH)-3624",
  "strain": "C57BL/6J",
  "structure_abbrev": "AN",
  "structure_id": 1017,
  "structure_name": "Ansiform lobule",
  "primary_injection_structure": 1017,
  "data_set_id": 168230532,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838361,
   "name": "Etv1-CreERT2"
  },
  "gender": "F",
  "injection_structures": "378/1002/1011",
  "injection_volume": 0.2378995864,
  "injection_x": 7570,
  "injection_y": 2400,
  "injection_z": 10240,
  "product_id": 5,
  "specimen_name": "Etv1-CreERT2-1176",
  "strain": "C57BL/6J",
  "structure_abbrev": "AUDd",
  "structure_id": 1011,
  "structure_name": "Dorsal auditory area",
  "primary_injection_structure": 378,
  "data_set_id": 156493815,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 266649644,
   "name": "Drd3-Cre_KI196"
  },
  "gender": "F",
  "injection_structures": "329/417/312782546",
  "injection_volume": 0.0629706301,
  "injection_x": 7520,
  "injection_y": 1210,
  "injection_z": 8500,
  "product_id": 5,
  "specimen_name": "Drd3-Cre_KI196_LURC-130785",
  "strain": null,
  "structure_abbrev": "VISrl",
  "structure_id": 417,
  "structure_name": "Rostrolateral visual area",
  "primary_injection_structure": 329,
  "data_set_id": 521402511,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 266649644,
   "name": "Drd3-Cre_KI196"
  },
  "gender": "M",
  "injection_structures": "345",
  "injection_volume": 0.0649930365,
  "injection_x": 4700,
  "injection_y": 1980,
  "injection_z": 9110,
  "product_id": 5,
  "specimen_name": "Drd3-Cre_KI196_LURC-125057",
  "strain": null,
  "structure_abbrev": "SSp-m",
  "structure_id": 345,
  "structure_name": "Primary somatosensory area, mouth",
  "primary_injection_structure": 345,
  "data_set_id": 309427991,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "F",
  "injection_structures": "879/886",
  "injection_volume": 0.0484091972,
  "injection_x": 7950,
  "injection_y": 650,
  "injection_z": 5990,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg3-Cre-4790",
  "strain": "B6.C3H",
  "structure_abbrev": "RSPd",
  "structure_id": 879,
  "structure_name": "Retrosplenial area, dorsal part",
  "primary_injection_structure": 879,
  "data_set_id": 166458363,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "F",
  "injection_structures": "329",
  "injection_volume": 0.0122414054,
  "injection_x": 7320,
  "injection_y": 1770,
  "injection_z": 8710,
  "product_id": 5,
  "specimen_name": "Syt6-Cre-3171",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 178487444,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838502,
   "name": "Grik4-Cre"
  },
  "gender": "F",
  "injection_structures": "463/726",
  "injection_volume": 0.118251553,
  "injection_x": 6770,
  "injection_y": 2570,
  "injection_z": 6010,
  "product_id": 5,
  "specimen_name": "Grik4-Cre-4955",
  "strain": "C57BL/6J",
  "structure_abbrev": "DG",
  "structure_id": 726,
  "structure_name": "Dentate gyrus",
  "primary_injection_structure": 463,
  "data_set_id": 114009636,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "385/409",
  "injection_volume": 0.1241475858,
  "injection_x": 9000,
  "injection_y": 2000,
  "injection_z": 2620,
  "product_id": 35,
  "specimen_name": "Cux2-IRES-Cre-239316",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 517962038,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838681,
   "name": "Slc6a4-Cre_ET33"
  },
  "gender": "F",
  "injection_structures": "115/795/872",
  "injection_volume": 0.1373016946,
  "injection_x": 9630,
  "injection_y": 3830,
  "injection_z": 5470,
  "product_id": 35,
  "specimen_name": "Slc6a4-Cre_ET33-191128",
  "strain": "C57BL/6J",
  "structure_abbrev": "DR",
  "structure_id": 872,
  "structure_name": "Dorsal nucleus raphe",
  "primary_injection_structure": 115,
  "data_set_id": 480074702,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182901722,
   "name": "Ppp1r17-Cre_NL146"
  },
  "gender": "M",
  "injection_structures": "294/302/1037",
  "injection_volume": 0.0729616125,
  "injection_x": 8760,
  "injection_y": 2080,
  "injection_z": 7160,
  "product_id": 5,
  "specimen_name": "Ppp1r17-Cre_NL146-122273",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "SCm",
  "structure_id": 294,
  "structure_name": "Superior colliculus, motor related",
  "primary_injection_structure": 294,
  "data_set_id": 307656580,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839090,
   "name": "Slc32a1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "395/429/1098",
  "injection_volume": 0.0921935686,
  "injection_x": 12960,
  "injection_y": 6200,
  "injection_z": 7390,
  "product_id": 5,
  "specimen_name": "Slc32a1-IRES-Cre-123248",
  "strain": "B6.129",
  "structure_abbrev": "SPVC",
  "structure_id": 429,
  "structure_name": "Spinal nucleus of the trigeminal, caudal part",
  "primary_injection_structure": 395,
  "data_set_id": 305319461,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 180526080,
   "name": "Slc17a7-IRES2-Cre"
  },
  "gender": "F",
  "injection_structures": "382/423/463/726",
  "injection_volume": 0.231244132,
  "injection_x": 7660,
  "injection_y": 2370,
  "injection_z": 8220,
  "product_id": 5,
  "specimen_name": "Slc17a7-IRES2-Cre-D-642",
  "strain": null,
  "structure_abbrev": "CA1",
  "structure_id": 382,
  "structure_name": "Field CA1",
  "primary_injection_structure": 382,
  "data_set_id": 267104205,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 695130294,
   "name": "Nxph4-T2A-CreERT2"
  },
  "gender": "M",
  "injection_structures": "398/661",
  "injection_volume": 0.02071345395,
  "injection_x": 10160,
  "injection_y": 6680,
  "injection_z": 7290,
  "product_id": 5,
  "specimen_name": "Nxph4-2A-CreERT2-D-3838",
  "strain": "B6.129",
  "structure_abbrev": "SOC",
  "structure_id": 398,
  "structure_name": "Superior olivary complex",
  "primary_injection_structure": 398,
  "data_set_id": 287668550,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838942,
   "name": "Htr2a-Cre_KM207"
  },
  "gender": "M",
  "injection_structures": "985/993",
  "injection_volume": 0.2551622668,
  "injection_x": 3800,
  "injection_y": 1710,
  "injection_z": 7310,
  "product_id": 5,
  "specimen_name": "Htr2a-Cre-3437",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 183617432,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837834,
   "name": "Crh-IRES-Cre_BL"
  },
  "gender": "F",
  "injection_structures": "83/230",
  "injection_volume": 0.1225552769,
  "injection_x": 11580,
  "injection_y": 7100,
  "injection_z": 5680,
  "product_id": 5,
  "specimen_name": "Crh-IRES-Cre-237",
  "strain": "B6.129",
  "structure_abbrev": "IO",
  "structure_id": 83,
  "structure_name": "Inferior olivary complex",
  "primary_injection_structure": 83,
  "data_set_id": 167665302,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "39/48",
  "injection_volume": 0.0565157131,
  "injection_x": 5680,
  "injection_y": 1470,
  "injection_z": 5480,
  "product_id": 36,
  "specimen_name": "Emx1-IRES-Cre-225285",
  "strain": null,
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 501483880,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 301021674,
   "name": "Oxtr-T2A-Cre"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0710505586,
  "injection_x": 9380,
  "injection_y": 1120,
  "injection_z": 8440,
  "product_id": 5,
  "specimen_name": "Oxtr-T2A-Cre;Ai14-318594",
  "strain": "unknown",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 596828910,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838899,
   "name": "Dbh-Cre_KH212"
  },
  "gender": "F",
  "injection_structures": "128/147/162/460/795/867/898",
  "injection_volume": 0.013244005425,
  "injection_x": 10620,
  "injection_y": 3990,
  "injection_z": 4850,
  "product_id": 36,
  "specimen_name": "Dbh-Cre_KH212-241303",
  "strain": null,
  "structure_abbrev": "LDT",
  "structure_id": 162,
  "structure_name": "Laterodorsal tegmental nucleus",
  "primary_injection_structure": 128,
  "data_set_id": 519735413,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "104/119/583/985",
  "injection_volume": 0.415792048,
  "injection_x": 3530,
  "injection_y": 4540,
  "injection_z": 8340,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre-103",
  "strain": null,
  "structure_abbrev": "AIv",
  "structure_id": 119,
  "structure_name": "Agranular insular area, ventral part",
  "primary_injection_structure": 104,
  "data_set_id": 166153483,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "382/385/417/533/312782546",
  "injection_volume": 0.5453169624,
  "injection_x": 7800,
  "injection_y": 1240,
  "injection_z": 3570,
  "product_id": 36,
  "specimen_name": "Emx1-IRES-Cre-321463",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 382,
  "data_set_id": 598605738,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "F",
  "injection_structures": "879/886",
  "injection_volume": 0.103035877,
  "injection_x": 8150,
  "injection_y": 1120,
  "injection_z": 6630,
  "product_id": 5,
  "specimen_name": "Syt6-Cre-603",
  "strain": "C57BL/6J",
  "structure_abbrev": "RSPv",
  "structure_id": 886,
  "structure_name": "Retrosplenial area, ventral part",
  "primary_injection_structure": 879,
  "data_set_id": 157711043,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837281,
   "name": "Calb2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "250/258/310/581",
  "injection_volume": 0.0464938207822266,
  "injection_x": 5510,
  "injection_y": 3560,
  "injection_z": 6040,
  "product_id": 5,
  "specimen_name": "Calb2-IRES-Cre-277",
  "strain": "C57BL/6J",
  "structure_abbrev": "TRS",
  "structure_id": 581,
  "structure_name": "Triangular nucleus of septum",
  "primary_injection_structure": 250,
  "data_set_id": 287950390,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "M",
  "injection_structures": "385/409",
  "injection_volume": 0.0299190962,
  "injection_x": 8850,
  "injection_y": 1490,
  "injection_z": 2260,
  "product_id": 36,
  "specimen_name": "Scnn1a-Tg3-Cre-195224",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 482639810,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 309779993,
   "name": "Ctgf-T2A-dgCre"
  },
  "gender": "M",
  "injection_structures": "985/993",
  "injection_volume": 0.018586117725,
  "injection_x": 3990,
  "injection_y": 2830,
  "injection_z": 7110,
  "product_id": 5,
  "specimen_name": "Ctgf-2A-dgCre-242375",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 524070052,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "312782574/312782628",
  "injection_volume": 0.1282535604,
  "injection_x": 9400,
  "injection_y": 2750,
  "injection_z": 2190,
  "product_id": 35,
  "specimen_name": "Ntsr1-Cre_GN220-317690",
  "strain": null,
  "structure_abbrev": "VISpor",
  "structure_id": 312782628,
  "structure_name": "Postrhinal area",
  "primary_injection_structure": 312782574,
  "data_set_id": 593407064,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837777,
   "name": "Sim1-Cre_KJ18"
  },
  "gender": "M",
  "injection_structures": "985/993",
  "injection_volume": 0.02529760975,
  "injection_x": 3380,
  "injection_y": 2220,
  "injection_z": 7650,
  "product_id": 5,
  "specimen_name": "Sim1-Cre_KJ18-324769",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 606570335,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 266649644,
   "name": "Drd3-Cre_KI196"
  },
  "gender": "F",
  "injection_structures": "12/100/197/749/946",
  "injection_volume": 0.042215068,
  "injection_x": 8610,
  "injection_y": 5570,
  "injection_z": 5270,
  "product_id": 5,
  "specimen_name": "Drd3-Cre_KI196-140776",
  "strain": null,
  "structure_abbrev": "IPN",
  "structure_id": 100,
  "structure_name": "Interpeduncular nucleus",
  "primary_injection_structure": 12,
  "data_set_id": 304859407,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 179697864,
   "name": "Rorb-IRES2-Cre"
  },
  "gender": "F",
  "injection_structures": "353",
  "injection_volume": 0.1594382972,
  "injection_x": 5630,
  "injection_y": 2320,
  "injection_z": 8650,
  "product_id": 5,
  "specimen_name": "Rorb-IRES2-Cre-D_LURC-131999",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-n",
  "structure_id": 353,
  "structure_name": "Primary somatosensory area, nose",
  "primary_injection_structure": 353,
  "data_set_id": 301583889,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838048,
   "name": "Slc17a6-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "147/280/460/795/867/898",
  "injection_volume": 0.083555437,
  "injection_x": 10570,
  "injection_y": 4210,
  "injection_z": 6560,
  "product_id": 5,
  "specimen_name": "Slc17a6-IRES-Cre-124428",
  "strain": "B6.129",
  "structure_abbrev": "PB",
  "structure_id": 867,
  "structure_name": "Parabrachial nucleus",
  "primary_injection_structure": 147,
  "data_set_id": 305404551,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838942,
   "name": "Htr2a-Cre_KM207"
  },
  "gender": "M",
  "injection_structures": "96/846/1007/1017/1033/1041",
  "injection_volume": 0.00645919225,
  "injection_x": 11160,
  "injection_y": 3220,
  "injection_z": 8380,
  "product_id": 5,
  "specimen_name": "Htr2a-Cre_KM207-130963",
  "strain": "unknown",
  "structure_abbrev": "DN",
  "structure_id": 846,
  "structure_name": "Dentate nucleus",
  "primary_injection_structure": 96,
  "data_set_id": 552283801,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "385/425/879/894/484682508",
  "injection_volume": 0.106491112,
  "injection_x": 9770,
  "injection_y": 2000,
  "injection_z": 3390,
  "product_id": 36,
  "specimen_name": "Rbp4-Cre_KL100-332692",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 606929366,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "385/417",
  "injection_volume": 0.4078438168,
  "injection_x": 7900,
  "injection_y": 1380,
  "injection_z": 8470,
  "product_id": 5,
  "specimen_name": "378-1141",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 113887162,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "F",
  "injection_structures": "385/425/879/894/1037",
  "injection_volume": 0.0078875465375,
  "injection_x": 9680,
  "injection_y": 1980,
  "injection_z": 8220,
  "product_id": 5,
  "specimen_name": "Syt6-Cre_KI148-3673",
  "strain": "C57BL/6J",
  "structure_abbrev": "RSPagl",
  "structure_id": 894,
  "structure_name": "Retrosplenial area, lateral agranular part",
  "primary_injection_structure": 385,
  "data_set_id": 267032286,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839285,
   "name": "Gpr26-Cre_KO250"
  },
  "gender": "F",
  "injection_structures": "329/361",
  "injection_volume": 0.1358939442,
  "injection_x": 7100,
  "injection_y": 1340,
  "injection_z": 8180,
  "product_id": 5,
  "specimen_name": "Gpr26-Cre-20",
  "strain": null,
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 175819113,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837328,
   "name": "Syt17-Cre_NO14"
  },
  "gender": "M",
  "injection_structures": "726",
  "injection_volume": 0.014211498175,
  "injection_x": 8920,
  "injection_y": 3240,
  "injection_z": 8370,
  "product_id": 5,
  "specimen_name": "Syt17-Cre_NO14_LURC-120290",
  "strain": "C57BL/6J",
  "structure_abbrev": "DG",
  "structure_id": 726,
  "structure_name": "Dentate gyrus",
  "primary_injection_structure": 726,
  "data_set_id": 293011759,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "918/312782574/312782628",
  "injection_volume": 0.1557133564,
  "injection_x": 9350,
  "injection_y": 2690,
  "injection_z": 9300,
  "product_id": 5,
  "specimen_name": "Ntsr1-Cre-3623",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISpor",
  "structure_id": 312782628,
  "structure_name": "Postrhinal area",
  "primary_injection_structure": 918,
  "data_set_id": 170261951,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0894824084,
  "injection_x": 8110,
  "injection_y": 1180,
  "injection_z": 3400,
  "product_id": 36,
  "specimen_name": "Cux2-IRES-Cre-214776",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 501117182,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761781,
   "name": "Efr3a-Cre_NO108"
  },
  "gender": "M",
  "injection_structures": "215",
  "injection_volume": 0.0858395132,
  "injection_x": 8370,
  "injection_y": 3410,
  "injection_z": 7210,
  "product_id": 5,
  "specimen_name": "Efr3a-Cre_NO108-132088",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "APN",
  "structure_id": 215,
  "structure_name": "Anterior pretectal nucleus",
  "primary_injection_structure": 215,
  "data_set_id": 299759175,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "329/382/417/423/463",
  "injection_volume": 0.1511302688,
  "injection_x": 7180,
  "injection_y": 2430,
  "injection_z": 8680,
  "product_id": 5,
  "specimen_name": "378-1859",
  "strain": "C57BL/6J",
  "structure_abbrev": "CA3",
  "structure_id": 463,
  "structure_name": "Field CA3",
  "primary_injection_structure": 329,
  "data_set_id": 272404772,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "385/409/425/312782574/312782628",
  "injection_volume": 0.814006032,
  "injection_x": 9290,
  "injection_y": 2220,
  "injection_z": 9410,
  "product_id": 5,
  "specimen_name": "378-1815",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 180296424,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 266649644,
   "name": "Drd3-Cre_KI196"
  },
  "gender": "M",
  "injection_structures": "879/886",
  "injection_volume": 0.1619951564,
  "injection_x": 6990,
  "injection_y": 1470,
  "injection_z": 6160,
  "product_id": 5,
  "specimen_name": "Drd3-Cre_KI196_LURC-128048",
  "strain": null,
  "structure_abbrev": "RSPv",
  "structure_id": 886,
  "structure_name": "Retrosplenial area, ventral part",
  "primary_injection_structure": 879,
  "data_set_id": 516838033,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "385/533",
  "injection_volume": 0.003658431875,
  "injection_x": 7980,
  "injection_y": 1100,
  "injection_z": 3470,
  "product_id": 36,
  "specimen_name": "A930038C07Rik-Tg1-Cre-299926",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 579628905,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0364038052,
  "injection_x": 9560,
  "injection_y": 1550,
  "injection_z": 3190,
  "product_id": 35,
  "specimen_name": "Tlx3-Cre_PL56-252236",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 528963991,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 266649644,
   "name": "Drd3-Cre_KI196"
  },
  "gender": "F",
  "injection_structures": "385/425/879/894/484682508",
  "injection_volume": 0.2014761616,
  "injection_x": 10070,
  "injection_y": 1970,
  "injection_z": 8240,
  "product_id": 5,
  "specimen_name": "Drd3-Cre_KI196-130851",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 298759552,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 308603484,
   "name": "Kcng4-Cre"
  },
  "gender": "M",
  "injection_structures": "128/214/749",
  "injection_volume": 0.3404979032,
  "injection_x": 8790,
  "injection_y": 4580,
  "injection_z": 6000,
  "product_id": 5,
  "specimen_name": "Kcng4-Cre-254802",
  "strain": "C57BL/6J",
  "structure_abbrev": "RN",
  "structure_id": 214,
  "structure_name": "Red nucleus",
  "primary_injection_structure": 128,
  "data_set_id": 543881679,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "402/409/541/1011/1027/312782574",
  "injection_volume": 0.016431171225,
  "injection_x": 8420,
  "injection_y": 1760,
  "injection_z": 2140,
  "product_id": 35,
  "specimen_name": "A930038C07Rik-Tg1-Cre-299928",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISal",
  "structure_id": 402,
  "structure_name": "Anterolateral visual area",
  "primary_injection_structure": 402,
  "data_set_id": 576684233,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838496,
   "name": "Grp-Cre_KH288"
  },
  "gender": "M",
  "injection_structures": "64/255",
  "injection_volume": 0.1016808016,
  "injection_x": 6010,
  "injection_y": 3820,
  "injection_z": 6930,
  "product_id": 5,
  "specimen_name": "Grp-Cre_KH288-3906",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "AV",
  "structure_id": 255,
  "structure_name": "Anteroventral nucleus of thalamus",
  "primary_injection_structure": 64,
  "data_set_id": 292478008,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837281,
   "name": "Calb2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "993",
  "injection_volume": 0.01083985105,
  "injection_x": 3640,
  "injection_y": 1970,
  "injection_z": 7320,
  "product_id": 5,
  "specimen_name": "Calb2-IRES-Cre-169",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 168454779,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.0450994873,
  "injection_x": 8680,
  "injection_y": 1690,
  "injection_z": 2930,
  "product_id": 35,
  "specimen_name": "Ntsr1-Cre_GN220-261410",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 543679575,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837446,
   "name": "Adcyap1-2A-Cre"
  },
  "gender": "M",
  "injection_structures": "119/583/952/961",
  "injection_volume": 0.2235440368,
  "injection_x": 4450,
  "injection_y": 5750,
  "injection_z": 8230,
  "product_id": 5,
  "specimen_name": "Adcyap1-2A-Cre-D-127414",
  "strain": "C57BL/6J",
  "structure_abbrev": "PIR",
  "structure_id": 961,
  "structure_name": "Piriform area",
  "primary_injection_structure": 119,
  "data_set_id": 551350026,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182693192,
   "name": "Chrna2-Cre_OE25"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.0054353733875,
  "injection_x": 9660,
  "injection_y": 1400,
  "injection_z": 8050,
  "product_id": 5,
  "specimen_name": "Chrna2-Cre_OE25-132664",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 299781566,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "F",
  "injection_structures": "394/312782546",
  "injection_volume": 0.0871407866,
  "injection_x": 7260,
  "injection_y": 1060,
  "injection_z": 7280,
  "product_id": 5,
  "specimen_name": "Tlx3-Cre_LURC-124582",
  "strain": null,
  "structure_abbrev": "VISa",
  "structure_id": 312782546,
  "structure_name": "Anterior area",
  "primary_injection_structure": 394,
  "data_set_id": 294533406,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "1020",
  "injection_volume": 0.015272418,
  "injection_x": 7350,
  "injection_y": 3930,
  "injection_z": 7040,
  "product_id": 5,
  "specimen_name": "378-1833",
  "strain": "C57BL/6J",
  "structure_abbrev": "PO",
  "structure_id": 1020,
  "structure_name": "Posterior complex of the thalamus",
  "primary_injection_structure": 1020,
  "data_set_id": 180708524,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182693192,
   "name": "Chrna2-Cre_OE25"
  },
  "gender": "M",
  "injection_structures": "151/159",
  "injection_volume": 0.3187141692,
  "injection_x": 2280,
  "injection_y": 5060,
  "injection_z": 6630,
  "product_id": 5,
  "specimen_name": "Chrna2-Cre_OE25-136326",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "AON",
  "structure_id": 159,
  "structure_name": "Anterior olfactory nucleus",
  "primary_injection_structure": 151,
  "data_set_id": 304719312,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "F",
  "injection_structures": "402/409/417",
  "injection_volume": 0.06180713,
  "injection_x": 8250,
  "injection_y": 1720,
  "injection_z": 2120,
  "product_id": 36,
  "specimen_name": "Tlx3-Cre_PL56-232875",
  "strain": null,
  "structure_abbrev": "VISal",
  "structure_id": 402,
  "structure_name": "Anterolateral visual area",
  "primary_injection_structure": 402,
  "data_set_id": 510848595,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182901722,
   "name": "Ppp1r17-Cre_NL146"
  },
  "gender": "F",
  "injection_structures": "382/463",
  "injection_volume": 0.02195976895,
  "injection_x": 6780,
  "injection_y": 1930,
  "injection_z": 6380,
  "product_id": 5,
  "specimen_name": "Ppp1r17-Cre_NL146-3963",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "CA1",
  "structure_id": 382,
  "structure_name": "Field CA1",
  "primary_injection_structure": 382,
  "data_set_id": 292479421,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "F",
  "injection_structures": "409/425/312782574/312782628",
  "injection_volume": 0.045411191,
  "injection_x": 9560,
  "injection_y": 2000,
  "injection_z": 2200,
  "product_id": 35,
  "specimen_name": "Scnn1a-Tg3-Cre-363349",
  "strain": null,
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 409,
  "data_set_id": 657162589,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182901722,
   "name": "Ppp1r17-Cre_NL146"
  },
  "gender": "M",
  "injection_structures": "329/382/423/463",
  "injection_volume": 0.0551873378,
  "injection_x": 7350,
  "injection_y": 2610,
  "injection_z": 8710,
  "product_id": 5,
  "specimen_name": "Ppp1r17-Cre_NL146-3920",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "CA3",
  "structure_id": 463,
  "structure_name": "Field CA3",
  "primary_injection_structure": 329,
  "data_set_id": 278399657,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "104/583/985/1057",
  "injection_volume": 0.3978940336,
  "injection_x": 3620,
  "injection_y": 4540,
  "injection_z": 8780,
  "product_id": 5,
  "specimen_name": "378-1006",
  "strain": "C57BL/6J",
  "structure_abbrev": "AId",
  "structure_id": 104,
  "structure_name": "Agranular insular area, dorsal part",
  "primary_injection_structure": 104,
  "data_set_id": 112596790,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "394/533/894",
  "injection_volume": 0.1173092242,
  "injection_x": 8200,
  "injection_y": 1030,
  "injection_z": 4250,
  "product_id": 35,
  "specimen_name": "Rbp4-Cre_KL100-315833",
  "strain": null,
  "structure_abbrev": "RSPagl",
  "structure_id": 894,
  "structure_name": "Retrosplenial area, lateral agranular part",
  "primary_injection_structure": 394,
  "data_set_id": 593018150,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839004,
   "name": "Cux2-CreERT2"
  },
  "gender": "M",
  "injection_structures": "918/926",
  "injection_volume": 0.2303402584,
  "injection_x": 9620,
  "injection_y": 4370,
  "injection_z": 9220,
  "product_id": 5,
  "specimen_name": "Cux2-CreERT2-184",
  "strain": "C57BL/6J",
  "structure_abbrev": "ENTm",
  "structure_id": 926,
  "structure_name": "Entorhinal area, medial part, dorsal zone",
  "primary_injection_structure": 918,
  "data_set_id": 264873809,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.0244599915,
  "injection_x": 9100,
  "injection_y": 1130,
  "injection_z": 8400,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg3-Cre-4781",
  "strain": "B6.C3H",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 166269819,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838496,
   "name": "Grp-Cre_KH288"
  },
  "gender": "F",
  "injection_structures": "48",
  "injection_volume": 0.0350964166,
  "injection_x": 5140,
  "injection_y": 2180,
  "injection_z": 6270,
  "product_id": 5,
  "specimen_name": "Grp-Cre_KH288-106",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "ACAv",
  "structure_id": 48,
  "structure_name": "Anterior cingulate area, ventral part",
  "primary_injection_structure": 48,
  "data_set_id": 265291552,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838942,
   "name": "Htr2a-Cre_KM207"
  },
  "gender": "M",
  "injection_structures": "39/48",
  "injection_volume": 0.0555524671839477,
  "injection_x": 4540,
  "injection_y": 2670,
  "injection_z": 6030,
  "product_id": 5,
  "specimen_name": "Htr2a-Cre_KM207-3915",
  "strain": null,
  "structure_abbrev": "ACAv",
  "structure_id": 48,
  "structure_name": "Anterior cingulate area, ventral part",
  "primary_injection_structure": 39,
  "data_set_id": 287714903,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761918,
   "name": "Oxtr-Cre_ON66"
  },
  "gender": "M",
  "injection_structures": "345/353/378/583/672",
  "injection_volume": 0.0419247136,
  "injection_x": 5390,
  "injection_y": 3560,
  "injection_z": 8980,
  "product_id": 5,
  "specimen_name": "Oxtr-Cre_ON66-3889",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "SSs",
  "structure_id": 378,
  "structure_name": "Supplemental somatosensory area",
  "primary_injection_structure": 345,
  "data_set_id": 286838022,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838622,
   "name": "Vip-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "1002",
  "injection_volume": 0.026473475,
  "injection_x": 8080,
  "injection_y": 2320,
  "injection_z": 10200,
  "product_id": 5,
  "specimen_name": "Vip-IRES-Cre-185",
  "strain": null,
  "structure_abbrev": "AUDp",
  "structure_id": 1002,
  "structure_name": "Primary auditory area",
  "primary_injection_structure": 1002,
  "data_set_id": 182040934,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "104/119",
  "injection_volume": 0.016414800325,
  "injection_x": 3550,
  "injection_y": 4760,
  "injection_z": 8550,
  "product_id": 5,
  "specimen_name": "Ntsr1-Cre-3561",
  "strain": "C57BL/6J",
  "structure_abbrev": "AId",
  "structure_id": 104,
  "structure_name": "Agranular insular area, dorsal part",
  "primary_injection_structure": 104,
  "data_set_id": 159322514,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837858,
   "name": "Pvalb-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "35/115/128/795/872/606826663",
  "injection_volume": 0.06011359720625,
  "injection_x": 9460,
  "injection_y": 3640,
  "injection_z": 5930,
  "product_id": 5,
  "specimen_name": "Pvalb-IRES-Cre-5101",
  "strain": "B6.129",
  "structure_abbrev": "PAG",
  "structure_id": 795,
  "structure_name": "Periaqueductal gray",
  "primary_injection_structure": 35,
  "data_set_id": 266099165,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761918,
   "name": "Oxtr-Cre_ON66"
  },
  "gender": "M",
  "injection_structures": "119/583/672/952/985",
  "injection_volume": 0.1418851448,
  "injection_x": 4370,
  "injection_y": 5180,
  "injection_z": 8420,
  "product_id": 5,
  "specimen_name": "Oxtr-Cre_ON66-3864",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 119,
  "data_set_id": 267762146,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837324,
   "name": "Slc18a2-Cre_OZ14"
  },
  "gender": "M",
  "injection_structures": "337/369/985",
  "injection_volume": 0.01663069555,
  "injection_x": 5700,
  "injection_y": 1610,
  "injection_z": 7430,
  "product_id": 5,
  "specimen_name": "Slc18a2-Cre_OZ14-3582",
  "strain": null,
  "structure_abbrev": "SSp-ll",
  "structure_id": 337,
  "structure_name": "Primary somatosensory area, lower limb",
  "primary_injection_structure": 337,
  "data_set_id": 266643194,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "429/589508451",
  "injection_volume": 0.1230446448,
  "injection_x": 12840,
  "injection_y": 5120,
  "injection_z": 7350,
  "product_id": 5,
  "specimen_name": "378-1187",
  "strain": "C57BL/6J",
  "structure_abbrev": "SPVC",
  "structure_id": 429,
  "structure_name": "Spinal nucleus of the trigeminal, caudal part",
  "primary_injection_structure": 429,
  "data_set_id": 114402050,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "507/731/746/972",
  "injection_volume": 0.349502953920312,
  "injection_x": 2400,
  "injection_y": 3560,
  "injection_z": 5690,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-285",
  "strain": "B6.129",
  "structure_abbrev": "ORBm",
  "structure_id": 731,
  "structure_name": "Orbital area, medial part",
  "primary_injection_structure": 507,
  "data_set_id": 272781246,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839285,
   "name": "Gpr26-Cre_KO250"
  },
  "gender": "M",
  "injection_structures": "879/886",
  "injection_volume": 0.0414482425,
  "injection_x": 8180,
  "injection_y": 570,
  "injection_z": 6310,
  "product_id": 5,
  "specimen_name": "Gpr26-Cre_KO250-123757",
  "strain": null,
  "structure_abbrev": "RSPd",
  "structure_id": 879,
  "structure_name": "Retrosplenial area, dorsal part",
  "primary_injection_structure": 879,
  "data_set_id": 298720191,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "39/44/731/746/972/993",
  "injection_volume": 0.2875039524,
  "injection_x": 2810,
  "injection_y": 2140,
  "injection_z": 5250,
  "product_id": 35,
  "specimen_name": "Rbp4-Cre_KL100-211660",
  "strain": null,
  "structure_abbrev": "ORBm",
  "structure_id": 731,
  "structure_name": "Orbital area, medial part",
  "primary_injection_structure": 39,
  "data_set_id": 496554237,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182693192,
   "name": "Chrna2-Cre_OE25"
  },
  "gender": "F",
  "injection_structures": "39/993",
  "injection_volume": 0.095908288,
  "injection_x": 4800,
  "injection_y": 1690,
  "injection_z": 6460,
  "product_id": 5,
  "specimen_name": "Chrna2-Cre_OE25-3906",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 39,
  "data_set_id": 292174974,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.044148902,
  "injection_x": 9220,
  "injection_y": 1860,
  "injection_z": 2990,
  "product_id": 35,
  "specimen_name": "Emx1-IRES-Cre-244142",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 522409371,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837979,
   "name": "Drd1a-Cre_EY262"
  },
  "gender": "M",
  "injection_structures": "56/342/754",
  "injection_volume": 0.2351549004,
  "injection_x": 3750,
  "injection_y": 6590,
  "injection_z": 6500,
  "product_id": 5,
  "specimen_name": "Drd1a-Cre-4932",
  "strain": "C57BL/6J",
  "structure_abbrev": "ACB",
  "structure_id": 56,
  "structure_name": "Nucleus accumbens",
  "primary_injection_structure": 56,
  "data_set_id": 170785775,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "993",
  "injection_volume": 0.014314692175,
  "injection_x": 3770,
  "injection_y": 1500,
  "injection_z": 6520,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-147",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 182461051,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838496,
   "name": "Grp-Cre_KH288"
  },
  "gender": "F",
  "injection_structures": "39/985/993",
  "injection_volume": 0.1140049974,
  "injection_x": 5040,
  "injection_y": 1640,
  "injection_z": 6290,
  "product_id": 5,
  "specimen_name": "Grp-Cre_KH288-108",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 39,
  "data_set_id": 265292679,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "475/1044/560581563",
  "injection_volume": 0.03604678875,
  "injection_x": 8690,
  "injection_y": 4060,
  "injection_z": 7980,
  "product_id": 5,
  "specimen_name": "378-1829",
  "strain": "C57BL/6J",
  "structure_abbrev": "MG",
  "structure_id": 475,
  "structure_name": "Medial geniculate complex",
  "primary_injection_structure": 475,
  "data_set_id": 180520257,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "502/918/926",
  "injection_volume": 0.1947015784,
  "injection_x": 9450,
  "injection_y": 5210,
  "injection_z": 9310,
  "product_id": 5,
  "specimen_name": "C57BL6J-314643",
  "strain": "C57BL/6J",
  "structure_abbrev": "ENTm",
  "structure_id": 926,
  "structure_name": "Entorhinal area, medial part, dorsal zone",
  "primary_injection_structure": 502,
  "data_set_id": 585051446,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "385/417/312782546",
  "injection_volume": 0.00233705101875,
  "injection_x": 7730,
  "injection_y": 1210,
  "injection_z": 3250,
  "product_id": 36,
  "specimen_name": "A930038C07Rik-Tg1-Cre-241723",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 519488493,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "985/993",
  "injection_volume": 0.3710439936,
  "injection_x": 3940,
  "injection_y": 3410,
  "injection_z": 7780,
  "product_id": 5,
  "specimen_name": "378-1862",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOp",
  "structure_id": 985,
  "structure_name": "Primary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 272697944,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "F",
  "injection_structures": "329/182305689",
  "injection_volume": 0.0981225392,
  "injection_x": 6600,
  "injection_y": 1910,
  "injection_z": 7960,
  "product_id": 5,
  "specimen_name": "Syt6-Cre-637",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 159223769,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "329/417",
  "injection_volume": 0.1088658676,
  "injection_x": 7510,
  "injection_y": 1160,
  "injection_z": 2610,
  "product_id": 36,
  "specimen_name": "Emx1-IRES-Cre-244146",
  "strain": null,
  "structure_abbrev": "VISrl",
  "structure_id": 417,
  "structure_name": "Rostrolateral visual area",
  "primary_injection_structure": 329,
  "data_set_id": 521600943,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "502/879/886",
  "injection_volume": 0.2557951704,
  "injection_x": 7880,
  "injection_y": 710,
  "injection_z": 4900,
  "product_id": 36,
  "specimen_name": "Emx1-IRES-Cre-225284",
  "strain": null,
  "structure_abbrev": "RSPv",
  "structure_id": 886,
  "structure_name": "Retrosplenial area, ventral part",
  "primary_injection_structure": 502,
  "data_set_id": 501484658,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 298341195,
   "name": "Htr3a-Cre_NO152"
  },
  "gender": "F",
  "injection_structures": "287/351",
  "injection_volume": 0.1164053016,
  "injection_x": 5470,
  "injection_y": 5070,
  "injection_z": 6240,
  "product_id": 5,
  "specimen_name": "Htr3a-Cre_NO152-145243",
  "strain": null,
  "structure_abbrev": "BST",
  "structure_id": 351,
  "structure_name": "Bed nuclei of the stria terminalis",
  "primary_injection_structure": 287,
  "data_set_id": 305645843,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837834,
   "name": "Crh-IRES-Cre_BL"
  },
  "gender": "M",
  "injection_structures": "44/731/746/972",
  "injection_volume": 0.02986397365,
  "injection_x": 3070,
  "injection_y": 3960,
  "injection_z": 6070,
  "product_id": 5,
  "specimen_name": "Crh-IRES-Cre-329",
  "strain": "B6.129",
  "structure_abbrev": "ORBm",
  "structure_id": 731,
  "structure_name": "Orbital area, medial part",
  "primary_injection_structure": 44,
  "data_set_id": 277855624,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "128/294/795",
  "injection_volume": 0.469684670767021,
  "injection_x": 9050,
  "injection_y": 3490,
  "injection_z": 5970,
  "product_id": 5,
  "specimen_name": "378-1095",
  "strain": "C57BL/6J",
  "structure_abbrev": "PAG",
  "structure_id": 795,
  "structure_name": "Periaqueductal gray",
  "primary_injection_structure": 128,
  "data_set_id": 113096571,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 179697864,
   "name": "Rorb-IRES2-Cre"
  },
  "gender": "F",
  "injection_structures": "345",
  "injection_volume": 0.01912547175,
  "injection_x": 4500,
  "injection_y": 2640,
  "injection_z": 9040,
  "product_id": 5,
  "specimen_name": "Rorb-IRES2-Cre-D_LURC-128094",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-m",
  "structure_id": 345,
  "structure_name": "Primary somatosensory area, mouth",
  "primary_injection_structure": 345,
  "data_set_id": 301581763,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839513,
   "name": "Drd2-Cre_ER44"
  },
  "gender": "M",
  "injection_structures": "672",
  "injection_volume": 0.114002077,
  "injection_x": 6430,
  "injection_y": 2700,
  "injection_z": 8540,
  "product_id": 5,
  "specimen_name": "Drd2-Cre-2067",
  "strain": "C57BL/6J",
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 672,
  "data_set_id": 175732996,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 301613169,
   "name": "Sepw1-Cre_NP39"
  },
  "gender": "M",
  "injection_structures": "378/1002/1011",
  "injection_volume": 0.0288807715,
  "injection_x": 7450,
  "injection_y": 2320,
  "injection_z": 9990,
  "product_id": 5,
  "specimen_name": "Sepw1-Cre_NP39-174463",
  "strain": null,
  "structure_abbrev": "AUDd",
  "structure_id": 1011,
  "structure_name": "Dorsal auditory area",
  "primary_injection_structure": 378,
  "data_set_id": 477834984,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 517749378,
   "name": "Rorb-IRES2-Cre-neo"
  },
  "gender": "F",
  "injection_structures": "993",
  "injection_volume": 0.012178689075,
  "injection_x": 3390,
  "injection_y": 2070,
  "injection_z": 4170,
  "product_id": 45,
  "specimen_name": "Rorb-IRES2-Cre-neo-330784",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 615543017,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182693192,
   "name": "Chrna2-Cre_OE25"
  },
  "gender": "F",
  "injection_structures": "879/886",
  "injection_volume": 0.0339766931,
  "injection_x": 7630,
  "injection_y": 820,
  "injection_z": 6240,
  "product_id": 5,
  "specimen_name": "Chrna2-Cre_OE25-3899",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "RSPv",
  "structure_id": 886,
  "structure_name": "Retrosplenial area, ventral part",
  "primary_injection_structure": 879,
  "data_set_id": 292172100,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "F",
  "injection_structures": "731/746/972/993",
  "injection_volume": 0.0052324986875,
  "injection_x": 2570,
  "injection_y": 3300,
  "injection_z": 6080,
  "product_id": 5,
  "specimen_name": "Tlx3-Cre_PL56-262307",
  "strain": null,
  "structure_abbrev": "PL",
  "structure_id": 972,
  "structure_name": "Prelimbic area",
  "primary_injection_structure": 731,
  "data_set_id": 545434921,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "394/312782546",
  "injection_volume": 0.1692370428,
  "injection_x": 7350,
  "injection_y": 1200,
  "injection_z": 7320,
  "product_id": 5,
  "specimen_name": "378-1348",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISam",
  "structure_id": 394,
  "structure_name": "Anteromedial visual area",
  "primary_injection_structure": 394,
  "data_set_id": 126861679,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839108,
   "name": "Wfs1-Tg2-CreERT2"
  },
  "gender": "F",
  "injection_structures": "918/926",
  "injection_volume": 0.41422346,
  "injection_x": 9590,
  "injection_y": 3770,
  "injection_z": 9360,
  "product_id": 5,
  "specimen_name": "Wfs1-Tg2-CreERT2-3541",
  "strain": "C57BL/6J",
  "structure_abbrev": "ENTm",
  "structure_id": 926,
  "structure_name": "Entorhinal area, medial part, dorsal zone",
  "primary_injection_structure": 918,
  "data_set_id": 278435152,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "385/879/886/894",
  "injection_volume": 0.013654659725,
  "injection_x": 9500,
  "injection_y": 1020,
  "injection_z": 3830,
  "product_id": 36,
  "specimen_name": "A930038C07Rik-Tg1-Cre-241721",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 519486602,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "502/918/926",
  "injection_volume": 0.2396796388,
  "injection_x": 9370,
  "injection_y": 5310,
  "injection_z": 9170,
  "product_id": 5,
  "specimen_name": "C57BL6J-314642",
  "strain": "C57BL/6J",
  "structure_abbrev": "ENTm",
  "structure_id": 926,
  "structure_name": "Entorhinal area, medial part, dorsal zone",
  "primary_injection_structure": 502,
  "data_set_id": 585026021,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837481,
   "name": "Scnn1a-Tg2-Cre"
  },
  "gender": "M",
  "injection_structures": "170/218/733",
  "injection_volume": 0.12810903,
  "injection_x": 7350,
  "injection_y": 3250,
  "injection_z": 7820,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg2-Cre-3488",
  "strain": "C57BL/6J",
  "structure_abbrev": "LGd",
  "structure_id": 170,
  "structure_name": "Dorsal part of the lateral geniculate complex",
  "primary_injection_structure": 170,
  "data_set_id": 263241470,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839331,
   "name": "Nr5a1-Cre"
  },
  "gender": "F",
  "injection_structures": "329/417",
  "injection_volume": 0.0341513879,
  "injection_x": 7610,
  "injection_y": 1190,
  "injection_z": 8710,
  "product_id": 5,
  "specimen_name": "Nr5a1-Cre-2541",
  "strain": "B6.FVB",
  "structure_abbrev": "VISrl",
  "structure_id": 417,
  "structure_name": "Rostrolateral visual area",
  "primary_injection_structure": 329,
  "data_set_id": 277851379,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "993",
  "injection_volume": 0.139516965,
  "injection_x": 3760,
  "injection_y": 2320,
  "injection_z": 6960,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-3756",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 156394513,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "39/48",
  "injection_volume": 0.0310888291,
  "injection_x": 4940,
  "injection_y": 1830,
  "injection_z": 5440,
  "product_id": 36,
  "specimen_name": "Rbp4-Cre_KL100-247905",
  "strain": null,
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 526783054,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "394/533/312782546",
  "injection_volume": 0.0059435438125,
  "injection_x": 7640,
  "injection_y": 1010,
  "injection_z": 3600,
  "product_id": 35,
  "specimen_name": "A930038C07Rik-Tg1-Cre-242038",
  "strain": null,
  "structure_abbrev": "VISa",
  "structure_id": 312782546,
  "structure_name": "Anterior area",
  "primary_injection_structure": 394,
  "data_set_id": 520012330,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "918/952/961",
  "injection_volume": 0.2115607144,
  "injection_x": 7400,
  "injection_y": 5620,
  "injection_z": 9720,
  "product_id": 5,
  "specimen_name": "378-1510",
  "strain": "C57BL/6J",
  "structure_abbrev": "PIR",
  "structure_id": 961,
  "structure_name": "Piriform area",
  "primary_injection_structure": 918,
  "data_set_id": 146984209,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.0437422706,
  "injection_x": 8520,
  "injection_y": 890,
  "injection_z": 8130,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg3-Cre-4824",
  "strain": "B6.C3H",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 182089608,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "1002/1027",
  "injection_volume": 0.1141708428,
  "injection_x": 8170,
  "injection_y": 2530,
  "injection_z": 9650,
  "product_id": 5,
  "specimen_name": "378-1506",
  "strain": "C57BL/6J",
  "structure_abbrev": "AUDp",
  "structure_id": 1002,
  "structure_name": "Primary auditory area",
  "primary_injection_structure": 1002,
  "data_set_id": 146858006,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "128/214",
  "injection_volume": 0.0795814145,
  "injection_x": 8530,
  "injection_y": 4720,
  "injection_z": 6530,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre_KL100-121999",
  "strain": "C57BL/6J",
  "structure_abbrev": "RN",
  "structure_id": 214,
  "structure_name": "Red nucleus",
  "primary_injection_structure": 128,
  "data_set_id": 300624130,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838899,
   "name": "Dbh-Cre_KH212"
  },
  "gender": "M",
  "injection_structures": "128/162/795/867/1052",
  "injection_volume": 0.0026497478875,
  "injection_x": 10180,
  "injection_y": 3810,
  "injection_z": 4550,
  "product_id": 36,
  "specimen_name": "Dbh-Cre_KH212-205739",
  "strain": null,
  "structure_abbrev": "PAG",
  "structure_id": 795,
  "structure_name": "Periaqueductal gray",
  "primary_injection_structure": 128,
  "data_set_id": 496114558,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "39/48/993",
  "injection_volume": 0.205057944,
  "injection_x": 4450,
  "injection_y": 2260,
  "injection_z": 6290,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-3928",
  "strain": null,
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 286774770,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 179697864,
   "name": "Rorb-IRES2-Cre"
  },
  "gender": "M",
  "injection_structures": "385/409",
  "injection_volume": 0.1141038598,
  "injection_x": 9260,
  "injection_y": 1680,
  "injection_z": 9300,
  "product_id": 5,
  "specimen_name": "Rorb-IRES2-Cre-D-131375",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 385,
  "data_set_id": 301617370,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "337/361",
  "injection_volume": 0.01629913705,
  "injection_x": 6130,
  "injection_y": 1310,
  "injection_z": 7430,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-3880",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-ll",
  "structure_id": 337,
  "structure_name": "Primary somatosensory area, lower limb",
  "primary_injection_structure": 337,
  "data_set_id": 264629246,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838302,
   "name": "Gng7-Cre_KH71"
  },
  "gender": "M",
  "injection_structures": "1002/1011",
  "injection_volume": 0.3384270848,
  "injection_x": 7620,
  "injection_y": 3120,
  "injection_z": 9760,
  "product_id": 5,
  "specimen_name": "Gng7-Cre-967",
  "strain": "C57BL/6J",
  "structure_abbrev": "AUDp",
  "structure_id": 1002,
  "structure_name": "Primary auditory area",
  "primary_injection_structure": 1002,
  "data_set_id": 157954927,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838942,
   "name": "Htr2a-Cre_KM207"
  },
  "gender": "M",
  "injection_structures": "159",
  "injection_volume": 0.02434342295,
  "injection_x": 2690,
  "injection_y": 6120,
  "injection_z": 6690,
  "product_id": 5,
  "specimen_name": "Htr2a-Cre_KM207-3918",
  "strain": null,
  "structure_abbrev": "AON",
  "structure_id": 159,
  "structure_name": "Anterior olfactory nucleus",
  "primary_injection_structure": 159,
  "data_set_id": 287715650,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837777,
   "name": "Sim1-Cre_KJ18"
  },
  "gender": "M",
  "injection_structures": "985/993",
  "injection_volume": 0.010591760375,
  "injection_x": 4880,
  "injection_y": 1700,
  "injection_z": 6700,
  "product_id": 5,
  "specimen_name": "Sim1-Cre_KJ18-123150",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 297854981,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "409/312782574",
  "injection_volume": 0.03766467075,
  "injection_x": 9140,
  "injection_y": 2300,
  "injection_z": 9430,
  "product_id": 5,
  "specimen_name": "Ntsr1-Cre-3554",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISli",
  "structure_id": 312782574,
  "structure_name": "Laterointermediate area",
  "primary_injection_structure": 409,
  "data_set_id": 159320367,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837834,
   "name": "Crh-IRES-Cre_BL"
  },
  "gender": "M",
  "injection_structures": "329/353",
  "injection_volume": 0.0657814857,
  "injection_x": 5920,
  "injection_y": 1790,
  "injection_z": 9230,
  "product_id": 5,
  "specimen_name": "Crh-IRES-Cre-301",
  "strain": "B6.129",
  "structure_abbrev": "SSp-n",
  "structure_id": 353,
  "structure_name": "Primary somatosensory area, nose",
  "primary_injection_structure": 329,
  "data_set_id": 266913240,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838828,
   "name": "Gal-Cre_KI87"
  },
  "gender": "M",
  "injection_structures": "1/126/830/980/1004/1126",
  "injection_volume": 0.0414527818887104,
  "injection_x": 7730,
  "injection_y": 6660,
  "injection_z": 6040,
  "product_id": 5,
  "specimen_name": "Gal-Cre-1285",
  "strain": "C57BL/6J",
  "structure_abbrev": "PVp",
  "structure_id": 126,
  "structure_name": "Periventricular hypothalamic nucleus, posterior part",
  "primary_injection_structure": 1,
  "data_set_id": 168005102,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838302,
   "name": "Gng7-Cre_KH71"
  },
  "gender": "F",
  "injection_structures": "329/353/182305689",
  "injection_volume": 0.5826105096,
  "injection_x": 6020,
  "injection_y": 2480,
  "injection_z": 8350,
  "product_id": 5,
  "specimen_name": "Gng7-Cre-970",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-n",
  "structure_id": 353,
  "structure_name": "Primary somatosensory area, nose",
  "primary_injection_structure": 329,
  "data_set_id": 158017916,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "394/533/312782546",
  "injection_volume": 0.3082546096,
  "injection_x": 7920,
  "injection_y": 690,
  "injection_z": 3810,
  "product_id": 36,
  "specimen_name": "Cux2-IRES-Cre-358006.0",
  "strain": null,
  "structure_abbrev": "VISa",
  "structure_id": 312782546,
  "structure_name": "Anterior area",
  "primary_injection_structure": 394,
  "data_set_id": 653191449,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "993",
  "injection_volume": 0.2707409348,
  "injection_x": 3600,
  "injection_y": 1650,
  "injection_z": 7370,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-93",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 168002073,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "F",
  "injection_structures": "886",
  "injection_volume": 0.1096448696,
  "injection_x": 8460,
  "injection_y": 1440,
  "injection_z": 6740,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg3-Cre-4831",
  "strain": "B6.C3H",
  "structure_abbrev": "RSPv",
  "structure_id": 886,
  "structure_name": "Retrosplenial area, ventral part",
  "primary_injection_structure": 886,
  "data_set_id": 181860879,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 553502711,
   "name": "Chat-IRES-Cre-neo"
  },
  "gender": "M",
  "injection_structures": "226/342/523/596",
  "injection_volume": 0.065585324,
  "injection_x": 4840,
  "injection_y": 6610,
  "injection_z": 6360,
  "product_id": 5,
  "specimen_name": "Chat-IRES-Cre-402",
  "strain": "B6.129",
  "structure_abbrev": "NDB",
  "structure_id": 596,
  "structure_name": "Diagonal band nucleus",
  "primary_injection_structure": 226,
  "data_set_id": 167200755,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 301613169,
   "name": "Sepw1-Cre_NP39"
  },
  "gender": "F",
  "injection_structures": "329",
  "injection_volume": 0.0425259828,
  "injection_x": 6800,
  "injection_y": 1270,
  "injection_z": 8670,
  "product_id": 5,
  "specimen_name": "Sepw1-Cre_NP39-211003",
  "strain": null,
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 511234957,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 553502711,
   "name": "Chat-IRES-Cre-neo"
  },
  "gender": "M",
  "injection_structures": "136/661/978",
  "injection_volume": 0.03751346165,
  "injection_x": 11180,
  "injection_y": 6600,
  "injection_z": 7230,
  "product_id": 5,
  "specimen_name": "Chat-IRES-Cre-374",
  "strain": "B6.129",
  "structure_abbrev": "VII",
  "structure_id": 661,
  "structure_name": "Facial motor nucleus",
  "primary_injection_structure": 136,
  "data_set_id": 161460153,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "385/409/425/312782574/312782628",
  "injection_volume": 0.2518286008,
  "injection_x": 9450,
  "injection_y": 2220,
  "injection_z": 9460,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-131049",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 385,
  "data_set_id": 298829455,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "918/926/312782628",
  "injection_volume": 0.0797880769,
  "injection_x": 9810,
  "injection_y": 3090,
  "injection_z": 10090,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre-181",
  "strain": "C57BL/6J",
  "structure_abbrev": "ENTl",
  "structure_id": 918,
  "structure_name": "Entorhinal area, lateral part",
  "primary_injection_structure": 918,
  "data_set_id": 182794184,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839159,
   "name": "Cck-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "366/685/741/907/946",
  "injection_volume": 0.0589133909,
  "injection_x": 7260,
  "injection_y": 4690,
  "injection_z": 6300,
  "product_id": 5,
  "specimen_name": "Cck-IRES-Cre-121991",
  "strain": null,
  "structure_abbrev": "VM",
  "structure_id": 685,
  "structure_name": "Ventral medial nucleus of the thalamus",
  "primary_injection_structure": 366,
  "data_set_id": 306444486,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838496,
   "name": "Grp-Cre_KH288"
  },
  "gender": "F",
  "injection_structures": "731/746/972",
  "injection_volume": 0.03730990585,
  "injection_x": 2530,
  "injection_y": 3780,
  "injection_z": 6280,
  "product_id": 5,
  "specimen_name": "Grp-Cre_KH288-118",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "ORBvl",
  "structure_id": 746,
  "structure_name": "Orbital area, ventrolateral part",
  "primary_injection_structure": 731,
  "data_set_id": 266564027,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "402/409/1027/312782574",
  "injection_volume": 0.0374164245,
  "injection_x": 8470,
  "injection_y": 1980,
  "injection_z": 1900,
  "product_id": 35,
  "specimen_name": "Tlx3-Cre_PL56-250454",
  "strain": null,
  "structure_abbrev": "VISal",
  "structure_id": 402,
  "structure_name": "Anterolateral visual area",
  "primary_injection_structure": 402,
  "data_set_id": 528509838,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "385/409",
  "injection_volume": 0.2073077692,
  "injection_x": 9200,
  "injection_y": 2160,
  "injection_z": 2590,
  "product_id": 36,
  "specimen_name": "Emx1-IRES-Cre-194297",
  "strain": "B6.129",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 479981981,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838361,
   "name": "Etv1-CreERT2"
  },
  "gender": "F",
  "injection_structures": "39/48/972/993",
  "injection_volume": 0.2472242276,
  "injection_x": 4210,
  "injection_y": 2160,
  "injection_z": 6120,
  "product_id": 5,
  "specimen_name": "Etv1-CreERT2-1147",
  "strain": "C57BL/6J",
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 114428632,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838048,
   "name": "Slc17a6-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "4/294",
  "injection_volume": 0.0438185587,
  "injection_x": 9840,
  "injection_y": 2310,
  "injection_z": 6960,
  "product_id": 5,
  "specimen_name": "Slc17a6-IRES-Cre-121218",
  "strain": "B6.129",
  "structure_abbrev": "SCm",
  "structure_id": 294,
  "structure_name": "Superior colliculus, motor related",
  "primary_injection_structure": 4,
  "data_set_id": 294174290,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "15/351",
  "injection_volume": 0.0311778425,
  "injection_x": 5540,
  "injection_y": 4870,
  "injection_z": 6550,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre_KL100-123745",
  "strain": null,
  "structure_abbrev": "BST",
  "structure_id": 351,
  "structure_name": "Bed nuclei of the stria terminalis",
  "primary_injection_structure": 15,
  "data_set_id": 308386222,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182693192,
   "name": "Chrna2-Cre_OE25"
  },
  "gender": "M",
  "injection_structures": "378/1011",
  "injection_volume": 0.0793555441,
  "injection_x": 6860,
  "injection_y": 2710,
  "injection_z": 9710,
  "product_id": 5,
  "specimen_name": "Chrna2-Cre_OE25-122355",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "SSs",
  "structure_id": 378,
  "structure_name": "Supplemental somatosensory area",
  "primary_injection_structure": 378,
  "data_set_id": 296047084,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "F",
  "injection_structures": "329/182305689",
  "injection_volume": 0.1167720862,
  "injection_x": 6820,
  "injection_y": 1460,
  "injection_z": 8440,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg3-Cre-4793",
  "strain": "B6.C3H",
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 166459070,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "F",
  "injection_structures": "39/48",
  "injection_volume": 0.02600043145,
  "injection_x": 5400,
  "injection_y": 2200,
  "injection_z": 6010,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg3-Cre-4899",
  "strain": "B6.C3H",
  "structure_abbrev": "ACAv",
  "structure_id": 48,
  "structure_name": "Anterior cingulate area, ventral part",
  "primary_injection_structure": 39,
  "data_set_id": 268038262,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182693192,
   "name": "Chrna2-Cre_OE25"
  },
  "gender": "M",
  "injection_structures": "385/533",
  "injection_volume": 0.0324014362,
  "injection_x": 8220,
  "injection_y": 1170,
  "injection_z": 7980,
  "product_id": 5,
  "specimen_name": "Chrna2-Cre_OE25-123212",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 297597186,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 309483400,
   "name": "Foxp2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "194/342/470/718/733/797",
  "injection_volume": 0.018603965975,
  "injection_x": 6950,
  "injection_y": 5960,
  "injection_z": 7450,
  "product_id": 5,
  "specimen_name": "Foxp2-IRES-Cre-276063",
  "strain": null,
  "structure_abbrev": "STN",
  "structure_id": 470,
  "structure_name": "Subthalamic nucleus",
  "primary_injection_structure": 194,
  "data_set_id": 564688610,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "993",
  "injection_volume": 0.001593030578125,
  "injection_x": 3580,
  "injection_y": 2780,
  "injection_z": 7250,
  "product_id": 5,
  "specimen_name": "Ntsr1-Cre-3562",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 159372889,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837858,
   "name": "Pvalb-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "4",
  "injection_volume": 0.4187302448,
  "injection_x": 10540,
  "injection_y": 2250,
  "injection_z": 7380,
  "product_id": 5,
  "specimen_name": "Pvalb-IRES-Cre-5126",
  "strain": "B6.129",
  "structure_abbrev": "IC",
  "structure_id": 4,
  "structure_name": "Inferior colliculus",
  "primary_injection_structure": 4,
  "data_set_id": 277959325,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "369/985",
  "injection_volume": 0.1615864474,
  "injection_x": 4840,
  "injection_y": 1620,
  "injection_z": 7960,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-304",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-ul",
  "structure_id": 369,
  "structure_name": "Primary somatosensory area, upper limb",
  "primary_injection_structure": 369,
  "data_set_id": 286300594,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839468,
   "name": "Gabra6-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "1007/1017",
  "injection_volume": 0.0077458679375,
  "injection_x": 11810,
  "injection_y": 2210,
  "injection_z": 8020,
  "product_id": 5,
  "specimen_name": "Gabra6-IRES-Cre-117",
  "strain": "B6.129",
  "structure_abbrev": "SIM",
  "structure_id": 1007,
  "structure_name": "Simple lobule",
  "primary_injection_structure": 1007,
  "data_set_id": 159945113,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 310459452,
   "name": "Slc17a8-IRES2-Cre"
  },
  "gender": "M",
  "injection_structures": "795/872",
  "injection_volume": 0.05703186195,
  "injection_x": 9690,
  "injection_y": 3520,
  "injection_z": 5640,
  "product_id": 5,
  "specimen_name": "Slc17a8-IRES2-Cre-257840",
  "strain": null,
  "structure_abbrev": "DR",
  "structure_id": 872,
  "structure_name": "Dorsal nucleus raphe",
  "primary_injection_structure": 795,
  "data_set_id": 547510030,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839075,
   "name": "Pomc-Cre_BL"
  },
  "gender": "M",
  "injection_structures": "126/223/693/830/1004/1126",
  "injection_volume": 0.0096459115375,
  "injection_x": 7310,
  "injection_y": 6860,
  "injection_z": 5990,
  "product_id": 5,
  "specimen_name": "Pomc-Cre (BL)-758",
  "strain": null,
  "structure_abbrev": "ARH",
  "structure_id": 223,
  "structure_name": "Arcuate hypothalamic nucleus",
  "primary_injection_structure": 126,
  "data_set_id": 586447435,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "337/361/894/985/993",
  "injection_volume": 1.015067536,
  "injection_x": 5760,
  "injection_y": 820,
  "injection_z": 6460,
  "product_id": 5,
  "specimen_name": "378-1823",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOp",
  "structure_id": 985,
  "structure_name": "Primary motor area",
  "primary_injection_structure": 337,
  "data_set_id": 180720175,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "353/378",
  "injection_volume": 0.5544967792,
  "injection_x": 5900,
  "injection_y": 3460,
  "injection_z": 9160,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-122232",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSs",
  "structure_id": 378,
  "structure_name": "Supplemental somatosensory area",
  "primary_injection_structure": 353,
  "data_set_id": 297855879,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182901722,
   "name": "Ppp1r17-Cre_NL146"
  },
  "gender": "M",
  "injection_structures": "463/726",
  "injection_volume": 0.0668345888,
  "injection_x": 6540,
  "injection_y": 2860,
  "injection_z": 6610,
  "product_id": 5,
  "specimen_name": "Ppp1r17-Cre_NL146-3917",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "DG",
  "structure_id": 726,
  "structure_name": "Dentate gyrus",
  "primary_injection_structure": 463,
  "data_set_id": 278504263,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 266649644,
   "name": "Drd3-Cre_KI196"
  },
  "gender": "F",
  "injection_structures": "382/502/484682470",
  "injection_volume": 0.165693402,
  "injection_x": 9140,
  "injection_y": 4650,
  "injection_z": 9540,
  "product_id": 5,
  "specimen_name": "Drd3-Cre_KI196_LURC-123358",
  "strain": null,
  "structure_abbrev": "ProS",
  "structure_id": 484682470,
  "structure_name": "Prosubiculum",
  "primary_injection_structure": 382,
  "data_set_id": 293701770,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837315,
   "name": "Avp-IRES2-Cre"
  },
  "gender": "F",
  "injection_structures": "390/403",
  "injection_volume": 0.0015520799,
  "injection_x": 6260,
  "injection_y": 6620,
  "injection_z": 7270,
  "product_id": 5,
  "specimen_name": "Avp-IRES2-Cre-D-3561",
  "strain": "B6.129",
  "structure_abbrev": "MEA",
  "structure_id": 403,
  "structure_name": "Medial amygdalar nucleus",
  "primary_injection_structure": 390,
  "data_set_id": 287666431,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837777,
   "name": "Sim1-Cre_KJ18"
  },
  "gender": "F",
  "injection_structures": "337/361/894/312782546",
  "injection_volume": 0.097948305,
  "injection_x": 6800,
  "injection_y": 920,
  "injection_z": 6950,
  "product_id": 5,
  "specimen_name": "Sim1-Cre_KJ18-132876",
  "strain": null,
  "structure_abbrev": "SSp-tr",
  "structure_id": 361,
  "structure_name": "Primary somatosensory area, trunk",
  "primary_injection_structure": 337,
  "data_set_id": 299784429,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838942,
   "name": "Htr2a-Cre_KM207"
  },
  "gender": "F",
  "injection_structures": "39/993",
  "injection_volume": 0.1529806166,
  "injection_x": 4310,
  "injection_y": 1940,
  "injection_z": 6250,
  "product_id": 5,
  "specimen_name": "Htr2a-Cre_KM207-3471",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 39,
  "data_set_id": 277957202,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838942,
   "name": "Htr2a-Cre_KM207"
  },
  "gender": "F",
  "injection_structures": "895/918/922",
  "injection_volume": 0.0465711729,
  "injection_x": 8690,
  "injection_y": 4970,
  "injection_z": 10400,
  "product_id": 5,
  "specimen_name": "Htr2a-Cre_KM207-123935",
  "strain": null,
  "structure_abbrev": "ENTl",
  "structure_id": 918,
  "structure_name": "Entorhinal area, lateral part",
  "primary_injection_structure": 895,
  "data_set_id": 301988879,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839285,
   "name": "Gpr26-Cre_KO250"
  },
  "gender": "F",
  "injection_structures": "993",
  "injection_volume": 0.2859407936,
  "injection_x": 2740,
  "injection_y": 2160,
  "injection_z": 7730,
  "product_id": 5,
  "specimen_name": "Gpr26-Cre_KO250-132716",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 301267162,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "39/993",
  "injection_volume": 0.1705569068,
  "injection_x": 4180,
  "injection_y": 2160,
  "injection_z": 6300,
  "product_id": 5,
  "specimen_name": "378-1351",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 39,
  "data_set_id": 141603190,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837779,
   "name": "Pcp2-Cre_GN135"
  },
  "gender": "F",
  "injection_structures": "936/1007/1017",
  "injection_volume": 0.018863009375,
  "injection_x": 11790,
  "injection_y": 2080,
  "injection_z": 7810,
  "product_id": 5,
  "specimen_name": "Pcp2-Cre (NH)-3571",
  "strain": "C57BL/6J",
  "structure_abbrev": "AN",
  "structure_id": 1017,
  "structure_name": "Ansiform lobule",
  "primary_injection_structure": 936,
  "data_set_id": 147633169,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 266649644,
   "name": "Drd3-Cre_KI196"
  },
  "gender": "F",
  "injection_structures": "409/425",
  "injection_volume": 0.00684846785,
  "injection_x": 9520,
  "injection_y": 1990,
  "injection_z": 9130,
  "product_id": 5,
  "specimen_name": "Drd3-Cre_KI196-130854",
  "strain": null,
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 409,
  "data_set_id": 530018580,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "337/985",
  "injection_volume": 0.1725958556,
  "injection_x": 5370,
  "injection_y": 1590,
  "injection_z": 7490,
  "product_id": 5,
  "specimen_name": "378-697",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOp",
  "structure_id": 985,
  "structure_name": "Primary motor area",
  "primary_injection_structure": 337,
  "data_set_id": 100141563,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838496,
   "name": "Grp-Cre_KH288"
  },
  "gender": "F",
  "injection_structures": "48",
  "injection_volume": 0.037047871,
  "injection_x": 4330,
  "injection_y": 2820,
  "injection_z": 6000,
  "product_id": 5,
  "specimen_name": "Grp-Cre-79",
  "strain": "B6.FVB(ICR)",
  "structure_abbrev": "ACAv",
  "structure_id": 48,
  "structure_name": "Anterior cingulate area, ventral part",
  "primary_injection_structure": 48,
  "data_set_id": 184213701,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "329",
  "injection_volume": 0.1754450684,
  "injection_x": 7310,
  "injection_y": 1890,
  "injection_z": 9080,
  "product_id": 5,
  "specimen_name": "378-1488",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 127866392,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "797/1031",
  "injection_volume": 0.0209611857,
  "injection_x": 6820,
  "injection_y": 5340,
  "injection_z": 7540,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre_KL100-137546",
  "strain": "C57BL/6J",
  "structure_abbrev": "GPi",
  "structure_id": 1031,
  "structure_name": "Globus pallidus, internal segment",
  "primary_injection_structure": 797,
  "data_set_id": 305024724,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "111/378/583/672/677",
  "injection_volume": 0.1041097806,
  "injection_x": 6020,
  "injection_y": 3820,
  "injection_z": 9380,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-293",
  "strain": "C57BL/6J",
  "structure_abbrev": "AIp",
  "structure_id": 111,
  "structure_name": "Agranular insular area, posterior part",
  "primary_injection_structure": 111,
  "data_set_id": 272827141,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837481,
   "name": "Scnn1a-Tg2-Cre"
  },
  "gender": "M",
  "injection_structures": "718/733",
  "injection_volume": 0.002297512,
  "injection_x": 7010,
  "injection_y": 4380,
  "injection_z": 7810,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg2-Cre-3371",
  "strain": "C57BL/6J",
  "structure_abbrev": "VPL",
  "structure_id": 718,
  "structure_name": "Ventral posterolateral nucleus of the thalamus",
  "primary_injection_structure": 718,
  "data_set_id": 156202272,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "F",
  "injection_structures": "993",
  "injection_volume": 0.0412774138,
  "injection_x": 3130,
  "injection_y": 2780,
  "injection_z": 3780,
  "product_id": 45,
  "specimen_name": "Syt6-Cre_KI148-270322",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 552757477,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "723/746",
  "injection_volume": 0.262893036,
  "injection_x": 2740,
  "injection_y": 3920,
  "injection_z": 6740,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-3812",
  "strain": "C57BL/6J",
  "structure_abbrev": "ORBl",
  "structure_id": 723,
  "structure_name": "Orbital area, lateral part",
  "primary_injection_structure": 723,
  "data_set_id": 168164972,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 301614880,
   "name": "Glt25d2-Cre_NF107"
  },
  "gender": "M",
  "injection_structures": "184/723/746/993",
  "injection_volume": 0.0860964888,
  "injection_x": 2410,
  "injection_y": 2770,
  "injection_z": 6990,
  "product_id": 5,
  "specimen_name": "Glt25d2-Cre_NF107-162216",
  "strain": null,
  "structure_abbrev": "ORBl",
  "structure_id": 723,
  "structure_name": "Orbital area, lateral part",
  "primary_injection_structure": 184,
  "data_set_id": 313327735,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 183238310,
   "name": "Ntrk1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "258/266/310/564",
  "injection_volume": 0.1531134703,
  "injection_x": 5120,
  "injection_y": 4710,
  "injection_z": 5440,
  "product_id": 5,
  "specimen_name": "Ntrk1-IRES-Cre-3534",
  "strain": "B6.129",
  "structure_abbrev": "MS",
  "structure_id": 564,
  "structure_name": "Medial septal nucleus",
  "primary_injection_structure": 258,
  "data_set_id": 265944939,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761781,
   "name": "Efr3a-Cre_NO108"
  },
  "gender": "F",
  "injection_structures": "39/985/993",
  "injection_volume": 0.1328556992,
  "injection_x": 4890,
  "injection_y": 1210,
  "injection_z": 6380,
  "product_id": 5,
  "specimen_name": "Efr3a-Cre_NO108-55",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 39,
  "data_set_id": 278258073,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "218/726/1020",
  "injection_volume": 0.1667998024,
  "injection_x": 7440,
  "injection_y": 3600,
  "injection_z": 6920,
  "product_id": 5,
  "specimen_name": "378-1771",
  "strain": "C57BL/6J",
  "structure_abbrev": "PO",
  "structure_id": 1020,
  "structure_name": "Posterior complex of the thalamus",
  "primary_injection_structure": 218,
  "data_set_id": 174781014,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 301613169,
   "name": "Sepw1-Cre_NP39"
  },
  "gender": "F",
  "injection_structures": "993",
  "injection_volume": 0.0055845440875,
  "injection_x": 2890,
  "injection_y": 2120,
  "injection_z": 3700,
  "product_id": 45,
  "specimen_name": "Sepw1-Cre_NP39-331945",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 616676552,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "385/894",
  "injection_volume": 0.0097904321375,
  "injection_x": 9540,
  "injection_y": 1600,
  "injection_z": 2900,
  "product_id": 36,
  "specimen_name": "A930038C07Rik-Tg1-Cre-231780",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 508889322,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "385/394/533/879/886/894",
  "injection_volume": 0.00748045515,
  "injection_x": 8260,
  "injection_y": 870,
  "injection_z": 4180,
  "product_id": 35,
  "specimen_name": "A930038C07Rik-Tg1-Cre-272202",
  "strain": null,
  "structure_abbrev": "RSPagl",
  "structure_id": 894,
  "structure_name": "Retrosplenial area, lateral agranular part",
  "primary_injection_structure": 385,
  "data_set_id": 554333581,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838584,
   "name": "Gad2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "202/968",
  "injection_volume": 0.0592934447,
  "injection_x": 11840,
  "injection_y": 4590,
  "injection_z": 6110,
  "product_id": 5,
  "specimen_name": "Gad2-IRES-Cre_LURC-122967",
  "strain": "C57BL/6J",
  "structure_abbrev": "NOD",
  "structure_id": 968,
  "structure_name": "Nodulus (X)",
  "primary_injection_structure": 202,
  "data_set_id": 305293222,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837979,
   "name": "Drd1a-Cre_EY262"
  },
  "gender": "F",
  "injection_structures": "369/672",
  "injection_volume": 0.1287834758,
  "injection_x": 5660,
  "injection_y": 3220,
  "injection_z": 8040,
  "product_id": 5,
  "specimen_name": "Drd1a-Cre-4894",
  "strain": "C57BL/6J",
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 369,
  "data_set_id": 159941339,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838108,
   "name": "Agrp-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "223/693/10671",
  "injection_volume": 0.015921031153125,
  "injection_x": 6790,
  "injection_y": 7200,
  "injection_z": 5920,
  "product_id": 5,
  "specimen_name": "Agrp-IRES-Cre-458",
  "strain": "C57BL/6J",
  "structure_abbrev": "ARH",
  "structure_id": 223,
  "structure_name": "Arcuate hypothalamic nucleus",
  "primary_injection_structure": 223,
  "data_set_id": 146554676,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "M",
  "injection_structures": "39/48",
  "injection_volume": 0.1226420216,
  "injection_x": 4260,
  "injection_y": 2390,
  "injection_z": 6320,
  "product_id": 5,
  "specimen_name": "Syt6-Cre_KI148-132911",
  "strain": "C57BL/6J",
  "structure_abbrev": "ACAv",
  "structure_id": 48,
  "structure_name": "Anterior cingulate area, ventral part",
  "primary_injection_structure": 39,
  "data_set_id": 299829892,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839494,
   "name": "Hdc-Cre_IM1"
  },
  "gender": "M",
  "injection_structures": "1/126/210/491/980/1004",
  "injection_volume": 0.0209397531,
  "injection_x": 7740,
  "injection_y": 6520,
  "injection_z": 6460,
  "product_id": 5,
  "specimen_name": "Hdc-Cre-88",
  "strain": null,
  "structure_abbrev": "PMv",
  "structure_id": 1004,
  "structure_name": "Ventral premammillary nucleus",
  "primary_injection_structure": 1,
  "data_set_id": 157952068,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 183941572,
   "name": "Plxnd1-Cre_OG1"
  },
  "gender": "F",
  "injection_structures": "985/993",
  "injection_volume": 0.129983868,
  "injection_x": 3820,
  "injection_y": 1530,
  "injection_z": 7200,
  "product_id": 5,
  "specimen_name": "Plxnd1-Cre-127749",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 297669605,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "128/294/580",
  "injection_volume": 0.30609124,
  "injection_x": 8690,
  "injection_y": 2750,
  "injection_z": 7620,
  "product_id": 5,
  "specimen_name": "378-1790",
  "strain": "C57BL/6J",
  "structure_abbrev": "SCm",
  "structure_id": 294,
  "structure_name": "Superior colliculus, motor related",
  "primary_injection_structure": 128,
  "data_set_id": 175158132,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "879/886",
  "injection_volume": 0.1114921794,
  "injection_x": 7780,
  "injection_y": 900,
  "injection_z": 6030,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-275",
  "strain": "C57BL/6J",
  "structure_abbrev": "RSPv",
  "structure_id": 886,
  "structure_name": "Retrosplenial area, ventral part",
  "primary_injection_structure": 879,
  "data_set_id": 272735744,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "329",
  "injection_volume": 0.1123848026,
  "injection_x": 7210,
  "injection_y": 1660,
  "injection_z": 2750,
  "product_id": 35,
  "specimen_name": "Rbp4-Cre_KL100-351928",
  "strain": null,
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 647806688,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761781,
   "name": "Efr3a-Cre_NO108"
  },
  "gender": "M",
  "injection_structures": "879/886",
  "injection_volume": 0.0329642061,
  "injection_x": 6740,
  "injection_y": 1000,
  "injection_z": 6100,
  "product_id": 5,
  "specimen_name": "Efr3a-Cre_NO108-16",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "RSPv",
  "structure_id": 886,
  "structure_name": "Retrosplenial area, ventral part",
  "primary_injection_structure": 879,
  "data_set_id": 267661018,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182693192,
   "name": "Chrna2-Cre_OE25"
  },
  "gender": "M",
  "injection_structures": "39/48",
  "injection_volume": 0.0610823759,
  "injection_x": 4410,
  "injection_y": 2270,
  "injection_z": 6150,
  "product_id": 5,
  "specimen_name": "Chrna2-Cre_OE25-133243",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "ACAv",
  "structure_id": 48,
  "structure_name": "Anterior cingulate area, ventral part",
  "primary_injection_structure": 39,
  "data_set_id": 299782983,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 310459452,
   "name": "Slc17a8-IRES2-Cre"
  },
  "gender": "M",
  "injection_structures": "385/894",
  "injection_volume": 0.0094288684875,
  "injection_x": 9790,
  "injection_y": 1640,
  "injection_z": 8430,
  "product_id": 5,
  "specimen_name": "Slc17a8-IRES2-Cre-288366",
  "strain": "unknown",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 564356750,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "394/312782546",
  "injection_volume": 0.015327211025,
  "injection_x": 7490,
  "injection_y": 990,
  "injection_z": 7620,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-3816",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISa",
  "structure_id": 312782546,
  "structure_name": "Anterior area",
  "primary_injection_structure": 394,
  "data_set_id": 168165712,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 183941572,
   "name": "Plxnd1-Cre_OG1"
  },
  "gender": "F",
  "injection_structures": "672",
  "injection_volume": 0.1616943062,
  "injection_x": 6590,
  "injection_y": 2560,
  "injection_z": 8330,
  "product_id": 5,
  "specimen_name": "Plxnd1-Cre-121492",
  "strain": null,
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 672,
  "data_set_id": 293366035,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838584,
   "name": "Gad2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "88/173/286/332/347/515/523",
  "injection_volume": 0.112704753,
  "injection_x": 5990,
  "injection_y": 6670,
  "injection_z": 5960,
  "product_id": 5,
  "specimen_name": "Gad2-IRES-Cre_LURC-124383",
  "strain": "C57BL/6J",
  "structure_abbrev": "AHN",
  "structure_id": 88,
  "structure_name": "Anterior hypothalamic nucleus",
  "primary_injection_structure": 88,
  "data_set_id": 300167479,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "385/533",
  "injection_volume": 0.1773006396,
  "injection_x": 8370,
  "injection_y": 610,
  "injection_z": 3720,
  "product_id": 35,
  "specimen_name": "Cux2-IRES-Cre-239317",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 517962765,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 266649644,
   "name": "Drd3-Cre_KI196"
  },
  "gender": "F",
  "injection_structures": "369/985",
  "injection_volume": 0.0916760894,
  "injection_x": 5020,
  "injection_y": 2230,
  "injection_z": 8010,
  "product_id": 5,
  "specimen_name": "Drd3-Cre_KI196_LURC-125343",
  "strain": null,
  "structure_abbrev": "SSp-ul",
  "structure_id": 369,
  "structure_name": "Primary somatosensory area, upper limb",
  "primary_injection_structure": 369,
  "data_set_id": 309428697,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838494,
   "name": "Prkcd-GluCla-CFP-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "351",
  "injection_volume": 0.02304374205,
  "injection_x": 5160,
  "injection_y": 5060,
  "injection_z": 6750,
  "product_id": 5,
  "specimen_name": "Prkcd-GluCla-CFP-IRES-Cre-237646",
  "strain": null,
  "structure_abbrev": "BST",
  "structure_id": 351,
  "structure_name": "Bed nuclei of the stria terminalis",
  "primary_injection_structure": 351,
  "data_set_id": 522078446,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838899,
   "name": "Dbh-Cre_KH212"
  },
  "gender": "M",
  "injection_structures": "147/202/280/350/358/534/867/898/1093/549009215",
  "injection_volume": 0.1435814072,
  "injection_x": 10670,
  "injection_y": 4430,
  "injection_z": 4710,
  "product_id": 35,
  "specimen_name": "Dbh-Cre_KH212-193690",
  "strain": null,
  "structure_abbrev": "MV",
  "structure_id": 202,
  "structure_name": "Medial vestibular nucleus",
  "primary_injection_structure": 147,
  "data_set_id": 480689656,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838048,
   "name": "Slc17a6-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "920/928",
  "injection_volume": 0.0392110054,
  "injection_x": 11460,
  "injection_y": 3210,
  "injection_z": 6370,
  "product_id": 5,
  "specimen_name": "Slc17a6-IRES-Cre-126491",
  "strain": "B6.129",
  "structure_abbrev": "CUL",
  "structure_id": 928,
  "structure_name": "Culmen",
  "primary_injection_structure": 920,
  "data_set_id": 304543821,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 530349719,
   "name": "Pdyn-T2A-CreERT2"
  },
  "gender": "F",
  "injection_structures": "38/63/88/347",
  "injection_volume": 0.0152887056,
  "injection_x": 6390,
  "injection_y": 5790,
  "injection_z": 6030,
  "product_id": 5,
  "specimen_name": "Pdyn-T2A-CreERT2-255279",
  "strain": null,
  "structure_abbrev": "PVH",
  "structure_id": 38,
  "structure_name": "Paraventricular hypothalamic nucleus",
  "primary_injection_structure": 38,
  "data_set_id": 540685246,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0968017148,
  "injection_x": 8750,
  "injection_y": 1220,
  "injection_z": 3940,
  "product_id": 35,
  "specimen_name": "Emx1-IRES-Cre-226609",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 502966396,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "104/119/985",
  "injection_volume": 0.4732626976,
  "injection_x": 3680,
  "injection_y": 4670,
  "injection_z": 8720,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-243",
  "strain": null,
  "structure_abbrev": "AId",
  "structure_id": 104,
  "structure_name": "Agranular insular area, dorsal part",
  "primary_injection_structure": 104,
  "data_set_id": 264076854,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "104/111/119/345/583/1057",
  "injection_volume": 0.6950145888,
  "injection_x": 4420,
  "injection_y": 4070,
  "injection_z": 9060,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-132732",
  "strain": "B6.129",
  "structure_abbrev": "AId",
  "structure_id": 104,
  "structure_name": "Agranular insular area, dorsal part",
  "primary_injection_structure": 104,
  "data_set_id": 299783689,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838022,
   "name": "Pmch-Cre"
  },
  "gender": "F",
  "injection_structures": "63/88/194/356/797/576073704",
  "injection_volume": 0.0306187966,
  "injection_x": 6940,
  "injection_y": 5800,
  "injection_z": 7120,
  "product_id": 5,
  "specimen_name": "Pmch-Cre-479",
  "strain": "B6.129.FVB",
  "structure_abbrev": "LHA",
  "structure_id": 194,
  "structure_name": "Lateral hypothalamic area",
  "primary_injection_structure": 63,
  "data_set_id": 113506174,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837446,
   "name": "Adcyap1-2A-Cre"
  },
  "gender": "F",
  "injection_structures": "128/422/475/1029/1044/560581563/563807435",
  "injection_volume": 0.1589623014,
  "injection_x": 8680,
  "injection_y": 4080,
  "injection_z": 7870,
  "product_id": 5,
  "specimen_name": "Adcyap1-2A-Cre-D-132352",
  "strain": "B6.129",
  "structure_abbrev": "PIL",
  "structure_id": 560581563,
  "structure_name": "Posterior intralaminar thalamic nucleus",
  "primary_injection_structure": 128,
  "data_set_id": 586054741,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "394/533/312782546",
  "injection_volume": 0.2036213228,
  "injection_x": 7800,
  "injection_y": 560,
  "injection_z": 3850,
  "product_id": 35,
  "specimen_name": "Emx1-IRES-Cre-240164",
  "strain": null,
  "structure_abbrev": "VISam",
  "structure_id": 394,
  "structure_name": "Anteromedial visual area",
  "primary_injection_structure": 394,
  "data_set_id": 518742338,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "337/369/985",
  "injection_volume": 0.1988448616,
  "injection_x": 5770,
  "injection_y": 1630,
  "injection_z": 7510,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre-176",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-ll",
  "structure_id": 337,
  "structure_name": "Primary somatosensory area, lower limb",
  "primary_injection_structure": 337,
  "data_set_id": 182803137,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "F",
  "injection_structures": "44/731/746/972",
  "injection_volume": 0.06117866335,
  "injection_x": 3130,
  "injection_y": 3620,
  "injection_z": 5170,
  "product_id": 5,
  "specimen_name": "Syt6-Cre-641",
  "strain": "C57BL/6J",
  "structure_abbrev": "ORBm",
  "structure_id": 731,
  "structure_name": "Orbital area, medial part",
  "primary_injection_structure": 44,
  "data_set_id": 159209586,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838584,
   "name": "Gad2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "39/48/993",
  "injection_volume": 0.146300068436279,
  "injection_x": 5020,
  "injection_y": 1500,
  "injection_z": 5980,
  "product_id": 5,
  "specimen_name": "Gad2-IRES-Cre-1146",
  "strain": "C57BL/6J",
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 167512351,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "104/119/583/672/985/1057",
  "injection_volume": 0.7621053104,
  "injection_x": 4000,
  "injection_y": 4010,
  "injection_z": 8110,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-126031",
  "strain": "C57BL/6J",
  "structure_abbrev": "AId",
  "structure_id": 104,
  "structure_name": "Agranular insular area, dorsal part",
  "primary_injection_structure": 104,
  "data_set_id": 296047806,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839004,
   "name": "Cux2-CreERT2"
  },
  "gender": "M",
  "injection_structures": "926",
  "injection_volume": 0.0788958555,
  "injection_x": 9680,
  "injection_y": 3920,
  "injection_z": 9140,
  "product_id": 5,
  "specimen_name": "Cux2-CreERT2-182",
  "strain": "C57BL/6J",
  "structure_abbrev": "ENTm",
  "structure_id": 926,
  "structure_name": "Entorhinal area, medial part, dorsal zone",
  "primary_injection_structure": 926,
  "data_set_id": 264873092,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 695130294,
   "name": "Nxph4-T2A-CreERT2"
  },
  "gender": "F",
  "injection_structures": "4/128/294/580",
  "injection_volume": 0.0617727712,
  "injection_x": 9620,
  "injection_y": 2740,
  "injection_z": 7710,
  "product_id": 5,
  "specimen_name": "Nxph4-2A-CreERT2-D-3844",
  "strain": "B6.129",
  "structure_abbrev": "IC",
  "structure_id": 4,
  "structure_name": "Inferior colliculus",
  "primary_injection_structure": 4,
  "data_set_id": 287714197,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839285,
   "name": "Gpr26-Cre_KO250"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0101814797,
  "injection_x": 9540,
  "injection_y": 1400,
  "injection_z": 8500,
  "product_id": 5,
  "specimen_name": "Gpr26-Cre-42",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 181601101,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761781,
   "name": "Efr3a-Cre_NO108"
  },
  "gender": "F",
  "injection_structures": "928/1007",
  "injection_volume": 0.03587778285,
  "injection_x": 10310,
  "injection_y": 3170,
  "injection_z": 7930,
  "product_id": 5,
  "specimen_name": "Efr3a-Cre_NO108-131948",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "CUL",
  "structure_id": 928,
  "structure_name": "Culmen",
  "primary_injection_structure": 928,
  "data_set_id": 299732033,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839294,
   "name": "Slc6a5-Cre_KF109"
  },
  "gender": "M",
  "injection_structures": "128/146/534/795/1093/549009215",
  "injection_volume": 0.416824968,
  "injection_x": 10090,
  "injection_y": 5190,
  "injection_z": 6440,
  "product_id": 5,
  "specimen_name": "Slc6a5-Cre-2929",
  "strain": "C57BL/6J",
  "structure_abbrev": "PRNr",
  "structure_id": 146,
  "structure_name": "Pontine reticular nucleus",
  "primary_injection_structure": 128,
  "data_set_id": 168515225,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839044,
   "name": "Lepr-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "118/693/830/946",
  "injection_volume": 0.0921946368,
  "injection_x": 7260,
  "injection_y": 6110,
  "injection_z": 5990,
  "product_id": 5,
  "specimen_name": "Lepr-IRES-Cre-Homo-306",
  "strain": "B6.129",
  "structure_abbrev": "DMH",
  "structure_id": 830,
  "structure_name": "Dorsomedial nucleus of the hypothalamus",
  "primary_injection_structure": 118,
  "data_set_id": 113314337,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839468,
   "name": "Gabra6-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "1017",
  "injection_volume": 0.079482067,
  "injection_x": 12090,
  "injection_y": 2280,
  "injection_z": 8190,
  "product_id": 5,
  "specimen_name": "Gabra6-IRES-Cre-179",
  "strain": "B6.129",
  "structure_abbrev": "AN",
  "structure_id": 1017,
  "structure_name": "Ansiform lobule",
  "primary_injection_structure": 1017,
  "data_set_id": 181116850,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "993",
  "injection_volume": 0.1661959264,
  "injection_x": 3370,
  "injection_y": 2030,
  "injection_z": 7490,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-260",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 266645328,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182693192,
   "name": "Chrna2-Cre_OE25"
  },
  "gender": "F",
  "injection_structures": "993",
  "injection_volume": 0.1783998664,
  "injection_x": 3580,
  "injection_y": 2060,
  "injection_z": 7490,
  "product_id": 5,
  "specimen_name": "Chrna2-Cre_OE25-3866",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 267750528,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "918/926",
  "injection_volume": 0.4845582168,
  "injection_x": 9580,
  "injection_y": 4660,
  "injection_z": 10410,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-242",
  "strain": null,
  "structure_abbrev": "ENTl",
  "structure_id": 918,
  "structure_name": "Entorhinal area, lateral part",
  "primary_injection_structure": 918,
  "data_set_id": 263974698,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.03930505845,
  "injection_x": 9060,
  "injection_y": 1340,
  "injection_z": 3420,
  "product_id": 35,
  "specimen_name": "Tlx3-Cre_PL56-259156",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 540146149,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "F",
  "injection_structures": "879/894",
  "injection_volume": 0.0105760865,
  "injection_x": 9680,
  "injection_y": 1190,
  "injection_z": 4250,
  "product_id": 35,
  "specimen_name": "Tlx3-Cre_PL56-313537",
  "strain": null,
  "structure_abbrev": "RSPd",
  "structure_id": 879,
  "structure_name": "Retrosplenial area, dorsal part",
  "primary_injection_structure": 879,
  "data_set_id": 591223633,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182693192,
   "name": "Chrna2-Cre_OE25"
  },
  "gender": "F",
  "injection_structures": "409/541/312782574",
  "injection_volume": 0.00720666275,
  "injection_x": 8870,
  "injection_y": 2150,
  "injection_z": 9440,
  "product_id": 5,
  "specimen_name": "Chrna2-Cre_OE25-3904",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "VISli",
  "structure_id": 312782574,
  "structure_name": "Laterointermediate area",
  "primary_injection_structure": 409,
  "data_set_id": 292174264,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "F",
  "injection_structures": "394/533",
  "injection_volume": 0.0238543907,
  "injection_x": 8010,
  "injection_y": 1000,
  "injection_z": 4080,
  "product_id": 35,
  "specimen_name": "Ntsr1-Cre_GN220-354934",
  "strain": null,
  "structure_abbrev": "VISpm",
  "structure_id": 533,
  "structure_name": "posteromedial visual area",
  "primary_injection_structure": 394,
  "data_set_id": 649361916,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838502,
   "name": "Grik4-Cre"
  },
  "gender": "M",
  "injection_structures": "1041/1049",
  "injection_volume": 0.02212650615,
  "injection_x": 11070,
  "injection_y": 5100,
  "injection_z": 8990,
  "product_id": 5,
  "specimen_name": "Grik4-Cre-5097",
  "strain": "C57BL/6J",
  "structure_abbrev": "PFL",
  "structure_id": 1041,
  "structure_name": "Paraflocculus",
  "primary_injection_structure": 1041,
  "data_set_id": 182685522,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "39/48/993",
  "injection_volume": 0.17721977,
  "injection_x": 5270,
  "injection_y": 1650,
  "injection_z": 6180,
  "product_id": 5,
  "specimen_name": "378-1487",
  "strain": "C57BL/6J",
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 146593590,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839285,
   "name": "Gpr26-Cre_KO250"
  },
  "gender": "F",
  "injection_structures": "329/353/182305689",
  "injection_volume": 0.2716958076,
  "injection_x": 5850,
  "injection_y": 2550,
  "injection_z": 8350,
  "product_id": 5,
  "specimen_name": "Gpr26-Cre-36",
  "strain": null,
  "structure_abbrev": "SSp-n",
  "structure_id": 353,
  "structure_name": "Primary somatosensory area, nose",
  "primary_injection_structure": 329,
  "data_set_id": 181599674,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "F",
  "injection_structures": "329/353/378",
  "injection_volume": 0.0651584311,
  "injection_x": 6180,
  "injection_y": 2380,
  "injection_z": 9400,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg3-Cre-4826",
  "strain": "B6.C3H",
  "structure_abbrev": "SSp-n",
  "structure_id": 353,
  "structure_name": "Primary somatosensory area, nose",
  "primary_injection_structure": 329,
  "data_set_id": 181818339,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "385/402/409/425/312782574/312782628",
  "injection_volume": 0.6053789672,
  "injection_x": 9140,
  "injection_y": 2290,
  "injection_z": 2390,
  "product_id": 35,
  "specimen_name": "Emx1-IRES-Cre-191135",
  "strain": "B6.129",
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 385,
  "data_set_id": 479673174,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838225,
   "name": "Cart-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "38/181/797/1077/560581559",
  "injection_volume": 0.1524744080125,
  "injection_x": 6020,
  "injection_y": 5390,
  "injection_z": 5690,
  "product_id": 5,
  "specimen_name": "Cart-Tg1-Cre-4477",
  "strain": "C57BL/6J",
  "structure_abbrev": "RE",
  "structure_id": 181,
  "structure_name": "Nucleus of reuniens",
  "primary_injection_structure": 38,
  "data_set_id": 265286700,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "385/533/894",
  "injection_volume": 0.2759925196,
  "injection_x": 8480,
  "injection_y": 510,
  "injection_z": 4080,
  "product_id": 35,
  "specimen_name": "Emx1-IRES-Cre-234273",
  "strain": null,
  "structure_abbrev": "VISpm",
  "structure_id": 533,
  "structure_name": "posteromedial visual area",
  "primary_injection_structure": 385,
  "data_set_id": 512314723,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "533/879/886/894",
  "injection_volume": 0.0422687867,
  "injection_x": 8770,
  "injection_y": 1250,
  "injection_z": 4760,
  "product_id": 35,
  "specimen_name": "A930038C07Rik-Tg1-Cre-332782",
  "strain": null,
  "structure_abbrev": "RSPv",
  "structure_id": 886,
  "structure_name": "Retrosplenial area, ventral part",
  "primary_injection_structure": 533,
  "data_set_id": 606778738,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839044,
   "name": "Lepr-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "194/830/576073704",
  "injection_volume": 0.03427058285,
  "injection_x": 6890,
  "injection_y": 5910,
  "injection_z": 6630,
  "product_id": 5,
  "specimen_name": "Lepr-IRES-Cre-Homo-305",
  "strain": "B6.129",
  "structure_abbrev": "LHA",
  "structure_id": 194,
  "structure_name": "Lateral hypothalamic area",
  "primary_injection_structure": 194,
  "data_set_id": 113225519,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "F",
  "injection_structures": "1002",
  "injection_volume": 0.012041410675,
  "injection_x": 7590,
  "injection_y": 3100,
  "injection_z": 10290,
  "product_id": 5,
  "specimen_name": "Tlx3-Cre-135572",
  "strain": null,
  "structure_abbrev": "AUDp",
  "structure_id": 1002,
  "structure_name": "Primary auditory area",
  "primary_injection_structure": 1002,
  "data_set_id": 303582161,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839513,
   "name": "Drd2-Cre_ER44"
  },
  "gender": "F",
  "injection_structures": "672",
  "injection_volume": 0.1608804848,
  "injection_x": 5110,
  "injection_y": 2660,
  "injection_z": 7290,
  "product_id": 5,
  "specimen_name": "Drd2-Cre-1976",
  "strain": "C57BL/6J",
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 672,
  "data_set_id": 127140981,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 183941572,
   "name": "Plxnd1-Cre_OG1"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.0749521885,
  "injection_x": 10070,
  "injection_y": 1250,
  "injection_z": 8240,
  "product_id": 5,
  "specimen_name": "Plxnd1-Cre-133039",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 299859225,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837777,
   "name": "Sim1-Cre_KJ18"
  },
  "gender": "M",
  "injection_structures": "993",
  "injection_volume": 0.016968130375,
  "injection_x": 3530,
  "injection_y": 2220,
  "injection_z": 4080,
  "product_id": 45,
  "specimen_name": "Sim1-Cre_KJ18-239626",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 518222748,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 183238310,
   "name": "Ntrk1-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "226/298/342/596/689",
  "injection_volume": 0.1378708982,
  "injection_x": 5560,
  "injection_y": 6880,
  "injection_z": 6720,
  "product_id": 5,
  "specimen_name": "Ntrk1-IRES-Cre-3563",
  "strain": null,
  "structure_abbrev": "NDB",
  "structure_id": 596,
  "structure_name": "Diagonal band nucleus",
  "primary_injection_structure": 226,
  "data_set_id": 286319739,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838502,
   "name": "Grik4-Cre"
  },
  "gender": "M",
  "injection_structures": "4/136/661/852",
  "injection_volume": 0.2064743968,
  "injection_x": 10720,
  "injection_y": 6120,
  "injection_z": 7100,
  "product_id": 5,
  "specimen_name": "Grik4-Cre-4950",
  "strain": "C57BL/6J",
  "structure_abbrev": "VII",
  "structure_id": 661,
  "structure_name": "Facial motor nucleus",
  "primary_injection_structure": 4,
  "data_set_id": 113766744,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 309483400,
   "name": "Foxp2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "173/194/223/614/693/576073704",
  "injection_volume": 0.1828740368,
  "injection_x": 6350,
  "injection_y": 6990,
  "injection_z": 4880,
  "product_id": 45,
  "specimen_name": "Foxp2-IRES-Cre-260828",
  "strain": null,
  "structure_abbrev": "TU",
  "structure_id": 614,
  "structure_name": "Tuberal nucleus",
  "primary_injection_structure": 173,
  "data_set_id": 547202505,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "F",
  "injection_structures": "385/879/894",
  "injection_volume": 0.0395584105,
  "injection_x": 9530,
  "injection_y": 1650,
  "injection_z": 3600,
  "product_id": 36,
  "specimen_name": "Ntsr1-Cre_GN220-280242",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 560799305,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 183238310,
   "name": "Ntrk1-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "56/258/266",
  "injection_volume": 0.1025289328,
  "injection_x": 4210,
  "injection_y": 5460,
  "injection_z": 6390,
  "product_id": 5,
  "specimen_name": "Ntrk1-IRES-Cre-3526",
  "strain": "B6.129",
  "structure_abbrev": "ACB",
  "structure_id": 56,
  "structure_name": "Nucleus accumbens",
  "primary_injection_structure": 56,
  "data_set_id": 265136608,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838502,
   "name": "Grik4-Cre"
  },
  "gender": "F",
  "injection_structures": "155/262/629/685/718",
  "injection_volume": 0.0574482174,
  "injection_x": 6090,
  "injection_y": 4550,
  "injection_z": 7080,
  "product_id": 5,
  "specimen_name": "Grik4-Cre-126985",
  "strain": "C57BL/6J",
  "structure_abbrev": "VAL",
  "structure_id": 629,
  "structure_name": "Ventral anterior-lateral complex of the thalamus",
  "primary_injection_structure": 155,
  "data_set_id": 300237470,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "M",
  "injection_structures": "44/814/972",
  "injection_volume": 0.1110733274,
  "injection_x": 3760,
  "injection_y": 3650,
  "injection_z": 6440,
  "product_id": 5,
  "specimen_name": "Syt6-Cre-3149",
  "strain": "C57BL/6J",
  "structure_abbrev": "ILA",
  "structure_id": 44,
  "structure_name": "Infralimbic area",
  "primary_injection_structure": 44,
  "data_set_id": 177889243,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837777,
   "name": "Sim1-Cre_KJ18"
  },
  "gender": "F",
  "injection_structures": "369/985",
  "injection_volume": 0.322666666,
  "injection_x": 5390,
  "injection_y": 2230,
  "injection_z": 7910,
  "product_id": 5,
  "specimen_name": "Sim1-Cre_KJ18-128571",
  "strain": null,
  "structure_abbrev": "SSp-ul",
  "structure_id": 369,
  "structure_name": "Primary somatosensory area, upper limb",
  "primary_injection_structure": 369,
  "data_set_id": 297951732,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 460133878,
   "name": "Ntng2-IRES2-Cre"
  },
  "gender": "M",
  "injection_structures": "104/119/345/583/672/952/1057",
  "injection_volume": 0.1822986984,
  "injection_x": 4570,
  "injection_y": 5310,
  "injection_z": 8600,
  "product_id": 5,
  "specimen_name": "Ntng2-IRES2-Cre-191954",
  "strain": null,
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 104,
  "data_set_id": 485846989,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761781,
   "name": "Efr3a-Cre_NO108"
  },
  "gender": "F",
  "injection_structures": "345/353/378/672",
  "injection_volume": 0.4846838136,
  "injection_x": 5850,
  "injection_y": 3290,
  "injection_z": 9020,
  "product_id": 5,
  "specimen_name": "Efr3a-Cre_NO108-81",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "SSs",
  "structure_id": 378,
  "structure_name": "Supplemental somatosensory area",
  "primary_injection_structure": 345,
  "data_set_id": 287996596,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182901722,
   "name": "Ppp1r17-Cre_NL146"
  },
  "gender": "M",
  "injection_structures": "294/302",
  "injection_volume": 0.0874090616,
  "injection_x": 9490,
  "injection_y": 1670,
  "injection_z": 6930,
  "product_id": 5,
  "specimen_name": "Ppp1r17-Cre_NL146-3923",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "SCm",
  "structure_id": 294,
  "structure_name": "Superior colliculus, motor related",
  "primary_injection_structure": 294,
  "data_set_id": 278511717,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 180643392,
   "name": "Rasgrf2-T2A-dCre"
  },
  "gender": "F",
  "injection_structures": "184/723/746/993",
  "injection_volume": 0.2191758436,
  "injection_x": 2520,
  "injection_y": 3710,
  "injection_z": 6820,
  "product_id": 5,
  "specimen_name": "Rasgrf2-2A-dCre-D-136966",
  "strain": null,
  "structure_abbrev": "ORBl",
  "structure_id": 723,
  "structure_name": "Orbital area, lateral part",
  "primary_injection_structure": 184,
  "data_set_id": 303783725,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839285,
   "name": "Gpr26-Cre_KO250"
  },
  "gender": "M",
  "injection_structures": "507",
  "injection_volume": 0.03638773565,
  "injection_x": 1530,
  "injection_y": 4670,
  "injection_z": 6130,
  "product_id": 5,
  "specimen_name": "Gpr26-Cre_KO250-135015",
  "strain": null,
  "structure_abbrev": "MOB",
  "structure_id": 507,
  "structure_name": "Main olfactory bulb",
  "primary_injection_structure": 507,
  "data_set_id": 304947087,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 266649644,
   "name": "Drd3-Cre_KI196"
  },
  "gender": "F",
  "injection_structures": "409/312782574/312782628",
  "injection_volume": 0.0072212672,
  "injection_x": 9270,
  "injection_y": 2050,
  "injection_z": 9370,
  "product_id": 5,
  "specimen_name": "Drd3-Cre_KI196-130852",
  "strain": null,
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 409,
  "data_set_id": 530008580,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839331,
   "name": "Nr5a1-Cre"
  },
  "gender": "M",
  "injection_structures": "337/361/369",
  "injection_volume": 0.0580780977,
  "injection_x": 6230,
  "injection_y": 1040,
  "injection_z": 7590,
  "product_id": 5,
  "specimen_name": "Nr5a1-Cre-2380",
  "strain": "B6.FVB",
  "structure_abbrev": "SSp-ll",
  "structure_id": 337,
  "structure_name": "Primary somatosensory area, lower limb",
  "primary_injection_structure": 337,
  "data_set_id": 166566678,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 301467016,
   "name": "Npr3-IRES2-Cre"
  },
  "gender": "F",
  "injection_structures": "385/394/417/533/312782546",
  "injection_volume": 0.0745511872,
  "injection_x": 7580,
  "injection_y": 1090,
  "injection_z": 7740,
  "product_id": 5,
  "specimen_name": "Npr3-IRES2-Cre-299664",
  "strain": null,
  "structure_abbrev": "VISa",
  "structure_id": 312782546,
  "structure_name": "Anterior area",
  "primary_injection_structure": 385,
  "data_set_id": 586041882,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "F",
  "injection_structures": "385/894",
  "injection_volume": 0.0691102272,
  "injection_x": 9480,
  "injection_y": 1760,
  "injection_z": 3340,
  "product_id": 35,
  "specimen_name": "Ntsr1-Cre_GN220-261411",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 544488252,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 553502711,
   "name": "Chat-IRES-Cre-neo"
  },
  "gender": "M",
  "injection_structures": "135/136/395/978/1098",
  "injection_volume": 0.013174854175,
  "injection_x": 12500,
  "injection_y": 6800,
  "injection_z": 6940,
  "product_id": 5,
  "specimen_name": "Chat-IRES-Cre-3206",
  "strain": "B6.129",
  "structure_abbrev": "IRN",
  "structure_id": 136,
  "structure_name": "Intermediate reticular nucleus",
  "primary_injection_structure": 135,
  "data_set_id": 264321572,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "385/879/894",
  "injection_volume": 0.04067586775,
  "injection_x": 9190,
  "injection_y": 1390,
  "injection_z": 7550,
  "product_id": 5,
  "specimen_name": "Ntsr1-Cre-3571",
  "strain": "C57BL/6J",
  "structure_abbrev": "RSPd",
  "structure_id": 879,
  "structure_name": "Retrosplenial area, dorsal part",
  "primary_injection_structure": 385,
  "data_set_id": 159554010,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "886",
  "injection_volume": 0.00442393315,
  "injection_x": 7910,
  "injection_y": 820,
  "injection_z": 5890,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-130",
  "strain": null,
  "structure_abbrev": "RSPv",
  "structure_id": 886,
  "structure_name": "Retrosplenial area, ventral part",
  "primary_injection_structure": 886,
  "data_set_id": 177907082,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "39/48/993",
  "injection_volume": 0.00722047585,
  "injection_x": 5280,
  "injection_y": 2320,
  "injection_z": 6300,
  "product_id": 5,
  "specimen_name": "Ntsr1-Cre_GN220-3728",
  "strain": "C57BL/6J",
  "structure_abbrev": "ACAv",
  "structure_id": 48,
  "structure_name": "Anterior cingulate area, ventral part",
  "primary_injection_structure": 39,
  "data_set_id": 266412788,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "F",
  "injection_structures": "394/533/312782546",
  "injection_volume": 0.0678906417,
  "injection_x": 7730,
  "injection_y": 1070,
  "injection_z": 3600,
  "product_id": 35,
  "specimen_name": "Ntsr1-Cre_GN220-269808",
  "strain": null,
  "structure_abbrev": "VISa",
  "structure_id": 312782546,
  "structure_name": "Anterior area",
  "primary_injection_structure": 394,
  "data_set_id": 552279683,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "385/394/533/312782546",
  "injection_volume": 0.007525482475,
  "injection_x": 7860,
  "injection_y": 1120,
  "injection_z": 3370,
  "product_id": 35,
  "specimen_name": "A930038C07Rik-Tg1-Cre-306723",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 584511119,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761918,
   "name": "Oxtr-Cre_ON66"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0309824697,
  "injection_x": 9580,
  "injection_y": 990,
  "injection_z": 8000,
  "product_id": 5,
  "specimen_name": "Oxtr-Cre_ON66-121548",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 294312537,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761781,
   "name": "Efr3a-Cre_NO108"
  },
  "gender": "F",
  "injection_structures": "385/402/409/417",
  "injection_volume": 0.0102249972125,
  "injection_x": 8470,
  "injection_y": 1630,
  "injection_z": 8990,
  "product_id": 5,
  "specimen_name": "Efr3a-Cre_NO108-66",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 385,
  "data_set_id": 286835688,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "M",
  "injection_structures": "345",
  "injection_volume": 0.0349378624,
  "injection_x": 5100,
  "injection_y": 2870,
  "injection_z": 9090,
  "product_id": 5,
  "specimen_name": "Syt6-Cre-2826",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-m",
  "structure_id": 345,
  "structure_name": "Primary somatosensory area, mouth",
  "primary_injection_structure": 345,
  "data_set_id": 168229820,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "382/394/726/312782546/484682470",
  "injection_volume": 0.016775256575,
  "injection_x": 7680,
  "injection_y": 1010,
  "injection_z": 3900,
  "product_id": 35,
  "specimen_name": "Ntsr1-Cre_GN220-252294",
  "strain": null,
  "structure_abbrev": "VISam",
  "structure_id": 394,
  "structure_name": "Anteromedial visual area",
  "primary_injection_structure": 382,
  "data_set_id": 528963283,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "1017",
  "injection_volume": 0.331598876,
  "injection_x": 11460,
  "injection_y": 3780,
  "injection_z": 8800,
  "product_id": 5,
  "specimen_name": "378-1072",
  "strain": "C57BL/6J",
  "structure_abbrev": "AN",
  "structure_id": 1017,
  "structure_name": "Ansiform lobule",
  "primary_injection_structure": 1017,
  "data_set_id": 112827872,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837834,
   "name": "Crh-IRES-Cre_BL"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.013660676925,
  "injection_x": 9490,
  "injection_y": 1120,
  "injection_z": 8370,
  "product_id": 5,
  "specimen_name": "Crh-IRES-Cre-302",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 266836749,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839513,
   "name": "Drd2-Cre_ER44"
  },
  "gender": "M",
  "injection_structures": "672",
  "injection_volume": 0.0774258555,
  "injection_x": 4130,
  "injection_y": 3570,
  "injection_z": 6570,
  "product_id": 5,
  "specimen_name": "Drd2-Cre-1980",
  "strain": "C57BL/6J",
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 672,
  "data_set_id": 127224133,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838584,
   "name": "Gad2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "463",
  "injection_volume": 0.263051502,
  "injection_x": 7630,
  "injection_y": 4070,
  "injection_z": 8920,
  "product_id": 5,
  "specimen_name": "Gad2-IRES-Cre-1211",
  "strain": "C57BL/6J",
  "structure_abbrev": "CA3",
  "structure_id": 463,
  "structure_name": "Field CA3",
  "primary_injection_structure": 463,
  "data_set_id": 266488504,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839159,
   "name": "Cck-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "395/429/1098",
  "injection_volume": 0.0495830461,
  "injection_x": 12920,
  "injection_y": 6520,
  "injection_z": 7450,
  "product_id": 5,
  "specimen_name": "Cck-IRES-Cre-178",
  "strain": "C57BL/6J",
  "structure_abbrev": "SPVC",
  "structure_id": 429,
  "structure_name": "Spinal nucleus of the trigeminal, caudal part",
  "primary_injection_structure": 395,
  "data_set_id": 287460307,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "329/369/182305689",
  "injection_volume": 0.247117486,
  "injection_x": 6240,
  "injection_y": 2060,
  "injection_z": 7790,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre_KL100-244",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-ul",
  "structure_id": 369,
  "structure_name": "Primary somatosensory area, upper limb",
  "primary_injection_structure": 329,
  "data_set_id": 266249483,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.1607514972,
  "injection_x": 9320,
  "injection_y": 1890,
  "injection_z": 3010,
  "product_id": 36,
  "specimen_name": "Emx1-IRES-Cre-212543",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 500837552,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "1041",
  "injection_volume": 0.013150929925,
  "injection_x": 10690,
  "injection_y": 4710,
  "injection_z": 8940,
  "product_id": 5,
  "specimen_name": "378-1643",
  "strain": "C57BL/6J",
  "structure_abbrev": "PFL",
  "structure_id": 1041,
  "structure_name": "Paraflocculus",
  "primary_injection_structure": 1041,
  "data_set_id": 158738894,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837858,
   "name": "Pvalb-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "294/302/795",
  "injection_volume": 0.0382624977,
  "injection_x": 9840,
  "injection_y": 1910,
  "injection_z": 6410,
  "product_id": 5,
  "specimen_name": "Pvalb-IRES-Cre-5084",
  "strain": "B6.129",
  "structure_abbrev": "SCm",
  "structure_id": 294,
  "structure_name": "Superior colliculus, motor related",
  "primary_injection_structure": 294,
  "data_set_id": 182613651,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 179697864,
   "name": "Rorb-IRES2-Cre"
  },
  "gender": "M",
  "injection_structures": "993",
  "injection_volume": 0.01613737825,
  "injection_x": 2860,
  "injection_y": 2780,
  "injection_z": 3750,
  "product_id": 45,
  "specimen_name": "Rorb-IRES2-Cre-324491",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 603574321,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "39/972/993",
  "injection_volume": 0.4832142448,
  "injection_x": 3380,
  "injection_y": 2230,
  "injection_z": 6410,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre-91",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 39,
  "data_set_id": 166055636,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 309779993,
   "name": "Ctgf-T2A-dgCre"
  },
  "gender": "M",
  "injection_structures": "48",
  "injection_volume": 0.0086925834625,
  "injection_x": 5480,
  "injection_y": 2320,
  "injection_z": 6260,
  "product_id": 5,
  "specimen_name": "Ctgf-2A-dgCre-270026",
  "strain": null,
  "structure_abbrev": "ACAv",
  "structure_id": 48,
  "structure_name": "Anterior cingulate area, ventral part",
  "primary_injection_structure": 48,
  "data_set_id": 553079356,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "39/48/993",
  "injection_volume": 0.1994364484,
  "injection_x": 4570,
  "injection_y": 2180,
  "injection_z": 6180,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre-49",
  "strain": null,
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 125833030,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "369/672",
  "injection_volume": 0.1725274124,
  "injection_x": 6020,
  "injection_y": 2520,
  "injection_z": 7930,
  "product_id": 5,
  "specimen_name": "378-1463",
  "strain": "C57BL/6J",
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 369,
  "data_set_id": 127762867,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838781,
   "name": "Slc6a4-CreERT2_EZ13"
  },
  "gender": "M",
  "injection_structures": "83/222",
  "injection_volume": 0.010844504825,
  "injection_x": 12460,
  "injection_y": 6480,
  "injection_z": 5660,
  "product_id": 5,
  "specimen_name": "Slc6a4-CreERT2-Homo-371",
  "strain": "C57BL/6J",
  "structure_abbrev": "RO",
  "structure_id": 222,
  "structure_name": "Nucleus raphe obscurus",
  "primary_injection_structure": 83,
  "data_set_id": 168227692,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "104/345/378/1057",
  "injection_volume": 0.163749768,
  "injection_x": 4330,
  "injection_y": 3920,
  "injection_z": 9320,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre_KL100-273",
  "strain": "B6.129",
  "structure_abbrev": "SSp-m",
  "structure_id": 345,
  "structure_name": "Primary somatosensory area, mouth",
  "primary_injection_structure": 104,
  "data_set_id": 272928602,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838361,
   "name": "Etv1-CreERT2"
  },
  "gender": "F",
  "injection_structures": "879/886",
  "injection_volume": 0.1652272552,
  "injection_x": 7200,
  "injection_y": 690,
  "injection_z": 6040,
  "product_id": 5,
  "specimen_name": "Etv1-CreERT2-1229",
  "strain": "C57BL/6J",
  "structure_abbrev": "RSPv",
  "structure_id": 886,
  "structure_name": "Retrosplenial area, ventral part",
  "primary_injection_structure": 879,
  "data_set_id": 159097209,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761781,
   "name": "Efr3a-Cre_NO108"
  },
  "gender": "M",
  "injection_structures": "985/993",
  "injection_volume": 0.3161012148,
  "injection_x": 4200,
  "injection_y": 1670,
  "injection_z": 7070,
  "product_id": 5,
  "specimen_name": "Efr3a-Cre_NO108-84",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "MOp",
  "structure_id": 985,
  "structure_name": "Primary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 288169135,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.1529364774,
  "injection_x": 9350,
  "injection_y": 1720,
  "injection_z": 2940,
  "product_id": 36,
  "specimen_name": "Cux2-IRES-Cre-230159",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 506891241,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 517749378,
   "name": "Rorb-IRES2-Cre-neo"
  },
  "gender": "F",
  "injection_structures": "993",
  "injection_volume": 0.00422896306875,
  "injection_x": 3510,
  "injection_y": 2050,
  "injection_z": 4240,
  "product_id": 45,
  "specimen_name": "Rorb-IRES2-Cre-neo-330148",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 616856399,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "985",
  "injection_volume": 0.3443178256,
  "injection_x": 5160,
  "injection_y": 2270,
  "injection_z": 6980,
  "product_id": 5,
  "specimen_name": "378-1401",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOp",
  "structure_id": 985,
  "structure_name": "Primary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 127084296,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838828,
   "name": "Gal-Cre_KI87"
  },
  "gender": "M",
  "injection_structures": "186/362/575/907",
  "injection_volume": 0.0879119094,
  "injection_x": 6600,
  "injection_y": 3760,
  "injection_z": 6340,
  "product_id": 5,
  "specimen_name": "Gal-Cre-1216",
  "strain": "C57BL/6J",
  "structure_abbrev": "MD",
  "structure_id": 362,
  "structure_name": "Mediodorsal nucleus of thalamus",
  "primary_injection_structure": 186,
  "data_set_id": 116904684,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837315,
   "name": "Avp-IRES2-Cre"
  },
  "gender": "F",
  "injection_structures": "38/63",
  "injection_volume": 0.02656256435,
  "injection_x": 6060,
  "injection_y": 5830,
  "injection_z": 5900,
  "product_id": 5,
  "specimen_name": "Avp-IRES2-Cre-D-3292",
  "strain": "B6.129",
  "structure_abbrev": "PVH",
  "structure_id": 38,
  "structure_name": "Paraventricular hypothalamic nucleus",
  "primary_injection_structure": 38,
  "data_set_id": 183459175,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "159/507/723/731/746/972",
  "injection_volume": 0.5335508864,
  "injection_x": 2770,
  "injection_y": 4220,
  "injection_z": 4910,
  "product_id": 36,
  "specimen_name": "Rbp4-Cre_KL100-195013",
  "strain": "C57BL/6J",
  "structure_abbrev": "ORBvl",
  "structure_id": 746,
  "structure_name": "Orbital area, ventrolateral part",
  "primary_injection_structure": 159,
  "data_set_id": 480994108,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "385/402/409",
  "injection_volume": 0.0661820313,
  "injection_x": 8520,
  "injection_y": 1230,
  "injection_z": 9270,
  "product_id": 5,
  "specimen_name": "378-1507",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 385,
  "data_set_id": 146858755,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "F",
  "injection_structures": "385/533",
  "injection_volume": 0.048918709,
  "injection_x": 8480,
  "injection_y": 900,
  "injection_z": 7540,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg3-Cre-4872",
  "strain": "B6.C3H",
  "structure_abbrev": "VISpm",
  "structure_id": 533,
  "structure_name": "posteromedial visual area",
  "primary_injection_structure": 385,
  "data_set_id": 257667830,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839331,
   "name": "Nr5a1-Cre"
  },
  "gender": "M",
  "injection_structures": "985/993",
  "injection_volume": 0.0344776789,
  "injection_x": 4090,
  "injection_y": 1570,
  "injection_z": 7180,
  "product_id": 5,
  "specimen_name": "Nr5a1-Cre-2518",
  "strain": "B6.FVB",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 265929196,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 553502711,
   "name": "Chat-IRES-Cre-neo"
  },
  "gender": "F",
  "injection_structures": "534/621/867/920/549009215",
  "injection_volume": 0.08363761,
  "injection_x": 10020,
  "injection_y": 5030,
  "injection_z": 7200,
  "product_id": 5,
  "specimen_name": "Chat-IRES-Cre-383",
  "strain": "B6.129",
  "structure_abbrev": "V",
  "structure_id": 621,
  "structure_name": "Motor nucleus of trigeminal",
  "primary_injection_structure": 534,
  "data_set_id": 166052101,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "F",
  "injection_structures": "1002/1011/1027",
  "injection_volume": 0.0671303234,
  "injection_x": 7980,
  "injection_y": 2480,
  "injection_z": 1850,
  "product_id": 35,
  "specimen_name": "Ntsr1-Cre_GN220-271816",
  "strain": null,
  "structure_abbrev": "AUDp",
  "structure_id": 1002,
  "structure_name": "Primary auditory area",
  "primary_injection_structure": 1002,
  "data_set_id": 553092069,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "44/731/972",
  "injection_volume": 0.4841804219,
  "injection_x": 2760,
  "injection_y": 3020,
  "injection_z": 5660,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-3923",
  "strain": "C57BL/6J",
  "structure_abbrev": "ILA",
  "structure_id": 44,
  "structure_name": "Infralimbic area",
  "primary_injection_structure": 44,
  "data_set_id": 283019341,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838496,
   "name": "Grp-Cre_KH288"
  },
  "gender": "M",
  "injection_structures": "463",
  "injection_volume": 0.008766664725,
  "injection_x": 7600,
  "injection_y": 3130,
  "injection_z": 8600,
  "product_id": 5,
  "specimen_name": "Grp-Cre-46",
  "strain": "B6.FVB(ICR)",
  "structure_abbrev": "CA3",
  "structure_id": 463,
  "structure_name": "Field CA3",
  "primary_injection_structure": 463,
  "data_set_id": 179644545,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "385/417/312782546",
  "injection_volume": 0.0887584922,
  "injection_x": 7580,
  "injection_y": 1360,
  "injection_z": 8310,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-305",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISrl",
  "structure_id": 417,
  "structure_name": "Rostrolateral visual area",
  "primary_injection_structure": 385,
  "data_set_id": 286301303,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "M",
  "injection_structures": "353/378",
  "injection_volume": 0.0865573044,
  "injection_x": 6090,
  "injection_y": 3060,
  "injection_z": 9530,
  "product_id": 5,
  "specimen_name": "Syt6-Cre-661",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSs",
  "structure_id": 378,
  "structure_name": "Supplemental somatosensory area",
  "primary_injection_structure": 353,
  "data_set_id": 159508775,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837777,
   "name": "Sim1-Cre_KJ18"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.01117183585,
  "injection_x": 7990,
  "injection_y": 1220,
  "injection_z": 3100,
  "product_id": 35,
  "specimen_name": "Sim1-Cre_KJ18-324772",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 602094339,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837262,
   "name": "Pomc-Cre_ST"
  },
  "gender": "F",
  "injection_structures": "223/10671",
  "injection_volume": 0.0011316161828125,
  "injection_x": 7250,
  "injection_y": 7050,
  "injection_z": 5800,
  "product_id": 5,
  "specimen_name": "Pomc-Cre (ST)-1267",
  "strain": "B6.FVB",
  "structure_abbrev": "ARH",
  "structure_id": 223,
  "structure_name": "Arcuate hypothalamic nucleus",
  "primary_injection_structure": 223,
  "data_set_id": 158142090,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "918/312782628",
  "injection_volume": 0.017703559125,
  "injection_x": 9670,
  "injection_y": 2930,
  "injection_z": 1570,
  "product_id": 35,
  "specimen_name": "Tlx3-Cre_PL56-352313",
  "strain": null,
  "structure_abbrev": "VISpor",
  "structure_id": 312782628,
  "structure_name": "Postrhinal area",
  "primary_injection_structure": 918,
  "data_set_id": 648048130,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837858,
   "name": "Pvalb-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "329/353/182305689",
  "injection_volume": 0.2148493004,
  "injection_x": 5850,
  "injection_y": 2150,
  "injection_z": 8440,
  "product_id": 5,
  "specimen_name": "Pvalb-IRES-Cre-5060",
  "strain": "B6.129",
  "structure_abbrev": "SSp-un",
  "structure_id": 182305689,
  "structure_name": "Primary somatosensory area, unassigned",
  "primary_injection_structure": 329,
  "data_set_id": 176496282,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 266649644,
   "name": "Drd3-Cre_KI196"
  },
  "gender": "M",
  "injection_structures": "879/886",
  "injection_volume": 0.1667299088,
  "injection_x": 8110,
  "injection_y": 620,
  "injection_z": 6110,
  "product_id": 5,
  "specimen_name": "Drd3-Cre_KI196-130865",
  "strain": null,
  "structure_abbrev": "RSPv",
  "structure_id": 886,
  "structure_name": "Retrosplenial area, ventral part",
  "primary_injection_structure": 879,
  "data_set_id": 538078619,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.1685982004,
  "injection_x": 8510,
  "injection_y": 1540,
  "injection_z": 8190,
  "product_id": 5,
  "specimen_name": "Ntsr1-Cre-3566",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 159550125,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "64/255",
  "injection_volume": 0.1711982188,
  "injection_x": 6000,
  "injection_y": 3610,
  "injection_z": 6360,
  "product_id": 5,
  "specimen_name": "378-1177",
  "strain": "C57BL/6J",
  "structure_abbrev": "AV",
  "structure_id": 255,
  "structure_name": "Anteroventral nucleus of thalamus",
  "primary_injection_structure": 64,
  "data_set_id": 114427219,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182901722,
   "name": "Ppp1r17-Cre_NL146"
  },
  "gender": "F",
  "injection_structures": "258/266",
  "injection_volume": 0.223286532,
  "injection_x": 4340,
  "injection_y": 4430,
  "injection_z": 6430,
  "product_id": 5,
  "specimen_name": "Ppp1r17-Cre_NL146-3891",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "LSr",
  "structure_id": 258,
  "structure_name": "Lateral septal nucleus, rostral (rostroventral) part",
  "primary_injection_structure": 258,
  "data_set_id": 267810394,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 301467016,
   "name": "Npr3-IRES2-Cre"
  },
  "gender": "F",
  "injection_structures": "1002/1011",
  "injection_volume": 0.0715360898,
  "injection_x": 7740,
  "injection_y": 2630,
  "injection_z": 9890,
  "product_id": 5,
  "specimen_name": "Npr3-IRES2-Cre-313838",
  "strain": null,
  "structure_abbrev": "AUDd",
  "structure_id": 1011,
  "structure_name": "Dorsal auditory area",
  "primary_injection_structure": 1002,
  "data_set_id": 592513871,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839513,
   "name": "Drd2-Cre_ER44"
  },
  "gender": "M",
  "injection_structures": "48/672",
  "injection_volume": 0.0325024203,
  "injection_x": 4590,
  "injection_y": 3290,
  "injection_z": 6690,
  "product_id": 5,
  "specimen_name": "Drd2-Cre-2010",
  "strain": "C57BL/6J",
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 48,
  "data_set_id": 155737254,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "91",
  "injection_volume": 0.0681951522,
  "injection_x": 11570,
  "injection_y": 3560,
  "injection_z": 7380,
  "product_id": 5,
  "specimen_name": "378-1242",
  "strain": "C57BL/6J",
  "structure_abbrev": "IP",
  "structure_id": 91,
  "structure_name": "Interposed nucleus",
  "primary_injection_structure": 91,
  "data_set_id": 120493315,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839406,
   "name": "Nos1-CreERT2"
  },
  "gender": "M",
  "injection_structures": "12/100/128/146/197/679/749/549009211/607344830",
  "injection_volume": 0.3515475404,
  "injection_x": 9000,
  "injection_y": 5060,
  "injection_z": 5630,
  "product_id": 5,
  "specimen_name": "Nos1-CreERT2-131827",
  "strain": "B6.129",
  "structure_abbrev": "IPN",
  "structure_id": 100,
  "structure_name": "Interpeduncular nucleus",
  "primary_injection_structure": 12,
  "data_set_id": 299761999,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 301613169,
   "name": "Sepw1-Cre_NP39"
  },
  "gender": "M",
  "injection_structures": "879/886",
  "injection_volume": 0.02983033025,
  "injection_x": 7090,
  "injection_y": 1070,
  "injection_z": 5790,
  "product_id": 5,
  "specimen_name": "Sepw1-Cre_NP39-244256",
  "strain": "B6.FVB",
  "structure_abbrev": "RSPv",
  "structure_id": 886,
  "structure_name": "Retrosplenial area, ventral part",
  "primary_injection_structure": 879,
  "data_set_id": 526502961,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.1259225422,
  "injection_x": 8840,
  "injection_y": 1050,
  "injection_z": 8430,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg3-Cre-4789",
  "strain": "B6.C3H",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 166326736,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "886",
  "injection_volume": 0.1341705554,
  "injection_x": 7450,
  "injection_y": 810,
  "injection_z": 6100,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-3792",
  "strain": "C57BL/6J",
  "structure_abbrev": "RSPv",
  "structure_id": 886,
  "structure_name": "Retrosplenial area, ventral part",
  "primary_injection_structure": 886,
  "data_set_id": 166271142,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0442807953,
  "injection_x": 9860,
  "injection_y": 1290,
  "injection_z": 8220,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg3-Cre-4860",
  "strain": "B6.C3H",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 184074388,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839331,
   "name": "Nr5a1-Cre"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0153661158,
  "injection_x": 9870,
  "injection_y": 1310,
  "injection_z": 8230,
  "product_id": 5,
  "specimen_name": "Nr5a1-Cre-2521",
  "strain": "B6.FVB",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 266100645,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "541/1002/1011/1027",
  "injection_volume": 0.0650656888,
  "injection_x": 8090,
  "injection_y": 2250,
  "injection_z": 1850,
  "product_id": 35,
  "specimen_name": "Rbp4-Cre_KL100-330691",
  "strain": null,
  "structure_abbrev": "AUDpo",
  "structure_id": 1027,
  "structure_name": "Posterior auditory area",
  "primary_injection_structure": 541,
  "data_set_id": 606100558,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "366/685/741/797/879/886",
  "injection_volume": 0.2300897312,
  "injection_x": 7150,
  "injection_y": 4790,
  "injection_z": 6240,
  "product_id": 5,
  "specimen_name": "378-1769",
  "strain": "C57BL/6J",
  "structure_abbrev": "VM",
  "structure_id": 685,
  "structure_name": "Ventral medial nucleus of the thalamus",
  "primary_injection_structure": 366,
  "data_set_id": 174736554,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "382/502/726/484682470",
  "injection_volume": 0.07194278,
  "injection_x": 8280,
  "injection_y": 2380,
  "injection_z": 7720,
  "product_id": 5,
  "specimen_name": "378-1154",
  "strain": "C57BL/6J",
  "structure_abbrev": "DG",
  "structure_id": 726,
  "structure_name": "Dentate gyrus",
  "primary_injection_structure": 382,
  "data_set_id": 114008220,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761918,
   "name": "Oxtr-Cre_ON66"
  },
  "gender": "F",
  "injection_structures": "463/726",
  "injection_volume": 0.0701872276,
  "injection_x": 7870,
  "injection_y": 2550,
  "injection_z": 8000,
  "product_id": 5,
  "specimen_name": "Oxtr-Cre_ON66-124762",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "CA3",
  "structure_id": 463,
  "structure_name": "Field CA3",
  "primary_injection_structure": 463,
  "data_set_id": 301421960,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0735476182,
  "injection_x": 9440,
  "injection_y": 1220,
  "injection_z": 8630,
  "product_id": 5,
  "specimen_name": "378-874",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 100147853,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182846862,
   "name": "Chrnb4-Cre_OL57"
  },
  "gender": "F",
  "injection_structures": "394/312782546",
  "injection_volume": 0.1244313252,
  "injection_x": 7310,
  "injection_y": 1230,
  "injection_z": 7460,
  "product_id": 5,
  "specimen_name": "Chrna2-Cre_OE25-3882",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "VISa",
  "structure_id": 312782546,
  "structure_name": "Anterior area",
  "primary_injection_structure": 394,
  "data_set_id": 278175580,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "329/378/1011",
  "injection_volume": 0.1500089724,
  "injection_x": 7520,
  "injection_y": 2220,
  "injection_z": 9390,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre-47",
  "strain": null,
  "structure_abbrev": "SSs",
  "structure_id": 378,
  "structure_name": "Supplemental somatosensory area",
  "primary_injection_structure": 329,
  "data_set_id": 120916102,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "985/993",
  "injection_volume": 0.8089883536,
  "injection_x": 2670,
  "injection_y": 2800,
  "injection_z": 8010,
  "product_id": 5,
  "specimen_name": "378-1812",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 180709942,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837834,
   "name": "Crh-IRES-Cre_BL"
  },
  "gender": "M",
  "injection_structures": "1002",
  "injection_volume": 0.01865791865,
  "injection_x": 7890,
  "injection_y": 2450,
  "injection_z": 10330,
  "product_id": 5,
  "specimen_name": "Crh-IRES-Cre-283",
  "strain": "B6.129",
  "structure_abbrev": "AUDp",
  "structure_id": 1002,
  "structure_name": "Primary auditory area",
  "primary_injection_structure": 1002,
  "data_set_id": 183105218,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "39/972/993",
  "injection_volume": 0.0946670494,
  "injection_x": 3260,
  "injection_y": 2540,
  "injection_z": 6410,
  "product_id": 5,
  "specimen_name": "Ntsr1-Cre_GN220-3737",
  "strain": "C57BL/6J",
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 266964075,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838502,
   "name": "Grik4-Cre"
  },
  "gender": "M",
  "injection_structures": "319/647/780",
  "injection_volume": 0.3258791648,
  "injection_x": 7200,
  "injection_y": 6540,
  "injection_z": 8440,
  "product_id": 5,
  "specimen_name": "Grik4-Cre-5027",
  "strain": "C57BL/6J",
  "structure_abbrev": "COAp",
  "structure_id": 647,
  "structure_name": "Cortical amygdalar area, posterior part",
  "primary_injection_structure": 319,
  "data_set_id": 168302154,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837777,
   "name": "Sim1-Cre_KJ18"
  },
  "gender": "F",
  "injection_structures": "394/533/312782546",
  "injection_volume": 0.01075071025,
  "injection_x": 7760,
  "injection_y": 890,
  "injection_z": 7620,
  "product_id": 5,
  "specimen_name": "Sim1-Cre_KJ18-120880",
  "strain": null,
  "structure_abbrev": "VISam",
  "structure_id": 394,
  "structure_name": "Anteromedial visual area",
  "primary_injection_structure": 394,
  "data_set_id": 293821389,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "385/409",
  "injection_volume": 0.8053739568,
  "injection_x": 8930,
  "injection_y": 1810,
  "injection_z": 8450,
  "product_id": 5,
  "specimen_name": "C57BL/6-152924",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 307321674,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 517749378,
   "name": "Rorb-IRES2-Cre-neo"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0161658693,
  "injection_x": 9440,
  "injection_y": 1320,
  "injection_z": 8340,
  "product_id": 5,
  "specimen_name": "Rorb-IRES2-Cre-neo-347763",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 648051857,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 553502711,
   "name": "Chat-IRES-Cre-neo"
  },
  "gender": "F",
  "injection_structures": "226/298/342",
  "injection_volume": 0.0649898466,
  "injection_x": 5520,
  "injection_y": 6290,
  "injection_z": 7160,
  "product_id": 5,
  "specimen_name": "Chat-IRES-Cre-328",
  "strain": "B6.129",
  "structure_abbrev": "SI",
  "structure_id": 342,
  "structure_name": "Substantia innominata",
  "primary_injection_structure": 226,
  "data_set_id": 126841788,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "F",
  "injection_structures": "184/993",
  "injection_volume": 0.0632311239,
  "injection_x": 2760,
  "injection_y": 2780,
  "injection_z": 4270,
  "product_id": 45,
  "specimen_name": "Syt6-Cre_KI148-270323",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 184,
  "data_set_id": 552758186,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "104/119/723/961",
  "injection_volume": 0.797135528,
  "injection_x": 3010,
  "injection_y": 4370,
  "injection_z": 7970,
  "product_id": 5,
  "specimen_name": "378-1810",
  "strain": "C57BL/6J",
  "structure_abbrev": "ORBl",
  "structure_id": 723,
  "structure_name": "Orbital area, lateral part",
  "primary_injection_structure": 104,
  "data_set_id": 180709230,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837777,
   "name": "Sim1-Cre_KJ18"
  },
  "gender": "F",
  "injection_structures": "329",
  "injection_volume": 0.062929181,
  "injection_x": 7110,
  "injection_y": 1500,
  "injection_z": 8510,
  "product_id": 5,
  "specimen_name": "Sim1-Cre_KJ18-125422",
  "strain": null,
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 297710633,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "385/425",
  "injection_volume": 0.2434185348,
  "injection_x": 9470,
  "injection_y": 2070,
  "injection_z": 2840,
  "product_id": 36,
  "specimen_name": "Emx1-IRES-Cre-296888",
  "strain": "unknown",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 574941472,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "F",
  "injection_structures": "329",
  "injection_volume": 0.1056058878,
  "injection_x": 7400,
  "injection_y": 1800,
  "injection_z": 8570,
  "product_id": 5,
  "specimen_name": "Ntsr1-Cre_GN220-121889",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 297628576,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839285,
   "name": "Gpr26-Cre_KO250"
  },
  "gender": "M",
  "injection_structures": "151/184/723/746",
  "injection_volume": 0.1266437242,
  "injection_x": 2370,
  "injection_y": 3550,
  "injection_z": 6780,
  "product_id": 5,
  "specimen_name": "Gpr26-Cre_KO250-86",
  "strain": null,
  "structure_abbrev": "ORBl",
  "structure_id": 723,
  "structure_name": "Orbital area, lateral part",
  "primary_injection_structure": 151,
  "data_set_id": 265713683,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837858,
   "name": "Pvalb-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "146/574/1093",
  "injection_volume": 0.0238899387548828,
  "injection_x": 9890,
  "injection_y": 5860,
  "injection_z": 6140,
  "product_id": 5,
  "specimen_name": "Pvalb-IRES-Cre-5095",
  "strain": "B6.129",
  "structure_abbrev": "PRNc",
  "structure_id": 1093,
  "structure_name": "Pontine reticular nucleus, caudal part",
  "primary_injection_structure": 146,
  "data_set_id": 258316862,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838048,
   "name": "Slc17a6-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "186/362",
  "injection_volume": 0.0863288174,
  "injection_x": 6280,
  "injection_y": 4040,
  "injection_z": 6110,
  "product_id": 5,
  "specimen_name": "Slc17a6-IRES-Cre-121240",
  "strain": "B6.129",
  "structure_abbrev": "MD",
  "structure_id": 362,
  "structure_name": "Mediodorsal nucleus of thalamus",
  "primary_injection_structure": 186,
  "data_set_id": 294356922,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "39/985/993",
  "injection_volume": 0.10717672,
  "injection_x": 5150,
  "injection_y": 1630,
  "injection_z": 6270,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-3829",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 39,
  "data_set_id": 176433237,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837777,
   "name": "Sim1-Cre_KJ18"
  },
  "gender": "M",
  "injection_structures": "329/385/417/312782546",
  "injection_volume": 0.0306255978,
  "injection_x": 7700,
  "injection_y": 1260,
  "injection_z": 8460,
  "product_id": 5,
  "specimen_name": "Sim1-Cre_KJ18-129326",
  "strain": null,
  "structure_abbrev": "VISrl",
  "structure_id": 417,
  "structure_name": "Rostrolateral visual area",
  "primary_injection_structure": 329,
  "data_set_id": 297712045,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839331,
   "name": "Nr5a1-Cre"
  },
  "gender": "M",
  "injection_structures": "223/693/830/1004",
  "injection_volume": 0.0659280398,
  "injection_x": 7330,
  "injection_y": 6640,
  "injection_z": 6180,
  "product_id": 5,
  "specimen_name": "Nr5a1-Cre-126",
  "strain": "B6.FVB",
  "structure_abbrev": "VMH",
  "structure_id": 693,
  "structure_name": "Ventromedial hypothalamic nucleus",
  "primary_injection_structure": 223,
  "data_set_id": 114290225,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839004,
   "name": "Cux2-CreERT2"
  },
  "gender": "F",
  "injection_structures": "672/952",
  "injection_volume": 0.0560834596,
  "injection_x": 4450,
  "injection_y": 4320,
  "injection_z": 8290,
  "product_id": 5,
  "specimen_name": "Cux2-CreERT2-3801",
  "strain": "C57BL/6J",
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 672,
  "data_set_id": 292620968,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.060788077,
  "injection_x": 8710,
  "injection_y": 1020,
  "injection_z": 8340,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre_KL100-128401",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 294482052,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839004,
   "name": "Cux2-CreERT2"
  },
  "gender": "M",
  "injection_structures": "295/952/961/966/1105",
  "injection_volume": 0.5665218104,
  "injection_x": 6310,
  "injection_y": 6430,
  "injection_z": 8770,
  "product_id": 5,
  "specimen_name": "Cux2-CreERT2-124836",
  "strain": null,
  "structure_abbrev": "PIR",
  "structure_id": 961,
  "structure_name": "Piriform area",
  "primary_injection_structure": 295,
  "data_set_id": 310439724,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838710,
   "name": "Sst-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "329",
  "injection_volume": 0.0335860308,
  "injection_x": 7070,
  "injection_y": 1410,
  "injection_z": 8330,
  "product_id": 5,
  "specimen_name": "Sst-IRES-Cre-138014",
  "strain": null,
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 304335875,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "723/746",
  "injection_volume": 0.2963280096,
  "injection_x": 2590,
  "injection_y": 3890,
  "injection_z": 6810,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre_KL100-307",
  "strain": "C57BL/6J",
  "structure_abbrev": "ORBvl",
  "structure_id": 746,
  "structure_name": "Orbital area, ventrolateral part",
  "primary_injection_structure": 723,
  "data_set_id": 287769286,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "385/533/879/886/894",
  "injection_volume": 0.2360113028,
  "injection_x": 8710,
  "injection_y": 530,
  "injection_z": 4050,
  "product_id": 36,
  "specimen_name": "Emx1-IRES-Cre-211823",
  "strain": null,
  "structure_abbrev": "VISpm",
  "structure_id": 533,
  "structure_name": "posteromedial visual area",
  "primary_injection_structure": 385,
  "data_set_id": 501006221,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838803,
   "name": "Grm2-Cre_MR90"
  },
  "gender": "F",
  "injection_structures": "39/972/993",
  "injection_volume": 0.218993936,
  "injection_x": 3500,
  "injection_y": 2310,
  "injection_z": 6310,
  "product_id": 5,
  "specimen_name": "Grm2-Cre-99",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 39,
  "data_set_id": 182814071,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182693192,
   "name": "Chrna2-Cre_OE25"
  },
  "gender": "M",
  "injection_structures": "394/533/894",
  "injection_volume": 0.0711644542,
  "injection_x": 8030,
  "injection_y": 1040,
  "injection_z": 7190,
  "product_id": 5,
  "specimen_name": "Chrna2-Cre_OE25-126080",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "VISpm",
  "structure_id": 533,
  "structure_name": "posteromedial visual area",
  "primary_injection_structure": 394,
  "data_set_id": 297946154,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838942,
   "name": "Htr2a-Cre_KM207"
  },
  "gender": "F",
  "injection_structures": "44/731/746/972",
  "injection_volume": 0.1314248502,
  "injection_x": 2860,
  "injection_y": 3340,
  "injection_z": 5910,
  "product_id": 5,
  "specimen_name": "Htr2a-Cre_KM207-3470",
  "strain": null,
  "structure_abbrev": "ORBm",
  "structure_id": 731,
  "structure_name": "Orbital area, medial part",
  "primary_injection_structure": 44,
  "data_set_id": 277956496,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "382/423/463",
  "injection_volume": 0.1130080238,
  "injection_x": 7500,
  "injection_y": 3390,
  "injection_z": 9160,
  "product_id": 5,
  "specimen_name": "378-932",
  "strain": "C57BL/6J",
  "structure_abbrev": "CA3",
  "structure_id": 463,
  "structure_name": "Field CA3",
  "primary_injection_structure": 382,
  "data_set_id": 112308468,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839331,
   "name": "Nr5a1-Cre"
  },
  "gender": "M",
  "injection_structures": "345/378/1057",
  "injection_volume": 0.1753032232,
  "injection_x": 4250,
  "injection_y": 3900,
  "injection_z": 9320,
  "product_id": 5,
  "specimen_name": "Nr5a1-Cre-2459",
  "strain": "B6.FVB",
  "structure_abbrev": "SSp-m",
  "structure_id": 345,
  "structure_name": "Primary somatosensory area, mouth",
  "primary_injection_structure": 345,
  "data_set_id": 177892379,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "385/879/1037/484682508",
  "injection_volume": 0.015742581575,
  "injection_x": 9740,
  "injection_y": 2310,
  "injection_z": 8070,
  "product_id": 5,
  "specimen_name": "378-1271",
  "strain": "C57BL/6J",
  "structure_abbrev": "APr",
  "structure_id": 484682508,
  "structure_name": "Area prostriata",
  "primary_injection_structure": 385,
  "data_set_id": 126115436,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838502,
   "name": "Grik4-Cre"
  },
  "gender": "F",
  "injection_structures": "409/425/918/926/312782628",
  "injection_volume": 0.2472984724,
  "injection_x": 9840,
  "injection_y": 3590,
  "injection_z": 9540,
  "product_id": 5,
  "specimen_name": "Grik4-Cre-5127",
  "strain": "C57BL/6J",
  "structure_abbrev": "ENTm",
  "structure_id": 926,
  "structure_name": "Entorhinal area, medial part, dorsal zone",
  "primary_injection_structure": 409,
  "data_set_id": 267998328,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838622,
   "name": "Vip-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "294/795/872",
  "injection_volume": 0.025071860625,
  "injection_x": 9620,
  "injection_y": 3160,
  "injection_z": 5690,
  "product_id": 5,
  "specimen_name": "Vip-IRES-Cre-210",
  "strain": null,
  "structure_abbrev": "PAG",
  "structure_id": 795,
  "structure_name": "Periaqueductal gray",
  "primary_injection_structure": 294,
  "data_set_id": 262188772,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "133/263/272/452/515/523/763",
  "injection_volume": 0.0590262616152344,
  "injection_x": 4940,
  "injection_y": 6770,
  "injection_z": 5850,
  "product_id": 5,
  "specimen_name": "378-1335",
  "strain": "C57BL/6J",
  "structure_abbrev": "AVPV",
  "structure_id": 272,
  "structure_name": "Anteroventral periventricular nucleus",
  "primary_injection_structure": 133,
  "data_set_id": 138059031,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "39/48/993",
  "injection_volume": 0.1685560212,
  "injection_x": 5140,
  "injection_y": 1730,
  "injection_z": 5440,
  "product_id": 35,
  "specimen_name": "Emx1-IRES-Cre-187653",
  "strain": null,
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 478491090,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "155",
  "injection_volume": 0.0297776528,
  "injection_x": 6440,
  "injection_y": 3270,
  "injection_z": 7400,
  "product_id": 5,
  "specimen_name": "378-1899",
  "strain": "C57BL/6J",
  "structure_abbrev": "LD",
  "structure_id": 155,
  "structure_name": "Lateral dorsal nucleus of thalamus",
  "primary_injection_structure": 155,
  "data_set_id": 272969333,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 256980970,
   "name": "Lypd6-Cre_KL156"
  },
  "gender": "F",
  "injection_structures": "1/194/210/364/525/946/1004",
  "injection_volume": 0.0184814672,
  "injection_x": 7930,
  "injection_y": 6220,
  "injection_z": 6750,
  "product_id": 5,
  "specimen_name": "Lypd6-Cre_KL156-126616",
  "strain": null,
  "structure_abbrev": "LM",
  "structure_id": 210,
  "structure_name": "Lateral mammillary nucleus",
  "primary_injection_structure": 1,
  "data_set_id": 298104533,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "726",
  "injection_volume": 0.0537394858,
  "injection_x": 8100,
  "injection_y": 2390,
  "injection_z": 7730,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre-199",
  "strain": "C57BL/6J",
  "structure_abbrev": "DG",
  "structure_id": 726,
  "structure_name": "Dentate gyrus",
  "primary_injection_structure": 726,
  "data_set_id": 183376982,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182846862,
   "name": "Chrnb4-Cre_OL57"
  },
  "gender": "M",
  "injection_structures": "918",
  "injection_volume": 0.146326838,
  "injection_x": 9510,
  "injection_y": 5110,
  "injection_z": 10260,
  "product_id": 5,
  "specimen_name": "Chrnb4-Cre_OL57-130639",
  "strain": null,
  "structure_abbrev": "ENTl",
  "structure_id": 918,
  "structure_name": "Entorhinal area, lateral part",
  "primary_injection_structure": 918,
  "data_set_id": 516278420,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "402/541/1002/1011/1027",
  "injection_volume": 0.4149535208,
  "injection_x": 8040,
  "injection_y": 2310,
  "injection_z": 1980,
  "product_id": 36,
  "specimen_name": "Cux2-IRES-Cre-309606",
  "strain": null,
  "structure_abbrev": "AUDpo",
  "structure_id": 1027,
  "structure_name": "Posterior auditory area",
  "primary_injection_structure": 402,
  "data_set_id": 587345518,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 530349719,
   "name": "Pdyn-T2A-CreERT2"
  },
  "gender": "M",
  "injection_structures": "867",
  "injection_volume": 0.0314199907,
  "injection_x": 10330,
  "injection_y": 4140,
  "injection_z": 7180,
  "product_id": 5,
  "specimen_name": "Pdyn-T2A-CreERT2-255283",
  "strain": null,
  "structure_abbrev": "PB",
  "structure_id": 867,
  "structure_name": "Parabrachial nucleus",
  "primary_injection_structure": 867,
  "data_set_id": 543876073,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837364,
   "name": "Kiss1-Cre"
  },
  "gender": "M",
  "injection_structures": "38/72/133/272/515/523/914",
  "injection_volume": 0.1075945234,
  "injection_x": 5190,
  "injection_y": 6280,
  "injection_z": 5870,
  "product_id": 5,
  "specimen_name": "Kiss1-Cre-3857",
  "strain": "C57BL/6J",
  "structure_abbrev": "MPN",
  "structure_id": 515,
  "structure_name": "Medial preoptic nucleus",
  "primary_injection_structure": 38,
  "data_set_id": 301989585,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839331,
   "name": "Nr5a1-Cre"
  },
  "gender": "F",
  "injection_structures": "353/378",
  "injection_volume": 0.01685015185,
  "injection_x": 5600,
  "injection_y": 3260,
  "injection_z": 9710,
  "product_id": 5,
  "specimen_name": "Nr5a1-Cre-2440",
  "strain": "B6.FVB",
  "structure_abbrev": "SSp-n",
  "structure_id": 353,
  "structure_name": "Primary somatosensory area, nose",
  "primary_injection_structure": 353,
  "data_set_id": 177322838,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837324,
   "name": "Slc18a2-Cre_OZ14"
  },
  "gender": "F",
  "injection_structures": "337/361/369/312782546",
  "injection_volume": 0.0074330151875,
  "injection_x": 6600,
  "injection_y": 1070,
  "injection_z": 7720,
  "product_id": 5,
  "specimen_name": "Slc18a2-Cre-120",
  "strain": null,
  "structure_abbrev": "SSp-tr",
  "structure_id": 361,
  "structure_name": "Primary somatosensory area, trunk",
  "primary_injection_structure": 337,
  "data_set_id": 263553163,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0675711568,
  "injection_x": 9010,
  "injection_y": 1690,
  "injection_z": 3200,
  "product_id": 35,
  "specimen_name": "Ntsr1-Cre_GN220-187357",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 478257959,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839425,
   "name": "Pdzk1ip1-Cre_KD31"
  },
  "gender": "M",
  "injection_structures": "612",
  "injection_volume": 0.0866187798,
  "injection_x": 9170,
  "injection_y": 5320,
  "injection_z": 7320,
  "product_id": 5,
  "specimen_name": "Pdzk1ip1-Cre-126",
  "strain": null,
  "structure_abbrev": "NLL",
  "structure_id": 612,
  "structure_name": "Nucleus of the lateral lemniscus",
  "primary_injection_structure": 612,
  "data_set_id": 157766259,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838225,
   "name": "Cart-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "250/258/310/581",
  "injection_volume": 0.0406226562,
  "injection_x": 5140,
  "injection_y": 3570,
  "injection_z": 6140,
  "product_id": 5,
  "specimen_name": "Cart-Tg1-Cre-4448",
  "strain": "C57BL/6J",
  "structure_abbrev": "LSr",
  "structure_id": 258,
  "structure_name": "Lateral septal nucleus, rostral (rostroventral) part",
  "primary_injection_structure": 250,
  "data_set_id": 178486024,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "369",
  "injection_volume": 0.1767896088,
  "injection_x": 5160,
  "injection_y": 2180,
  "injection_z": 7910,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-129787",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-ul",
  "structure_id": 369,
  "structure_name": "Primary somatosensory area, upper limb",
  "primary_injection_structure": 369,
  "data_set_id": 298106713,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838496,
   "name": "Grp-Cre_KH288"
  },
  "gender": "M",
  "injection_structures": "151/723/746",
  "injection_volume": 0.1413367564,
  "injection_x": 2180,
  "injection_y": 3380,
  "injection_z": 6750,
  "product_id": 5,
  "specimen_name": "Grp-Cre_KH288-3896",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "ORBl",
  "structure_id": 723,
  "structure_name": "Orbital area, lateral part",
  "primary_injection_structure": 151,
  "data_set_id": 288324211,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838074,
   "name": "Slc6a3-Cre"
  },
  "gender": "M",
  "injection_structures": "507",
  "injection_volume": 0.246995182,
  "injection_x": 850,
  "injection_y": 5900,
  "injection_z": 6430,
  "product_id": 5,
  "specimen_name": "Slc6a3-Cre-4501",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOB",
  "structure_id": 507,
  "structure_name": "Main olfactory bulb",
  "primary_injection_structure": 507,
  "data_set_id": 166154193,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838953,
   "name": "Oxt-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "194/390",
  "injection_volume": 0.00470493743125,
  "injection_x": 5950,
  "injection_y": 6690,
  "injection_z": 6900,
  "product_id": 5,
  "specimen_name": "Oxt-IRES-Cre-1211",
  "strain": "B6.129",
  "structure_abbrev": "SO",
  "structure_id": 390,
  "structure_name": "Supraoptic nucleus",
  "primary_injection_structure": 194,
  "data_set_id": 147790922,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837585,
   "name": "Wfs1-Tg3-CreERT2"
  },
  "gender": "M",
  "injection_structures": "843/926/1084",
  "injection_volume": 0.0270182668,
  "injection_x": 9590,
  "injection_y": 3830,
  "injection_z": 8500,
  "product_id": 5,
  "specimen_name": "Wfs1-Tg3-CreERT2-5317",
  "strain": "C57BL/6J",
  "structure_abbrev": "PAR",
  "structure_id": 843,
  "structure_name": "Parasubiculum",
  "primary_injection_structure": 843,
  "data_set_id": 182617186,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0978884564,
  "injection_x": 8670,
  "injection_y": 1410,
  "injection_z": 3110,
  "product_id": 35,
  "specimen_name": "Rbp4-Cre_KL100-253238",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 530574594,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837858,
   "name": "Pvalb-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "1022",
  "injection_volume": 0.1791424712,
  "injection_x": 5810,
  "injection_y": 4520,
  "injection_z": 7640,
  "product_id": 5,
  "specimen_name": "Pvalb-IRES-Cre-211480",
  "strain": null,
  "structure_abbrev": "GPe",
  "structure_id": 1022,
  "structure_name": "Globus pallidus, external segment",
  "primary_injection_structure": 1022,
  "data_set_id": 511942270,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "385/409",
  "injection_volume": 0.102772649,
  "injection_x": 9140,
  "injection_y": 1840,
  "injection_z": 2630,
  "product_id": 35,
  "specimen_name": "Rbp4-Cre_KL100-253236",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 530555292,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.2089061884,
  "injection_x": 8510,
  "injection_y": 1540,
  "injection_z": 8190,
  "product_id": 5,
  "specimen_name": "C57BL/6-144798",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 304586645,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "402/409/541/1027/312782574",
  "injection_volume": 0.2577877652,
  "injection_x": 8410,
  "injection_y": 2090,
  "injection_z": 1960,
  "product_id": 36,
  "specimen_name": "Cux2-IRES-Cre-239320",
  "strain": null,
  "structure_abbrev": "VISli",
  "structure_id": 312782574,
  "structure_name": "Laterointermediate area",
  "primary_injection_structure": 402,
  "data_set_id": 517078399,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838622,
   "name": "Vip-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "39/993",
  "injection_volume": 0.01494668315,
  "injection_x": 5090,
  "injection_y": 1540,
  "injection_z": 6290,
  "product_id": 5,
  "specimen_name": "Vip-IRES-Cre-206",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 39,
  "data_set_id": 182935487,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839406,
   "name": "Nos1-CreERT2"
  },
  "gender": "M",
  "injection_structures": "843/926/484682508",
  "injection_volume": 0.0089436717125,
  "injection_x": 9940,
  "injection_y": 3400,
  "injection_z": 8530,
  "product_id": 5,
  "specimen_name": "Nos1-CreERT2-141390",
  "strain": "B6.129",
  "structure_abbrev": "PAR",
  "structure_id": 843,
  "structure_name": "Parasubiculum",
  "primary_injection_structure": 843,
  "data_set_id": 305071285,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "385/879/894",
  "injection_volume": 0.1580234418,
  "injection_x": 9330,
  "injection_y": 1670,
  "injection_z": 3330,
  "product_id": 35,
  "specimen_name": "Cux2-IRES-Cre-247057",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 526502209,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 180643392,
   "name": "Rasgrf2-T2A-dCre"
  },
  "gender": "F",
  "injection_structures": "1002/1011/1018",
  "injection_volume": 0.3451186816,
  "injection_x": 7490,
  "injection_y": 3040,
  "injection_z": 9830,
  "product_id": 5,
  "specimen_name": "Rasgrf2-2A-dCre-D-130403",
  "strain": null,
  "structure_abbrev": "AUDp",
  "structure_id": 1002,
  "structure_name": "Primary auditory area",
  "primary_injection_structure": 1002,
  "data_set_id": 298719484,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838266,
   "name": "Erbb4-T2A-CreERT2"
  },
  "gender": "M",
  "injection_structures": "39/48/993",
  "injection_volume": 0.2999076948,
  "injection_x": 4900,
  "injection_y": 1630,
  "injection_z": 6240,
  "product_id": 5,
  "specimen_name": "Erbb4-2A-CreERT2-D-5750",
  "strain": "C57BL/6J",
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 182183683,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0431919663,
  "injection_x": 9040,
  "injection_y": 1440,
  "injection_z": 3260,
  "product_id": 35,
  "specimen_name": "Rbp4-Cre_KL100-195016",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 482578964,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838584,
   "name": "Gad2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "385/1037/1084",
  "injection_volume": 0.1261776852,
  "injection_x": 9400,
  "injection_y": 2820,
  "injection_z": 8320,
  "product_id": 5,
  "specimen_name": "Gad2-IRES-Cre_LURC-126574",
  "strain": "C57BL/6J",
  "structure_abbrev": "PRE",
  "structure_id": 1084,
  "structure_name": "Presubiculum",
  "primary_injection_structure": 385,
  "data_set_id": 302217570,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838361,
   "name": "Etv1-CreERT2"
  },
  "gender": "F",
  "injection_structures": "184/723/746/993",
  "injection_volume": 0.1495061344,
  "injection_x": 3050,
  "injection_y": 3730,
  "injection_z": 6820,
  "product_id": 5,
  "specimen_name": "Etv1-CreERT2-122207",
  "strain": "C57BL/6J",
  "structure_abbrev": "ORBl",
  "structure_id": 723,
  "structure_name": "Orbital area, lateral part",
  "primary_injection_structure": 184,
  "data_set_id": 307766627,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837777,
   "name": "Sim1-Cre_KJ18"
  },
  "gender": "M",
  "injection_structures": "329/353/378/182305689",
  "injection_volume": 0.03424174145,
  "injection_x": 5960,
  "injection_y": 2160,
  "injection_z": 8710,
  "product_id": 5,
  "specimen_name": "Sim1-Cre_KJ18-125424",
  "strain": null,
  "structure_abbrev": "SSp-n",
  "structure_id": 353,
  "structure_name": "Primary somatosensory area, nose",
  "primary_injection_structure": 329,
  "data_set_id": 297713365,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182693192,
   "name": "Chrna2-Cre_OE25"
  },
  "gender": "F",
  "injection_structures": "184/731/746/972",
  "injection_volume": 0.0239339814,
  "injection_x": 2720,
  "injection_y": 3460,
  "injection_z": 6340,
  "product_id": 5,
  "specimen_name": "Chrna2-Cre_OE25-124227",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "ORBvl",
  "structure_id": 746,
  "structure_name": "Orbital area, ventrolateral part",
  "primary_injection_structure": 184,
  "data_set_id": 297892843,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838710,
   "name": "Sst-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "39/993",
  "injection_volume": 0.0904501388,
  "injection_x": 4710,
  "injection_y": 1670,
  "injection_z": 6030,
  "product_id": 5,
  "specimen_name": "Sst-IRES-Cre-231",
  "strain": "C57BL/6J",
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 288262635,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "402/541/1002/1011/1027",
  "injection_volume": 0.001366926509375,
  "injection_x": 8330,
  "injection_y": 2310,
  "injection_z": 2040,
  "product_id": 35,
  "specimen_name": "A930038C07Rik-Tg1-Cre-306719",
  "strain": null,
  "structure_abbrev": "AUDpo",
  "structure_id": 1027,
  "structure_name": "Posterior auditory area",
  "primary_injection_structure": 402,
  "data_set_id": 584896065,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "566",
  "injection_volume": 0.2247933996,
  "injection_x": 8290,
  "injection_y": 6110,
  "injection_z": 9290,
  "product_id": 5,
  "specimen_name": "378-1287",
  "strain": "C57BL/6J",
  "structure_abbrev": "TR",
  "structure_id": 566,
  "structure_name": "Postpiriform transition area",
  "primary_injection_structure": 566,
  "data_set_id": 126653015,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "M",
  "injection_structures": "394/312782546",
  "injection_volume": 0.0572495028,
  "injection_x": 7530,
  "injection_y": 1240,
  "injection_z": 7440,
  "product_id": 5,
  "specimen_name": "Syt6-Cre-617",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISa",
  "structure_id": 312782546,
  "structure_name": "Anterior area",
  "primary_injection_structure": 394,
  "data_set_id": 157769139,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837797,
   "name": "Th-Cre_FI172"
  },
  "gender": "F",
  "injection_structures": "128/214/749/795",
  "injection_volume": 0.01061472545,
  "injection_x": 8410,
  "injection_y": 4830,
  "injection_z": 6180,
  "product_id": 5,
  "specimen_name": "Th-Cre_FI172-135967",
  "strain": "B6.FVB",
  "structure_abbrev": "VTA",
  "structure_id": 749,
  "structure_name": "Ventral tegmental area",
  "primary_injection_structure": 128,
  "data_set_id": 304337288,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839294,
   "name": "Slc6a5-Cre_KF109"
  },
  "gender": "M",
  "injection_structures": "96/209/217/781",
  "injection_volume": 0.1288537614,
  "injection_x": 11370,
  "injection_y": 4590,
  "injection_z": 8120,
  "product_id": 5,
  "specimen_name": "Slc6a5-Cre-358",
  "strain": "C57BL/6J",
  "structure_abbrev": "DCO",
  "structure_id": 96,
  "structure_name": "Dorsal cochlear nucleus",
  "primary_injection_structure": 96,
  "data_set_id": 160152987,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "385/533/886/894",
  "injection_volume": 0.013946175425,
  "injection_x": 8640,
  "injection_y": 1260,
  "injection_z": 4580,
  "product_id": 36,
  "specimen_name": "A930038C07Rik-Tg1-Cre-321959",
  "strain": null,
  "structure_abbrev": "RSPv",
  "structure_id": 886,
  "structure_name": "Retrosplenial area, ventral part",
  "primary_injection_structure": 385,
  "data_set_id": 599076623,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "394/533/312782546",
  "injection_volume": 0.1270347834,
  "injection_x": 7690,
  "injection_y": 660,
  "injection_z": 7410,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-206",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISam",
  "structure_id": 394,
  "structure_name": "Anteromedial visual area",
  "primary_injection_structure": 394,
  "data_set_id": 184167484,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 180643392,
   "name": "Rasgrf2-T2A-dCre"
  },
  "gender": "M",
  "injection_structures": "39/993",
  "injection_volume": 0.0636154652,
  "injection_x": 4730,
  "injection_y": 1670,
  "injection_z": 6050,
  "product_id": 5,
  "specimen_name": "Rasgrf2-2A-dCre-D-131326",
  "strain": null,
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 298758841,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "382/423/463/726",
  "injection_volume": 0.0695466555,
  "injection_x": 8040,
  "injection_y": 5550,
  "injection_z": 8970,
  "product_id": 5,
  "specimen_name": "378-1250",
  "strain": "C57BL/6J",
  "structure_abbrev": "CA1",
  "structure_id": 382,
  "structure_name": "Field CA1",
  "primary_injection_structure": 382,
  "data_set_id": 122641784,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "215/294",
  "injection_volume": 0.1715720692,
  "injection_x": 8520,
  "injection_y": 3560,
  "injection_z": 7410,
  "product_id": 5,
  "specimen_name": "378-1375",
  "strain": "C57BL/6J",
  "structure_abbrev": "APN",
  "structure_id": 215,
  "structure_name": "Anterior pretectal nucleus",
  "primary_injection_structure": 215,
  "data_set_id": 127083591,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 266649644,
   "name": "Drd3-Cre_KI196"
  },
  "gender": "M",
  "injection_structures": "329",
  "injection_volume": 0.1303750154,
  "injection_x": 7000,
  "injection_y": 1530,
  "injection_z": 8570,
  "product_id": 5,
  "specimen_name": "Drd3-Cre_KI196_LURC-125055",
  "strain": null,
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 297671018,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838048,
   "name": "Slc17a6-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "502/726/918/484682470",
  "injection_volume": 0.0765572325,
  "injection_x": 9170,
  "injection_y": 3900,
  "injection_z": 9290,
  "product_id": 5,
  "specimen_name": "Slc17a6-IRES-Cre-246450",
  "strain": null,
  "structure_abbrev": "SUB",
  "structure_id": 502,
  "structure_name": "Subiculum",
  "primary_injection_structure": 502,
  "data_set_id": 527051458,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838022,
   "name": "Pmch-Cre"
  },
  "gender": "M",
  "injection_structures": "63/88/194/797/576073704",
  "injection_volume": 0.0173016844,
  "injection_x": 6570,
  "injection_y": 6070,
  "injection_z": 6500,
  "product_id": 5,
  "specimen_name": "Pmch-Cre-545",
  "strain": "B6.129.FVB",
  "structure_abbrev": "LHA",
  "structure_id": 194,
  "structure_name": "Lateral hypothalamic area",
  "primary_injection_structure": 63,
  "data_set_id": 171020416,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837324,
   "name": "Slc18a2-Cre_OZ14"
  },
  "gender": "M",
  "injection_structures": "250/258/310/581",
  "injection_volume": 0.0851999848,
  "injection_x": 5060,
  "injection_y": 3460,
  "injection_z": 6230,
  "product_id": 5,
  "specimen_name": "Slc18a2-Cre_OZ14-122348",
  "strain": null,
  "structure_abbrev": "LSc",
  "structure_id": 250,
  "structure_name": "Lateral septal nucleus, caudal (caudodorsal) part",
  "primary_injection_structure": 250,
  "data_set_id": 301672044,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837324,
   "name": "Slc18a2-Cre_OZ14"
  },
  "gender": "F",
  "injection_structures": "329",
  "injection_volume": 0.0095419764125,
  "injection_x": 6590,
  "injection_y": 1450,
  "injection_z": 8700,
  "product_id": 5,
  "specimen_name": "Slc18a2-Cre_OZ14-3586",
  "strain": null,
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 265820951,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838502,
   "name": "Grik4-Cre"
  },
  "gender": "M",
  "injection_structures": "463/726",
  "injection_volume": 0.0654581592,
  "injection_x": 7620,
  "injection_y": 2210,
  "injection_z": 7390,
  "product_id": 5,
  "specimen_name": "Grik4-Cre-4954",
  "strain": "C57BL/6J",
  "structure_abbrev": "DG",
  "structure_id": 726,
  "structure_name": "Dentate gyrus",
  "primary_injection_structure": 463,
  "data_set_id": 113933871,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837834,
   "name": "Crh-IRES-Cre_BL"
  },
  "gender": "M",
  "injection_structures": "91/96/209/781/846",
  "injection_volume": 0.011759944875,
  "injection_x": 11260,
  "injection_y": 4440,
  "injection_z": 7930,
  "product_id": 5,
  "specimen_name": "Crh-IRES-Cre-310",
  "strain": "C57BL/6J",
  "structure_abbrev": "DCO",
  "structure_id": 96,
  "structure_name": "Dorsal cochlear nucleus",
  "primary_injection_structure": 91,
  "data_set_id": 266839784,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "39/48",
  "injection_volume": 0.0262950023,
  "injection_x": 5170,
  "injection_y": 1710,
  "injection_z": 5530,
  "product_id": 35,
  "specimen_name": "Cux2-IRES-Cre-259097",
  "strain": null,
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 539521918,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "402/541/1002/1027/312782574",
  "injection_volume": 0.2591280524,
  "injection_x": 8430,
  "injection_y": 2120,
  "injection_z": 1860,
  "product_id": 36,
  "specimen_name": "Emx1-IRES-Cre-302360",
  "strain": null,
  "structure_abbrev": "AUDpo",
  "structure_id": 1027,
  "structure_name": "Posterior auditory area",
  "primary_injection_structure": 402,
  "data_set_id": 579203888,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "993",
  "injection_volume": 0.1000800108,
  "injection_x": 3620,
  "injection_y": 1910,
  "injection_z": 7220,
  "product_id": 5,
  "specimen_name": "Tlx3-Cre_LURC-3598",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 288169842,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "1017",
  "injection_volume": 0.2430607564,
  "injection_x": 12010,
  "injection_y": 2400,
  "injection_z": 8850,
  "product_id": 5,
  "specimen_name": "378-1358",
  "strain": "C57BL/6J",
  "structure_abbrev": "AN",
  "structure_id": 1017,
  "structure_name": "Ansiform lobule",
  "primary_injection_structure": 1017,
  "data_set_id": 142654808,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837777,
   "name": "Sim1-Cre_KJ18"
  },
  "gender": "F",
  "injection_structures": "1002/1011",
  "injection_volume": 0.0154417669,
  "injection_x": 7740,
  "injection_y": 2670,
  "injection_z": 10030,
  "product_id": 5,
  "specimen_name": "Sim1-Cre_KJ18-120878",
  "strain": null,
  "structure_abbrev": "AUDp",
  "structure_id": 1002,
  "structure_name": "Primary auditory area",
  "primary_injection_structure": 1002,
  "data_set_id": 293820681,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "385/533/894",
  "injection_volume": 0.0441435806,
  "injection_x": 8810,
  "injection_y": 970,
  "injection_z": 4100,
  "product_id": 35,
  "specimen_name": "Tlx3-Cre_PL56-240883",
  "strain": null,
  "structure_abbrev": "VISpm",
  "structure_id": 533,
  "structure_name": "posteromedial visual area",
  "primary_injection_structure": 385,
  "data_set_id": 519186737,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.03348199645,
  "injection_x": 9430,
  "injection_y": 1480,
  "injection_z": 2850,
  "product_id": 35,
  "specimen_name": "A930038C07Rik-Tg1-Cre-242566",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 520985971,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839406,
   "name": "Nos1-CreERT2"
  },
  "gender": "M",
  "injection_structures": "382/463/726",
  "injection_volume": 0.0510964307,
  "injection_x": 7370,
  "injection_y": 2410,
  "injection_z": 6440,
  "product_id": 5,
  "specimen_name": "Nos1-CreERT2-126106",
  "strain": "B6.129",
  "structure_abbrev": "DG",
  "structure_id": 726,
  "structure_name": "Dentate gyrus",
  "primary_injection_structure": 382,
  "data_set_id": 301327022,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837281,
   "name": "Calb2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "218/1029",
  "injection_volume": 0.089214006,
  "injection_x": 7580,
  "injection_y": 2770,
  "injection_z": 7430,
  "product_id": 5,
  "specimen_name": "Calb2-IRES-Cre-218",
  "strain": "C57BL/6J",
  "structure_abbrev": "LP",
  "structure_id": 218,
  "structure_name": "Lateral posterior nucleus of the thalamus",
  "primary_injection_structure": 218,
  "data_set_id": 183282970,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.103709921,
  "injection_x": 9980,
  "injection_y": 1420,
  "injection_z": 8360,
  "product_id": 5,
  "specimen_name": "Ntsr1-Cre_GN220-123373",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 297629282,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "541/1002/1018/1027",
  "injection_volume": 0.423433696,
  "injection_x": 8860,
  "injection_y": 2800,
  "injection_z": 1180,
  "product_id": 35,
  "specimen_name": "Emx1-IRES-Cre-298030",
  "strain": null,
  "structure_abbrev": "AUDp",
  "structure_id": 1002,
  "structure_name": "Primary auditory area",
  "primary_injection_structure": 541,
  "data_set_id": 576332845,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "39/48/993",
  "injection_volume": 0.2223386172,
  "injection_x": 4220,
  "injection_y": 1940,
  "injection_z": 6210,
  "product_id": 5,
  "specimen_name": "378-963",
  "strain": "C57BL/6J",
  "structure_abbrev": "ACAv",
  "structure_id": 48,
  "structure_name": "Anterior cingulate area, ventral part",
  "primary_injection_structure": 39,
  "data_set_id": 112514202,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838108,
   "name": "Agrp-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "223/693",
  "injection_volume": 0.00386130383003696,
  "injection_x": 6800,
  "injection_y": 7160,
  "injection_z": 5790,
  "product_id": 5,
  "specimen_name": "Agrp-IRES-Cre-461",
  "strain": "C57BL/6J",
  "structure_abbrev": "ARH",
  "structure_id": 223,
  "structure_name": "Arcuate hypothalamic nucleus",
  "primary_injection_structure": 223,
  "data_set_id": 146660999,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "329/385/417/312782546",
  "injection_volume": 0.0420604485,
  "injection_x": 7800,
  "injection_y": 1220,
  "injection_z": 8570,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-3959",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISrl",
  "structure_id": 417,
  "structure_name": "Rostrolateral visual area",
  "primary_injection_structure": 329,
  "data_set_id": 292792724,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "F",
  "injection_structures": "39/993",
  "injection_volume": 0.02039886905,
  "injection_x": 5110,
  "injection_y": 1890,
  "injection_z": 6660,
  "product_id": 5,
  "specimen_name": "Ntsr1-Cre-3563",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 39,
  "data_set_id": 159373612,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "385/409",
  "injection_volume": 1.0584762832,
  "injection_x": 8740,
  "injection_y": 1590,
  "injection_z": 8140,
  "product_id": 5,
  "specimen_name": "C57BL/6-153763",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 307558646,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837328,
   "name": "Syt17-Cre_NO14"
  },
  "gender": "M",
  "injection_structures": "726",
  "injection_volume": 0.0129417477,
  "injection_x": 7080,
  "injection_y": 2560,
  "injection_z": 6370,
  "product_id": 5,
  "specimen_name": "Syt17-Cre_NO14-219",
  "strain": "C57BL/6J",
  "structure_abbrev": "DG",
  "structure_id": 726,
  "structure_name": "Dentate gyrus",
  "primary_injection_structure": 726,
  "data_set_id": 288322797,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "104/119/985",
  "injection_volume": 0.02818553745,
  "injection_x": 3620,
  "injection_y": 4390,
  "injection_z": 8420,
  "product_id": 5,
  "specimen_name": "Tlx3-Cre_LURC-126303",
  "strain": null,
  "structure_abbrev": "AId",
  "structure_id": 104,
  "structure_name": "Agranular insular area, dorsal part",
  "primary_injection_structure": 104,
  "data_set_id": 294525229,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "329/378",
  "injection_volume": 0.412418496,
  "injection_x": 6720,
  "injection_y": 2440,
  "injection_z": 8720,
  "product_id": 5,
  "specimen_name": "378-1091",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 112951804,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 179697864,
   "name": "Rorb-IRES2-Cre"
  },
  "gender": "F",
  "injection_structures": "329/353/182305689",
  "injection_volume": 0.0703398773,
  "injection_x": 5960,
  "injection_y": 1800,
  "injection_z": 8940,
  "product_id": 5,
  "specimen_name": "Rorb-IRES2-Cre-D-686",
  "strain": null,
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 287174103,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837777,
   "name": "Sim1-Cre_KJ18"
  },
  "gender": "M",
  "injection_structures": "345/353/378",
  "injection_volume": 0.1803522028,
  "injection_x": 4980,
  "injection_y": 3180,
  "injection_z": 9510,
  "product_id": 5,
  "specimen_name": "Sim1-Cre_KJ18-129327",
  "strain": null,
  "structure_abbrev": "SSp-m",
  "structure_id": 345,
  "structure_name": "Primary somatosensory area, mouth",
  "primary_injection_structure": 345,
  "data_set_id": 297985755,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "726",
  "injection_volume": 0.0817523644,
  "injection_x": 7510,
  "injection_y": 2270,
  "injection_z": 7470,
  "product_id": 5,
  "specimen_name": "378-1057",
  "strain": "C57BL/6J",
  "structure_abbrev": "DG",
  "structure_id": 726,
  "structure_name": "Dentate gyrus",
  "primary_injection_structure": 726,
  "data_set_id": 112745787,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "723/746",
  "injection_volume": 0.1920021684,
  "injection_x": 2640,
  "injection_y": 3360,
  "injection_z": 6860,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-197",
  "strain": null,
  "structure_abbrev": "ORBl",
  "structure_id": 723,
  "structure_name": "Orbital area, lateral part",
  "primary_injection_structure": 723,
  "data_set_id": 183471174,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761781,
   "name": "Efr3a-Cre_NO108"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.0206600562,
  "injection_x": 8720,
  "injection_y": 1570,
  "injection_z": 8370,
  "product_id": 5,
  "specimen_name": "Efr3a-Cre_NO108-122553",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 298350212,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "533/879/886/894",
  "injection_volume": 0.00726527165,
  "injection_x": 8800,
  "injection_y": 1210,
  "injection_z": 4520,
  "product_id": 36,
  "specimen_name": "A930038C07Rik-Tg1-Cre-242323",
  "strain": null,
  "structure_abbrev": "RSPd",
  "structure_id": 879,
  "structure_name": "Retrosplenial area, dorsal part",
  "primary_injection_structure": 533,
  "data_set_id": 520018181,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839004,
   "name": "Cux2-CreERT2"
  },
  "gender": "F",
  "injection_structures": "926",
  "injection_volume": 0.213855796,
  "injection_x": 9740,
  "injection_y": 4070,
  "injection_z": 9290,
  "product_id": 5,
  "specimen_name": "Cux2-CreERT2-178",
  "strain": "C57BL/6J",
  "structure_abbrev": "ENTm",
  "structure_id": 926,
  "structure_name": "Entorhinal area, medial part, dorsal zone",
  "primary_injection_structure": 926,
  "data_set_id": 194947823,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837481,
   "name": "Scnn1a-Tg2-Cre"
  },
  "gender": "M",
  "injection_structures": "926",
  "injection_volume": 0.03203068015,
  "injection_x": 9680,
  "injection_y": 4490,
  "injection_z": 9060,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg2-Cre-122199",
  "strain": "C57BL/6J",
  "structure_abbrev": "ENTm",
  "structure_id": 926,
  "structure_name": "Entorhinal area, medial part, dorsal zone",
  "primary_injection_structure": 926,
  "data_set_id": 299447153,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "329/385/417/312782546",
  "injection_volume": 0.2006779124,
  "injection_x": 7400,
  "injection_y": 1020,
  "injection_z": 2670,
  "product_id": 35,
  "specimen_name": "Rbp4-Cre_KL100-257446",
  "strain": null,
  "structure_abbrev": "VISrl",
  "structure_id": 417,
  "structure_name": "Rostrolateral visual area",
  "primary_injection_structure": 329,
  "data_set_id": 540145406,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.5142510016,
  "injection_x": 9340,
  "injection_y": 1860,
  "injection_z": 8300,
  "product_id": 5,
  "specimen_name": "C57BL/6-152923",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 307320960,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "394/312782546",
  "injection_volume": 0.0583555308,
  "injection_x": 7660,
  "injection_y": 820,
  "injection_z": 3670,
  "product_id": 35,
  "specimen_name": "Tlx3-Cre_PL56-250456",
  "strain": null,
  "structure_abbrev": "VISam",
  "structure_id": 394,
  "structure_name": "Anteromedial visual area",
  "primary_injection_structure": 394,
  "data_set_id": 528510546,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177835613,
   "name": "Nkx2-1-CreERT2"
  },
  "gender": "F",
  "injection_structures": "614/693",
  "injection_volume": 0.186065054,
  "injection_x": 6670,
  "injection_y": 6950,
  "injection_z": 5000,
  "product_id": 45,
  "specimen_name": "Nkx2-1-CreERT2-256332",
  "strain": null,
  "structure_abbrev": "VMH",
  "structure_id": 693,
  "structure_name": "Ventromedial hypothalamic nucleus",
  "primary_injection_structure": 614,
  "data_set_id": 540139629,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837797,
   "name": "Th-Cre_FI172"
  },
  "gender": "F",
  "injection_structures": "797",
  "injection_volume": 0.02218877535,
  "injection_x": 6770,
  "injection_y": 5440,
  "injection_z": 6810,
  "product_id": 5,
  "specimen_name": "Th-Cre_FI172-146203",
  "strain": "B6.FVB",
  "structure_abbrev": "ZI",
  "structure_id": 797,
  "structure_name": "Zona incerta",
  "primary_injection_structure": 797,
  "data_set_id": 306270474,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "337/361",
  "injection_volume": 0.015437811375,
  "injection_x": 6440,
  "injection_y": 1120,
  "injection_z": 7650,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-132",
  "strain": null,
  "structure_abbrev": "SSp-ll",
  "structure_id": 337,
  "structure_name": "Primary somatosensory area, lower limb",
  "primary_injection_structure": 337,
  "data_set_id": 178284661,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "394/533/879/886/894",
  "injection_volume": 0.4357063536,
  "injection_x": 8180,
  "injection_y": 570,
  "injection_z": 4120,
  "product_id": 35,
  "specimen_name": "Emx1-IRES-Cre-294568",
  "strain": null,
  "structure_abbrev": "VISpm",
  "structure_id": 533,
  "structure_name": "posteromedial visual area",
  "primary_injection_structure": 394,
  "data_set_id": 572588941,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "337/369",
  "injection_volume": 0.0725938332,
  "injection_x": 5720,
  "injection_y": 1450,
  "injection_z": 7700,
  "product_id": 5,
  "specimen_name": "Tlx3-Cre-125620",
  "strain": null,
  "structure_abbrev": "SSp-ll",
  "structure_id": 337,
  "structure_name": "Primary somatosensory area, lower limb",
  "primary_injection_structure": 337,
  "data_set_id": 296052839,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839425,
   "name": "Pdzk1ip1-Cre_KD31"
  },
  "gender": "F",
  "injection_structures": "128/214/294/795",
  "injection_volume": 0.0439620062,
  "injection_x": 8620,
  "injection_y": 3820,
  "injection_z": 5870,
  "product_id": 5,
  "specimen_name": "Pdzk1ip1-Cre_KD31-253",
  "strain": "C57BL/6J",
  "structure_abbrev": "PAG",
  "structure_id": 795,
  "structure_name": "Periaqueductal gray",
  "primary_injection_structure": 128,
  "data_set_id": 267029447,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 553502711,
   "name": "Chat-IRES-Cre-neo"
  },
  "gender": "M",
  "injection_structures": "653",
  "injection_volume": 0.00202166895,
  "injection_x": 10770,
  "injection_y": 5230,
  "injection_z": 6110,
  "product_id": 5,
  "specimen_name": "Chat-IRES-Cre-412",
  "strain": "B6.129",
  "structure_abbrev": "VI",
  "structure_id": 653,
  "structure_name": "Abducens nucleus",
  "primary_injection_structure": 653,
  "data_set_id": 168616111,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "879/886",
  "injection_volume": 0.0437349157,
  "injection_x": 6170,
  "injection_y": 1270,
  "injection_z": 5960,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-138",
  "strain": null,
  "structure_abbrev": "RSPv",
  "structure_id": 886,
  "structure_name": "Retrosplenial area, ventral part",
  "primary_injection_structure": 879,
  "data_set_id": 182338356,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "329",
  "injection_volume": 0.0653194794,
  "injection_x": 7200,
  "injection_y": 1900,
  "injection_z": 2470,
  "product_id": 35,
  "specimen_name": "Rbp4-Cre_KL100-351929",
  "strain": null,
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 648253235,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838266,
   "name": "Erbb4-T2A-CreERT2"
  },
  "gender": "M",
  "injection_structures": "378/1011",
  "injection_volume": 0.245991858,
  "injection_x": 7450,
  "injection_y": 2240,
  "injection_z": 10010,
  "product_id": 5,
  "specimen_name": "Erbb4-2A-CreERT2-D-5679",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSs",
  "structure_id": 378,
  "structure_name": "Supplemental somatosensory area",
  "primary_injection_structure": 378,
  "data_set_id": 156743264,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "184/507/723/993",
  "injection_volume": 0.139684006,
  "injection_x": 2090,
  "injection_y": 3180,
  "injection_z": 7380,
  "product_id": 5,
  "specimen_name": "378-1816",
  "strain": "C57BL/6J",
  "structure_abbrev": "ORBl",
  "structure_id": 723,
  "structure_name": "Orbital area, lateral part",
  "primary_injection_structure": 184,
  "data_set_id": 180673746,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839376,
   "name": "Otof-Cre"
  },
  "gender": "M",
  "injection_structures": "918",
  "injection_volume": 0.281341928,
  "injection_x": 8760,
  "injection_y": 5150,
  "injection_z": 10460,
  "product_id": 5,
  "specimen_name": "Otof-Cre-2855",
  "strain": "C57BL/6J",
  "structure_abbrev": "ENTl",
  "structure_id": 918,
  "structure_name": "Entorhinal area, lateral part",
  "primary_injection_structure": 918,
  "data_set_id": 182226133,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 310459452,
   "name": "Slc17a8-IRES2-Cre"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0017588501,
  "injection_x": 8590,
  "injection_y": 1130,
  "injection_z": 8070,
  "product_id": 5,
  "specimen_name": "Slc17a8-IRES2-Cre-336351",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 623289575,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 298341195,
   "name": "Htr3a-Cre_NO152"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0563692276,
  "injection_x": 9500,
  "injection_y": 950,
  "injection_z": 8320,
  "product_id": 5,
  "specimen_name": "Htr3a-Cre_NO152-145246",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 305489377,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "918/926",
  "injection_volume": 0.3643517304,
  "injection_x": 9850,
  "injection_y": 4180,
  "injection_z": 10270,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-299",
  "strain": "B6.129",
  "structure_abbrev": "ENTl",
  "structure_id": 918,
  "structure_name": "Entorhinal area, lateral part",
  "primary_injection_structure": 918,
  "data_set_id": 278317239,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 183238310,
   "name": "Ntrk1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "226/263/298/342/523/596/689",
  "injection_volume": 0.03897464165,
  "injection_x": 5090,
  "injection_y": 6650,
  "injection_z": 6760,
  "product_id": 5,
  "specimen_name": "Ntrk1-IRES-Cre_LURC-3545",
  "strain": null,
  "structure_abbrev": "NDB",
  "structure_id": 596,
  "structure_name": "Diagonal band nucleus",
  "primary_injection_structure": 226,
  "data_set_id": 292961470,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "329",
  "injection_volume": 0.1088838702,
  "injection_x": 7130,
  "injection_y": 1740,
  "injection_z": 8320,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre_KL100-265",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 272735030,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 286615058,
   "name": "Drd3-Cre_KI198"
  },
  "gender": "M",
  "injection_structures": "382",
  "injection_volume": 0.002360455103125,
  "injection_x": 7590,
  "injection_y": 1910,
  "injection_z": 8290,
  "product_id": 5,
  "specimen_name": "Drd3-Cre_KI198-125969",
  "strain": null,
  "structure_abbrev": "CA1",
  "structure_id": 382,
  "structure_name": "Field CA1",
  "primary_injection_structure": 382,
  "data_set_id": 301119459,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.1720099136,
  "injection_x": 8740,
  "injection_y": 1540,
  "injection_z": 8020,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre_KL100-245",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 266250195,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838074,
   "name": "Slc6a3-Cre"
  },
  "gender": "F",
  "injection_structures": "403/536",
  "injection_volume": 0.0786254098,
  "injection_x": 6820,
  "injection_y": 5830,
  "injection_z": 7940,
  "product_id": 5,
  "specimen_name": "Slc6a3-Cre-4607",
  "strain": "C57BL/6J",
  "structure_abbrev": "MEA",
  "structure_id": 403,
  "structure_name": "Medial amygdalar nucleus",
  "primary_injection_structure": 403,
  "data_set_id": 286303000,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "F",
  "injection_structures": "345",
  "injection_volume": 0.03281842865,
  "injection_x": 4910,
  "injection_y": 2610,
  "injection_z": 8800,
  "product_id": 5,
  "specimen_name": "Ntsr1-Cre-3575",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-m",
  "structure_id": 345,
  "structure_name": "Primary somatosensory area, mouth",
  "primary_injection_structure": 345,
  "data_set_id": 159651770,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "136/203/773/852/1048",
  "injection_volume": 0.3297127288,
  "injection_x": 11910,
  "injection_y": 6240,
  "injection_z": 6370,
  "product_id": 5,
  "specimen_name": "378-1278",
  "strain": "C57BL/6J",
  "structure_abbrev": "IRN",
  "structure_id": 136,
  "structure_name": "Intermediate reticular nucleus",
  "primary_injection_structure": 136,
  "data_set_id": 126188607,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837481,
   "name": "Scnn1a-Tg2-Cre"
  },
  "gender": "M",
  "injection_structures": "170/178",
  "injection_volume": 0.1042503812,
  "injection_x": 7070,
  "injection_y": 3210,
  "injection_z": 3350,
  "product_id": 35,
  "specimen_name": "Scnn1a-Tg2-Cre-190462",
  "strain": "C57BL/6J",
  "structure_abbrev": "LGd",
  "structure_id": 170,
  "structure_name": "Dorsal part of the lateral geniculate complex",
  "primary_injection_structure": 170,
  "data_set_id": 479891303,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "F",
  "injection_structures": "394/533/312782546",
  "injection_volume": 0.045142524,
  "injection_x": 7640,
  "injection_y": 670,
  "injection_z": 7460,
  "product_id": 5,
  "specimen_name": "Tlx3-Cre_LURC-3610",
  "strain": null,
  "structure_abbrev": "VISam",
  "structure_id": 394,
  "structure_name": "Anteromedial visual area",
  "primary_injection_structure": 394,
  "data_set_id": 516491813,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "128/246/271/374/381",
  "injection_volume": 0.150937346,
  "injection_x": 9100,
  "injection_y": 4250,
  "injection_z": 7540,
  "product_id": 5,
  "specimen_name": "378-1306",
  "strain": "C57BL/6J",
  "structure_abbrev": "MRN",
  "structure_id": 128,
  "structure_name": "Midbrain reticular nucleus",
  "primary_injection_structure": 128,
  "data_set_id": 127557915,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "F",
  "injection_structures": "402/409",
  "injection_volume": 0.02803235855,
  "injection_x": 8450,
  "injection_y": 1700,
  "injection_z": 9390,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg3-Cre-4794",
  "strain": "B6.C3H",
  "structure_abbrev": "VISal",
  "structure_id": 402,
  "structure_name": "Anterolateral visual area",
  "primary_injection_structure": 402,
  "data_set_id": 166459778,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839406,
   "name": "Nos1-CreERT2"
  },
  "gender": "F",
  "injection_structures": "184/723/746/993",
  "injection_volume": 0.0628100032,
  "injection_x": 2700,
  "injection_y": 3160,
  "injection_z": 7150,
  "product_id": 5,
  "specimen_name": "Nos1-CreERT2-120796",
  "strain": "B6.129",
  "structure_abbrev": "ORBl",
  "structure_id": 723,
  "structure_name": "Orbital area, lateral part",
  "primary_injection_structure": 184,
  "data_set_id": 293889216,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "104/985/993",
  "injection_volume": 1.0470485424,
  "injection_x": 2460,
  "injection_y": 2960,
  "injection_z": 8170,
  "product_id": 5,
  "specimen_name": "378-1822",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 104,
  "data_set_id": 180719293,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 180526080,
   "name": "Slc17a7-IRES2-Cre"
  },
  "gender": "M",
  "injection_structures": "463/726",
  "injection_volume": 0.0188192242,
  "injection_x": 7260,
  "injection_y": 2570,
  "injection_z": 7110,
  "product_id": 5,
  "specimen_name": "Slc17a7-IRES2-Cre-D-120702",
  "strain": null,
  "structure_abbrev": "DG",
  "structure_id": 726,
  "structure_name": "Dentate gyrus",
  "primary_injection_structure": 463,
  "data_set_id": 293787994,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "F",
  "injection_structures": "39/985/993",
  "injection_volume": 0.267696898,
  "injection_x": 4780,
  "injection_y": 2310,
  "injection_z": 6640,
  "product_id": 5,
  "specimen_name": "Syt6-Cre-3146",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 39,
  "data_set_id": 177319974,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761781,
   "name": "Efr3a-Cre_NO108"
  },
  "gender": "M",
  "injection_structures": "369/985",
  "injection_volume": 0.0867229146,
  "injection_x": 5180,
  "injection_y": 2090,
  "injection_z": 7830,
  "product_id": 5,
  "specimen_name": "Efr3a-Cre_NO108-123435",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "SSp-ul",
  "structure_id": 369,
  "structure_name": "Primary somatosensory area, upper limb",
  "primary_injection_structure": 369,
  "data_set_id": 298352336,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "402/409/1027/312782574",
  "injection_volume": 0.0977309606,
  "injection_x": 8200,
  "injection_y": 1920,
  "injection_z": 2010,
  "product_id": 36,
  "specimen_name": "Rbp4-Cre_KL100-233302",
  "strain": null,
  "structure_abbrev": "VISal",
  "structure_id": 402,
  "structure_name": "Anterolateral visual area",
  "primary_injection_structure": 402,
  "data_set_id": 510834706,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838502,
   "name": "Grik4-Cre"
  },
  "gender": "F",
  "injection_structures": "218/1020",
  "injection_volume": 0.0393260378,
  "injection_x": 7560,
  "injection_y": 3450,
  "injection_z": 7110,
  "product_id": 5,
  "specimen_name": "Grik4-Cre-5155",
  "strain": "C57BL/6J",
  "structure_abbrev": "PO",
  "structure_id": 1020,
  "structure_name": "Posterior complex of the thalamus",
  "primary_injection_structure": 218,
  "data_set_id": 292212456,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839331,
   "name": "Nr5a1-Cre"
  },
  "gender": "M",
  "injection_structures": "723/746",
  "injection_volume": 0.0908053594,
  "injection_x": 2830,
  "injection_y": 3580,
  "injection_z": 6880,
  "product_id": 5,
  "specimen_name": "Nr5a1-Cre-2364",
  "strain": "B6.FVB",
  "structure_abbrev": "ORBl",
  "structure_id": 723,
  "structure_name": "Orbital area, lateral part",
  "primary_injection_structure": 723,
  "data_set_id": 159887627,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "F",
  "injection_structures": "39/48",
  "injection_volume": 0.009907085825,
  "injection_x": 4810,
  "injection_y": 2270,
  "injection_z": 6050,
  "product_id": 5,
  "specimen_name": "Ntsr1-Cre-3505",
  "strain": "C57BL/6J",
  "structure_abbrev": "ACAv",
  "structure_id": 48,
  "structure_name": "Anterior cingulate area, ventral part",
  "primary_injection_structure": 39,
  "data_set_id": 156546627,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182693192,
   "name": "Chrna2-Cre_OE25"
  },
  "gender": "M",
  "injection_structures": "337/361/894/312782546",
  "injection_volume": 0.1305199672,
  "injection_x": 6840,
  "injection_y": 1060,
  "injection_z": 6920,
  "product_id": 5,
  "specimen_name": "Chrna2-Cre_OE25-3901",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "SSp-tr",
  "structure_id": 361,
  "structure_name": "Primary somatosensory area, trunk",
  "primary_injection_structure": 337,
  "data_set_id": 292124765,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 503823716,
   "name": "Gnb4-IRES2-CreERT2"
  },
  "gender": "M",
  "injection_structures": "345/353",
  "injection_volume": 0.0021475475,
  "injection_x": 5700,
  "injection_y": 3240,
  "injection_z": 2330,
  "product_id": 35,
  "specimen_name": "Gnb4-IRES2-CreERT2-360657",
  "strain": null,
  "structure_abbrev": "SSp-n",
  "structure_id": 353,
  "structure_name": "Primary somatosensory area, nose",
  "primary_injection_structure": 345,
  "data_set_id": 656688345,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "337/369",
  "injection_volume": 0.1502545604,
  "injection_x": 5900,
  "injection_y": 1550,
  "injection_z": 7440,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-258",
  "strain": null,
  "structure_abbrev": "SSp-ll",
  "structure_id": 337,
  "structure_name": "Primary somatosensory area, lower limb",
  "primary_injection_structure": 337,
  "data_set_id": 266644610,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 183941572,
   "name": "Plxnd1-Cre_OG1"
  },
  "gender": "F",
  "injection_structures": "672",
  "injection_volume": 0.0967674736,
  "injection_x": 6030,
  "injection_y": 2530,
  "injection_z": 7870,
  "product_id": 5,
  "specimen_name": "Plxnd1-Cre-121493",
  "strain": null,
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 672,
  "data_set_id": 293366741,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 183238310,
   "name": "Ntrk1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "59/149/362",
  "injection_volume": 0.1181757598,
  "injection_x": 6510,
  "injection_y": 3680,
  "injection_z": 5590,
  "product_id": 5,
  "specimen_name": "Ntrk1-IRES-Cre-3514",
  "strain": "B6.129",
  "structure_abbrev": "PVT",
  "structure_id": 149,
  "structure_name": "Paraventricular nucleus of the thalamus",
  "primary_injection_structure": 59,
  "data_set_id": 263106751,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839285,
   "name": "Gpr26-Cre_KO250"
  },
  "gender": "M",
  "injection_structures": "39/993",
  "injection_volume": 0.196274204,
  "injection_x": 3050,
  "injection_y": 2180,
  "injection_z": 6280,
  "product_id": 5,
  "specimen_name": "Gpr26-Cre_KO250-123754",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 39,
  "data_set_id": 298601161,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "F",
  "injection_structures": "993",
  "injection_volume": 0.0577383905,
  "injection_x": 3040,
  "injection_y": 2340,
  "injection_z": 4500,
  "product_id": 45,
  "specimen_name": "Tlx3-Cre_PL56;Sst-IRES-FlpO;Ai65F-239012",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 517082898,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "385/425",
  "injection_volume": 0.01066742985,
  "injection_x": 10070,
  "injection_y": 2080,
  "injection_z": 2520,
  "product_id": 35,
  "specimen_name": "Tlx3-Cre_PL56-273879",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISpl",
  "structure_id": 425,
  "structure_name": "Posterolateral visual area",
  "primary_injection_structure": 385,
  "data_set_id": 555745687,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.1073505916,
  "injection_x": 8520,
  "injection_y": 1090,
  "injection_z": 7930,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-3796",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 166324604,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "44/48/814/972",
  "injection_volume": 0.2048747042125,
  "injection_x": 3750,
  "injection_y": 3670,
  "injection_z": 5690,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre_KL100-295",
  "strain": "C57BL/6J",
  "structure_abbrev": "ILA",
  "structure_id": 44,
  "structure_name": "Infralimbic area",
  "primary_injection_structure": 44,
  "data_set_id": 286313491,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0142754591,
  "injection_x": 9440,
  "injection_y": 1230,
  "injection_z": 7990,
  "product_id": 5,
  "specimen_name": "Tlx3-Cre-125625",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 297593235,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 304561261,
   "name": "Ndnf-IRES2-dgCre"
  },
  "gender": "F",
  "injection_structures": "250/258",
  "injection_volume": 0.0447424537,
  "injection_x": 5260,
  "injection_y": 3340,
  "injection_z": 6250,
  "product_id": 5,
  "specimen_name": "Ndnf-IRES2-dgCre-248358",
  "strain": null,
  "structure_abbrev": "LSc",
  "structure_id": 250,
  "structure_name": "Lateral septal nucleus, caudal (caudodorsal) part",
  "primary_injection_structure": 250,
  "data_set_id": 527808181,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 553502711,
   "name": "Chat-IRES-Cre-neo"
  },
  "gender": "F",
  "injection_structures": "147/460/867",
  "injection_volume": 0.0077850691625,
  "injection_x": 10710,
  "injection_y": 4130,
  "injection_z": 6900,
  "product_id": 5,
  "specimen_name": "Chat-IRES-Cre-3201",
  "strain": "B6.129",
  "structure_abbrev": "PB",
  "structure_id": 867,
  "structure_name": "Parabrachial nucleus",
  "primary_injection_structure": 147,
  "data_set_id": 183901489,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838048,
   "name": "Slc17a6-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "718/733",
  "injection_volume": 0.1799635544,
  "injection_x": 7390,
  "injection_y": 4200,
  "injection_z": 7540,
  "product_id": 5,
  "specimen_name": "Slc17a6-IRES-Cre-125962",
  "strain": "B6.129",
  "structure_abbrev": "VPM",
  "structure_id": 733,
  "structure_name": "Ventral posteromedial nucleus of the thalamus",
  "primary_injection_structure": 718,
  "data_set_id": 312240825,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 503823716,
   "name": "Gnb4-IRES2-CreERT2"
  },
  "gender": "M",
  "injection_structures": "378",
  "injection_volume": 0.01987531875,
  "injection_x": 6420,
  "injection_y": 3980,
  "injection_z": 9690,
  "product_id": 5,
  "specimen_name": "Gnb4-IRES2-CreERT2-285316",
  "strain": null,
  "structure_abbrev": "SSs",
  "structure_id": 378,
  "structure_name": "Supplemental somatosensory area",
  "primary_injection_structure": 378,
  "data_set_id": 581327676,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "353",
  "injection_volume": 0.3585781192,
  "injection_x": 5830,
  "injection_y": 2430,
  "injection_z": 8740,
  "product_id": 5,
  "specimen_name": "378-1353",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-n",
  "structure_id": 353,
  "structure_name": "Primary somatosensory area, nose",
  "primary_injection_structure": 353,
  "data_set_id": 126908007,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "362",
  "injection_volume": 0.0765251081,
  "injection_x": 7100,
  "injection_y": 4220,
  "injection_z": 6190,
  "product_id": 5,
  "specimen_name": "378-1901",
  "strain": "C57BL/6J",
  "structure_abbrev": "MD",
  "structure_id": 362,
  "structure_name": "Mediodorsal nucleus of thalamus",
  "primary_injection_structure": 362,
  "data_set_id": 272970747,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "F",
  "injection_structures": "385/394/417/533/312782546",
  "injection_volume": 0.2946945848,
  "injection_x": 8030,
  "injection_y": 1120,
  "injection_z": 3850,
  "product_id": 36,
  "specimen_name": "Ntsr1-Cre_GN220-275964",
  "strain": "unknown",
  "structure_abbrev": "VISa",
  "structure_id": 312782546,
  "structure_name": "Anterior area",
  "primary_injection_structure": 385,
  "data_set_id": 557347149,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.01708920815,
  "injection_x": 9240,
  "injection_y": 1810,
  "injection_z": 3110,
  "product_id": 36,
  "specimen_name": "Ntsr1-Cre_GN220-235603",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 514314707,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "541/312782574/312782628",
  "injection_volume": 0.073378578,
  "injection_x": 9270,
  "injection_y": 2860,
  "injection_z": 9500,
  "product_id": 5,
  "specimen_name": "378-1607",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISpor",
  "structure_id": 312782628,
  "structure_name": "Postrhinal area",
  "primary_injection_structure": 541,
  "data_set_id": 157062358,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838361,
   "name": "Etv1-CreERT2"
  },
  "gender": "F",
  "injection_structures": "337/361/382/423/463",
  "injection_volume": 0.0724139346,
  "injection_x": 6590,
  "injection_y": 2050,
  "injection_z": 7510,
  "product_id": 5,
  "specimen_name": "Etv1-CreERT2-2896",
  "strain": "C57BL/6J",
  "structure_abbrev": "CA1",
  "structure_id": 382,
  "structure_name": "Field CA1",
  "primary_injection_structure": 337,
  "data_set_id": 171019004,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 553502711,
   "name": "Chat-IRES-Cre-neo"
  },
  "gender": "F",
  "injection_structures": "672",
  "injection_volume": 0.00217951908125,
  "injection_x": 6440,
  "injection_y": 2840,
  "injection_z": 8510,
  "product_id": 5,
  "specimen_name": "Chat-IRES-Cre-122857",
  "strain": "B6.129",
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 672,
  "data_set_id": 307692311,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "541/312782628",
  "injection_volume": 0.0730558738,
  "injection_x": 9130,
  "injection_y": 3000,
  "injection_z": 9700,
  "product_id": 5,
  "specimen_name": "Ntsr1-Cre-3560",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISpor",
  "structure_id": 312782628,
  "structure_name": "Postrhinal area",
  "primary_injection_structure": 541,
  "data_set_id": 159321806,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "F",
  "injection_structures": "394/312782546",
  "injection_volume": 0.0363448778,
  "injection_x": 7460,
  "injection_y": 830,
  "injection_z": 4300,
  "product_id": 35,
  "specimen_name": "Tlx3-Cre_PL56-330021",
  "strain": null,
  "structure_abbrev": "VISam",
  "structure_id": 394,
  "structure_name": "Anteromedial visual area",
  "primary_injection_structure": 394,
  "data_set_id": 605661910,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837834,
   "name": "Crh-IRES-Cre_BL"
  },
  "gender": "M",
  "injection_structures": "39/993",
  "injection_volume": 0.02993805675,
  "injection_x": 5090,
  "injection_y": 1500,
  "injection_z": 6040,
  "product_id": 5,
  "specimen_name": "Crh-IRES-Cre-285",
  "strain": "B6.129",
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 183104511,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "329",
  "injection_volume": 0.1312833186,
  "injection_x": 6920,
  "injection_y": 1600,
  "injection_z": 8450,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-264",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 266486371,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "111/131/295/672/952",
  "injection_volume": 0.1401703604,
  "injection_x": 6910,
  "injection_y": 5130,
  "injection_z": 9450,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-121656",
  "strain": "C57BL/6J",
  "structure_abbrev": "LA",
  "structure_id": 131,
  "structure_name": "Lateral amygdalar nucleus",
  "primary_injection_structure": 111,
  "data_set_id": 516846757,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182901722,
   "name": "Ppp1r17-Cre_NL146"
  },
  "gender": "M",
  "injection_structures": "329/382/423/463",
  "injection_volume": 0.117110049,
  "injection_x": 7350,
  "injection_y": 2430,
  "injection_z": 8820,
  "product_id": 5,
  "specimen_name": "Ppp1r17-Cre_NL146-3912",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "CA3",
  "structure_id": 463,
  "structure_name": "Field CA3",
  "primary_injection_structure": 329,
  "data_set_id": 278398243,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "409",
  "injection_volume": 0.0714350371,
  "injection_x": 9170,
  "injection_y": 1860,
  "injection_z": 2070,
  "product_id": 35,
  "specimen_name": "Rbp4-Cre_KL100-202029",
  "strain": null,
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 409,
  "data_set_id": 495562600,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.017221837675,
  "injection_x": 9040,
  "injection_y": 1530,
  "injection_z": 8090,
  "product_id": 5,
  "specimen_name": "Syt6-Cre-601",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 156784823,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "345",
  "injection_volume": 0.1382585568,
  "injection_x": 4370,
  "injection_y": 3460,
  "injection_z": 9310,
  "product_id": 5,
  "specimen_name": "378-1618",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-m",
  "structure_id": 345,
  "structure_name": "Primary somatosensory area, mouth",
  "primary_injection_structure": 345,
  "data_set_id": 157654817,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0303851891,
  "injection_x": 8650,
  "injection_y": 1380,
  "injection_z": 3140,
  "product_id": 36,
  "specimen_name": "Tlx3-Cre_PL56-244757",
  "strain": "B6.FVB",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 522408611,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838502,
   "name": "Grik4-Cre"
  },
  "gender": "F",
  "injection_structures": "502/484682470",
  "injection_volume": 0.064708273,
  "injection_x": 9150,
  "injection_y": 4250,
  "injection_z": 9110,
  "product_id": 5,
  "specimen_name": "Grik4-Cre-5123",
  "strain": "C57BL/6J",
  "structure_abbrev": "SUB",
  "structure_id": 502,
  "structure_name": "Subiculum",
  "primary_injection_structure": 502,
  "data_set_id": 264945172,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839331,
   "name": "Nr5a1-Cre"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0306428409,
  "injection_x": 8540,
  "injection_y": 750,
  "injection_z": 7690,
  "product_id": 5,
  "specimen_name": "Nr5a1-Cre-2446",
  "strain": "B6.FVB",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 177781006,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "127/262/351/797",
  "injection_volume": 0.1770259652,
  "injection_x": 5570,
  "injection_y": 4500,
  "injection_z": 6440,
  "product_id": 5,
  "specimen_name": "378-1527",
  "strain": "C57BL/6J",
  "structure_abbrev": "BST",
  "structure_id": 351,
  "structure_name": "Bed nuclei of the stria terminalis",
  "primary_injection_structure": 127,
  "data_set_id": 147049515,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "463/726",
  "injection_volume": 0.1479260804,
  "injection_x": 8750,
  "injection_y": 2930,
  "injection_z": 8310,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre_KL100-248",
  "strain": "C57BL/6J",
  "structure_abbrev": "DG",
  "structure_id": 726,
  "structure_name": "Dentate gyrus",
  "primary_injection_structure": 463,
  "data_set_id": 266409949,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838633,
   "name": "Trib2-F2A-CreERT2"
  },
  "gender": "M",
  "injection_structures": "337/361",
  "injection_volume": 0.17444637,
  "injection_x": 6160,
  "injection_y": 1290,
  "injection_z": 6990,
  "product_id": 5,
  "specimen_name": "Trib2-2A-CreERT2-D-3302",
  "strain": "B6.129",
  "structure_abbrev": "SSp-ll",
  "structure_id": 337,
  "structure_name": "Primary somatosensory area, lower limb",
  "primary_injection_structure": 337,
  "data_set_id": 292620251,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.03720122875,
  "injection_x": 8050,
  "injection_y": 1170,
  "injection_z": 3450,
  "product_id": 35,
  "specimen_name": "Tlx3-Cre_PL56-319677",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 596252304,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "394/894",
  "injection_volume": 0.1241232524,
  "injection_x": 7870,
  "injection_y": 540,
  "injection_z": 4300,
  "product_id": 36,
  "specimen_name": "Cux2-IRES-Cre-274796",
  "strain": "C57BL/6J",
  "structure_abbrev": "RSPagl",
  "structure_id": 894,
  "structure_name": "Retrosplenial area, lateral agranular part",
  "primary_injection_structure": 394,
  "data_set_id": 557187751,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "39",
  "injection_volume": 0.01122412375,
  "injection_x": 4720,
  "injection_y": 1810,
  "injection_z": 5430,
  "product_id": 35,
  "specimen_name": "Cux2-IRES-Cre-243007",
  "strain": "C57BL/6J",
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 522774029,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "39/48/972/993",
  "injection_volume": 0.5722432464,
  "injection_x": 3810,
  "injection_y": 2550,
  "injection_z": 5570,
  "product_id": 36,
  "specimen_name": "Emx1-IRES-Cre-305958",
  "strain": null,
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 583747816,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0843142608,
  "injection_x": 8210,
  "injection_y": 1400,
  "injection_z": 3400,
  "product_id": 35,
  "specimen_name": "Ntsr1-Cre_GN220-257920",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 539739321,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "F",
  "injection_structures": "337/369",
  "injection_volume": 0.243838504,
  "injection_x": 6060,
  "injection_y": 2060,
  "injection_z": 7580,
  "product_id": 5,
  "specimen_name": "Syt6-Cre-3144",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-ll",
  "structure_id": 337,
  "structure_name": "Primary somatosensory area, lower limb",
  "primary_injection_structure": 337,
  "data_set_id": 177319236,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "F",
  "injection_structures": "385/409/425",
  "injection_volume": 0.0276260677,
  "injection_x": 9860,
  "injection_y": 2020,
  "injection_z": 9010,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg3-Cre-4871",
  "strain": "B6.C3H",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 257667118,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838104,
   "name": "Pvalb-T2A-CreERT2"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.1036669774,
  "injection_x": 8050,
  "injection_y": 750,
  "injection_z": 8300,
  "product_id": 5,
  "specimen_name": "Pvalb-T2A-CreERT2-337653",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 638313933,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "385/409",
  "injection_volume": 0.5686671088,
  "injection_x": 8810,
  "injection_y": 1890,
  "injection_z": 8690,
  "product_id": 5,
  "specimen_name": "C57BL/6-153764",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 307743253,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838942,
   "name": "Htr2a-Cre_KM207"
  },
  "gender": "F",
  "injection_structures": "44/48/814/972",
  "injection_volume": 0.198731243792078,
  "injection_x": 3800,
  "injection_y": 3350,
  "injection_z": 5980,
  "product_id": 5,
  "specimen_name": "Htr2a-Cre_KM207-3460",
  "strain": null,
  "structure_abbrev": "PL",
  "structure_id": 972,
  "structure_name": "Prelimbic area",
  "primary_injection_structure": 44,
  "data_set_id": 266816894,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761781,
   "name": "Efr3a-Cre_NO108"
  },
  "gender": "F",
  "injection_structures": "226/298/342/985",
  "injection_volume": 0.0856565472,
  "injection_x": 5290,
  "injection_y": 6260,
  "injection_z": 7440,
  "product_id": 5,
  "specimen_name": "Efr3a-Cre_NO108-126004",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "SI",
  "structure_id": 342,
  "structure_name": "Substantia innominata",
  "primary_injection_structure": 226,
  "data_set_id": 298835152,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837779,
   "name": "Pcp2-Cre_GN135"
  },
  "gender": "F",
  "injection_structures": "920",
  "injection_volume": 0.00438186879375,
  "injection_x": 11160,
  "injection_y": 2070,
  "injection_z": 6250,
  "product_id": 5,
  "specimen_name": "Pcp2-Cre (NH)-3569",
  "strain": "C57BL/6J",
  "structure_abbrev": "CENT",
  "structure_id": 920,
  "structure_name": "Central lobule",
  "primary_injection_structure": 920,
  "data_set_id": 147708352,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839513,
   "name": "Drd2-Cre_ER44"
  },
  "gender": "F",
  "injection_structures": "672",
  "injection_volume": 0.3153216444,
  "injection_x": 5230,
  "injection_y": 4570,
  "injection_z": 7190,
  "product_id": 5,
  "specimen_name": "Drd2-Cre-2028",
  "strain": "C57BL/6J",
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 672,
  "data_set_id": 159329308,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 183238310,
   "name": "Ntrk1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "342/754",
  "injection_volume": 0.0237153434,
  "injection_x": 3810,
  "injection_y": 6730,
  "injection_z": 7050,
  "product_id": 5,
  "specimen_name": "Ntrk1-IRES-Cre_LURC-3547",
  "strain": null,
  "structure_abbrev": "OT",
  "structure_id": 754,
  "structure_name": "Olfactory tubercle",
  "primary_injection_structure": 342,
  "data_set_id": 292962283,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "104/119/345/583/1057",
  "injection_volume": 0.243130748,
  "injection_x": 4140,
  "injection_y": 3980,
  "injection_z": 9230,
  "product_id": 5,
  "specimen_name": "378-1864",
  "strain": "C57BL/6J",
  "structure_abbrev": "AId",
  "structure_id": 104,
  "structure_name": "Agranular insular area, dorsal part",
  "primary_injection_structure": 104,
  "data_set_id": 272737914,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839285,
   "name": "Gpr26-Cre_KO250"
  },
  "gender": "M",
  "injection_structures": "961",
  "injection_volume": 0.0784926688,
  "injection_x": 4460,
  "injection_y": 5940,
  "injection_z": 8640,
  "product_id": 5,
  "specimen_name": "Gpr26-Cre_KO250-213396",
  "strain": null,
  "structure_abbrev": "PIR",
  "structure_id": 961,
  "structure_name": "Piriform area",
  "primary_injection_structure": 961,
  "data_set_id": 518745077,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "329/361/385/417/533/182305689/312782546",
  "injection_volume": 0.2203097428,
  "injection_x": 6900,
  "injection_y": 1480,
  "injection_z": 3290,
  "product_id": 36,
  "specimen_name": "Rbp4-Cre_KL100-233303",
  "strain": null,
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 511550172,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182693192,
   "name": "Chrna2-Cre_OE25"
  },
  "gender": "M",
  "injection_structures": "39/48",
  "injection_volume": 0.0500157945,
  "injection_x": 4770,
  "injection_y": 2260,
  "injection_z": 6060,
  "product_id": 5,
  "specimen_name": "Chrna2-Cre_OE25-3879",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "ACAv",
  "structure_id": 48,
  "structure_name": "Anterior cingulate area, ventral part",
  "primary_injection_structure": 39,
  "data_set_id": 278173743,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "39/731/746/972/993",
  "injection_volume": 0.3800914712,
  "injection_x": 2450,
  "injection_y": 2140,
  "injection_z": 5260,
  "product_id": 35,
  "specimen_name": "Cux2-IRES-Cre-243005",
  "strain": "C57BL/6J",
  "structure_abbrev": "ORBvl",
  "structure_id": 746,
  "structure_name": "Orbital area, ventrolateral part",
  "primary_injection_structure": 39,
  "data_set_id": 522773270,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 553502711,
   "name": "Chat-IRES-Cre-neo"
  },
  "gender": "F",
  "injection_structures": "23/194/298/342/536/1022",
  "injection_volume": 0.03276091735,
  "injection_x": 5660,
  "injection_y": 6170,
  "injection_z": 3900,
  "product_id": 36,
  "specimen_name": "Chat-IRES-Cre-187663",
  "strain": null,
  "structure_abbrev": "SI",
  "structure_id": 342,
  "structure_name": "Substantia innominata",
  "primary_injection_structure": 23,
  "data_set_id": 478581786,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "M",
  "injection_structures": "385/425/879/894/1037/484682508",
  "injection_volume": 0.00346031783125,
  "injection_x": 9720,
  "injection_y": 2130,
  "injection_z": 8330,
  "product_id": 5,
  "specimen_name": "Syt6-Cre_KI148-124890",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISpl",
  "structure_id": 425,
  "structure_name": "Posterolateral visual area",
  "primary_injection_structure": 385,
  "data_set_id": 297671724,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "985/993",
  "injection_volume": 0.2685053,
  "injection_x": 4850,
  "injection_y": 1240,
  "injection_z": 6660,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-241",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 263781454,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839075,
   "name": "Pomc-Cre_BL"
  },
  "gender": "M",
  "injection_structures": "223/614/693/10671",
  "injection_volume": 0.13664530683125,
  "injection_x": 6690,
  "injection_y": 6920,
  "injection_z": 5860,
  "product_id": 5,
  "specimen_name": "Pomc-Cre-757",
  "strain": "B6.FVB",
  "structure_abbrev": "ARH",
  "structure_id": 223,
  "structure_name": "Arcuate hypothalamic nucleus",
  "primary_injection_structure": 223,
  "data_set_id": 263369222,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839513,
   "name": "Drd2-Cre_ER44"
  },
  "gender": "F",
  "injection_structures": "56/342/672/754",
  "injection_volume": 0.0861825916,
  "injection_x": 4480,
  "injection_y": 7250,
  "injection_z": 7400,
  "product_id": 5,
  "specimen_name": "Drd2-Cre-2051",
  "strain": "C57BL/6J",
  "structure_abbrev": "OT",
  "structure_id": 754,
  "structure_name": "Olfactory tubercle",
  "primary_injection_structure": 56,
  "data_set_id": 167904255,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 179697864,
   "name": "Rorb-IRES2-Cre"
  },
  "gender": "M",
  "injection_structures": "294/302",
  "injection_volume": 0.0440261619,
  "injection_x": 9280,
  "injection_y": 1370,
  "injection_z": 6470,
  "product_id": 5,
  "specimen_name": "Rorb-IRES2-Cre-208962",
  "strain": null,
  "structure_abbrev": "SCs",
  "structure_id": 302,
  "structure_name": "Superior colliculus, sensory related",
  "primary_injection_structure": 294,
  "data_set_id": 509880387,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.016397721375,
  "injection_x": 8380,
  "injection_y": 1130,
  "injection_z": 3380,
  "product_id": 36,
  "specimen_name": "Rbp4-Cre_KL100-227921",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 503809372,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "879/886",
  "injection_volume": 0.0390158139,
  "injection_x": 8250,
  "injection_y": 790,
  "injection_z": 4970,
  "product_id": 35,
  "specimen_name": "Emx1-IRES-Cre-305954",
  "strain": null,
  "structure_abbrev": "RSPd",
  "structure_id": 879,
  "structure_name": "Retrosplenial area, dorsal part",
  "primary_injection_structure": 879,
  "data_set_id": 584895127,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761918,
   "name": "Oxtr-Cre_ON66"
  },
  "gender": "F",
  "injection_structures": "44/814/972",
  "injection_volume": 0.1112240514,
  "injection_x": 3730,
  "injection_y": 3940,
  "injection_z": 6390,
  "product_id": 5,
  "specimen_name": "Oxtr-Cre_ON66-121549",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "ILA",
  "structure_id": 44,
  "structure_name": "Infralimbic area",
  "primary_injection_structure": 44,
  "data_set_id": 294399325,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "39/985/993",
  "injection_volume": 0.2552866876,
  "injection_x": 4680,
  "injection_y": 1280,
  "injection_z": 6710,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre-178",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 39,
  "data_set_id": 182793477,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 298725927,
   "name": "Penk-IRES2-Cre-neo"
  },
  "gender": "M",
  "injection_structures": "385/409/425",
  "injection_volume": 0.04103320025,
  "injection_x": 9470,
  "injection_y": 2090,
  "injection_z": 8880,
  "product_id": 5,
  "specimen_name": "Penk-IRES2-Cre-neo-302797",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 586054033,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838496,
   "name": "Grp-Cre_KH288"
  },
  "gender": "F",
  "injection_structures": "362/575/629/907/1020",
  "injection_volume": 0.043255044,
  "injection_x": 6840,
  "injection_y": 4200,
  "injection_z": 6640,
  "product_id": 5,
  "specimen_name": "Grp-Cre-65",
  "strain": "B6.FVB(ICR)",
  "structure_abbrev": "PCN",
  "structure_id": 907,
  "structure_name": "Paracentral nucleus",
  "primary_injection_structure": 362,
  "data_set_id": 183071513,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "409/425/312782574/312782628",
  "injection_volume": 0.1785615076,
  "injection_x": 9180,
  "injection_y": 2370,
  "injection_z": 9440,
  "product_id": 5,
  "specimen_name": "378-796",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISli",
  "structure_id": 312782574,
  "structure_name": "Laterointermediate area",
  "primary_injection_structure": 409,
  "data_set_id": 100141796,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "F",
  "injection_structures": "394/533/312782546",
  "injection_volume": 0.01360473485,
  "injection_x": 7790,
  "injection_y": 890,
  "injection_z": 7510,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg3-Cre-4900",
  "strain": "B6.C3H",
  "structure_abbrev": "VISam",
  "structure_id": 394,
  "structure_name": "Anteromedial visual area",
  "primary_injection_structure": 394,
  "data_set_id": 268038969,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 298725927,
   "name": "Penk-IRES2-Cre-neo"
  },
  "gender": "M",
  "injection_structures": "672/1022",
  "injection_volume": 0.1148923874,
  "injection_x": 5790,
  "injection_y": 4390,
  "injection_z": 7390,
  "product_id": 5,
  "specimen_name": "Penk-IRES2-Cre-neo-292009",
  "strain": "unknown",
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 672,
  "data_set_id": 575683020,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "39/48",
  "injection_volume": 0.125346312,
  "injection_x": 4930,
  "injection_y": 1800,
  "injection_z": 6080,
  "product_id": 5,
  "specimen_name": "378-1343",
  "strain": "C57BL/6J",
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 139426984,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838710,
   "name": "Sst-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "394/533/312782546",
  "injection_volume": 0.2095367988,
  "injection_x": 7700,
  "injection_y": 590,
  "injection_z": 7200,
  "product_id": 5,
  "specimen_name": "Sst-IRES-Cre-233",
  "strain": null,
  "structure_abbrev": "VISam",
  "structure_id": 394,
  "structure_name": "Anteromedial visual area",
  "primary_injection_structure": 394,
  "data_set_id": 288264047,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839331,
   "name": "Nr5a1-Cre"
  },
  "gender": "F",
  "injection_structures": "614/693/830/1004",
  "injection_volume": 0.211092098,
  "injection_x": 6810,
  "injection_y": 6320,
  "injection_z": 6180,
  "product_id": 5,
  "specimen_name": "Nr5a1-Cre-2476",
  "strain": "B6.FVB",
  "structure_abbrev": "VMH",
  "structure_id": 693,
  "structure_name": "Ventromedial hypothalamic nucleus",
  "primary_injection_structure": 614,
  "data_set_id": 182337561,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "F",
  "injection_structures": "329/417",
  "injection_volume": 0.0700897813,
  "injection_x": 7350,
  "injection_y": 1420,
  "injection_z": 2650,
  "product_id": 36,
  "specimen_name": "Tlx3-Cre_PL56-372776",
  "strain": null,
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 670228985,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837315,
   "name": "Avp-IRES2-Cre"
  },
  "gender": "M",
  "injection_structures": "38/63/181/366/1077",
  "injection_volume": 0.0118468084,
  "injection_x": 6230,
  "injection_y": 5530,
  "injection_z": 5940,
  "product_id": 5,
  "specimen_name": "Avp-IRES2-Cre-D-3545",
  "strain": "B6.129",
  "structure_abbrev": "PVH",
  "structure_id": 38,
  "structure_name": "Paraventricular hypothalamic nucleus",
  "primary_injection_structure": 38,
  "data_set_id": 267997620,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838803,
   "name": "Grm2-Cre_MR90"
  },
  "gender": "F",
  "injection_structures": "215/218/325/1029/563807435",
  "injection_volume": 0.0792803291,
  "injection_x": 7830,
  "injection_y": 3420,
  "injection_z": 7330,
  "product_id": 5,
  "specimen_name": "Grm2-Cre-46",
  "strain": null,
  "structure_abbrev": "LP",
  "structure_id": 218,
  "structure_name": "Lateral posterior nucleus of the thalamus",
  "primary_injection_structure": 215,
  "data_set_id": 166267651,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0801850847,
  "injection_x": 8350,
  "injection_y": 1210,
  "injection_z": 3600,
  "product_id": 35,
  "specimen_name": "Cux2-IRES-Cre-246905",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 524656772,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839186,
   "name": "Ucn3-Cre_KF43"
  },
  "gender": "M",
  "injection_structures": "194/364/470/525",
  "injection_volume": 0.0061278217875,
  "injection_x": 7470,
  "injection_y": 5610,
  "injection_z": 7000,
  "product_id": 5,
  "specimen_name": "Ucn3-Cre-1980",
  "strain": null,
  "structure_abbrev": "PSTN",
  "structure_id": 364,
  "structure_name": "Parasubthalamic nucleus",
  "primary_injection_structure": 194,
  "data_set_id": 165974379,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0059079391875,
  "injection_x": 9370,
  "injection_y": 1320,
  "injection_z": 3350,
  "product_id": 36,
  "specimen_name": "A930038C07Rik-Tg1-Cre-186365",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 478257195,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838496,
   "name": "Grp-Cre_KH288"
  },
  "gender": "M",
  "injection_structures": "215",
  "injection_volume": 0.051320542,
  "injection_x": 8260,
  "injection_y": 3430,
  "injection_z": 6880,
  "product_id": 5,
  "specimen_name": "Grp-Cre_KH288-112",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "APN",
  "structure_id": 215,
  "structure_name": "Anterior pretectal nucleus",
  "primary_injection_structure": 215,
  "data_set_id": 265293391,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "385/533/879/894",
  "injection_volume": 0.0774230968,
  "injection_x": 9700,
  "injection_y": 960,
  "injection_z": 7200,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-122234",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 294484177,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "F",
  "injection_structures": "345/378/1057",
  "injection_volume": 0.00280216698125,
  "injection_x": 4570,
  "injection_y": 3730,
  "injection_z": 9470,
  "product_id": 5,
  "specimen_name": "Syt6-Cre-3158",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSs",
  "structure_id": 378,
  "structure_name": "Supplemental somatosensory area",
  "primary_injection_structure": 345,
  "data_set_id": 177783918,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "385/402/409",
  "injection_volume": 0.1688734824,
  "injection_x": 8490,
  "injection_y": 1430,
  "injection_z": 9120,
  "product_id": 5,
  "specimen_name": "378-1192",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 385,
  "data_set_id": 116903968,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839331,
   "name": "Nr5a1-Cre"
  },
  "gender": "M",
  "injection_structures": "385/402/409/417",
  "injection_volume": 0.0287673463,
  "injection_x": 8310,
  "injection_y": 1310,
  "injection_z": 8970,
  "product_id": 5,
  "specimen_name": "Nr5a1-Cre-2458",
  "strain": "B6.FVB",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 176882966,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837328,
   "name": "Syt17-Cre_NO14"
  },
  "gender": "F",
  "injection_structures": "726",
  "injection_volume": 0.0118690005,
  "injection_x": 7540,
  "injection_y": 2260,
  "injection_z": 7380,
  "product_id": 5,
  "specimen_name": "Syt17-Cre_NO14_LURC-120288",
  "strain": "C57BL/6J",
  "structure_abbrev": "DG",
  "structure_id": 726,
  "structure_name": "Dentate gyrus",
  "primary_injection_structure": 726,
  "data_set_id": 293011049,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837446,
   "name": "Adcyap1-2A-Cre"
  },
  "gender": "M",
  "injection_structures": "202/209/217/534/781/867",
  "injection_volume": 0.3317439552,
  "injection_x": 10390,
  "injection_y": 4410,
  "injection_z": 7520,
  "product_id": 5,
  "specimen_name": "Adcyap1-2A-Cre-D-127769",
  "strain": "B6.129",
  "structure_abbrev": "PB",
  "structure_id": 867,
  "structure_name": "Parabrachial nucleus",
  "primary_injection_structure": 202,
  "data_set_id": 552284594,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "385/533/879/886/894/1037/484682508",
  "injection_volume": 0.2050248592,
  "injection_x": 9410,
  "injection_y": 1710,
  "injection_z": 3940,
  "product_id": 36,
  "specimen_name": "Rbp4-Cre_KL100-195017",
  "strain": null,
  "structure_abbrev": "RSPd",
  "structure_id": 879,
  "structure_name": "Retrosplenial area, dorsal part",
  "primary_injection_structure": 385,
  "data_set_id": 482581199,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837563,
   "name": "Slc17a8-iCre"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.015978617025,
  "injection_x": 8490,
  "injection_y": 1030,
  "injection_z": 8070,
  "product_id": 5,
  "specimen_name": "Slc17a8-iCre-350811",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 648318728,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837481,
   "name": "Scnn1a-Tg2-Cre"
  },
  "gender": "M",
  "injection_structures": "170/218",
  "injection_volume": 0.0545025187,
  "injection_x": 7530,
  "injection_y": 3250,
  "injection_z": 7780,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg2-Cre-3357",
  "strain": "C57BL/6J",
  "structure_abbrev": "LGd",
  "structure_id": 170,
  "structure_name": "Dorsal part of the lateral geniculate complex",
  "primary_injection_structure": 170,
  "data_set_id": 156198187,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "329/378/1002/1011",
  "injection_volume": 0.0781312203,
  "injection_x": 7330,
  "injection_y": 2260,
  "injection_z": 9670,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-3762",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSs",
  "structure_id": 378,
  "structure_name": "Supplemental somatosensory area",
  "primary_injection_structure": 329,
  "data_set_id": 156395997,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839159,
   "name": "Cck-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "287/351",
  "injection_volume": 0.01033028535,
  "injection_x": 5510,
  "injection_y": 4800,
  "injection_z": 6390,
  "product_id": 5,
  "specimen_name": "Cck-IRES-Cre-64",
  "strain": null,
  "structure_abbrev": "BST",
  "structure_id": 351,
  "structure_name": "Bed nuclei of the stria terminalis",
  "primary_injection_structure": 287,
  "data_set_id": 159433187,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "329/378",
  "injection_volume": 0.1749166132,
  "injection_x": 7310,
  "injection_y": 2270,
  "injection_z": 8990,
  "product_id": 5,
  "specimen_name": "378-1073",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 112882565,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "111/378/677/952/961/1057",
  "injection_volume": 0.2799105596,
  "injection_x": 5930,
  "injection_y": 4760,
  "injection_z": 9880,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-3929",
  "strain": null,
  "structure_abbrev": "AIp",
  "structure_id": 111,
  "structure_name": "Agranular insular area, posterior part",
  "primary_injection_structure": 111,
  "data_set_id": 286775476,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838710,
   "name": "Sst-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "879/886",
  "injection_volume": 0.1181503092,
  "injection_x": 7260,
  "injection_y": 1050,
  "injection_z": 6130,
  "product_id": 5,
  "specimen_name": "Sst-IRES-Cre-234",
  "strain": null,
  "structure_abbrev": "RSPv",
  "structure_id": 886,
  "structure_name": "Retrosplenial area, ventral part",
  "primary_injection_structure": 879,
  "data_set_id": 288264753,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838828,
   "name": "Gal-Cre_KI87"
  },
  "gender": "F",
  "injection_structures": "64/155",
  "injection_volume": 0.0718350878,
  "injection_x": 6400,
  "injection_y": 3110,
  "injection_z": 7090,
  "product_id": 5,
  "specimen_name": "Gal-Cre-1199",
  "strain": "C57BL/6J",
  "structure_abbrev": "LD",
  "structure_id": 155,
  "structure_name": "Lateral dorsal nucleus of thalamus",
  "primary_injection_structure": 64,
  "data_set_id": 113554008,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838715,
   "name": "Tac1-IRES2-Cre"
  },
  "gender": "F",
  "injection_structures": "502/926/484682470",
  "injection_volume": 0.18830798,
  "injection_x": 9180,
  "injection_y": 3910,
  "injection_z": 9480,
  "product_id": 5,
  "specimen_name": "Tac1-IRES2-Cre-D-121210",
  "strain": "B6.129",
  "structure_abbrev": "SUB",
  "structure_id": 502,
  "structure_name": "Subiculum",
  "primary_injection_structure": 502,
  "data_set_id": 294314037,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 543338518,
   "name": "Vipr2-IRES2-Cre"
  },
  "gender": "F",
  "injection_structures": "170",
  "injection_volume": 0.0131094502,
  "injection_x": 7490,
  "injection_y": 3170,
  "injection_z": 7860,
  "product_id": 5,
  "specimen_name": "Vipr2-IRES2-Cre-332910",
  "strain": null,
  "structure_abbrev": "LGd",
  "structure_id": 170,
  "structure_name": "Dorsal part of the lateral geniculate complex",
  "primary_injection_structure": 170,
  "data_set_id": 642480973,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 301614880,
   "name": "Glt25d2-Cre_NF107"
  },
  "gender": "M",
  "injection_structures": "104/119",
  "injection_volume": 0.2116917796,
  "injection_x": 3350,
  "injection_y": 4600,
  "injection_z": 8460,
  "product_id": 5,
  "specimen_name": "Glt25d2-Cre_NF107-162215",
  "strain": null,
  "structure_abbrev": "AId",
  "structure_id": 104,
  "structure_name": "Agranular insular area, dorsal part",
  "primary_injection_structure": 104,
  "data_set_id": 313327028,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838622,
   "name": "Vip-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "985/993",
  "injection_volume": 0.1288322406,
  "injection_x": 3750,
  "injection_y": 1600,
  "injection_z": 7380,
  "product_id": 5,
  "specimen_name": "Vip-IRES-Cre-163",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 179904912,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "385/402/409/417",
  "injection_volume": 0.0830408194,
  "injection_x": 8260,
  "injection_y": 1650,
  "injection_z": 8870,
  "product_id": 5,
  "specimen_name": "Tlx3-Cre_LURC-124585",
  "strain": null,
  "structure_abbrev": "VISal",
  "structure_id": 402,
  "structure_name": "Anterolateral visual area",
  "primary_injection_structure": 385,
  "data_set_id": 297231636,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838259,
   "name": "Cort-T2A-Cre"
  },
  "gender": "F",
  "injection_structures": "985/993",
  "injection_volume": 0.2286342744,
  "injection_x": 3880,
  "injection_y": 1540,
  "injection_z": 7290,
  "product_id": 5,
  "specimen_name": "Cort-T2A-Cre-88",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 157826227,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838331,
   "name": "Cdhr1-Cre_KG66"
  },
  "gender": "M",
  "injection_structures": "507",
  "injection_volume": 0.0684507313,
  "injection_x": 2540,
  "injection_y": 5240,
  "injection_z": 5850,
  "product_id": 5,
  "specimen_name": "Cdhr1-Cre-124",
  "strain": null,
  "structure_abbrev": "MOB",
  "structure_id": 507,
  "structure_name": "Main olfactory bulb",
  "primary_injection_structure": 507,
  "data_set_id": 175728165,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182901722,
   "name": "Ppp1r17-Cre_NL146"
  },
  "gender": "F",
  "injection_structures": "1007",
  "injection_volume": 0.023310035,
  "injection_x": 11270,
  "injection_y": 1810,
  "injection_z": 8010,
  "product_id": 5,
  "specimen_name": "Ppp1r17-Cre_NL146-3874",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "SIM",
  "structure_id": 1007,
  "structure_name": "Simple lobule",
  "primary_injection_structure": 1007,
  "data_set_id": 267811103,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839285,
   "name": "Gpr26-Cre_KO250"
  },
  "gender": "M",
  "injection_structures": "337/361/312782546",
  "injection_volume": 0.1672525036,
  "injection_x": 6900,
  "injection_y": 1100,
  "injection_z": 7020,
  "product_id": 5,
  "specimen_name": "Gpr26-Cre_KO250-123676",
  "strain": null,
  "structure_abbrev": "SSp-tr",
  "structure_id": 361,
  "structure_name": "Primary somatosensory area, trunk",
  "primary_injection_structure": 337,
  "data_set_id": 298404154,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "337/985",
  "injection_volume": 0.2219102396,
  "injection_x": 5690,
  "injection_y": 1590,
  "injection_z": 7270,
  "product_id": 5,
  "specimen_name": "378-1076",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-ll",
  "structure_id": 337,
  "structure_name": "Primary somatosensory area, lower limb",
  "primary_injection_structure": 337,
  "data_set_id": 112935169,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837858,
   "name": "Pvalb-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "920/928",
  "injection_volume": 0.238528715755859,
  "injection_x": 11430,
  "injection_y": 2170,
  "injection_z": 6250,
  "product_id": 5,
  "specimen_name": "Pvalb-IRES-Cre-5034",
  "strain": "B6.129",
  "structure_abbrev": "CUL",
  "structure_id": 928,
  "structure_name": "Culmen",
  "primary_injection_structure": 920,
  "data_set_id": 167202174,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838633,
   "name": "Trib2-F2A-CreERT2"
  },
  "gender": "M",
  "injection_structures": "394/312782546",
  "injection_volume": 0.12358535,
  "injection_x": 7390,
  "injection_y": 1100,
  "injection_z": 7400,
  "product_id": 5,
  "specimen_name": "Trib2-2A-CreERT2-D-3290",
  "strain": "B6.129",
  "structure_abbrev": "VISam",
  "structure_id": 394,
  "structure_name": "Anteromedial visual area",
  "primary_injection_structure": 394,
  "data_set_id": 287601808,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837834,
   "name": "Crh-IRES-Cre_BL"
  },
  "gender": "M",
  "injection_structures": "83/235/978",
  "injection_volume": 0.1061924864,
  "injection_x": 12080,
  "injection_y": 7090,
  "injection_z": 6870,
  "product_id": 5,
  "specimen_name": "Crh-IRES-Cre-341",
  "strain": "B6.129",
  "structure_abbrev": "LRN",
  "structure_id": 235,
  "structure_name": "Lateral reticular nucleus",
  "primary_injection_structure": 83,
  "data_set_id": 277906744,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "39/993",
  "injection_volume": 0.00188460645625,
  "injection_x": 5100,
  "injection_y": 1530,
  "injection_z": 6050,
  "product_id": 5,
  "specimen_name": "Tlx3-Cre_LURC-3606",
  "strain": null,
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 293431869,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "329/353/182305689",
  "injection_volume": 0.0191213239,
  "injection_x": 5560,
  "injection_y": 2100,
  "injection_z": 8710,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-3820",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-n",
  "structure_id": 353,
  "structure_name": "Primary somatosensory area, nose",
  "primary_injection_structure": 329,
  "data_set_id": 171276330,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "985/993",
  "injection_volume": 0.2090960536,
  "injection_x": 3970,
  "injection_y": 1810,
  "injection_z": 7400,
  "product_id": 5,
  "specimen_name": "378-1350",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 141602484,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "F",
  "injection_structures": "879/886/894",
  "injection_volume": 0.0083249964875,
  "injection_x": 8450,
  "injection_y": 830,
  "injection_z": 4490,
  "product_id": 35,
  "specimen_name": "Ntsr1-Cre_GN220-307264",
  "strain": null,
  "structure_abbrev": "RSPagl",
  "structure_id": 894,
  "structure_name": "Retrosplenial area, lateral agranular part",
  "primary_injection_structure": 879,
  "data_set_id": 585760423,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "378/677",
  "injection_volume": 0.03828131615,
  "injection_x": 6550,
  "injection_y": 3400,
  "injection_z": 9950,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-3930",
  "strain": null,
  "structure_abbrev": "SSs",
  "structure_id": 378,
  "structure_name": "Supplemental somatosensory area",
  "primary_injection_structure": 378,
  "data_set_id": 286834976,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839513,
   "name": "Drd2-Cre_ER44"
  },
  "gender": "M",
  "injection_structures": "672",
  "injection_volume": 0.2368059652,
  "injection_x": 5600,
  "injection_y": 2950,
  "injection_z": 8490,
  "product_id": 5,
  "specimen_name": "Drd2-Cre-2008",
  "strain": "C57BL/6J",
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 672,
  "data_set_id": 156670520,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.0449954064,
  "injection_x": 8500,
  "injection_y": 830,
  "injection_z": 8380,
  "product_id": 5,
  "specimen_name": "Tlx3-Cre-124475",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 297592527,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "541/895/918/922",
  "injection_volume": 0.6441443176,
  "injection_x": 8990,
  "injection_y": 4940,
  "injection_z": 10530,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-3809",
  "strain": "C57BL/6J",
  "structure_abbrev": "ENTl",
  "structure_id": 918,
  "structure_name": "Entorhinal area, lateral part",
  "primary_injection_structure": 541,
  "data_set_id": 287952517,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838953,
   "name": "Oxt-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "38",
  "injection_volume": 0.0025100207125,
  "injection_x": 6020,
  "injection_y": 5530,
  "injection_z": 5890,
  "product_id": 5,
  "specimen_name": "Oxt-IRES-Cre-1203",
  "strain": "B6.129",
  "structure_abbrev": "PVH",
  "structure_id": 38,
  "structure_name": "Paraventricular hypothalamic nucleus",
  "primary_injection_structure": 38,
  "data_set_id": 147136518,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "394/312782546",
  "injection_volume": 0.0888404986,
  "injection_x": 7420,
  "injection_y": 1180,
  "injection_z": 7290,
  "product_id": 5,
  "specimen_name": "378-897",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISam",
  "structure_id": 394,
  "structure_name": "Anteromedial visual area",
  "primary_injection_structure": 394,
  "data_set_id": 100148503,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "337/361/985",
  "injection_volume": 0.009420510925,
  "injection_x": 6400,
  "injection_y": 1220,
  "injection_z": 7310,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-3918",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-tr",
  "structure_id": 361,
  "structure_name": "Primary somatosensory area, trunk",
  "primary_injection_structure": 337,
  "data_set_id": 281459203,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839285,
   "name": "Gpr26-Cre_KO250"
  },
  "gender": "F",
  "injection_structures": "402/409",
  "injection_volume": 0.0458771859,
  "injection_x": 8460,
  "injection_y": 1640,
  "injection_z": 9320,
  "product_id": 5,
  "specimen_name": "Gpr26-Cre_KO250-128695",
  "strain": null,
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 402,
  "data_set_id": 301210923,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "541/312782628",
  "injection_volume": 0.1068234398,
  "injection_x": 9240,
  "injection_y": 2790,
  "injection_z": 1340,
  "product_id": 36,
  "specimen_name": "Rbp4-Cre_KL100-351927",
  "strain": null,
  "structure_abbrev": "VISpor",
  "structure_id": 312782628,
  "structure_name": "Postrhinal area",
  "primary_injection_structure": 541,
  "data_set_id": 648252402,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "F",
  "injection_structures": "402/409",
  "injection_volume": 0.0089057591875,
  "injection_x": 8600,
  "injection_y": 1730,
  "injection_z": 8990,
  "product_id": 5,
  "specimen_name": "Ntsr1-Cre_GN220-3795",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 402,
  "data_set_id": 287036898,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0149129834,
  "injection_x": 8980,
  "injection_y": 1200,
  "injection_z": 3300,
  "product_id": 36,
  "specimen_name": "A930038C07Rik-Tg1-Cre-306720",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 584507699,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "345/353/378",
  "injection_volume": 0.241480722,
  "injection_x": 5240,
  "injection_y": 2710,
  "injection_z": 9530,
  "product_id": 5,
  "specimen_name": "Ntsr1-Cre_GN220-136213",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-m",
  "structure_id": 345,
  "structure_name": "Primary somatosensory area, mouth",
  "primary_injection_structure": 345,
  "data_set_id": 303617548,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "128/215/634/930",
  "injection_volume": 0.1202778696,
  "injection_x": 7780,
  "injection_y": 3610,
  "injection_z": 6320,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre_KL100-123748",
  "strain": null,
  "structure_abbrev": "PF",
  "structure_id": 930,
  "structure_name": "Parafascicular nucleus",
  "primary_injection_structure": 128,
  "data_set_id": 300642574,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838927,
   "name": "Pcdh9-Cre_NP276"
  },
  "gender": "F",
  "injection_structures": "534/621",
  "injection_volume": 0.0420113407,
  "injection_x": 10270,
  "injection_y": 5310,
  "injection_z": 7160,
  "product_id": 5,
  "specimen_name": "Pcdh9-Cre-86",
  "strain": null,
  "structure_abbrev": "V",
  "structure_id": 621,
  "structure_name": "Motor nucleus of trigeminal",
  "primary_injection_structure": 534,
  "data_set_id": 272829024,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 183238310,
   "name": "Ntrk1-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "56",
  "injection_volume": 0.0976463082,
  "injection_x": 3700,
  "injection_y": 5270,
  "injection_z": 6610,
  "product_id": 5,
  "specimen_name": "Ntrk1-IRES-Cre-3567",
  "strain": "B6.129",
  "structure_abbrev": "ACB",
  "structure_id": 56,
  "structure_name": "Nucleus accumbens",
  "primary_injection_structure": 56,
  "data_set_id": 287993060,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 179697864,
   "name": "Rorb-IRES2-Cre"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.081151987,
  "injection_x": 8200,
  "injection_y": 830,
  "injection_z": 8270,
  "product_id": 5,
  "specimen_name": "Rorb-IRES2-Cre-D_LURC-124656",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 300929973,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838633,
   "name": "Trib2-F2A-CreERT2"
  },
  "gender": "M",
  "injection_structures": "985/993",
  "injection_volume": 0.3250964192,
  "injection_x": 3620,
  "injection_y": 1950,
  "injection_z": 7510,
  "product_id": 5,
  "specimen_name": "Trib2-2A-CreERT2-D-3233",
  "strain": "B6.129",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 266176167,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838584,
   "name": "Gad2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "63/194/262/685/797",
  "injection_volume": 0.1576285802,
  "injection_x": 6790,
  "injection_y": 5650,
  "injection_z": 6680,
  "product_id": 5,
  "specimen_name": "Gad2-IRES-Cre-1079",
  "strain": "C57BL/6J",
  "structure_abbrev": "ZI",
  "structure_id": 797,
  "structure_name": "Zona incerta",
  "primary_injection_structure": 63,
  "data_set_id": 156315468,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "329",
  "injection_volume": 0.2405140696,
  "injection_x": 6870,
  "injection_y": 1420,
  "injection_z": 2270,
  "product_id": 36,
  "specimen_name": "Emx1-IRES-Cre-363522",
  "strain": null,
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 657041814,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838803,
   "name": "Grm2-Cre_MR90"
  },
  "gender": "M",
  "injection_structures": "726",
  "injection_volume": 0.0671065976,
  "injection_x": 6950,
  "injection_y": 2560,
  "injection_z": 6470,
  "product_id": 5,
  "specimen_name": "Grm2-Cre_MR90-3837",
  "strain": null,
  "structure_abbrev": "DG",
  "structure_id": 726,
  "structure_name": "Dentate gyrus",
  "primary_injection_structure": 726,
  "data_set_id": 287097454,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182693192,
   "name": "Chrna2-Cre_OE25"
  },
  "gender": "F",
  "injection_structures": "337/369/985",
  "injection_volume": 0.0368971176,
  "injection_x": 5150,
  "injection_y": 1720,
  "injection_z": 7680,
  "product_id": 5,
  "specimen_name": "Chrna2-Cre_OE25-3903",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "SSp-ul",
  "structure_id": 369,
  "structure_name": "Primary somatosensory area, upper limb",
  "primary_injection_structure": 337,
  "data_set_id": 292173552,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "533/879/886/894",
  "injection_volume": 0.1603770196,
  "injection_x": 8560,
  "injection_y": 940,
  "injection_z": 4230,
  "product_id": 36,
  "specimen_name": "Rbp4-Cre_KL100-208501",
  "strain": null,
  "structure_abbrev": "RSPv",
  "structure_id": 886,
  "structure_name": "Retrosplenial area, ventral part",
  "primary_injection_structure": 533,
  "data_set_id": 496113850,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839331,
   "name": "Nr5a1-Cre"
  },
  "gender": "F",
  "injection_structures": "378/1002/1011",
  "injection_volume": 0.01873352565,
  "injection_x": 7640,
  "injection_y": 2270,
  "injection_z": 10010,
  "product_id": 5,
  "specimen_name": "Nr5a1-Cre-144",
  "strain": "B6.FVB",
  "structure_abbrev": "AUDd",
  "structure_id": 1011,
  "structure_name": "Dorsal auditory area",
  "primary_injection_structure": 378,
  "data_set_id": 114752488,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838715,
   "name": "Tac1-IRES2-Cre"
  },
  "gender": "M",
  "injection_structures": "202/867/920",
  "injection_volume": 0.0243788769,
  "injection_x": 10500,
  "injection_y": 4190,
  "injection_z": 7380,
  "product_id": 5,
  "specimen_name": "Tac1-IRES2-Cre-D-122059",
  "strain": "B6.129",
  "structure_abbrev": "PB",
  "structure_id": 867,
  "structure_name": "Parabrachial nucleus",
  "primary_injection_structure": 202,
  "data_set_id": 294356216,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "385/425/879/894",
  "injection_volume": 0.314177318,
  "injection_x": 9980,
  "injection_y": 1490,
  "injection_z": 7920,
  "product_id": 5,
  "specimen_name": "378-1142",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 113887868,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838048,
   "name": "Slc17a6-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "15/149/181",
  "injection_volume": 0.0966703954775394,
  "injection_x": 5500,
  "injection_y": 4660,
  "injection_z": 6130,
  "product_id": 5,
  "specimen_name": "Slc17a6-IRES-Cre-125144",
  "strain": "B6.129",
  "structure_abbrev": "PT",
  "structure_id": 15,
  "structure_name": "Parataenial nucleus",
  "primary_injection_structure": 15,
  "data_set_id": 305449231,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837858,
   "name": "Pvalb-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "235/978",
  "injection_volume": 0.01124574255,
  "injection_x": 11720,
  "injection_y": 7020,
  "injection_z": 6920,
  "product_id": 5,
  "specimen_name": "Pvalb-IRES-Cre-5132",
  "strain": "B6.129",
  "structure_abbrev": "PGRNl",
  "structure_id": 978,
  "structure_name": "Paragigantocellular reticular nucleus, lateral part",
  "primary_injection_structure": 235,
  "data_set_id": 286725359,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 287226763,
   "name": "Calb1-T2A-dgCre"
  },
  "gender": "F",
  "injection_structures": "1002/1011",
  "injection_volume": 0.0057788444,
  "injection_x": 7790,
  "injection_y": 2870,
  "injection_z": 10160,
  "product_id": 5,
  "specimen_name": "Calb1-2A-dgCre-189011",
  "strain": "C57BL/6J",
  "structure_abbrev": "AUDp",
  "structure_id": 1002,
  "structure_name": "Primary auditory area",
  "primary_injection_structure": 1002,
  "data_set_id": 485022815,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838259,
   "name": "Cort-T2A-Cre"
  },
  "gender": "M",
  "injection_structures": "329/353/182305689",
  "injection_volume": 0.2030188776,
  "injection_x": 6030,
  "injection_y": 2220,
  "injection_z": 8360,
  "product_id": 5,
  "specimen_name": "Cort-T2A-Cre-125",
  "strain": null,
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 167374643,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 301467016,
   "name": "Npr3-IRES2-Cre"
  },
  "gender": "M",
  "injection_structures": "39/993",
  "injection_volume": 0.1093359638,
  "injection_x": 5080,
  "injection_y": 1560,
  "injection_z": 6070,
  "product_id": 5,
  "specimen_name": "Npr3-IRES2-Cre-313835",
  "strain": null,
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 592391705,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0331066638,
  "injection_x": 9320,
  "injection_y": 1250,
  "injection_z": 3430,
  "product_id": 36,
  "specimen_name": "Tlx3-Cre_PL56-226271",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 503017948,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "146/574/931/1093",
  "injection_volume": 0.244815999832031,
  "injection_x": 9040,
  "injection_y": 6160,
  "injection_z": 6080,
  "product_id": 5,
  "specimen_name": "378-1228",
  "strain": "C57BL/6J",
  "structure_abbrev": "TRN",
  "structure_id": 574,
  "structure_name": "Tegmental reticular nucleus",
  "primary_injection_structure": 146,
  "data_set_id": 121145045,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838633,
   "name": "Trib2-F2A-CreERT2"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.1124945724,
  "injection_x": 8990,
  "injection_y": 1050,
  "injection_z": 8290,
  "product_id": 5,
  "specimen_name": "Trib2-2A-CreERT2-D-123726",
  "strain": "B6.129",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 308879859,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "F",
  "injection_structures": "39/993",
  "injection_volume": 0.0125432993,
  "injection_x": 5120,
  "injection_y": 1900,
  "injection_z": 6680,
  "product_id": 5,
  "specimen_name": "Ntsr1-Cre-3538",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 39,
  "data_set_id": 158139883,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838048,
   "name": "Slc17a6-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "170",
  "injection_volume": 0.073006129,
  "injection_x": 7520,
  "injection_y": 3190,
  "injection_z": 3150,
  "product_id": 36,
  "specimen_name": "Slc17a6-IRES-Cre-190069",
  "strain": "C57BL/6J",
  "structure_abbrev": "LGd",
  "structure_id": 170,
  "structure_name": "Dorsal part of the lateral geniculate complex",
  "primary_injection_structure": 170,
  "data_set_id": 479670988,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "39/48",
  "injection_volume": 0.135139795,
  "injection_x": 4900,
  "injection_y": 2110,
  "injection_z": 6200,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre-50",
  "strain": null,
  "structure_abbrev": "ACAv",
  "structure_id": 48,
  "structure_name": "Anterior cingulate area, ventral part",
  "primary_injection_structure": 39,
  "data_set_id": 126117554,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "345/378/1057",
  "injection_volume": 0.989409568,
  "injection_x": 4620,
  "injection_y": 3340,
  "injection_z": 9380,
  "product_id": 5,
  "specimen_name": "378-1813",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-m",
  "structure_id": 345,
  "structure_name": "Primary somatosensory area, mouth",
  "primary_injection_structure": 345,
  "data_set_id": 180717881,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "35/294/795/872/975",
  "injection_volume": 0.0672325416875,
  "injection_x": 8990,
  "injection_y": 3680,
  "injection_z": 5790,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre_KL100-121996",
  "strain": "C57BL/6J",
  "structure_abbrev": "PAG",
  "structure_id": 795,
  "structure_name": "Periaqueductal gray",
  "primary_injection_structure": 35,
  "data_set_id": 301540850,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 301613169,
   "name": "Sepw1-Cre_NP39"
  },
  "gender": "F",
  "injection_structures": "985/993",
  "injection_volume": 0.0717065216,
  "injection_x": 3640,
  "injection_y": 1650,
  "injection_z": 7330,
  "product_id": 5,
  "specimen_name": "Sepw1-Cre_NP39-174461",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 477037203,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839494,
   "name": "Hdc-Cre_IM1"
  },
  "gender": "F",
  "injection_structures": "1/210/491/525/1004",
  "injection_volume": 0.013686122625,
  "injection_x": 8190,
  "injection_y": 6090,
  "injection_z": 6490,
  "product_id": 5,
  "specimen_name": "Hdc-Cre_IM1-204100",
  "strain": null,
  "structure_abbrev": "SUM",
  "structure_id": 525,
  "structure_name": "Supramammillary nucleus",
  "primary_injection_structure": 1,
  "data_set_id": 519164644,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838710,
   "name": "Sst-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "184/723/746/993",
  "injection_volume": 0.4225427584,
  "injection_x": 2580,
  "injection_y": 3850,
  "injection_z": 6610,
  "product_id": 5,
  "specimen_name": "Sst-IRES-Cre-232",
  "strain": null,
  "structure_abbrev": "ORBvl",
  "structure_id": 746,
  "structure_name": "Orbital area, ventrolateral part",
  "primary_injection_structure": 184,
  "data_set_id": 288263341,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 266649644,
   "name": "Drd3-Cre_KI196"
  },
  "gender": "F",
  "injection_structures": "56",
  "injection_volume": 0.1553988156,
  "injection_x": 4230,
  "injection_y": 4990,
  "injection_z": 6300,
  "product_id": 5,
  "specimen_name": "Drd3-Cre_KI196_LURC-132028",
  "strain": null,
  "structure_abbrev": "ACB",
  "structure_id": 56,
  "structure_name": "Nucleus accumbens",
  "primary_injection_structure": 56,
  "data_set_id": 300078901,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838828,
   "name": "Gal-Cre_KI87"
  },
  "gender": "M",
  "injection_structures": "83/177/222/307/395/773/1048/1107",
  "injection_volume": 0.0185106448625,
  "injection_x": 12270,
  "injection_y": 6820,
  "injection_z": 5650,
  "product_id": 5,
  "specimen_name": "Gal-Cre_KI87-130251",
  "strain": "C57BL/6J",
  "structure_abbrev": "IO",
  "structure_id": 83,
  "structure_name": "Inferior olivary complex",
  "primary_injection_structure": 83,
  "data_set_id": 300209456,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "502/895/918/922/312782628",
  "injection_volume": 0.1678063408,
  "injection_x": 9400,
  "injection_y": 3440,
  "injection_z": 9930,
  "product_id": 5,
  "specimen_name": "378-1357",
  "strain": "C57BL/6J",
  "structure_abbrev": "ENTl",
  "structure_id": 918,
  "structure_name": "Entorhinal area, lateral part",
  "primary_injection_structure": 502,
  "data_set_id": 142654100,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838496,
   "name": "Grp-Cre_KH288"
  },
  "gender": "M",
  "injection_structures": "39/993",
  "injection_volume": 0.1675574796,
  "injection_x": 4220,
  "injection_y": 2080,
  "injection_z": 6280,
  "product_id": 5,
  "specimen_name": "Grp-Cre_KH288-119",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 39,
  "data_set_id": 266503540,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "394/533/312782546",
  "injection_volume": 0.0775600567,
  "injection_x": 7830,
  "injection_y": 480,
  "injection_z": 7360,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-126033",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISam",
  "structure_id": 394,
  "structure_name": "Anteromedial visual area",
  "primary_injection_structure": 394,
  "data_set_id": 294434867,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "39/993",
  "injection_volume": 0.0950428892,
  "injection_x": 4540,
  "injection_y": 1660,
  "injection_z": 6160,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-3819",
  "strain": "C57BL/6J",
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 171275617,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "F",
  "injection_structures": "385/394/417/533/312782546",
  "injection_volume": 0.0834376704,
  "injection_x": 7870,
  "injection_y": 1220,
  "injection_z": 3680,
  "product_id": 36,
  "specimen_name": "Ntsr1-Cre_GN220-233656",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 511817919,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.5278938168,
  "injection_x": 8740,
  "injection_y": 1590,
  "injection_z": 8140,
  "product_id": 5,
  "specimen_name": "C57BL/6-153762",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 307557934,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "155/218/463",
  "injection_volume": 0.107922108,
  "injection_x": 6700,
  "injection_y": 2870,
  "injection_z": 6840,
  "product_id": 5,
  "specimen_name": "378-1134",
  "strain": "C57BL/6J",
  "structure_abbrev": "LD",
  "structure_id": 155,
  "structure_name": "Lateral dorsal nucleus of thalamus",
  "primary_injection_structure": 155,
  "data_set_id": 113784293,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "F",
  "injection_structures": "337/985",
  "injection_volume": 0.0202537139,
  "injection_x": 5620,
  "injection_y": 1890,
  "injection_z": 7290,
  "product_id": 5,
  "specimen_name": "Ntsr1-Cre-3531",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOp",
  "structure_id": 985,
  "structure_name": "Primary motor area",
  "primary_injection_structure": 337,
  "data_set_id": 157909001,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "394/533/894",
  "injection_volume": 0.1006299868,
  "injection_x": 8330,
  "injection_y": 590,
  "injection_z": 4120,
  "product_id": 35,
  "specimen_name": "Cux2-IRES-Cre-292891",
  "strain": "unknown",
  "structure_abbrev": "VISpm",
  "structure_id": 533,
  "structure_name": "posteromedial visual area",
  "primary_injection_structure": 394,
  "data_set_id": 570460301,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 298725927,
   "name": "Penk-IRES2-Cre-neo"
  },
  "gender": "F",
  "injection_structures": "993",
  "injection_volume": 0.015558579225,
  "injection_x": 2940,
  "injection_y": 2570,
  "injection_z": 4380,
  "product_id": 45,
  "specimen_name": "Penk-IRES2-Cre-neo-321019",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 603331422,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 266649644,
   "name": "Drd3-Cre_KI196"
  },
  "gender": "M",
  "injection_structures": "39/48/993",
  "injection_volume": 0.008532434925,
  "injection_x": 4770,
  "injection_y": 1890,
  "injection_z": 6100,
  "product_id": 5,
  "specimen_name": "Drd3-Cre_KI196_LURC-127396",
  "strain": null,
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 516848198,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "918/922/312782628",
  "injection_volume": 0.2178542156,
  "injection_x": 9420,
  "injection_y": 3490,
  "injection_z": 10060,
  "product_id": 5,
  "specimen_name": "378-1819",
  "strain": "C57BL/6J",
  "structure_abbrev": "ENTl",
  "structure_id": 918,
  "structure_name": "Entorhinal area, lateral part",
  "primary_injection_structure": 918,
  "data_set_id": 180297139,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.02762890725,
  "injection_x": 9510,
  "injection_y": 1720,
  "injection_z": 2920,
  "product_id": 35,
  "specimen_name": "Tlx3-Cre_PL56-246075",
  "strain": "B6.FVB",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 522636747,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838502,
   "name": "Grik4-Cre"
  },
  "gender": "M",
  "injection_structures": "502/918/926/484682470",
  "injection_volume": 0.0872722634,
  "injection_x": 9140,
  "injection_y": 4450,
  "injection_z": 9100,
  "product_id": 5,
  "specimen_name": "Grik4-Cre-261553",
  "strain": null,
  "structure_abbrev": "SUB",
  "structure_id": 502,
  "structure_name": "Subiculum",
  "primary_injection_structure": 502,
  "data_set_id": 549809266,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837777,
   "name": "Sim1-Cre_KJ18"
  },
  "gender": "F",
  "injection_structures": "993",
  "injection_volume": 0.0515946088,
  "injection_x": 3950,
  "injection_y": 1970,
  "injection_z": 7120,
  "product_id": 5,
  "specimen_name": "Sim1-Cre_KJ18-3810",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 287807030,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 298725927,
   "name": "Penk-IRES2-Cre-neo"
  },
  "gender": "M",
  "injection_structures": "385/409",
  "injection_volume": 0.0458860206,
  "injection_x": 9250,
  "injection_y": 1430,
  "injection_z": 9070,
  "product_id": 5,
  "specimen_name": "Penk-IRES2-Cre-neo-298362",
  "strain": "unknown",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 575772121,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839285,
   "name": "Gpr26-Cre_KO250"
  },
  "gender": "F",
  "injection_structures": "382",
  "injection_volume": 0.0419554611,
  "injection_x": 7620,
  "injection_y": 2030,
  "injection_z": 8650,
  "product_id": 5,
  "specimen_name": "Gpr26-Cre_KO250-118",
  "strain": null,
  "structure_abbrev": "CA1",
  "structure_id": 382,
  "structure_name": "Field CA1",
  "primary_injection_structure": 382,
  "data_set_id": 286554017,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "394/533/879/894",
  "injection_volume": 0.1944418588,
  "injection_x": 8030,
  "injection_y": 980,
  "injection_z": 4130,
  "product_id": 36,
  "specimen_name": "Rbp4-Cre_KL100-296541",
  "strain": "unknown",
  "structure_abbrev": "RSPagl",
  "structure_id": 894,
  "structure_name": "Retrosplenial area, lateral agranular part",
  "primary_injection_structure": 394,
  "data_set_id": 576341623,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0584606162,
  "injection_x": 8390,
  "injection_y": 1040,
  "injection_z": 8120,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-3924",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 283020912,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "F",
  "injection_structures": "612/928",
  "injection_volume": 0.018580806125,
  "injection_x": 10060,
  "injection_y": 4170,
  "injection_z": 8160,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg3-Cre-4759",
  "strain": "B6.C3H",
  "structure_abbrev": "CUL",
  "structure_id": 928,
  "structure_name": "Culmen",
  "primary_injection_structure": 612,
  "data_set_id": 156255078,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838665,
   "name": "Vipr2-Cre_KE2"
  },
  "gender": "F",
  "injection_structures": "382",
  "injection_volume": 0.123578539,
  "injection_x": 7680,
  "injection_y": 1520,
  "injection_z": 7400,
  "product_id": 5,
  "specimen_name": "Vipr2-Cre-3283",
  "strain": null,
  "structure_abbrev": "CA1",
  "structure_id": 382,
  "structure_name": "Field CA1",
  "primary_injection_structure": 382,
  "data_set_id": 180520968,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "928",
  "injection_volume": 0.129003077875,
  "injection_x": 11210,
  "injection_y": 1650,
  "injection_z": 5690,
  "product_id": 5,
  "specimen_name": "378-1179",
  "strain": "C57BL/6J",
  "structure_abbrev": "CUL",
  "structure_id": 928,
  "structure_name": "Culmen",
  "primary_injection_structure": 928,
  "data_set_id": 114473794,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "385/879/894",
  "injection_volume": 0.2252064892,
  "injection_x": 9620,
  "injection_y": 1460,
  "injection_z": 3760,
  "product_id": 35,
  "specimen_name": "Emx1-IRES-Cre-255365",
  "strain": "B6.129",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 536298726,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "385/394/417/533/312782546",
  "injection_volume": 0.1335441786,
  "injection_x": 7820,
  "injection_y": 1150,
  "injection_z": 3540,
  "product_id": 36,
  "specimen_name": "Tlx3-Cre_PL56-319679",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 597256577,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838104,
   "name": "Pvalb-T2A-CreERT2"
  },
  "gender": "M",
  "injection_structures": "993",
  "injection_volume": 0.1455665442,
  "injection_x": 2970,
  "injection_y": 2150,
  "injection_z": 4410,
  "product_id": 45,
  "specimen_name": "Pvalb-T2A-CreERT2;Ai166-338827",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 636803027,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "394/894",
  "injection_volume": 0.03655322825,
  "injection_x": 7450,
  "injection_y": 970,
  "injection_z": 7020,
  "product_id": 5,
  "specimen_name": "378-927",
  "strain": "C57BL/6J",
  "structure_abbrev": "RSPagl",
  "structure_id": 894,
  "structure_name": "Retrosplenial area, lateral agranular part",
  "primary_injection_structure": 394,
  "data_set_id": 112229103,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838494,
   "name": "Prkcd-GluCla-CFP-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "536",
  "injection_volume": 0.02468381615,
  "injection_x": 7380,
  "injection_y": 5350,
  "injection_z": 8560,
  "product_id": 5,
  "specimen_name": "Prkcd-GluCla-CFP-IRES-Cre-113",
  "strain": "C57BL/6J",
  "structure_abbrev": "CEA",
  "structure_id": 536,
  "structure_name": "Central amygdalar nucleus",
  "primary_injection_structure": 536,
  "data_set_id": 265945645,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182693192,
   "name": "Chrna2-Cre_OE25"
  },
  "gender": "M",
  "injection_structures": "353/378",
  "injection_volume": 0.1426368832,
  "injection_x": 5780,
  "injection_y": 3780,
  "injection_z": 9850,
  "product_id": 5,
  "specimen_name": "Chrna2-Cre_OE25-124948",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "SSs",
  "structure_id": 378,
  "structure_name": "Supplemental somatosensory area",
  "primary_injection_structure": 353,
  "data_set_id": 297945448,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838633,
   "name": "Trib2-F2A-CreERT2"
  },
  "gender": "M",
  "injection_structures": "502/484682470",
  "injection_volume": 0.0760175514,
  "injection_x": 9150,
  "injection_y": 4490,
  "injection_z": 9230,
  "product_id": 5,
  "specimen_name": "Trib2-2A-CreERT2-D-262854-Snap25-LSL-2A-GFP(BT)",
  "strain": null,
  "structure_abbrev": "SUB",
  "structure_id": 502,
  "structure_name": "Subiculum",
  "primary_injection_structure": 502,
  "data_set_id": 550892495,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "879/886",
  "injection_volume": 0.122177531,
  "injection_x": 6830,
  "injection_y": 1260,
  "injection_z": 6160,
  "product_id": 5,
  "specimen_name": "378-892",
  "strain": "C57BL/6J",
  "structure_abbrev": "RSPv",
  "structure_id": 886,
  "structure_name": "Retrosplenial area, ventral part",
  "primary_injection_structure": 879,
  "data_set_id": 100148142,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838942,
   "name": "Htr2a-Cre_KM207"
  },
  "gender": "M",
  "injection_structures": "184/723/746/993",
  "injection_volume": 0.3857294112,
  "injection_x": 2630,
  "injection_y": 2030,
  "injection_z": 6780,
  "product_id": 5,
  "specimen_name": "Htr2a-Cre-3438",
  "strain": null,
  "structure_abbrev": "ORBvl",
  "structure_id": 746,
  "structure_name": "Orbital area, ventrolateral part",
  "primary_injection_structure": 184,
  "data_set_id": 183618845,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182846862,
   "name": "Chrnb4-Cre_OL57"
  },
  "gender": "M",
  "injection_structures": "993",
  "injection_volume": 0.2240168672,
  "injection_x": 3060,
  "injection_y": 2140,
  "injection_z": 7690,
  "product_id": 5,
  "specimen_name": "Chrnb4-Cre_OL57-125947",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 297987980,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839406,
   "name": "Nos1-CreERT2"
  },
  "gender": "F",
  "injection_structures": "39/993",
  "injection_volume": 0.0090034762125,
  "injection_x": 4950,
  "injection_y": 1420,
  "injection_z": 6070,
  "product_id": 5,
  "specimen_name": "Nos1-CreERT2-183",
  "strain": "B6.129",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 39,
  "data_set_id": 267930978,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "F",
  "injection_structures": "345/353/369/182305689",
  "injection_volume": 0.0626496801,
  "injection_x": 4860,
  "injection_y": 2260,
  "injection_z": 8650,
  "product_id": 5,
  "specimen_name": "Tlx3-Cre-135574",
  "strain": null,
  "structure_abbrev": "SSp-m",
  "structure_id": 345,
  "structure_name": "Primary somatosensory area, mouth",
  "primary_injection_structure": 345,
  "data_set_id": 303614706,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 310459452,
   "name": "Slc17a8-IRES2-Cre"
  },
  "gender": "M",
  "injection_structures": "100/238/604/679",
  "injection_volume": 0.1703311001,
  "injection_x": 9890,
  "injection_y": 5060,
  "injection_z": 5680,
  "product_id": 5,
  "specimen_name": "Slc17a8-IRES2-Cre-279945",
  "strain": null,
  "structure_abbrev": "CS",
  "structure_id": 679,
  "structure_name": "Superior central nucleus raphe",
  "primary_injection_structure": 100,
  "data_set_id": 562963492,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "369/985",
  "injection_volume": 0.3054036132,
  "injection_x": 5080,
  "injection_y": 2360,
  "injection_z": 7800,
  "product_id": 5,
  "specimen_name": "378-1354",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOp",
  "structure_id": 985,
  "structure_name": "Primary motor area",
  "primary_injection_structure": 369,
  "data_set_id": 126909424,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839331,
   "name": "Nr5a1-Cre"
  },
  "gender": "F",
  "injection_structures": "993",
  "injection_volume": 0.00285467966875,
  "injection_x": 3160,
  "injection_y": 1790,
  "injection_z": 6650,
  "product_id": 5,
  "specimen_name": "Nr5a1-Cre-2395",
  "strain": "B6.FVB",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 167791990,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837281,
   "name": "Calb2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "63/194/356/830/946/576073704",
  "injection_volume": 0.0469379232,
  "injection_x": 6900,
  "injection_y": 5950,
  "injection_z": 6790,
  "product_id": 5,
  "specimen_name": "Calb2-IRES-Cre-249",
  "strain": null,
  "structure_abbrev": "LHA",
  "structure_id": 194,
  "structure_name": "Lateral hypothalamic area",
  "primary_injection_structure": 63,
  "data_set_id": 267213087,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838899,
   "name": "Dbh-Cre_KH212"
  },
  "gender": "M",
  "injection_structures": "128/147/162/460/795/867/898",
  "injection_volume": 0.018416800675,
  "injection_x": 10260,
  "injection_y": 3910,
  "injection_z": 6430,
  "product_id": 5,
  "specimen_name": "Dbh-Cre-94",
  "strain": null,
  "structure_abbrev": "LDT",
  "structure_id": 162,
  "structure_name": "Laterodorsal tegmental nucleus",
  "primary_injection_structure": 128,
  "data_set_id": 159510205,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.3140588164,
  "injection_x": 8870,
  "injection_y": 1740,
  "injection_z": 8360,
  "product_id": 5,
  "specimen_name": "C57BL/6-154703",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 309003780,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "385/894",
  "injection_volume": 0.02146006205,
  "injection_x": 9890,
  "injection_y": 1410,
  "injection_z": 3250,
  "product_id": 36,
  "specimen_name": "Cux2-IRES-Cre-230161",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 506946046,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "F",
  "injection_structures": "385/879/894",
  "injection_volume": 0.015256594675,
  "injection_x": 9620,
  "injection_y": 980,
  "injection_z": 7200,
  "product_id": 5,
  "specimen_name": "Tlx3-Cre_PL56-336162",
  "strain": null,
  "structure_abbrev": "RSPagl",
  "structure_id": 894,
  "structure_name": "Retrosplenial area, lateral agranular part",
  "primary_injection_structure": 385,
  "data_set_id": 614094233,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "F",
  "injection_structures": "895/918/312782628",
  "injection_volume": 0.02444719025,
  "injection_x": 9270,
  "injection_y": 3130,
  "injection_z": 9730,
  "product_id": 5,
  "specimen_name": "Syt6-Cre-626",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISpor",
  "structure_id": 312782628,
  "structure_name": "Postrhinal area",
  "primary_injection_structure": 895,
  "data_set_id": 158141324,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "39/993",
  "injection_volume": 0.0132889372,
  "injection_x": 4430,
  "injection_y": 2190,
  "injection_z": 6640,
  "product_id": 5,
  "specimen_name": "Ntsr1-Cre-3569",
  "strain": "C57BL/6J",
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 159551564,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "385/894",
  "injection_volume": 0.2120955592,
  "injection_x": 10030,
  "injection_y": 1640,
  "injection_z": 8110,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-307",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 286302294,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.03871535815,
  "injection_x": 8370,
  "injection_y": 930,
  "injection_z": 8250,
  "product_id": 5,
  "specimen_name": "378-1853",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 277712166,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 309779993,
   "name": "Ctgf-T2A-dgCre"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0047398024625,
  "injection_x": 9420,
  "injection_y": 1860,
  "injection_z": 8240,
  "product_id": 5,
  "specimen_name": "Ctgf-T2A-dgCre-325054",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 605657236,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "F",
  "injection_structures": "385/1037/1084/484682508",
  "injection_volume": 0.0972645198,
  "injection_x": 9480,
  "injection_y": 2730,
  "injection_z": 8100,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg3-Cre-4763",
  "strain": "B6.C3H",
  "structure_abbrev": "POST",
  "structure_id": 1037,
  "structure_name": "Postsubiculum",
  "primary_injection_structure": 385,
  "data_set_id": 156314054,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.1685275228,
  "injection_x": 8870,
  "injection_y": 1560,
  "injection_z": 3320,
  "product_id": 35,
  "specimen_name": "Emx1-IRES-Cre-191140",
  "strain": "B6.129",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 479755622,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 553502711,
   "name": "Chat-IRES-Cre-neo"
  },
  "gender": "F",
  "injection_structures": "35/115/128/197/795/975/606826663",
  "injection_volume": 0.02450095795,
  "injection_x": 9090,
  "injection_y": 3900,
  "injection_z": 5770,
  "product_id": 5,
  "specimen_name": "Chat-IRES-Cre-3151",
  "strain": "B6.129",
  "structure_abbrev": "III",
  "structure_id": 35,
  "structure_name": "Oculomotor nucleus",
  "primary_injection_structure": 35,
  "data_set_id": 177460028,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.013981313325,
  "injection_x": 9040,
  "injection_y": 1080,
  "injection_z": 3330,
  "product_id": 36,
  "specimen_name": "Scnn1a-Tg3-Cre-227543",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 504099316,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "385/879/894",
  "injection_volume": 0.0425947102,
  "injection_x": 9460,
  "injection_y": 1410,
  "injection_z": 3820,
  "product_id": 36,
  "specimen_name": "Ntsr1-Cre_GN220-238034",
  "strain": null,
  "structure_abbrev": "RSPd",
  "structure_id": 879,
  "structure_name": "Retrosplenial area, dorsal part",
  "primary_injection_structure": 385,
  "data_set_id": 516273371,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839331,
   "name": "Nr5a1-Cre"
  },
  "gender": "F",
  "injection_structures": "329/385/417",
  "injection_volume": 0.0526850793,
  "injection_x": 7640,
  "injection_y": 1170,
  "injection_z": 8440,
  "product_id": 5,
  "specimen_name": "Nr5a1-Cre-2579",
  "strain": "B6.FVB",
  "structure_abbrev": "VISrl",
  "structure_id": 417,
  "structure_name": "Rostrolateral visual area",
  "primary_injection_structure": 329,
  "data_set_id": 293548317,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 301613169,
   "name": "Sepw1-Cre_NP39"
  },
  "gender": "M",
  "injection_structures": "39/993",
  "injection_volume": 0.0513577575,
  "injection_x": 4720,
  "injection_y": 1190,
  "injection_z": 6060,
  "product_id": 5,
  "specimen_name": "Sepw1-Cre_NP39-174465",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 39,
  "data_set_id": 477835774,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837446,
   "name": "Adcyap1-2A-Cre"
  },
  "gender": "M",
  "injection_structures": "395/651/711/1039/1098",
  "injection_volume": 0.04882158273125,
  "injection_x": 12990,
  "injection_y": 5180,
  "injection_z": 5980,
  "product_id": 5,
  "specimen_name": "Adcyap1-2A-Cre-D-131310",
  "strain": "B6.129",
  "structure_abbrev": "GR",
  "structure_id": 1039,
  "structure_name": "Gracile nucleus",
  "primary_injection_structure": 395,
  "data_set_id": 558579356,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "329",
  "injection_volume": 0.0855762362,
  "injection_x": 7250,
  "injection_y": 1400,
  "injection_z": 8400,
  "product_id": 5,
  "specimen_name": "Tlx3-Cre_LURC-128441",
  "strain": null,
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 297654263,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "329/417",
  "injection_volume": 0.178945256,
  "injection_x": 7530,
  "injection_y": 1210,
  "injection_z": 2500,
  "product_id": 35,
  "specimen_name": "Emx1-IRES-Cre-240165",
  "strain": null,
  "structure_abbrev": "VISrl",
  "structure_id": 417,
  "structure_name": "Rostrolateral visual area",
  "primary_injection_structure": 329,
  "data_set_id": 518606617,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "111/378/583/677/952",
  "injection_volume": 0.2447804016,
  "injection_x": 6160,
  "injection_y": 5310,
  "injection_z": 9400,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre-197",
  "strain": "C57BL/6J",
  "structure_abbrev": "AIp",
  "structure_id": 111,
  "structure_name": "Agranular insular area, posterior part",
  "primary_injection_structure": 111,
  "data_set_id": 183330908,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "502/484682470",
  "injection_volume": 0.02879613135,
  "injection_x": 9180,
  "injection_y": 3530,
  "injection_z": 9440,
  "product_id": 5,
  "specimen_name": "C57BL6J_OLD-343578",
  "strain": "C57BL/6J",
  "structure_abbrev": "SUB",
  "structure_id": 502,
  "structure_name": "Subiculum",
  "primary_injection_structure": 502,
  "data_set_id": 640285199,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182693192,
   "name": "Chrna2-Cre_OE25"
  },
  "gender": "F",
  "injection_structures": "879/886",
  "injection_volume": 0.1440313644,
  "injection_x": 8340,
  "injection_y": 1260,
  "injection_z": 6830,
  "product_id": 5,
  "specimen_name": "Chrna2-Cre_OE25-130471",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "RSPv",
  "structure_id": 886,
  "structure_name": "Retrosplenial area, ventral part",
  "primary_injection_structure": 879,
  "data_set_id": 298275548,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837328,
   "name": "Syt17-Cre_NO14"
  },
  "gender": "M",
  "injection_structures": "382/484682470",
  "injection_volume": 0.1847073424,
  "injection_x": 9080,
  "injection_y": 3800,
  "injection_z": 9490,
  "product_id": 5,
  "specimen_name": "Syt17-Cre_NO14-203",
  "strain": "C57BL/6J",
  "structure_abbrev": "CA1",
  "structure_id": 382,
  "structure_name": "Field CA1",
  "primary_injection_structure": 382,
  "data_set_id": 286610923,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "385/425",
  "injection_volume": 0.002035490471875,
  "injection_x": 9860,
  "injection_y": 2110,
  "injection_z": 2580,
  "product_id": 35,
  "specimen_name": "A930038C07Rik-Tg1-Cre-345969",
  "strain": null,
  "structure_abbrev": "VISpl",
  "structure_id": 425,
  "structure_name": "Posterolateral visual area",
  "primary_injection_structure": 385,
  "data_set_id": 643166866,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838715,
   "name": "Tac1-IRES2-Cre"
  },
  "gender": "F",
  "injection_structures": "378/1011",
  "injection_volume": 0.2314943456,
  "injection_x": 7290,
  "injection_y": 2380,
  "injection_z": 9700,
  "product_id": 5,
  "specimen_name": "Tac1-IRES2-Cre-D-49",
  "strain": "B6.129",
  "structure_abbrev": "SSs",
  "structure_id": 378,
  "structure_name": "Supplemental somatosensory area",
  "primary_injection_structure": 378,
  "data_set_id": 181372049,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839285,
   "name": "Gpr26-Cre_KO250"
  },
  "gender": "F",
  "injection_structures": "385/894/1037",
  "injection_volume": 0.124728618,
  "injection_x": 9390,
  "injection_y": 1840,
  "injection_z": 8190,
  "product_id": 5,
  "specimen_name": "Gpr26-Cre_KO250-123679",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 298404860,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "F",
  "injection_structures": "879/886/894",
  "injection_volume": 0.03361001385,
  "injection_x": 9260,
  "injection_y": 1100,
  "injection_z": 4170,
  "product_id": 36,
  "specimen_name": "Ntsr1-Cre_GN220-280243",
  "strain": null,
  "structure_abbrev": "RSPd",
  "structure_id": 879,
  "structure_name": "Retrosplenial area, dorsal part",
  "primary_injection_structure": 879,
  "data_set_id": 560800029,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838361,
   "name": "Etv1-CreERT2"
  },
  "gender": "M",
  "injection_structures": "985/993",
  "injection_volume": 0.1667484112,
  "injection_x": 3790,
  "injection_y": 1720,
  "injection_z": 7190,
  "product_id": 5,
  "specimen_name": "Etv1-CreERT2-2882",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 167569313,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "F",
  "injection_structures": "44/972",
  "injection_volume": 0.0729349712,
  "injection_x": 3360,
  "injection_y": 3200,
  "injection_z": 6180,
  "product_id": 5,
  "specimen_name": "Ntsr1-Cre-3595",
  "strain": "C57BL/6J",
  "structure_abbrev": "PL",
  "structure_id": 972,
  "structure_name": "Prelimbic area",
  "primary_injection_structure": 44,
  "data_set_id": 161463521,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838496,
   "name": "Grp-Cre_KH288"
  },
  "gender": "M",
  "injection_structures": "382/423/463",
  "injection_volume": 0.00216671048125,
  "injection_x": 6710,
  "injection_y": 2420,
  "injection_z": 7760,
  "product_id": 5,
  "specimen_name": "Grp-Cre-49",
  "strain": "B6.FVB(ICR)",
  "structure_abbrev": "CA3",
  "structure_id": 463,
  "structure_name": "Field CA3",
  "primary_injection_structure": 382,
  "data_set_id": 179902073,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "394/533/879/886/894",
  "injection_volume": 0.5184697448,
  "injection_x": 8180,
  "injection_y": 570,
  "injection_z": 4120,
  "product_id": 36,
  "specimen_name": "Emx1-IRES-Cre-294569",
  "strain": null,
  "structure_abbrev": "VISam",
  "structure_id": 394,
  "structure_name": "Anteromedial visual area",
  "primary_injection_structure": 394,
  "data_set_id": 571100135,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837625,
   "name": "Sst-Cre"
  },
  "gender": "M",
  "injection_structures": "250/258",
  "injection_volume": 0.062105834,
  "injection_x": 4920,
  "injection_y": 3400,
  "injection_z": 5960,
  "product_id": 5,
  "specimen_name": "Sst-Cre-D-806",
  "strain": "C57BL/6J",
  "structure_abbrev": "LSc",
  "structure_id": 250,
  "structure_name": "Lateral septal nucleus, caudal (caudodorsal) part",
  "primary_injection_structure": 250,
  "data_set_id": 160294327,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "402/409",
  "injection_volume": 0.012784659825,
  "injection_x": 8560,
  "injection_y": 1610,
  "injection_z": 9070,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-3922",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 402,
  "data_set_id": 283017324,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 298341195,
   "name": "Htr3a-Cre_NO152"
  },
  "gender": "M",
  "injection_structures": "35/128/795/975",
  "injection_volume": 0.016895674075,
  "injection_x": 8960,
  "injection_y": 3880,
  "injection_z": 5880,
  "product_id": 5,
  "specimen_name": "Htr3a-Cre_NO152-158481",
  "strain": null,
  "structure_abbrev": "III",
  "structure_id": 35,
  "structure_name": "Oculomotor nucleus",
  "primary_injection_structure": 35,
  "data_set_id": 311845265,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "F",
  "injection_structures": "879/886",
  "injection_volume": 0.090878144,
  "injection_x": 6800,
  "injection_y": 1030,
  "injection_z": 6050,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg3-Cre-4782",
  "strain": "B6.C3H",
  "structure_abbrev": "RSPv",
  "structure_id": 886,
  "structure_name": "Retrosplenial area, ventral part",
  "primary_injection_structure": 879,
  "data_set_id": 166269090,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838584,
   "name": "Gad2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.2103427508,
  "injection_x": 9650,
  "injection_y": 1120,
  "injection_z": 8490,
  "product_id": 5,
  "specimen_name": "Gad2-IRES-Cre-1145",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 167511639,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "329/353/182305689",
  "injection_volume": 0.250750542,
  "injection_x": 5900,
  "injection_y": 2130,
  "injection_z": 8670,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-94",
  "strain": null,
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 168003640,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837777,
   "name": "Sim1-Cre_KJ18"
  },
  "gender": "M",
  "injection_structures": "38/63/88/332/347/351/515/523/576073704",
  "injection_volume": 0.2943930976,
  "injection_x": 6180,
  "injection_y": 5810,
  "injection_z": 6370,
  "product_id": 5,
  "specimen_name": "Sim1-Cre-Homo-256",
  "strain": "B6.FVB",
  "structure_abbrev": "AHN",
  "structure_id": 88,
  "structure_name": "Anterior hypothalamic nucleus",
  "primary_injection_structure": 38,
  "data_set_id": 159375743,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837777,
   "name": "Sim1-Cre_KJ18"
  },
  "gender": "F",
  "injection_structures": "1/194/210/364/946/980",
  "injection_volume": 0.2174696832,
  "injection_x": 7440,
  "injection_y": 5870,
  "injection_z": 6340,
  "product_id": 5,
  "specimen_name": "Sim1-Cre-Homo-222",
  "strain": "B6.FVB",
  "structure_abbrev": "LHA",
  "structure_id": 194,
  "structure_name": "Lateral hypothalamic area",
  "primary_injection_structure": 1,
  "data_set_id": 156195758,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839425,
   "name": "Pdzk1ip1-Cre_KD31"
  },
  "gender": "F",
  "injection_structures": "302",
  "injection_volume": 0.03835169485,
  "injection_x": 8870,
  "injection_y": 1640,
  "injection_z": 6290,
  "product_id": 5,
  "specimen_name": "Pdzk1ip1-Cre-173",
  "strain": "C57BL/6J",
  "structure_abbrev": "SCs",
  "structure_id": 302,
  "structure_name": "Superior colliculus, sensory related",
  "primary_injection_structure": 302,
  "data_set_id": 176900059,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838730,
   "name": "Tac2-IRES2-Cre"
  },
  "gender": "M",
  "injection_structures": "149/186/483",
  "injection_volume": 0.00840878908105469,
  "injection_x": 6710,
  "injection_y": 3030,
  "injection_z": 5970,
  "product_id": 5,
  "specimen_name": "Tac2-IRES2-Cre-D-101",
  "strain": "B6.129",
  "structure_abbrev": "MH",
  "structure_id": 483,
  "structure_name": "Medial habenula",
  "primary_injection_structure": 149,
  "data_set_id": 265287564,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "918/926",
  "injection_volume": 0.5450253536,
  "injection_x": 9510,
  "injection_y": 4410,
  "injection_z": 9390,
  "product_id": 5,
  "specimen_name": "378-1101",
  "strain": "C57BL/6J",
  "structure_abbrev": "ENTm",
  "structure_id": 926,
  "structure_name": "Entorhinal area, medial part, dorsal zone",
  "primary_injection_structure": 918,
  "data_set_id": 113226232,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839481,
   "name": "Pnmt-Cre"
  },
  "gender": "M",
  "injection_structures": "96/101",
  "injection_volume": 0.034884521,
  "injection_x": 11270,
  "injection_y": 4620,
  "injection_z": 8010,
  "product_id": 5,
  "specimen_name": "Pnmt-Cre-157",
  "strain": "B6.129",
  "structure_abbrev": "DCO",
  "structure_id": 96,
  "structure_name": "Dorsal cochlear nucleus",
  "primary_injection_structure": 96,
  "data_set_id": 177903648,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839406,
   "name": "Nos1-CreERT2"
  },
  "gender": "M",
  "injection_structures": "128/215/294/795",
  "injection_volume": 0.0645537025,
  "injection_x": 8080,
  "injection_y": 3740,
  "injection_z": 6280,
  "product_id": 5,
  "specimen_name": "Nos1-CreERT2-285466",
  "strain": null,
  "structure_abbrev": "MRN",
  "structure_id": 128,
  "structure_name": "Midbrain reticular nucleus",
  "primary_injection_structure": 128,
  "data_set_id": 581328778,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838781,
   "name": "Slc6a4-CreERT2_EZ13"
  },
  "gender": "F",
  "injection_structures": "115/591/795/872",
  "injection_volume": 0.1324498322,
  "injection_x": 9630,
  "injection_y": 3820,
  "injection_z": 5510,
  "product_id": 5,
  "specimen_name": "Slc6a4-CreERT2-Homo-271",
  "strain": "C57BL/6J",
  "structure_abbrev": "DR",
  "structure_id": 872,
  "structure_name": "Dorsal nucleus raphe",
  "primary_injection_structure": 115,
  "data_set_id": 114155190,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "385/533/886",
  "injection_volume": 0.0194409362,
  "injection_x": 8560,
  "injection_y": 890,
  "injection_z": 7500,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-121786",
  "strain": null,
  "structure_abbrev": "VISpm",
  "structure_id": 533,
  "structure_name": "posteromedial visual area",
  "primary_injection_structure": 385,
  "data_set_id": 298324391,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "F",
  "injection_structures": "312782574/312782628",
  "injection_volume": 0.0188072878,
  "injection_x": 9190,
  "injection_y": 2900,
  "injection_z": 1840,
  "product_id": 35,
  "specimen_name": "Ntsr1-Cre_GN220-353476",
  "strain": null,
  "structure_abbrev": "VISpor",
  "structure_id": 312782628,
  "structure_name": "Postrhinal area",
  "primary_injection_structure": 312782574,
  "data_set_id": 649183036,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "1002/1011",
  "injection_volume": 0.0846237252,
  "injection_x": 7660,
  "injection_y": 2840,
  "injection_z": 9790,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre-160",
  "strain": null,
  "structure_abbrev": "AUDp",
  "structure_id": 1002,
  "structure_name": "Primary auditory area",
  "primary_injection_structure": 1002,
  "data_set_id": 182090318,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "136/235/307/395/429/661/978/1048/1069/1098",
  "injection_volume": 0.4345008752,
  "injection_x": 11330,
  "injection_y": 6830,
  "injection_z": 6790,
  "product_id": 5,
  "specimen_name": "378-1365",
  "strain": "C57BL/6J",
  "structure_abbrev": "PGRNl",
  "structure_id": 978,
  "structure_name": "Paragigantocellular reticular nucleus, lateral part",
  "primary_injection_structure": 136,
  "data_set_id": 127042540,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "337/361/369",
  "injection_volume": 0.4762145752,
  "injection_x": 6380,
  "injection_y": 1320,
  "injection_z": 7570,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-131050",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-ll",
  "structure_id": 337,
  "structure_name": "Primary somatosensory area, lower limb",
  "primary_injection_structure": 337,
  "data_set_id": 298830161,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "184/723/993",
  "injection_volume": 0.0936015934,
  "injection_x": 2170,
  "injection_y": 2840,
  "injection_z": 7460,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-302",
  "strain": "C57BL/6J",
  "structure_abbrev": "FRP",
  "structure_id": 184,
  "structure_name": "Frontal pole, cerebral cortex",
  "primary_injection_structure": 184,
  "data_set_id": 286299886,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "385/879/894",
  "injection_volume": 0.0764074052,
  "injection_x": 9530,
  "injection_y": 1210,
  "injection_z": 3740,
  "product_id": 36,
  "specimen_name": "Rbp4-Cre_KL100-233301",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 510417993,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "4",
  "injection_volume": 0.2893827692,
  "injection_x": 9810,
  "injection_y": 2500,
  "injection_z": 7450,
  "product_id": 5,
  "specimen_name": "378-729",
  "strain": "C57BL/6J",
  "structure_abbrev": "IC",
  "structure_id": 4,
  "structure_name": "Inferior colliculus",
  "primary_injection_structure": 4,
  "data_set_id": 100141596,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838710,
   "name": "Sst-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "342/536/1031",
  "injection_volume": 0.02690498125,
  "injection_x": 6430,
  "injection_y": 5640,
  "injection_z": 3370,
  "product_id": 45,
  "specimen_name": "Sst-IRES-Cre-257932",
  "strain": null,
  "structure_abbrev": "CEA",
  "structure_id": 536,
  "structure_name": "Central amygdalar nucleus",
  "primary_injection_structure": 342,
  "data_set_id": 539641136,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "39/972/993",
  "injection_volume": 0.17871182,
  "injection_x": 3160,
  "injection_y": 2460,
  "injection_z": 6510,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-3802",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 39,
  "data_set_id": 166460484,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "409",
  "injection_volume": 0.0977995606,
  "injection_x": 9050,
  "injection_y": 2170,
  "injection_z": 2410,
  "product_id": 35,
  "specimen_name": "Emx1-IRES-Cre-191134",
  "strain": "B6.129",
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 409,
  "data_set_id": 479700629,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.0552280519,
  "injection_x": 9270,
  "injection_y": 1060,
  "injection_z": 3430,
  "product_id": 36,
  "specimen_name": "Cux2-IRES-Cre-196737",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 483014695,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838022,
   "name": "Pmch-Cre"
  },
  "gender": "F",
  "injection_structures": "63/88/194/356/364/830/946/576073704",
  "injection_volume": 0.148942213,
  "injection_x": 7080,
  "injection_y": 6140,
  "injection_z": 6900,
  "product_id": 5,
  "specimen_name": "Pmch-Cre-475",
  "strain": "B6.129.FVB",
  "structure_abbrev": "LHA",
  "structure_id": 194,
  "structure_name": "Lateral hypothalamic area",
  "primary_injection_structure": 63,
  "data_set_id": 113444277,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182693192,
   "name": "Chrna2-Cre_OE25"
  },
  "gender": "F",
  "injection_structures": "184/723/746/993",
  "injection_volume": 0.0402646181,
  "injection_x": 2750,
  "injection_y": 3270,
  "injection_z": 6940,
  "product_id": 5,
  "specimen_name": "Chrna2-Cre_OE25-3881",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "ORBl",
  "structure_id": 723,
  "structure_name": "Orbital area, lateral part",
  "primary_injection_structure": 184,
  "data_set_id": 278174451,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "56/672/961/998",
  "injection_volume": 0.1866928616,
  "injection_x": 5260,
  "injection_y": 5950,
  "injection_z": 8110,
  "product_id": 5,
  "specimen_name": "378-1120",
  "strain": "C57BL/6J",
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 56,
  "data_set_id": 113505468,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 179697864,
   "name": "Rorb-IRES2-Cre"
  },
  "gender": "M",
  "injection_structures": "294/302",
  "injection_volume": 0.02469503225,
  "injection_x": 8620,
  "injection_y": 1580,
  "injection_z": 6250,
  "product_id": 5,
  "specimen_name": "Rorb-IRES2-Cre-D-139383",
  "strain": null,
  "structure_abbrev": "SCs",
  "structure_id": 302,
  "structure_name": "Superior colliculus, sensory related",
  "primary_injection_structure": 294,
  "data_set_id": 304338014,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "312782628",
  "injection_volume": 0.0296091957,
  "injection_x": 9780,
  "injection_y": 2730,
  "injection_z": 1600,
  "product_id": 35,
  "specimen_name": "Emx1-IRES-Cre-313075",
  "strain": null,
  "structure_abbrev": "VISpor",
  "structure_id": 312782628,
  "structure_name": "Postrhinal area",
  "primary_injection_structure": 312782628,
  "data_set_id": 590988082,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838048,
   "name": "Slc17a6-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "170",
  "injection_volume": 0.0110926249,
  "injection_x": 7610,
  "injection_y": 3330,
  "injection_z": 7900,
  "product_id": 5,
  "specimen_name": "Slc17a6-IRES-Cre-121012",
  "strain": "B6.129",
  "structure_abbrev": "LGd",
  "structure_id": 170,
  "structure_name": "Dorsal part of the lateral geniculate complex",
  "primary_injection_structure": 170,
  "data_set_id": 293914766,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182693192,
   "name": "Chrna2-Cre_OE25"
  },
  "gender": "M",
  "injection_structures": "44/731/972",
  "injection_volume": 0.2095124164,
  "injection_x": 3420,
  "injection_y": 3610,
  "injection_z": 6330,
  "product_id": 5,
  "specimen_name": "Chrna2-Cre_OE25-130473",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "ILA",
  "structure_id": 44,
  "structure_name": "Infralimbic area",
  "primary_injection_structure": 44,
  "data_set_id": 298274021,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.1323604464,
  "injection_x": 8990,
  "injection_y": 1500,
  "injection_z": 3510,
  "product_id": 35,
  "specimen_name": "Emx1-IRES-Cre-191139",
  "strain": "B6.129",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 479701339,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839513,
   "name": "Drd2-Cre_ER44"
  },
  "gender": "F",
  "injection_structures": "56",
  "injection_volume": 0.1697668896,
  "injection_x": 4130,
  "injection_y": 5880,
  "injection_z": 6040,
  "product_id": 5,
  "specimen_name": "Drd2-Cre-2059",
  "strain": "C57BL/6J",
  "structure_abbrev": "ACB",
  "structure_id": 56,
  "structure_name": "Nucleus accumbens",
  "primary_injection_structure": 56,
  "data_set_id": 175732001,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "F",
  "injection_structures": "385/409",
  "injection_volume": 0.0512315433,
  "injection_x": 9090,
  "injection_y": 1820,
  "injection_z": 2140,
  "product_id": 36,
  "specimen_name": "Tlx3-Cre_PL56-226272",
  "strain": null,
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 385,
  "data_set_id": 503018656,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.02384042325,
  "injection_x": 9610,
  "injection_y": 1280,
  "injection_z": 3310,
  "product_id": 36,
  "specimen_name": "Tlx3-Cre_PL56-232872",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 510582887,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "M",
  "injection_structures": "993",
  "injection_volume": 0.03092558315,
  "injection_x": 3730,
  "injection_y": 2710,
  "injection_z": 7090,
  "product_id": 5,
  "specimen_name": "Syt6-Cre_KI148-303004",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 583749274,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837858,
   "name": "Pvalb-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "797",
  "injection_volume": 0.1364341496,
  "injection_x": 7540,
  "injection_y": 4970,
  "injection_z": 7760,
  "product_id": 5,
  "specimen_name": "Pvalb-IRES-Cre-121709",
  "strain": "B6.129",
  "structure_abbrev": "ZI",
  "structure_id": 797,
  "structure_name": "Zona incerta",
  "primary_injection_structure": 797,
  "data_set_id": 301539438,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "385/879/894",
  "injection_volume": 0.2987955908,
  "injection_x": 9470,
  "injection_y": 1440,
  "injection_z": 3890,
  "product_id": 36,
  "specimen_name": "Emx1-IRES-Cre-191137",
  "strain": "B6.129",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 479984127,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182693192,
   "name": "Chrna2-Cre_OE25"
  },
  "gender": "M",
  "injection_structures": "4",
  "injection_volume": 0.1855025732,
  "injection_x": 10010,
  "injection_y": 2260,
  "injection_z": 7290,
  "product_id": 5,
  "specimen_name": "Chrna2-Cre_OE25-138099",
  "strain": "B6.FVB(ICR)",
  "structure_abbrev": "IC",
  "structure_id": 4,
  "structure_name": "Inferior colliculus",
  "primary_injection_structure": 4,
  "data_set_id": 304857992,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837281,
   "name": "Calb2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "128/422/1029/1044/560581563/563807435",
  "injection_volume": 0.0416169593,
  "injection_x": 8330,
  "injection_y": 4210,
  "injection_z": 7650,
  "product_id": 5,
  "specimen_name": "Calb2-IRES-Cre-126519",
  "strain": "C57BL/6J",
  "structure_abbrev": "PIL",
  "structure_id": 560581563,
  "structure_name": "Posterior intralaminar thalamic nucleus",
  "primary_injection_structure": 128,
  "data_set_id": 301063301,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "402/409/312782574",
  "injection_volume": 0.2127034336,
  "injection_x": 8700,
  "injection_y": 2170,
  "injection_z": 2220,
  "product_id": 35,
  "specimen_name": "Emx1-IRES-Cre-240163",
  "strain": null,
  "structure_abbrev": "VISal",
  "structure_id": 402,
  "structure_name": "Anterolateral visual area",
  "primary_injection_structure": 402,
  "data_set_id": 518605900,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "385/879/894",
  "injection_volume": 0.017242709225,
  "injection_x": 9670,
  "injection_y": 1420,
  "injection_z": 3180,
  "product_id": 35,
  "specimen_name": "A930038C07Rik-Tg1-Cre-255582",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 536298009,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838494,
   "name": "Prkcd-GluCla-CFP-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "96/101/1007",
  "injection_volume": 0.0832956782,
  "injection_x": 11410,
  "injection_y": 5230,
  "injection_z": 8160,
  "product_id": 5,
  "specimen_name": "Prkcd-GluCla-CFP-IRES-Cre-142",
  "strain": "C57BL/6J",
  "structure_abbrev": "DCO",
  "structure_id": 96,
  "structure_name": "Dorsal cochlear nucleus",
  "primary_injection_structure": 96,
  "data_set_id": 292319155,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838633,
   "name": "Trib2-F2A-CreERT2"
  },
  "gender": "F",
  "injection_structures": "385/425/894",
  "injection_volume": 0.1890134624,
  "injection_x": 10200,
  "injection_y": 1970,
  "injection_z": 8350,
  "product_id": 5,
  "specimen_name": "Trib2-2A-CreERT2-D-3274",
  "strain": "B6.129",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 268320515,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "F",
  "injection_structures": "385/879/894",
  "injection_volume": 0.0535932208,
  "injection_x": 9680,
  "injection_y": 1220,
  "injection_z": 4210,
  "product_id": 36,
  "specimen_name": "Tlx3-Cre_PL56-232873",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 510835921,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182901722,
   "name": "Ppp1r17-Cre_NL146"
  },
  "gender": "M",
  "injection_structures": "186/362",
  "injection_volume": 0.0435837213,
  "injection_x": 6610,
  "injection_y": 3520,
  "injection_z": 6070,
  "product_id": 5,
  "specimen_name": "Ppp1r17-Cre_NL146-3921",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "MD",
  "structure_id": 362,
  "structure_name": "Mediodorsal nucleus of thalamus",
  "primary_injection_structure": 186,
  "data_set_id": 278400363,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 266649644,
   "name": "Drd3-Cre_KI196"
  },
  "gender": "F",
  "injection_structures": "886",
  "injection_volume": 0.1878788576,
  "injection_x": 8240,
  "injection_y": 1600,
  "injection_z": 6150,
  "product_id": 5,
  "specimen_name": "Drd3-Cre_KI196_LURC-130782",
  "strain": null,
  "structure_abbrev": "RSPv",
  "structure_id": 886,
  "structure_name": "Retrosplenial area, ventral part",
  "primary_injection_structure": 886,
  "data_set_id": 521264566,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 180643392,
   "name": "Rasgrf2-T2A-dCre"
  },
  "gender": "F",
  "injection_structures": "985/993",
  "injection_volume": 0.101913238,
  "injection_x": 4000,
  "injection_y": 1790,
  "injection_z": 7310,
  "product_id": 5,
  "specimen_name": "Rasgrf2-2A-dCre-D-130399",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 298718072,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "329/378/1011",
  "injection_volume": 0.8573985504,
  "injection_x": 7440,
  "injection_y": 2680,
  "injection_z": 1940,
  "product_id": 35,
  "specimen_name": "Emx1-IRES-Cre-305953",
  "strain": null,
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 583748537,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 305451902,
   "name": "Dlg3-Cre_KG118"
  },
  "gender": "M",
  "injection_structures": "131/295/319",
  "injection_volume": 0.02070995535,
  "injection_x": 7320,
  "injection_y": 5980,
  "injection_z": 8910,
  "product_id": 5,
  "specimen_name": "Dlg3-Cre_KG118-190512",
  "strain": null,
  "structure_abbrev": "BLA",
  "structure_id": 295,
  "structure_name": "Basolateral amygdalar nucleus",
  "primary_injection_structure": 131,
  "data_set_id": 485105742,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "533/894",
  "injection_volume": 0.061059292,
  "injection_x": 8310,
  "injection_y": 580,
  "injection_z": 7070,
  "product_id": 5,
  "specimen_name": "378-1408",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISpm",
  "structure_id": 533,
  "structure_name": "posteromedial visual area",
  "primary_injection_structure": 533,
  "data_set_id": 146077302,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 180924870,
   "name": "Fezf1-T2A-dCre"
  },
  "gender": "M",
  "injection_structures": "88/614/693/830/576073704",
  "injection_volume": 0.4241856304,
  "injection_x": 6610,
  "injection_y": 6360,
  "injection_z": 6760,
  "product_id": 5,
  "specimen_name": "Fezf1-2A-DD-Cre-D-704",
  "strain": "B6.129",
  "structure_abbrev": "VMH",
  "structure_id": 693,
  "structure_name": "Ventromedial hypothalamic nucleus",
  "primary_injection_structure": 88,
  "data_set_id": 292211026,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "395/429/1098",
  "injection_volume": 0.0198809856,
  "injection_x": 12900,
  "injection_y": 5140,
  "injection_z": 7140,
  "product_id": 5,
  "specimen_name": "378-1500",
  "strain": "C57BL/6J",
  "structure_abbrev": "SPVC",
  "structure_id": 429,
  "structure_name": "Spinal nucleus of the trigeminal, caudal part",
  "primary_injection_structure": 395,
  "data_set_id": 146856593,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838953,
   "name": "Oxt-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "194/390/576073704",
  "injection_volume": 0.001471759865625,
  "injection_x": 6190,
  "injection_y": 6130,
  "injection_z": 6860,
  "product_id": 5,
  "specimen_name": "Oxt-IRES-Cre-3326",
  "strain": "C57BL/6J",
  "structure_abbrev": "LHA",
  "structure_id": 194,
  "structure_name": "Lateral hypothalamic area",
  "primary_injection_structure": 194,
  "data_set_id": 265125894,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "262/1022",
  "injection_volume": 0.0903202104,
  "injection_x": 5700,
  "injection_y": 4890,
  "injection_z": 7220,
  "product_id": 5,
  "specimen_name": "378-1634",
  "strain": "C57BL/6J",
  "structure_abbrev": "GPe",
  "structure_id": 1022,
  "structure_name": "Globus pallidus, external segment",
  "primary_injection_structure": 262,
  "data_set_id": 158373958,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 298725927,
   "name": "Penk-IRES2-Cre-neo"
  },
  "gender": "F",
  "injection_structures": "385/409/425/894",
  "injection_volume": 0.0677600616,
  "injection_x": 9890,
  "injection_y": 2020,
  "injection_z": 8660,
  "product_id": 5,
  "specimen_name": "Penk-IRES2-Cre-neo-298368",
  "strain": "unknown",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 585910380,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838803,
   "name": "Grm2-Cre_MR90"
  },
  "gender": "M",
  "injection_structures": "184/993",
  "injection_volume": 0.0798218232,
  "injection_x": 2410,
  "injection_y": 2520,
  "injection_z": 7440,
  "product_id": 5,
  "specimen_name": "Grm2-Cre_MR90-121734",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 184,
  "data_set_id": 300235349,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 301614880,
   "name": "Glt25d2-Cre_NF107"
  },
  "gender": "M",
  "injection_structures": "44/731/972",
  "injection_volume": 0.4241469344875,
  "injection_x": 2950,
  "injection_y": 3110,
  "injection_z": 5690,
  "product_id": 5,
  "specimen_name": "Glt25d2-Cre_NF107-144892",
  "strain": null,
  "structure_abbrev": "PL",
  "structure_id": 972,
  "structure_name": "Prelimbic area",
  "primary_injection_structure": 44,
  "data_set_id": 305618479,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837281,
   "name": "Calb2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "325/475",
  "injection_volume": 0.016596864725,
  "injection_x": 8890,
  "injection_y": 3590,
  "injection_z": 8080,
  "product_id": 5,
  "specimen_name": "Calb2-IRES-Cre-122689",
  "strain": "C57BL/6J",
  "structure_abbrev": "MG",
  "structure_id": 475,
  "structure_name": "Medial geniculate complex",
  "primary_injection_structure": 325,
  "data_set_id": 299996344,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "394/879/894/312782546",
  "injection_volume": 0.2752741208,
  "injection_x": 7870,
  "injection_y": 540,
  "injection_z": 4300,
  "product_id": 36,
  "specimen_name": "Emx1-IRES-Cre-296884",
  "strain": "unknown",
  "structure_abbrev": "RSPagl",
  "structure_id": 894,
  "structure_name": "Retrosplenial area, lateral agranular part",
  "primary_injection_structure": 394,
  "data_set_id": 575782182,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "39/993",
  "injection_volume": 0.2878095948,
  "injection_x": 3430,
  "injection_y": 2280,
  "injection_z": 6210,
  "product_id": 5,
  "specimen_name": "378-1085",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 39,
  "data_set_id": 112952510,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.1914851204,
  "injection_x": 8200,
  "injection_y": 950,
  "injection_z": 8580,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-120543",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 293472335,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "385/402/409",
  "injection_volume": 0.2040635772,
  "injection_x": 8920,
  "injection_y": 1830,
  "injection_z": 2480,
  "product_id": 36,
  "specimen_name": "Cux2-IRES-Cre-198364",
  "strain": null,
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 385,
  "data_set_id": 484503464,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "F",
  "injection_structures": "329",
  "injection_volume": 0.1089223058,
  "injection_x": 6800,
  "injection_y": 2180,
  "injection_z": 2850,
  "product_id": 36,
  "specimen_name": "Ntsr1-Cre_GN220-328056",
  "strain": null,
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 603717226,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838074,
   "name": "Slc6a3-Cre"
  },
  "gender": "M",
  "injection_structures": "795/872",
  "injection_volume": 0.02331371275625,
  "injection_x": 9460,
  "injection_y": 3340,
  "injection_z": 5830,
  "product_id": 5,
  "specimen_name": "Slc6a3-Cre-4469",
  "strain": "C57BL/6J",
  "structure_abbrev": "PAG",
  "structure_id": 795,
  "structure_name": "Periaqueductal gray",
  "primary_injection_structure": 795,
  "data_set_id": 160538548,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "39/48/993",
  "injection_volume": 0.1686680548,
  "injection_x": 5230,
  "injection_y": 1880,
  "injection_z": 6100,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-196",
  "strain": null,
  "structure_abbrev": "ACAv",
  "structure_id": 48,
  "structure_name": "Anterior cingulate area, ventral part",
  "primary_injection_structure": 39,
  "data_set_id": 183470468,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "337/369",
  "injection_volume": 0.1827469112,
  "injection_x": 6180,
  "injection_y": 1240,
  "injection_z": 7680,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre_KL100-263",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-ll",
  "structure_id": 337,
  "structure_name": "Primary somatosensory area, lower limb",
  "primary_injection_structure": 337,
  "data_set_id": 272698650,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838803,
   "name": "Grm2-Cre_MR90"
  },
  "gender": "M",
  "injection_structures": "726",
  "injection_volume": 0.0727904212,
  "injection_x": 8080,
  "injection_y": 2340,
  "injection_z": 7550,
  "product_id": 5,
  "specimen_name": "Grm2-Cre_MR90-3838",
  "strain": null,
  "structure_abbrev": "DG",
  "structure_id": 726,
  "structure_name": "Dentate gyrus",
  "primary_injection_structure": 726,
  "data_set_id": 287171983,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838803,
   "name": "Grm2-Cre_MR90"
  },
  "gender": "F",
  "injection_structures": "184/723/746/993",
  "injection_volume": 0.1639175048,
  "injection_x": 2550,
  "injection_y": 3710,
  "injection_z": 6840,
  "product_id": 5,
  "specimen_name": "Grm2-Cre_MR90-115",
  "strain": null,
  "structure_abbrev": "ORBl",
  "structure_id": 723,
  "structure_name": "Orbital area, lateral part",
  "primary_injection_structure": 184,
  "data_set_id": 264708349,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "394/533/894/312782546",
  "injection_volume": 0.1156418914,
  "injection_x": 7580,
  "injection_y": 720,
  "injection_z": 3820,
  "product_id": 35,
  "specimen_name": "Rbp4-Cre_KL100-247183",
  "strain": null,
  "structure_abbrev": "VISam",
  "structure_id": 394,
  "structure_name": "Anteromedial visual area",
  "primary_injection_structure": 394,
  "data_set_id": 524667618,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "409/425/541/312782574/312782628",
  "injection_volume": 0.403550084,
  "injection_x": 9270,
  "injection_y": 2460,
  "injection_z": 2250,
  "product_id": 35,
  "specimen_name": "Cux2-IRES-Cre-314798",
  "strain": null,
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 409,
  "data_set_id": 592518697,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "385/409/425",
  "injection_volume": 0.220694236,
  "injection_x": 9590,
  "injection_y": 1730,
  "injection_z": 9290,
  "product_id": 5,
  "specimen_name": "C57BL/6-145450",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 304565427,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "F",
  "injection_structures": "541/312782628",
  "injection_volume": 0.02099796265,
  "injection_x": 9030,
  "injection_y": 2870,
  "injection_z": 1710,
  "product_id": 36,
  "specimen_name": "Ntsr1-Cre_GN220-353477",
  "strain": null,
  "structure_abbrev": "VISpor",
  "structure_id": 312782628,
  "structure_name": "Postrhinal area",
  "primary_injection_structure": 541,
  "data_set_id": 649484051,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.001581774971875,
  "injection_x": 9720,
  "injection_y": 1940,
  "injection_z": 2610,
  "product_id": 36,
  "specimen_name": "A930038C07Rik-Tg1-Cre-299921",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 577241399,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838496,
   "name": "Grp-Cre_KH288"
  },
  "gender": "M",
  "injection_structures": "39/972/993",
  "injection_volume": 0.1126419546,
  "injection_x": 2880,
  "injection_y": 2320,
  "injection_z": 6330,
  "product_id": 5,
  "specimen_name": "Grp-Cre-75",
  "strain": "B6.FVB(ICR)",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 39,
  "data_set_id": 184169615,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837777,
   "name": "Sim1-Cre_KJ18"
  },
  "gender": "F",
  "injection_structures": "382",
  "injection_volume": 0.0322490315,
  "injection_x": 7280,
  "injection_y": 2180,
  "injection_z": 6040,
  "product_id": 5,
  "specimen_name": "Sim1-Cre-Homo-312",
  "strain": "B6.FVB",
  "structure_abbrev": "CA1",
  "structure_id": 382,
  "structure_name": "Field CA1",
  "primary_injection_structure": 382,
  "data_set_id": 181777177,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "402/1027/312782574",
  "injection_volume": 0.000233143378125,
  "injection_x": 8410,
  "injection_y": 2060,
  "injection_z": 1810,
  "product_id": 36,
  "specimen_name": "Cux2-IRES-Cre-280837",
  "strain": null,
  "structure_abbrev": "VISal",
  "structure_id": 402,
  "structure_name": "Anterolateral visual area",
  "primary_injection_structure": 402,
  "data_set_id": 560736987,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "329/353/369/182305689",
  "injection_volume": 0.1653628088,
  "injection_x": 6050,
  "injection_y": 2500,
  "injection_z": 8310,
  "product_id": 5,
  "specimen_name": "Ntsr1-Cre-3665",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-un",
  "structure_id": 182305689,
  "structure_name": "Primary somatosensory area, unassigned",
  "primary_injection_structure": 329,
  "data_set_id": 181180790,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "409/541/312782574/312782628",
  "injection_volume": 0.1974412664,
  "injection_x": 9150,
  "injection_y": 2360,
  "injection_z": 9560,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-276",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISpor",
  "structure_id": 312782628,
  "structure_name": "Postrhinal area",
  "primary_injection_structure": 409,
  "data_set_id": 272736450,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "329/182305689",
  "injection_volume": 0.01606406445,
  "injection_x": 6740,
  "injection_y": 1410,
  "injection_z": 8220,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-3840",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 177907797,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838828,
   "name": "Gal-Cre_KI87"
  },
  "gender": "F",
  "injection_structures": "118/830/946",
  "injection_volume": 0.1284860948,
  "injection_x": 7210,
  "injection_y": 6020,
  "injection_z": 6070,
  "product_id": 5,
  "specimen_name": "Gal-Cre-1318",
  "strain": "C57BL/6J",
  "structure_abbrev": "DMH",
  "structure_id": 830,
  "structure_name": "Dorsomedial nucleus of the hypothalamus",
  "primary_injection_structure": 118,
  "data_set_id": 178283239,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838225,
   "name": "Cart-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "15/149/181/262",
  "injection_volume": 0.0737405214,
  "injection_x": 5500,
  "injection_y": 4570,
  "injection_z": 6060,
  "product_id": 5,
  "specimen_name": "Cart-Tg1-Cre-4445",
  "strain": "C57BL/6J",
  "structure_abbrev": "PT",
  "structure_id": 15,
  "structure_name": "Parataenial nucleus",
  "primary_injection_structure": 15,
  "data_set_id": 176897793,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "F",
  "injection_structures": "985/993",
  "injection_volume": 0.000866476953125,
  "injection_x": 3630,
  "injection_y": 2920,
  "injection_z": 3960,
  "product_id": 45,
  "specimen_name": "Ntsr1-Cre_GN220-239693",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 518217838,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838331,
   "name": "Cdhr1-Cre_KG66"
  },
  "gender": "M",
  "injection_structures": "507",
  "injection_volume": 0.2776992092,
  "injection_x": 590,
  "injection_y": 5750,
  "injection_z": 6500,
  "product_id": 5,
  "specimen_name": "Cdhr1-Cre-60",
  "strain": null,
  "structure_abbrev": "MOB",
  "structure_id": 507,
  "structure_name": "Main olfactory bulb",
  "primary_injection_structure": 507,
  "data_set_id": 146859480,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 183238310,
   "name": "Ntrk1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "672",
  "injection_volume": 0.00508081153125,
  "injection_x": 5410,
  "injection_y": 3490,
  "injection_z": 7360,
  "product_id": 5,
  "specimen_name": "Ntrk1-IRES-Cre-123872",
  "strain": "B6.129",
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 672,
  "data_set_id": 308395312,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837858,
   "name": "Pvalb-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.0373642542,
  "injection_x": 9910,
  "injection_y": 1510,
  "injection_z": 8530,
  "product_id": 5,
  "specimen_name": "Pvalb-IRES-Cre-5024",
  "strain": "B6.129",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 166562678,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 266649644,
   "name": "Drd3-Cre_KI196"
  },
  "gender": "M",
  "injection_structures": "115/128/197/795/872/975",
  "injection_volume": 0.02103888745,
  "injection_x": 8870,
  "injection_y": 3530,
  "injection_z": 5970,
  "product_id": 5,
  "specimen_name": "Drd3-Cre_KI196-128334",
  "strain": null,
  "structure_abbrev": "PAG",
  "structure_id": 795,
  "structure_name": "Periaqueductal gray",
  "primary_injection_structure": 115,
  "data_set_id": 300111793,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "329/417",
  "injection_volume": 0.0610473703,
  "injection_x": 7720,
  "injection_y": 1840,
  "injection_z": 2620,
  "product_id": 36,
  "specimen_name": "Ntsr1-Cre_GN220-239691",
  "strain": null,
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 517974408,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838022,
   "name": "Pmch-Cre"
  },
  "gender": "F",
  "injection_structures": "63/88/194/797/830/576073704",
  "injection_volume": 0.00589506995,
  "injection_x": 6550,
  "injection_y": 6040,
  "injection_z": 6630,
  "product_id": 5,
  "specimen_name": "Pmch-Cre-487",
  "strain": "B6.129.FVB",
  "structure_abbrev": "LHA",
  "structure_id": 194,
  "structure_name": "Lateral hypothalamic area",
  "primary_injection_structure": 63,
  "data_set_id": 142653395,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837324,
   "name": "Slc18a2-Cre_OZ14"
  },
  "gender": "F",
  "injection_structures": "369",
  "injection_volume": 0.00266382283125,
  "injection_x": 4750,
  "injection_y": 2020,
  "injection_z": 8300,
  "product_id": 5,
  "specimen_name": "Slc18a2-Cre_OZ14-121808",
  "strain": null,
  "structure_abbrev": "SSp-ul",
  "structure_id": 369,
  "structure_name": "Primary somatosensory area, upper limb",
  "primary_injection_structure": 369,
  "data_set_id": 306990880,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "944/951",
  "injection_volume": 0.1578326162,
  "injection_x": 12830,
  "injection_y": 3150,
  "injection_z": 6300,
  "product_id": 5,
  "specimen_name": "378-1265",
  "strain": "C57BL/6J",
  "structure_abbrev": "PYR",
  "structure_id": 951,
  "structure_name": "Pyramus (VIII)",
  "primary_injection_structure": 944,
  "data_set_id": 125831616,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "463/726",
  "injection_volume": 0.3478825952,
  "injection_x": 8500,
  "injection_y": 4310,
  "injection_z": 8910,
  "product_id": 5,
  "specimen_name": "378-1153",
  "strain": "C57BL/6J",
  "structure_abbrev": "CA3",
  "structure_id": 463,
  "structure_name": "Field CA3",
  "primary_injection_structure": 463,
  "data_set_id": 113935285,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761781,
   "name": "Efr3a-Cre_NO108"
  },
  "gender": "F",
  "injection_structures": "385/409",
  "injection_volume": 0.0030883027875,
  "injection_x": 9140,
  "injection_y": 1710,
  "injection_z": 9030,
  "product_id": 5,
  "specimen_name": "Efr3a-Cre_NO108-125131",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 385,
  "data_set_id": 294266031,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "409/541/312782574/312782628",
  "injection_volume": 0.0539580993,
  "injection_x": 8960,
  "injection_y": 1940,
  "injection_z": 1910,
  "product_id": 36,
  "specimen_name": "Tlx3-Cre_PL56-356306.0",
  "strain": null,
  "structure_abbrev": "VISpor",
  "structure_id": 312782628,
  "structure_name": "Postrhinal area",
  "primary_injection_structure": 409,
  "data_set_id": 652753758,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0597378698,
  "injection_x": 8790,
  "injection_y": 1310,
  "injection_z": 3390,
  "product_id": 35,
  "specimen_name": "Tlx3-Cre_PL56-254852",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 531441947,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "502/541/312782628/484682470",
  "injection_volume": 0.3097371732,
  "injection_x": 9240,
  "injection_y": 2830,
  "injection_z": 1310,
  "product_id": 36,
  "specimen_name": "Emx1-IRES-Cre-363521",
  "strain": null,
  "structure_abbrev": "VISpor",
  "structure_id": 312782628,
  "structure_name": "Postrhinal area",
  "primary_injection_structure": 502,
  "data_set_id": 657172447,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 298725927,
   "name": "Penk-IRES2-Cre-neo"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.01990000005,
  "injection_x": 9370,
  "injection_y": 1670,
  "injection_z": 8320,
  "product_id": 5,
  "specimen_name": "Penk-IRES2-Cre-neo-298367",
  "strain": "unknown",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 585931259,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "F",
  "injection_structures": "39",
  "injection_volume": 0.0046038531875,
  "injection_x": 4000,
  "injection_y": 2300,
  "injection_z": 6160,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg3-Cre-4905",
  "strain": "B6.C3H",
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 268039675,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "F",
  "injection_structures": "541/1027/312782574/312782628",
  "injection_volume": 0.03661777105,
  "injection_x": 8490,
  "injection_y": 2470,
  "injection_z": 1810,
  "product_id": 35,
  "specimen_name": "Ntsr1-Cre_GN220-328058",
  "strain": null,
  "structure_abbrev": "TEa",
  "structure_id": 541,
  "structure_name": "Temporal association areas",
  "primary_injection_structure": 541,
  "data_set_id": 603890605,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.065161915,
  "injection_x": 8430,
  "injection_y": 1470,
  "injection_z": 3240,
  "product_id": 35,
  "specimen_name": "Rbp4-Cre_KL100-319623",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 596253012,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838710,
   "name": "Sst-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "985/993",
  "injection_volume": 0.295936186,
  "injection_x": 3930,
  "injection_y": 1650,
  "injection_z": 7420,
  "product_id": 5,
  "specimen_name": "Sst-IRES-Cre-203",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 277800995,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761781,
   "name": "Efr3a-Cre_NO108"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.01423080785,
  "injection_x": 9190,
  "injection_y": 1530,
  "injection_z": 8150,
  "product_id": 5,
  "specimen_name": "Efr3a-Cre_NO108-23",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 267704653,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838496,
   "name": "Grp-Cre_KH288"
  },
  "gender": "F",
  "injection_structures": "985/993",
  "injection_volume": 0.2718516472,
  "injection_x": 3700,
  "injection_y": 1520,
  "injection_z": 7030,
  "product_id": 5,
  "specimen_name": "Grp-Cre_KH288-104",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 265289624,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761781,
   "name": "Efr3a-Cre_NO108"
  },
  "gender": "F",
  "injection_structures": "329/672",
  "injection_volume": 0.04021443475,
  "injection_x": 6630,
  "injection_y": 2690,
  "injection_z": 8510,
  "product_id": 5,
  "specimen_name": "Efr3a-Cre_NO108-131548",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 329,
  "data_set_id": 301180385,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "541/312782628",
  "injection_volume": 0.0306279498,
  "injection_x": 9170,
  "injection_y": 2540,
  "injection_z": 1480,
  "product_id": 36,
  "specimen_name": "Tlx3-Cre_PL56-313535",
  "strain": null,
  "structure_abbrev": "VISpor",
  "structure_id": 312782628,
  "structure_name": "Postrhinal area",
  "primary_injection_structure": 541,
  "data_set_id": 590983389,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837315,
   "name": "Avp-IRES2-Cre"
  },
  "gender": "M",
  "injection_structures": "390",
  "injection_volume": 0.0142585492,
  "injection_x": 6040,
  "injection_y": 6680,
  "injection_z": 6900,
  "product_id": 5,
  "specimen_name": "Avp-IRES2-Cre-D-3563",
  "strain": "B6.129",
  "structure_abbrev": "SO",
  "structure_id": 390,
  "structure_name": "Supraoptic nucleus",
  "primary_injection_structure": 390,
  "data_set_id": 286726065,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "385/402/409/417",
  "injection_volume": 0.1453511304,
  "injection_x": 8270,
  "injection_y": 1710,
  "injection_z": 8820,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre-127",
  "strain": null,
  "structure_abbrev": "VISal",
  "structure_id": 402,
  "structure_name": "Anterolateral visual area",
  "primary_injection_structure": 385,
  "data_set_id": 167794131,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838584,
   "name": "Gad2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "128/374/422/475/797/560581563/563807435",
  "injection_volume": 0.090647844,
  "injection_x": 8070,
  "injection_y": 4700,
  "injection_z": 7590,
  "product_id": 5,
  "specimen_name": "Gad2-IRES-Cre-3866",
  "strain": null,
  "structure_abbrev": "ZI",
  "structure_id": 797,
  "structure_name": "Zona incerta",
  "primary_injection_structure": 128,
  "data_set_id": 286662551,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "312782628",
  "injection_volume": 0.015251907825,
  "injection_x": 9460,
  "injection_y": 2690,
  "injection_z": 9260,
  "product_id": 5,
  "specimen_name": "Ntsr1-Cre_GN220-131616",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISpor",
  "structure_id": 312782628,
  "structure_name": "Postrhinal area",
  "primary_injection_structure": 312782628,
  "data_set_id": 298830868,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "928/1007",
  "injection_volume": 0.0147394842,
  "injection_x": 11030,
  "injection_y": 3440,
  "injection_z": 8000,
  "product_id": 5,
  "specimen_name": "378-961",
  "strain": "C57BL/6J",
  "structure_abbrev": "SIM",
  "structure_id": 1007,
  "structure_name": "Simple lobule",
  "primary_injection_structure": 928,
  "data_set_id": 112460257,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839090,
   "name": "Slc32a1-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "126/223/693/830/1004/1126",
  "injection_volume": 0.2569419027,
  "injection_x": 7640,
  "injection_y": 6840,
  "injection_z": 5500,
  "product_id": 5,
  "specimen_name": "Slc32a1-IRES-Cre-2908",
  "strain": "B6.129",
  "structure_abbrev": "PVp",
  "structure_id": 126,
  "structure_name": "Periventricular hypothalamic nucleus, posterior part",
  "primary_injection_structure": 126,
  "data_set_id": 264078267,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838803,
   "name": "Grm2-Cre_MR90"
  },
  "gender": "M",
  "injection_structures": "1017",
  "injection_volume": 0.02335465685,
  "injection_x": 11430,
  "injection_y": 2220,
  "injection_z": 8430,
  "product_id": 5,
  "specimen_name": "Grm2-Cre_MR90-121733",
  "strain": null,
  "structure_abbrev": "AN",
  "structure_id": 1017,
  "structure_name": "Ansiform lobule",
  "primary_injection_structure": 1017,
  "data_set_id": 301324157,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "879/886",
  "injection_volume": 0.2483320392,
  "injection_x": 8410,
  "injection_y": 820,
  "injection_z": 4940,
  "product_id": 36,
  "specimen_name": "Emx1-IRES-Cre-314295",
  "strain": null,
  "structure_abbrev": "RSPv",
  "structure_id": 886,
  "structure_name": "Retrosplenial area, ventral part",
  "primary_injection_structure": 879,
  "data_set_id": 592540591,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837777,
   "name": "Sim1-Cre_KJ18"
  },
  "gender": "F",
  "injection_structures": "88/194/351/515/523/576073704",
  "injection_volume": 0.0640863944,
  "injection_x": 6010,
  "injection_y": 5840,
  "injection_z": 6300,
  "product_id": 5,
  "specimen_name": "Sim1-Cre_KJ18-128744",
  "strain": null,
  "structure_abbrev": "AHN",
  "structure_id": 88,
  "structure_name": "Anterior hypothalamic nucleus",
  "primary_injection_structure": 88,
  "data_set_id": 515198413,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0681208633,
  "injection_x": 8440,
  "injection_y": 860,
  "injection_z": 8160,
  "product_id": 5,
  "specimen_name": "378-1920",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 277616630,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182901722,
   "name": "Ppp1r17-Cre_NL146"
  },
  "gender": "M",
  "injection_structures": "382/423/463/726",
  "injection_volume": 0.0331439577,
  "injection_x": 6650,
  "injection_y": 2600,
  "injection_z": 6420,
  "product_id": 5,
  "specimen_name": "Ppp1r17-Cre_NL146-124168",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "DG",
  "structure_id": 726,
  "structure_name": "Dentate gyrus",
  "primary_injection_structure": 382,
  "data_set_id": 308550233,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "329/378",
  "injection_volume": 0.1607936274,
  "injection_x": 7380,
  "injection_y": 1940,
  "injection_z": 9400,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre_KL100-129771",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSs",
  "structure_id": 378,
  "structure_name": "Supplemental somatosensory area",
  "primary_injection_structure": 329,
  "data_set_id": 298179622,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "385/402/409",
  "injection_volume": 0.1263602886,
  "injection_x": 8430,
  "injection_y": 1160,
  "injection_z": 9300,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-266",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 385,
  "data_set_id": 266487079,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "39/48",
  "injection_volume": 0.1509594156,
  "injection_x": 4620,
  "injection_y": 2020,
  "injection_z": 5520,
  "product_id": 36,
  "specimen_name": "Emx1-IRES-Cre-187652",
  "strain": "B6.129",
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 478376911,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "M",
  "injection_structures": "541/312782628",
  "injection_volume": 0.0456965964,
  "injection_x": 9280,
  "injection_y": 2940,
  "injection_z": 9560,
  "product_id": 5,
  "specimen_name": "Syt6-Cre-610",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISpor",
  "structure_id": 312782628,
  "structure_name": "Postrhinal area",
  "primary_injection_structure": 541,
  "data_set_id": 156786939,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "425/541/312782574/312782628",
  "injection_volume": 0.1927823268,
  "injection_x": 9360,
  "injection_y": 2390,
  "injection_z": 9570,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre-97",
  "strain": null,
  "structure_abbrev": "VISpor",
  "structure_id": 312782628,
  "structure_name": "Postrhinal area",
  "primary_injection_structure": 425,
  "data_set_id": 166083557,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838953,
   "name": "Oxt-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "38/63",
  "injection_volume": 0.01350117059375,
  "injection_x": 6490,
  "injection_y": 5620,
  "injection_z": 5960,
  "product_id": 5,
  "specimen_name": "Oxt-IRES-Cre-1243",
  "strain": "B6.129",
  "structure_abbrev": "PVHd",
  "structure_id": 63,
  "structure_name": "Paraventricular hypothalamic nucleus, descending division",
  "primary_injection_structure": 38,
  "data_set_id": 166532512,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 301467016,
   "name": "Npr3-IRES2-Cre"
  },
  "gender": "M",
  "injection_structures": "39/48",
  "injection_volume": 0.132603595860449,
  "injection_x": 4890,
  "injection_y": 2380,
  "injection_z": 6520,
  "product_id": 5,
  "specimen_name": "Npr3-IRES2-Cre-299660",
  "strain": null,
  "structure_abbrev": "ACAv",
  "structure_id": 48,
  "structure_name": "Anterior cingulate area, ventral part",
  "primary_injection_structure": 39,
  "data_set_id": 585911240,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "672/993",
  "injection_volume": 0.1238697362,
  "injection_x": 4460,
  "injection_y": 3170,
  "injection_z": 6690,
  "product_id": 5,
  "specimen_name": "378-1464",
  "strain": "C57BL/6J",
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 672,
  "data_set_id": 146553266,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761781,
   "name": "Efr3a-Cre_NO108"
  },
  "gender": "M",
  "injection_structures": "329/353/182305689",
  "injection_volume": 0.0098071950375,
  "injection_x": 5900,
  "injection_y": 1990,
  "injection_z": 8730,
  "product_id": 5,
  "specimen_name": "Efr3a-Cre_NO108-36",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 267660272,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "329",
  "injection_volume": 0.0978455618,
  "injection_x": 7440,
  "injection_y": 1830,
  "injection_z": 2780,
  "product_id": 35,
  "specimen_name": "Ntsr1-Cre_GN220-256590",
  "strain": "B6.FVB",
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 538079359,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 179697864,
   "name": "Rorb-IRES2-Cre"
  },
  "gender": "F",
  "injection_structures": "1002/1011",
  "injection_volume": 0.0816380278,
  "injection_x": 7640,
  "injection_y": 2410,
  "injection_z": 10100,
  "product_id": 5,
  "specimen_name": "Rorb-IRES2-Cre-D-687",
  "strain": null,
  "structure_abbrev": "AUDp",
  "structure_id": 1002,
  "structure_name": "Primary auditory area",
  "primary_injection_structure": 1002,
  "data_set_id": 287223629,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838048,
   "name": "Slc17a6-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "4",
  "injection_volume": 0.009170353675,
  "injection_x": 10250,
  "injection_y": 2270,
  "injection_z": 6950,
  "product_id": 5,
  "specimen_name": "Slc17a6-IRES-Cre-3634",
  "strain": "B6.129",
  "structure_abbrev": "IC",
  "structure_id": 4,
  "structure_name": "Inferior colliculus",
  "primary_injection_structure": 4,
  "data_set_id": 287452840,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761781,
   "name": "Efr3a-Cre_NO108"
  },
  "gender": "M",
  "injection_structures": "48/258",
  "injection_volume": 0.1719575228,
  "injection_x": 4500,
  "injection_y": 4490,
  "injection_z": 6090,
  "product_id": 5,
  "specimen_name": "Efr3a-Cre_NO108-129943",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "LSr",
  "structure_id": 258,
  "structure_name": "Lateral septal nucleus, rostral (rostroventral) part",
  "primary_injection_structure": 48,
  "data_set_id": 300164356,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 298341195,
   "name": "Htr3a-Cre_NO152"
  },
  "gender": "F",
  "injection_structures": "1002/1011",
  "injection_volume": 0.03508239035,
  "injection_x": 7680,
  "injection_y": 2860,
  "injection_z": 10030,
  "product_id": 5,
  "specimen_name": "Htr3a-Cre_NO152-145245",
  "strain": null,
  "structure_abbrev": "AUDp",
  "structure_id": 1002,
  "structure_name": "Primary auditory area",
  "primary_injection_structure": 1002,
  "data_set_id": 305488671,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "266/287/351",
  "injection_volume": 0.213229478,
  "injection_x": 5270,
  "injection_y": 5030,
  "injection_z": 6260,
  "product_id": 5,
  "specimen_name": "378-1201",
  "strain": "C57BL/6J",
  "structure_abbrev": "BST",
  "structure_id": 351,
  "structure_name": "Bed nuclei of the stria terminalis",
  "primary_injection_structure": 266,
  "data_set_id": 117312486,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 183238310,
   "name": "Ntrk1-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "56/672",
  "injection_volume": 0.0840069622,
  "injection_x": 4690,
  "injection_y": 5420,
  "injection_z": 7120,
  "product_id": 5,
  "specimen_name": "Ntrk1-IRES-Cre_LURC-125681",
  "strain": null,
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 56,
  "data_set_id": 309739641,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761781,
   "name": "Efr3a-Cre_NO108"
  },
  "gender": "M",
  "injection_structures": "345/353/369/182305689",
  "injection_volume": 0.0492496697,
  "injection_x": 5060,
  "injection_y": 2230,
  "injection_z": 8720,
  "product_id": 5,
  "specimen_name": "Efr3a-Cre_NO108-89",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "SSp-m",
  "structure_id": 345,
  "structure_name": "Primary somatosensory area, mouth",
  "primary_injection_structure": 345,
  "data_set_id": 292318449,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "128/162/280/358/604/757/880/898/599626927",
  "injection_volume": 0.1822477384,
  "injection_x": 10570,
  "injection_y": 4570,
  "injection_z": 6010,
  "product_id": 5,
  "specimen_name": "378-1640",
  "strain": "C57BL/6J",
  "structure_abbrev": "PCG",
  "structure_id": 898,
  "structure_name": "Pontine central gray",
  "primary_injection_structure": 128,
  "data_set_id": 158434409,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 287226763,
   "name": "Calb1-T2A-dgCre"
  },
  "gender": "M",
  "injection_structures": "181/189/366/599/1077/1120/560581559",
  "injection_volume": 0.113252070290625,
  "injection_x": 6350,
  "injection_y": 5350,
  "injection_z": 5690,
  "product_id": 5,
  "specimen_name": "Calb1-2A-dgCre-249634",
  "strain": null,
  "structure_abbrev": "RE",
  "structure_id": 181,
  "structure_name": "Nucleus of reuniens",
  "primary_injection_structure": 181,
  "data_set_id": 538833505,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "345/378/1057",
  "injection_volume": 0.3366557544,
  "injection_x": 4420,
  "injection_y": 2970,
  "injection_z": 9130,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-126035",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-m",
  "structure_id": 345,
  "structure_name": "Primary somatosensory area, mouth",
  "primary_injection_structure": 345,
  "data_set_id": 294397199,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "382/385/409/502/484682470",
  "injection_volume": 0.0763924357,
  "injection_x": 8870,
  "injection_y": 1970,
  "injection_z": 2550,
  "product_id": 36,
  "specimen_name": "Emx1-IRES-Cre-194298",
  "strain": "B6.129",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 382,
  "data_set_id": 479982715,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.0105243131,
  "injection_x": 8750,
  "injection_y": 1300,
  "injection_z": 3200,
  "product_id": 36,
  "specimen_name": "A930038C07Rik-Tg1-Cre-212449",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 496252297,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "385/402/409/312782574",
  "injection_volume": 0.014853804875,
  "injection_x": 8610,
  "injection_y": 1640,
  "injection_z": 2270,
  "product_id": 35,
  "specimen_name": "A930038C07Rik-Tg1-Cre-242326",
  "strain": null,
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 385,
  "data_set_id": 520341802,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "385/879/894",
  "injection_volume": 0.0334296375,
  "injection_x": 9690,
  "injection_y": 1040,
  "injection_z": 7300,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-3931",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 286836394,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182693192,
   "name": "Chrna2-Cre_OE25"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0261185925,
  "injection_x": 9430,
  "injection_y": 1450,
  "injection_z": 8290,
  "product_id": 5,
  "specimen_name": "Chrna2-Cre_OE25-3871",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 278068196,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 298725927,
   "name": "Penk-IRES2-Cre-neo"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.02970216095,
  "injection_x": 8250,
  "injection_y": 730,
  "injection_z": 8370,
  "product_id": 5,
  "specimen_name": "Penk-IRES2-Cre-neo-368516",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 670657874,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837777,
   "name": "Sim1-Cre_KJ18"
  },
  "gender": "M",
  "injection_structures": "993",
  "injection_volume": 0.00814042145,
  "injection_x": 3410,
  "injection_y": 2520,
  "injection_z": 4000,
  "product_id": 45,
  "specimen_name": "Sim1-Cre_KJ18-230435",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 507798246,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "F",
  "injection_structures": "329",
  "injection_volume": 0.1194858728,
  "injection_x": 7270,
  "injection_y": 1500,
  "injection_z": 8740,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg3-Cre-4850",
  "strain": "B6.C3H",
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 183374804,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "993",
  "injection_volume": 0.0974537088,
  "injection_x": 2520,
  "injection_y": 2780,
  "injection_z": 7880,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-3954",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 292476595,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "312782574/312782628",
  "injection_volume": 0.0562352371,
  "injection_x": 9360,
  "injection_y": 2740,
  "injection_z": 2100,
  "product_id": 35,
  "specimen_name": "Rbp4-Cre_KL100-272634",
  "strain": null,
  "structure_abbrev": "VISpor",
  "structure_id": 312782628,
  "structure_name": "Postrhinal area",
  "primary_injection_structure": 312782574,
  "data_set_id": 554334685,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761781,
   "name": "Efr3a-Cre_NO108"
  },
  "gender": "M",
  "injection_structures": "151/159/507/723/746",
  "injection_volume": 0.1339345616,
  "injection_x": 2470,
  "injection_y": 3870,
  "injection_z": 6480,
  "product_id": 5,
  "specimen_name": "Efr3a-Cre_NO108-31",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "AOB",
  "structure_id": 151,
  "structure_name": "Accessory olfactory bulb",
  "primary_injection_structure": 151,
  "data_set_id": 267760731,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "329/385/417/312782546",
  "injection_volume": 0.163515156,
  "injection_x": 7740,
  "injection_y": 1390,
  "injection_z": 3330,
  "product_id": 36,
  "specimen_name": "Emx1-IRES-Cre-363519",
  "strain": null,
  "structure_abbrev": "VISrl",
  "structure_id": 417,
  "structure_name": "Rostrolateral visual area",
  "primary_injection_structure": 329,
  "data_set_id": 657334568,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "337/985",
  "injection_volume": 0.1691859652,
  "injection_x": 5920,
  "injection_y": 1280,
  "injection_z": 7050,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-148",
  "strain": null,
  "structure_abbrev": "MOp",
  "structure_id": 985,
  "structure_name": "Primary motor area",
  "primary_injection_structure": 337,
  "data_set_id": 182616478,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838584,
   "name": "Gad2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "1002/1011",
  "injection_volume": 0.03302066145,
  "injection_x": 7620,
  "injection_y": 2790,
  "injection_z": 10110,
  "product_id": 5,
  "specimen_name": "Gad2-IRES-Cre-1186",
  "strain": "C57BL/6J",
  "structure_abbrev": "AUDp",
  "structure_id": 1002,
  "structure_name": "Primary auditory area",
  "primary_injection_structure": 1002,
  "data_set_id": 182934777,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837324,
   "name": "Slc18a2-Cre_OZ14"
  },
  "gender": "F",
  "injection_structures": "329",
  "injection_volume": 0.01427992545,
  "injection_x": 6180,
  "injection_y": 1790,
  "injection_z": 8940,
  "product_id": 5,
  "specimen_name": "Slc18a2-Cre-119",
  "strain": null,
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 263370720,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "295/536/672",
  "injection_volume": 0.3307742256,
  "injection_x": 7240,
  "injection_y": 5730,
  "injection_z": 8580,
  "product_id": 5,
  "specimen_name": "378-1098",
  "strain": "C57BL/6J",
  "structure_abbrev": "BLA",
  "structure_id": 295,
  "structure_name": "Basolateral amygdalar nucleus",
  "primary_injection_structure": 295,
  "data_set_id": 113144533,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839513,
   "name": "Drd2-Cre_ER44"
  },
  "gender": "F",
  "injection_structures": "672",
  "injection_volume": 0.1825198648,
  "injection_x": 4800,
  "injection_y": 3400,
  "injection_z": 7500,
  "product_id": 5,
  "specimen_name": "Drd2-Cre-2021",
  "strain": "C57BL/6J",
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 672,
  "data_set_id": 158019342,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "385/879/894",
  "injection_volume": 0.1049293546,
  "injection_x": 9720,
  "injection_y": 1120,
  "injection_z": 7160,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre-138",
  "strain": null,
  "structure_abbrev": "RSPagl",
  "structure_id": 894,
  "structure_name": "Retrosplenial area, lateral agranular part",
  "primary_injection_structure": 385,
  "data_set_id": 168164230,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182693192,
   "name": "Chrna2-Cre_OE25"
  },
  "gender": "M",
  "injection_structures": "385/533/894",
  "injection_volume": 0.00530382615,
  "injection_x": 9130,
  "injection_y": 890,
  "injection_z": 7220,
  "product_id": 5,
  "specimen_name": "Chrna2-Cre_OE25-124949",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 297627858,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "886/894/1037",
  "injection_volume": 0.1173559408,
  "injection_x": 8450,
  "injection_y": 660,
  "injection_z": 6810,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-3949",
  "strain": null,
  "structure_abbrev": "RSPv",
  "structure_id": 886,
  "structure_name": "Retrosplenial area, ventral part",
  "primary_injection_structure": 886,
  "data_set_id": 292373346,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761781,
   "name": "Efr3a-Cre_NO108"
  },
  "gender": "F",
  "injection_structures": "337/369",
  "injection_volume": 0.0583332358,
  "injection_x": 5890,
  "injection_y": 1600,
  "injection_z": 7390,
  "product_id": 5,
  "specimen_name": "Efr3a-Cre_NO108-54",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "SSp-ll",
  "structure_id": 337,
  "structure_name": "Primary somatosensory area, lower limb",
  "primary_injection_structure": 337,
  "data_set_id": 278257366,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837281,
   "name": "Calb2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "155/218/463/726",
  "injection_volume": 0.03393013085,
  "injection_x": 6900,
  "injection_y": 2880,
  "injection_z": 6740,
  "product_id": 5,
  "specimen_name": "Calb2-IRES-Cre-215",
  "strain": "C57BL/6J",
  "structure_abbrev": "LP",
  "structure_id": 218,
  "structure_name": "Lateral posterior nucleus of the thalamus",
  "primary_injection_structure": 155,
  "data_set_id": 183282261,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "159/507",
  "injection_volume": 0.2497250504,
  "injection_x": 1640,
  "injection_y": 5300,
  "injection_z": 6930,
  "product_id": 5,
  "specimen_name": "378-638",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOB",
  "structure_id": 507,
  "structure_name": "Main olfactory bulb",
  "primary_injection_structure": 159,
  "data_set_id": 100141222,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761781,
   "name": "Efr3a-Cre_NO108"
  },
  "gender": "F",
  "injection_structures": "993",
  "injection_volume": 0.0700994686,
  "injection_x": 3370,
  "injection_y": 2700,
  "injection_z": 7440,
  "product_id": 5,
  "specimen_name": "Efr3a-Cre_NO108-59",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 278258780,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761781,
   "name": "Efr3a-Cre_NO108"
  },
  "gender": "M",
  "injection_structures": "128/218/362/575/930",
  "injection_volume": 0.168969444,
  "injection_x": 7760,
  "injection_y": 3920,
  "injection_z": 6820,
  "product_id": 5,
  "specimen_name": "Efr3a-Cre_NO108-131952",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "PF",
  "structure_id": 930,
  "structure_name": "Parafascicular nucleus",
  "primary_injection_structure": 128,
  "data_set_id": 299732738,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837481,
   "name": "Scnn1a-Tg2-Cre"
  },
  "gender": "F",
  "injection_structures": "170/218",
  "injection_volume": 0.0454280519,
  "injection_x": 7670,
  "injection_y": 3420,
  "injection_z": 7970,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg2-Cre-338215",
  "strain": null,
  "structure_abbrev": "LGd",
  "structure_id": 170,
  "structure_name": "Dorsal part of the lateral geniculate complex",
  "primary_injection_structure": 170,
  "data_set_id": 642811309,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "4/920",
  "injection_volume": 0.0726238359,
  "injection_x": 10860,
  "injection_y": 2600,
  "injection_z": 6290,
  "product_id": 5,
  "specimen_name": "378-1299",
  "strain": "C57BL/6J",
  "structure_abbrev": "CENT",
  "structure_id": 920,
  "structure_name": "Central lobule",
  "primary_injection_structure": 4,
  "data_set_id": 127398177,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "369/985",
  "injection_volume": 0.0140316988,
  "injection_x": 5250,
  "injection_y": 1890,
  "injection_z": 8190,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-3955",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-ul",
  "structure_id": 369,
  "structure_name": "Primary somatosensory area, upper limb",
  "primary_injection_structure": 369,
  "data_set_id": 292792016,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "F",
  "injection_structures": "409",
  "injection_volume": 0.0186894428,
  "injection_x": 9000,
  "injection_y": 2220,
  "injection_z": 2340,
  "product_id": 36,
  "specimen_name": "Ntsr1-Cre_GN220-204251",
  "strain": null,
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 409,
  "data_set_id": 495754475,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "385/394/533",
  "injection_volume": 0.1498509278,
  "injection_x": 7920,
  "injection_y": 610,
  "injection_z": 3780,
  "product_id": 36,
  "specimen_name": "Emx1-IRES-Cre-226608",
  "strain": null,
  "structure_abbrev": "VISpm",
  "structure_id": 533,
  "structure_name": "posteromedial visual area",
  "primary_injection_structure": 385,
  "data_set_id": 502074651,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182693192,
   "name": "Chrna2-Cre_OE25"
  },
  "gender": "F",
  "injection_structures": "985",
  "injection_volume": 0.0643295618,
  "injection_x": 4350,
  "injection_y": 2100,
  "injection_z": 8000,
  "product_id": 5,
  "specimen_name": "Chrna2-Cre_OE25-130478",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "MOp",
  "structure_id": 985,
  "structure_name": "Primary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 298325807,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 179697864,
   "name": "Rorb-IRES2-Cre"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0937516412,
  "injection_x": 9250,
  "injection_y": 1010,
  "injection_z": 8210,
  "product_id": 5,
  "specimen_name": "Rorb-IRES2-Cre-D_LURC-132002",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 301616660,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839285,
   "name": "Gpr26-Cre_KO250"
  },
  "gender": "M",
  "injection_structures": "39/48",
  "injection_volume": 0.126259181728944,
  "injection_x": 4850,
  "injection_y": 2100,
  "injection_z": 6150,
  "product_id": 5,
  "specimen_name": "Gpr26-Cre_KO250-85",
  "strain": null,
  "structure_abbrev": "ACAv",
  "structure_id": 48,
  "structure_name": "Anterior cingulate area, ventral part",
  "primary_injection_structure": 39,
  "data_set_id": 265712971,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "F",
  "injection_structures": "345",
  "injection_volume": 0.3033490824,
  "injection_x": 4700,
  "injection_y": 2680,
  "injection_z": 9270,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg3-Cre-4827",
  "strain": "B6.C3H",
  "structure_abbrev": "SSp-m",
  "structure_id": 345,
  "structure_name": "Primary somatosensory area, mouth",
  "primary_injection_structure": 345,
  "data_set_id": 181819064,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761781,
   "name": "Efr3a-Cre_NO108"
  },
  "gender": "F",
  "injection_structures": "985/993",
  "injection_volume": 0.138290103,
  "injection_x": 3730,
  "injection_y": 1870,
  "injection_z": 7240,
  "product_id": 5,
  "specimen_name": "Efr3a-Cre_NO108-12",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 267659565,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "337/369/985",
  "injection_volume": 0.9984581824,
  "injection_x": 5950,
  "injection_y": 2080,
  "injection_z": 7500,
  "product_id": 5,
  "specimen_name": "378-1814",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-ul",
  "structure_id": 369,
  "structure_name": "Primary somatosensory area, upper limb",
  "primary_injection_structure": 337,
  "data_set_id": 180718587,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839406,
   "name": "Nos1-CreERT2"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.018558589525,
  "injection_x": 9290,
  "injection_y": 1670,
  "injection_z": 8320,
  "product_id": 5,
  "specimen_name": "Nos1-CreERT2-122405",
  "strain": "B6.129",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 308385516,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "672/998/1022",
  "injection_volume": 0.0641407305,
  "injection_x": 5880,
  "injection_y": 4830,
  "injection_z": 7940,
  "product_id": 5,
  "specimen_name": "378-1443",
  "strain": "C57BL/6J",
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 672,
  "data_set_id": 127711803,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.02642247335,
  "injection_x": 9150,
  "injection_y": 1210,
  "injection_z": 3360,
  "product_id": 35,
  "specimen_name": "A930038C07Rik-Tg1-Cre-186363",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 478256439,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837834,
   "name": "Crh-IRES-Cre_BL"
  },
  "gender": "M",
  "injection_structures": "39/993",
  "injection_volume": 0.02159988355,
  "injection_x": 5370,
  "injection_y": 1610,
  "injection_z": 6210,
  "product_id": 5,
  "specimen_name": "Crh-IRES-Cre-286",
  "strain": "B6.129",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 39,
  "data_set_id": 183172820,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "378/1011",
  "injection_volume": 0.0251200999,
  "injection_x": 7490,
  "injection_y": 2640,
  "injection_z": 9360,
  "product_id": 5,
  "specimen_name": "Ntsr1-Cre-3485",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSs",
  "structure_id": 378,
  "structure_name": "Supplemental somatosensory area",
  "primary_injection_structure": 378,
  "data_set_id": 120494024,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "39/48/993",
  "injection_volume": 0.2290490888,
  "injection_x": 3910,
  "injection_y": 1950,
  "injection_z": 6000,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre_KL100-287",
  "strain": "C57BL/6J",
  "structure_abbrev": "ACAv",
  "structure_id": 48,
  "structure_name": "Anterior cingulate area, ventral part",
  "primary_injection_structure": 39,
  "data_set_id": 286482701,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "672",
  "injection_volume": 0.1065769992,
  "injection_x": 5370,
  "injection_y": 3650,
  "injection_z": 7270,
  "product_id": 5,
  "specimen_name": "378-824",
  "strain": "C57BL/6J",
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 672,
  "data_set_id": 100142580,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837281,
   "name": "Calb2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "1002/1011",
  "injection_volume": 0.017401608875,
  "injection_x": 7780,
  "injection_y": 2520,
  "injection_z": 10140,
  "product_id": 5,
  "specimen_name": "Calb2-IRES-Cre-171",
  "strain": "C57BL/6J",
  "structure_abbrev": "AUDp",
  "structure_id": 1002,
  "structure_name": "Primary auditory area",
  "primary_injection_structure": 1002,
  "data_set_id": 168455487,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 460133878,
   "name": "Ntng2-IRES2-Cre"
  },
  "gender": "M",
  "injection_structures": "104/111/119/583/672/952/1057",
  "injection_volume": 0.4214990584,
  "injection_x": 4360,
  "injection_y": 5200,
  "injection_z": 8340,
  "product_id": 5,
  "specimen_name": "Ntng2-IRES2-Cre-229432",
  "strain": null,
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 104,
  "data_set_id": 513775257,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838942,
   "name": "Htr2a-Cre_KM207"
  },
  "gender": "M",
  "injection_structures": "104/119/583/985",
  "injection_volume": 0.2007844776,
  "injection_x": 3390,
  "injection_y": 4550,
  "injection_z": 8250,
  "product_id": 5,
  "specimen_name": "Htr2a-Cre_KM207-3446",
  "strain": null,
  "structure_abbrev": "AId",
  "structure_id": 104,
  "structure_name": "Agranular insular area, dorsal part",
  "primary_injection_structure": 104,
  "data_set_id": 262536037,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "337/985",
  "injection_volume": 0.3771202288,
  "injection_x": 6180,
  "injection_y": 1180,
  "injection_z": 7090,
  "product_id": 5,
  "specimen_name": "378-1163",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-ll",
  "structure_id": 337,
  "structure_name": "Primary somatosensory area, lower limb",
  "primary_injection_structure": 337,
  "data_set_id": 114292355,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "329/417/312782546",
  "injection_volume": 0.1067473232,
  "injection_x": 7510,
  "injection_y": 1490,
  "injection_z": 8350,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre_KL100-276",
  "strain": "B6.129",
  "structure_abbrev": "VISrl",
  "structure_id": 417,
  "structure_name": "Rostrolateral visual area",
  "primary_injection_structure": 329,
  "data_set_id": 272929308,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "378",
  "injection_volume": 0.3927437416,
  "injection_x": 6170,
  "injection_y": 3870,
  "injection_z": 9450,
  "product_id": 5,
  "specimen_name": "378-1199",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSs",
  "structure_id": 378,
  "structure_name": "Supplemental somatosensory area",
  "primary_injection_structure": 378,
  "data_set_id": 117298988,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838048,
   "name": "Slc17a6-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "619",
  "injection_volume": 0.0241640119,
  "injection_x": 6040,
  "injection_y": 6640,
  "injection_z": 7840,
  "product_id": 5,
  "specimen_name": "Slc17a6-IRES-Cre-121216",
  "strain": "B6.129",
  "structure_abbrev": "NLOT",
  "structure_id": 619,
  "structure_name": "Nucleus of the lateral olfactory tract",
  "primary_injection_structure": 619,
  "data_set_id": 294174996,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 301613169,
   "name": "Sepw1-Cre_NP39"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0269665718,
  "injection_x": 9040,
  "injection_y": 1100,
  "injection_z": 8900,
  "product_id": 5,
  "specimen_name": "Sepw1-Cre_NP39-174464",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 477269754,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837797,
   "name": "Th-Cre_FI172"
  },
  "gender": "F",
  "injection_structures": "115/128/591/795/872",
  "injection_volume": 0.03817875915,
  "injection_x": 9630,
  "injection_y": 3610,
  "injection_z": 5550,
  "product_id": 5,
  "specimen_name": "Th-Cre_FI172-6198",
  "strain": "B6.FVB",
  "structure_abbrev": "PAG",
  "structure_id": 795,
  "structure_name": "Periaqueductal gray",
  "primary_injection_structure": 115,
  "data_set_id": 266500714,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839406,
   "name": "Nos1-CreERT2"
  },
  "gender": "F",
  "injection_structures": "926",
  "injection_volume": 0.0710677429,
  "injection_x": 9990,
  "injection_y": 4580,
  "injection_z": 9170,
  "product_id": 5,
  "specimen_name": "Nos1-CreERT2-126843",
  "strain": "B6.129",
  "structure_abbrev": "ENTm",
  "structure_id": 926,
  "structure_name": "Entorhinal area, medial part, dorsal zone",
  "primary_injection_structure": 926,
  "data_set_id": 299403823,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 301467016,
   "name": "Npr3-IRES2-Cre"
  },
  "gender": "F",
  "injection_structures": "147/202/350/460/867/898",
  "injection_volume": 0.0620947208,
  "injection_x": 10790,
  "injection_y": 4150,
  "injection_z": 6810,
  "product_id": 5,
  "specimen_name": "Npr3-IRES2-Cre-257245",
  "strain": "C57BL/6J",
  "structure_abbrev": "PB",
  "structure_id": 867,
  "structure_name": "Parabrachial nucleus",
  "primary_injection_structure": 147,
  "data_set_id": 545427588,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "F",
  "injection_structures": "345/353/378",
  "injection_volume": 0.1063221012,
  "injection_x": 5380,
  "injection_y": 3200,
  "injection_z": 9660,
  "product_id": 5,
  "specimen_name": "Tlx3-Cre-137311",
  "strain": null,
  "structure_abbrev": "SSs",
  "structure_id": 378,
  "structure_name": "Supplemental somatosensory area",
  "primary_injection_structure": 345,
  "data_set_id": 303616833,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.02481324965,
  "injection_x": 9440,
  "injection_y": 1270,
  "injection_z": 8150,
  "product_id": 5,
  "specimen_name": "Tlx3-Cre_LURC-3605",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 288171964,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "409/541/312782574/312782628",
  "injection_volume": 0.0321471213,
  "injection_x": 9200,
  "injection_y": 2270,
  "injection_z": 9450,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-3881",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISli",
  "structure_id": 312782574,
  "structure_name": "Laterointermediate area",
  "primary_injection_structure": 409,
  "data_set_id": 264630019,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "345",
  "injection_volume": 0.3689216272,
  "injection_x": 4530,
  "injection_y": 2470,
  "injection_z": 8980,
  "product_id": 5,
  "specimen_name": "378-1165",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-m",
  "structure_id": 345,
  "structure_name": "Primary somatosensory area, mouth",
  "primary_injection_structure": 345,
  "data_set_id": 114290938,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "329/353/182305689",
  "injection_volume": 0.1318428202,
  "injection_x": 5650,
  "injection_y": 2250,
  "injection_z": 8560,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre-45",
  "strain": null,
  "structure_abbrev": "SSp-n",
  "structure_id": 353,
  "structure_name": "Primary somatosensory area, nose",
  "primary_injection_structure": 329,
  "data_set_id": 120875816,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "111/378/672/677",
  "injection_volume": 0.4478761896,
  "injection_x": 6370,
  "injection_y": 4360,
  "injection_z": 10270,
  "product_id": 5,
  "specimen_name": "378-1811",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISC",
  "structure_id": 677,
  "structure_name": "Visceral area",
  "primary_injection_structure": 111,
  "data_set_id": 180074890,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 298341195,
   "name": "Htr3a-Cre_NO152"
  },
  "gender": "M",
  "injection_structures": "985/993",
  "injection_volume": 0.0902452796,
  "injection_x": 3640,
  "injection_y": 1900,
  "injection_z": 7300,
  "product_id": 5,
  "specimen_name": "Htr3a-Cre_NO152-145237",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 305487964,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 298725927,
   "name": "Penk-IRES2-Cre-neo"
  },
  "gender": "M",
  "injection_structures": "329/353/182305689",
  "injection_volume": 0.0234667174,
  "injection_x": 5800,
  "injection_y": 2220,
  "injection_z": 8650,
  "product_id": 5,
  "specimen_name": "Penk-IRES2-Cre-neo-145870",
  "strain": null,
  "structure_abbrev": "SSp-n",
  "structure_id": 353,
  "structure_name": "Primary somatosensory area, nose",
  "primary_injection_structure": 329,
  "data_set_id": 306268688,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "879/886",
  "injection_volume": 0.3094256312,
  "injection_x": 8320,
  "injection_y": 900,
  "injection_z": 6050,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre-90",
  "strain": null,
  "structure_abbrev": "RSPv",
  "structure_id": 886,
  "structure_name": "Retrosplenial area, ventral part",
  "primary_injection_structure": 879,
  "data_set_id": 166054929,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "88/118/173/223/614/693/993",
  "injection_volume": 0.6516929832,
  "injection_x": 6370,
  "injection_y": 7110,
  "injection_z": 6470,
  "product_id": 5,
  "specimen_name": "378-1779",
  "strain": "C57BL/6J",
  "structure_abbrev": "VMH",
  "structure_id": 693,
  "structure_name": "Ventromedial hypothalamic nucleus",
  "primary_injection_structure": 88,
  "data_set_id": 175106769,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "M",
  "injection_structures": "985/993",
  "injection_volume": 0.151070626,
  "injection_x": 4300,
  "injection_y": 2690,
  "injection_z": 7050,
  "product_id": 5,
  "specimen_name": "Syt6-Cre-585",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 122642490,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "319/403/639/647",
  "injection_volume": 0.3504543504,
  "injection_x": 7100,
  "injection_y": 6700,
  "injection_z": 8200,
  "product_id": 5,
  "specimen_name": "378-1512",
  "strain": "C57BL/6J",
  "structure_abbrev": "MEA",
  "structure_id": 403,
  "structure_name": "Medial amygdalar nucleus",
  "primary_injection_structure": 319,
  "data_set_id": 146985623,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "F",
  "injection_structures": "385/879/894",
  "injection_volume": 0.149606261,
  "injection_x": 9840,
  "injection_y": 1570,
  "injection_z": 3660,
  "product_id": 35,
  "specimen_name": "Tlx3-Cre_PL56-313272",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 591169915,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838074,
   "name": "Slc6a3-Cre"
  },
  "gender": "F",
  "injection_structures": "351",
  "injection_volume": 0.1885843008,
  "injection_x": 5660,
  "injection_y": 5050,
  "injection_z": 6340,
  "product_id": 5,
  "specimen_name": "Slc6a3-Cre-4528",
  "strain": "C57BL/6J",
  "structure_abbrev": "BST",
  "structure_id": 351,
  "structure_name": "Bed nuclei of the stria terminalis",
  "primary_injection_structure": 351,
  "data_set_id": 175739085,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837777,
   "name": "Sim1-Cre_KJ18"
  },
  "gender": "F",
  "injection_structures": "993",
  "injection_volume": 0.064394085,
  "injection_x": 3890,
  "injection_y": 2070,
  "injection_z": 7130,
  "product_id": 5,
  "specimen_name": "Sim1-Cre_KJ18-3811",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 287807743,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "F",
  "injection_structures": "385/417",
  "injection_volume": 0.0324413516,
  "injection_x": 8010,
  "injection_y": 1340,
  "injection_z": 3000,
  "product_id": 36,
  "specimen_name": "Tlx3-Cre_PL56-226275",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 503058936,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "F",
  "injection_structures": "39/993",
  "injection_volume": 0.0709428419,
  "injection_x": 3200,
  "injection_y": 2270,
  "injection_z": 6390,
  "product_id": 5,
  "specimen_name": "Tlx3-Cre-123690",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 39,
  "data_set_id": 294532700,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 266649644,
   "name": "Drd3-Cre_KI196"
  },
  "gender": "F",
  "injection_structures": "178/321/797",
  "injection_volume": 0.1131240558,
  "injection_x": 7710,
  "injection_y": 4310,
  "injection_z": 8050,
  "product_id": 5,
  "specimen_name": "Drd3-Cre_KI196-141698",
  "strain": null,
  "structure_abbrev": "ZI",
  "structure_id": 797,
  "structure_name": "Zona incerta",
  "primary_injection_structure": 178,
  "data_set_id": 304858700,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "385/879/894",
  "injection_volume": 0.0981237446,
  "injection_x": 9110,
  "injection_y": 1190,
  "injection_z": 4020,
  "product_id": 35,
  "specimen_name": "Ntsr1-Cre_GN220-245425",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 523714193,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "541/895/918/312782628",
  "injection_volume": 0.0245244804,
  "injection_x": 9490,
  "injection_y": 3110,
  "injection_z": 9800,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-123230",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISpor",
  "structure_id": 312782628,
  "structure_name": "Postrhinal area",
  "primary_injection_structure": 541,
  "data_set_id": 298350922,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "402/1011/1027",
  "injection_volume": 0.1135668198,
  "injection_x": 8220,
  "injection_y": 1520,
  "injection_z": 9700,
  "product_id": 5,
  "specimen_name": "378-1208",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISal",
  "structure_id": 402,
  "structure_name": "Anterolateral visual area",
  "primary_injection_structure": 402,
  "data_set_id": 120437703,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "985/993",
  "injection_volume": 0.1875109656,
  "injection_x": 3590,
  "injection_y": 1860,
  "injection_z": 7570,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-308464",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 587659400,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761918,
   "name": "Oxtr-Cre_ON66"
  },
  "gender": "M",
  "injection_structures": "184/723/746/993",
  "injection_volume": 0.0613104709,
  "injection_x": 2630,
  "injection_y": 3550,
  "injection_z": 6840,
  "product_id": 5,
  "specimen_name": "Oxtr-Cre_ON66-121546",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "ORBl",
  "structure_id": 723,
  "structure_name": "Orbital area, lateral part",
  "primary_injection_structure": 184,
  "data_set_id": 294311830,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837834,
   "name": "Crh-IRES-Cre_BL"
  },
  "gender": "F",
  "injection_structures": "128/280/350/358/898/1093",
  "injection_volume": 0.011760300125,
  "injection_x": 10600,
  "injection_y": 4540,
  "injection_z": 6510,
  "product_id": 5,
  "specimen_name": "Crh-IRES-Cre-337",
  "strain": "B6.129",
  "structure_abbrev": "SLD",
  "structure_id": 358,
  "structure_name": "Sublaterodorsal nucleus",
  "primary_injection_structure": 128,
  "data_set_id": 278180500,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 256980970,
   "name": "Lypd6-Cre_KL156"
  },
  "gender": "F",
  "injection_structures": "162/795",
  "injection_volume": 0.02204763085,
  "injection_x": 10150,
  "injection_y": 3670,
  "injection_z": 6190,
  "product_id": 5,
  "specimen_name": "Lypd6-Cre_KL156-126615",
  "strain": null,
  "structure_abbrev": "PAG",
  "structure_id": 795,
  "structure_name": "Periaqueductal gray",
  "primary_injection_structure": 162,
  "data_set_id": 298079928,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "1002/1011/1027",
  "injection_volume": 0.0453458299,
  "injection_x": 7840,
  "injection_y": 2110,
  "injection_z": 1660,
  "product_id": 36,
  "specimen_name": "Tlx3-Cre_PL56-295769",
  "strain": "unknown",
  "structure_abbrev": "AUDd",
  "structure_id": 1011,
  "structure_name": "Dorsal auditory area",
  "primary_injection_structure": 1002,
  "data_set_id": 572969377,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837328,
   "name": "Syt17-Cre_NO14"
  },
  "gender": "M",
  "injection_structures": "463/726",
  "injection_volume": 0.0048416538625,
  "injection_x": 6820,
  "injection_y": 2760,
  "injection_z": 6620,
  "product_id": 5,
  "specimen_name": "Syt17-Cre_NO14_LURC-121250",
  "strain": "C57BL/6J",
  "structure_abbrev": "DG",
  "structure_id": 726,
  "structure_name": "Dentate gyrus",
  "primary_injection_structure": 463,
  "data_set_id": 293115527,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837858,
   "name": "Pvalb-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "262",
  "injection_volume": 0.0618032345,
  "injection_x": 6020,
  "injection_y": 4510,
  "injection_z": 7160,
  "product_id": 5,
  "specimen_name": "Pvalb-IRES-Cre-4973",
  "strain": "B6.129",
  "structure_abbrev": "RT",
  "structure_id": 262,
  "structure_name": "Reticular nucleus of the thalamus",
  "primary_injection_structure": 262,
  "data_set_id": 127085005,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838361,
   "name": "Etv1-CreERT2"
  },
  "gender": "F",
  "injection_structures": "382/423/463",
  "injection_volume": 0.2152007088,
  "injection_x": 7980,
  "injection_y": 5110,
  "injection_z": 8870,
  "product_id": 5,
  "specimen_name": "Etv1-CreERT2-2898",
  "strain": "C57BL/6J",
  "structure_abbrev": "CA1",
  "structure_id": 382,
  "structure_name": "Field CA1",
  "primary_injection_structure": 382,
  "data_set_id": 175718634,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "F",
  "injection_structures": "533/894",
  "injection_volume": 0.0084865899125,
  "injection_x": 8810,
  "injection_y": 760,
  "injection_z": 4270,
  "product_id": 36,
  "specimen_name": "Tlx3-Cre_PL56-226273",
  "strain": null,
  "structure_abbrev": "VISpm",
  "structure_id": 533,
  "structure_name": "posteromedial visual area",
  "primary_injection_structure": 533,
  "data_set_id": 502180994,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.03188078915,
  "injection_x": 8720,
  "injection_y": 1510,
  "injection_z": 3340,
  "product_id": 36,
  "specimen_name": "Ntsr1-Cre_GN220-204250",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 495753765,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "337/985",
  "injection_volume": 0.1412620216,
  "injection_x": 5590,
  "injection_y": 1400,
  "injection_z": 7430,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre-93",
  "strain": null,
  "structure_abbrev": "MOp",
  "structure_id": 985,
  "structure_name": "Primary motor area",
  "primary_injection_structure": 337,
  "data_set_id": 166082842,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "329/361/417/312782546",
  "injection_volume": 0.6276065824,
  "injection_x": 7460,
  "injection_y": 1530,
  "injection_z": 8220,
  "product_id": 5,
  "specimen_name": "378-1352",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 126907302,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.1024400076,
  "injection_x": 9400,
  "injection_y": 1780,
  "injection_z": 3180,
  "product_id": 35,
  "specimen_name": "Emx1-IRES-Cre-255363",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 535692871,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "88/614/693/576073704",
  "injection_volume": 0.1513029742,
  "injection_x": 6570,
  "injection_y": 6370,
  "injection_z": 6250,
  "product_id": 5,
  "specimen_name": "378-1918",
  "strain": "C57BL/6J",
  "structure_abbrev": "VMH",
  "structure_id": 693,
  "structure_name": "Ventromedial hypothalamic nucleus",
  "primary_injection_structure": 88,
  "data_set_id": 277615922,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0823954404,
  "injection_x": 8280,
  "injection_y": 1530,
  "injection_z": 8290,
  "product_id": 5,
  "specimen_name": "Syt6-Cre-654",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 159330030,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "409/425/312782574/312782628",
  "injection_volume": 0.0784402143,
  "injection_x": 9420,
  "injection_y": 2400,
  "injection_z": 9460,
  "product_id": 5,
  "specimen_name": "378-936",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISli",
  "structure_id": 312782574,
  "structure_name": "Laterointermediate area",
  "primary_injection_structure": 409,
  "data_set_id": 112373830,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838048,
   "name": "Slc17a6-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "4",
  "injection_volume": 0.0870143274,
  "injection_x": 10620,
  "injection_y": 2630,
  "injection_z": 6990,
  "product_id": 5,
  "specimen_name": "Slc17a6-IRES-Cre-3632",
  "strain": "B6.129",
  "structure_abbrev": "IC",
  "structure_id": 4,
  "structure_name": "Inferior colliculus",
  "primary_injection_structure": 4,
  "data_set_id": 287448038,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839090,
   "name": "Slc32a1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "797",
  "injection_volume": 0.2654951124,
  "injection_x": 7480,
  "injection_y": 4850,
  "injection_z": 7870,
  "product_id": 5,
  "specimen_name": "Slc32a1-IRES-Cre-2855",
  "strain": "B6.129",
  "structure_abbrev": "ZI",
  "structure_id": 797,
  "structure_name": "Zona incerta",
  "primary_injection_structure": 797,
  "data_set_id": 171065906,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839285,
   "name": "Gpr26-Cre_KO250"
  },
  "gender": "F",
  "injection_structures": "39/993",
  "injection_volume": 0.103667144,
  "injection_x": 5040,
  "injection_y": 1570,
  "injection_z": 6070,
  "product_id": 5,
  "specimen_name": "Gpr26-Cre_KO250-69",
  "strain": null,
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 258316155,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 179697864,
   "name": "Rorb-IRES2-Cre"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.0245048265,
  "injection_x": 9720,
  "injection_y": 1380,
  "injection_z": 8480,
  "product_id": 5,
  "specimen_name": "Rorb-IRES2-Cre-D-688",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 287224335,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837777,
   "name": "Sim1-Cre_KJ18"
  },
  "gender": "M",
  "injection_structures": "369",
  "injection_volume": 0.00542749235,
  "injection_x": 5630,
  "injection_y": 1760,
  "injection_z": 8130,
  "product_id": 5,
  "specimen_name": "Sim1-Cre_KJ18-128746",
  "strain": null,
  "structure_abbrev": "SSp-ul",
  "structure_id": 369,
  "structure_name": "Primary somatosensory area, upper limb",
  "primary_injection_structure": 369,
  "data_set_id": 515254107,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 266649644,
   "name": "Drd3-Cre_KI196"
  },
  "gender": "F",
  "injection_structures": "329",
  "injection_volume": 0.2393745256,
  "injection_x": 5960,
  "injection_y": 2160,
  "injection_z": 8550,
  "product_id": 5,
  "specimen_name": "Drd3-Cre_KI196_LURC-124108",
  "strain": null,
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 293728197,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 553502711,
   "name": "Chat-IRES-Cre-neo"
  },
  "gender": "F",
  "injection_structures": "258/266/310/564",
  "injection_volume": 0.039264223075,
  "injection_x": 5140,
  "injection_y": 4680,
  "injection_z": 5690,
  "product_id": 5,
  "specimen_name": "Chat-IRES-Cre-344",
  "strain": "B6.129",
  "structure_abbrev": "MS",
  "structure_id": 564,
  "structure_name": "Medial septal nucleus",
  "primary_injection_structure": 258,
  "data_set_id": 156819600,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838331,
   "name": "Cdhr1-Cre_KG66"
  },
  "gender": "F",
  "injection_structures": "507",
  "injection_volume": 0.0845376028,
  "injection_x": 580,
  "injection_y": 3320,
  "injection_z": 7150,
  "product_id": 5,
  "specimen_name": "Cdhr1-Cre-63",
  "strain": null,
  "structure_abbrev": "MOB",
  "structure_id": 507,
  "structure_name": "Main olfactory bulb",
  "primary_injection_structure": 507,
  "data_set_id": 146921849,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "541/1002/1027",
  "injection_volume": 0.02880013955,
  "injection_x": 8210,
  "injection_y": 2480,
  "injection_z": 1810,
  "product_id": 36,
  "specimen_name": "Ntsr1-Cre_GN220-275952",
  "strain": "unknown",
  "structure_abbrev": "AUDpo",
  "structure_id": 1027,
  "structure_name": "Posterior auditory area",
  "primary_injection_structure": 541,
  "data_set_id": 557341233,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 183238310,
   "name": "Ntrk1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "672",
  "injection_volume": 0.0102187423625,
  "injection_x": 4510,
  "injection_y": 3330,
  "injection_z": 6770,
  "product_id": 5,
  "specimen_name": "Ntrk1-IRES-Cre_LURC-125687",
  "strain": null,
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 672,
  "data_set_id": 310175667,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 459995486,
   "name": "Gnb4-IRES2-Cre"
  },
  "gender": "M",
  "injection_structures": "111/583/672/677/1057",
  "injection_volume": 0.0206020598,
  "injection_x": 5950,
  "injection_y": 4740,
  "injection_z": 9450,
  "product_id": 5,
  "specimen_name": "Gnb4-IRES2-Cre-neo-193744",
  "strain": null,
  "structure_abbrev": "CLA",
  "structure_id": 583,
  "structure_name": "Claustrum",
  "primary_injection_structure": 111,
  "data_set_id": 485902743,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "159/952/961",
  "injection_volume": 0.147868133,
  "injection_x": 3240,
  "injection_y": 5050,
  "injection_z": 7140,
  "product_id": 5,
  "specimen_name": "378-1118",
  "strain": "C57BL/6J",
  "structure_abbrev": "AON",
  "structure_id": 159,
  "structure_name": "Anterior olfactory nucleus",
  "primary_injection_structure": 159,
  "data_set_id": 113443572,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0549243254,
  "injection_x": 8360,
  "injection_y": 900,
  "injection_z": 8230,
  "product_id": 5,
  "specimen_name": "378-1925",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 277714322,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0053502346625,
  "injection_x": 9120,
  "injection_y": 1170,
  "injection_z": 3230,
  "product_id": 35,
  "specimen_name": "Scnn1a-Tg3-Cre-189170",
  "strain": "B6.C3H",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 479085232,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838710,
   "name": "Sst-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "250/258/310",
  "injection_volume": 0.126238504,
  "injection_x": 5280,
  "injection_y": 3630,
  "injection_z": 6430,
  "product_id": 5,
  "specimen_name": "Sst-IRES-Cre-123831",
  "strain": null,
  "structure_abbrev": "LSr",
  "structure_id": 258,
  "structure_name": "Lateral septal nucleus, rostral (rostroventral) part",
  "primary_injection_structure": 250,
  "data_set_id": 300841699,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "44/48/589/814/972",
  "injection_volume": 0.1281030128,
  "injection_x": 3930,
  "injection_y": 3740,
  "injection_z": 6340,
  "product_id": 5,
  "specimen_name": "378-1614",
  "strain": "C57BL/6J",
  "structure_abbrev": "ILA",
  "structure_id": 44,
  "structure_name": "Infralimbic area",
  "primary_injection_structure": 44,
  "data_set_id": 157556400,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "202/225/765",
  "injection_volume": 0.1466467786,
  "injection_x": 11960,
  "injection_y": 4630,
  "injection_z": 6820,
  "product_id": 5,
  "specimen_name": "378-1329",
  "strain": "C57BL/6J",
  "structure_abbrev": "SPIV",
  "structure_id": 225,
  "structure_name": "Spinal vestibular nucleus",
  "primary_injection_structure": 202,
  "data_set_id": 128056535,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "385/409",
  "injection_volume": 0.0547515171,
  "injection_x": 8850,
  "injection_y": 1710,
  "injection_z": 2120,
  "product_id": 35,
  "specimen_name": "Tlx3-Cre_PL56-240882",
  "strain": null,
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 385,
  "data_set_id": 520750912,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838104,
   "name": "Pvalb-T2A-CreERT2"
  },
  "gender": "F",
  "injection_structures": "993",
  "injection_volume": 0.113024282,
  "injection_x": 2940,
  "injection_y": 2700,
  "injection_z": 3730,
  "product_id": 45,
  "specimen_name": "Pvalb-T2A-CreERT2-337659",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 636967182,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "985/993",
  "injection_volume": 0.1389593352,
  "injection_x": 3300,
  "injection_y": 2840,
  "injection_z": 7590,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-3942",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 292374068,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.4885201608,
  "injection_x": 8780,
  "injection_y": 1850,
  "injection_z": 8650,
  "product_id": 5,
  "specimen_name": "C57BL/6-152139",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 307137980,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 298725927,
   "name": "Penk-IRES2-Cre-neo"
  },
  "gender": "M",
  "injection_structures": "184/723/993",
  "injection_volume": 0.0442653358,
  "injection_x": 2620,
  "injection_y": 2160,
  "injection_z": 4350,
  "product_id": 45,
  "specimen_name": "Penk-IRES2-Cre-neo;Slc17a6-IRES2-FlpO-365244",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 184,
  "data_set_id": 671464291,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839406,
   "name": "Nos1-CreERT2"
  },
  "gender": "M",
  "injection_structures": "4/795/920",
  "injection_volume": 0.222054966,
  "injection_x": 10850,
  "injection_y": 2360,
  "injection_z": 6490,
  "product_id": 5,
  "specimen_name": "Nos1-CreERT2-131826",
  "strain": "B6.129",
  "structure_abbrev": "IC",
  "structure_id": 4,
  "structure_name": "Inferior colliculus",
  "primary_injection_structure": 4,
  "data_set_id": 299761293,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "423/463",
  "injection_volume": 0.014785660575,
  "injection_x": 7110,
  "injection_y": 2660,
  "injection_z": 8290,
  "product_id": 5,
  "specimen_name": "378-1832",
  "strain": "C57BL/6J",
  "structure_abbrev": "CA3",
  "structure_id": 463,
  "structure_name": "Field CA3",
  "primary_injection_structure": 423,
  "data_set_id": 180523704,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "F",
  "injection_structures": "541/312782574/312782628",
  "injection_volume": 0.0460086872,
  "injection_x": 9230,
  "injection_y": 2340,
  "injection_z": 9680,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg3-Cre-4862",
  "strain": "B6.C3H",
  "structure_abbrev": "VISpor",
  "structure_id": 312782628,
  "structure_name": "Postrhinal area",
  "primary_injection_structure": 541,
  "data_set_id": 184075100,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "329/361/417/312782546",
  "injection_volume": 0.095199356,
  "injection_x": 7320,
  "injection_y": 1290,
  "injection_z": 3040,
  "product_id": 36,
  "specimen_name": "Tlx3-Cre_PL56-356308.0",
  "strain": null,
  "structure_abbrev": "VISa",
  "structure_id": 312782546,
  "structure_name": "Anterior area",
  "primary_injection_structure": 329,
  "data_set_id": 651702588,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182901722,
   "name": "Ppp1r17-Cre_NL146"
  },
  "gender": "M",
  "injection_structures": "15/56/72/149/266/351/1109",
  "injection_volume": 0.2380153636,
  "injection_x": 5030,
  "injection_y": 5330,
  "injection_z": 6160,
  "product_id": 5,
  "specimen_name": "Ppp1r17-Cre_NL146-3879",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "BST",
  "structure_id": 351,
  "structure_name": "Bed nuclei of the stria terminalis",
  "primary_injection_structure": 15,
  "data_set_id": 267764292,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837328,
   "name": "Syt17-Cre_NO14"
  },
  "gender": "F",
  "injection_structures": "151/159/507",
  "injection_volume": 0.2040710056,
  "injection_x": 2070,
  "injection_y": 3770,
  "injection_z": 7040,
  "product_id": 5,
  "specimen_name": "Syt17-Cre-56",
  "strain": null,
  "structure_abbrev": "AOB",
  "structure_id": 151,
  "structure_name": "Accessory olfactory bulb",
  "primary_injection_structure": 151,
  "data_set_id": 161178152,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "385/879/886/894",
  "injection_volume": 0.02836379945,
  "injection_x": 9400,
  "injection_y": 890,
  "injection_z": 3870,
  "product_id": 35,
  "specimen_name": "A930038C07Rik-Tg1-Cre-249275",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 527576192,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837777,
   "name": "Sim1-Cre_KJ18"
  },
  "gender": "M",
  "injection_structures": "985",
  "injection_volume": 0.03132449215,
  "injection_x": 5230,
  "injection_y": 1390,
  "injection_z": 7250,
  "product_id": 5,
  "specimen_name": "Sim1-Cre_KJ18-318908",
  "strain": "unknown",
  "structure_abbrev": "MOp",
  "structure_id": 985,
  "structure_name": "Primary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 597007858,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "1011/1027",
  "injection_volume": 0.0577743908,
  "injection_x": 8270,
  "injection_y": 2330,
  "injection_z": 1860,
  "product_id": 36,
  "specimen_name": "Emx1-IRES-Cre-283755",
  "strain": null,
  "structure_abbrev": "AUDpo",
  "structure_id": 1027,
  "structure_name": "Posterior auditory area",
  "primary_injection_structure": 1011,
  "data_set_id": 563179067,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761918,
   "name": "Oxtr-Cre_ON66"
  },
  "gender": "M",
  "injection_structures": "258/266/564/596",
  "injection_volume": 0.316862861,
  "injection_x": 5140,
  "injection_y": 4740,
  "injection_z": 5540,
  "product_id": 5,
  "specimen_name": "Oxtr-Cre_ON66-3874",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "MS",
  "structure_id": 564,
  "structure_name": "Medial septal nucleus",
  "primary_injection_structure": 258,
  "data_set_id": 278259822,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0894132792,
  "injection_x": 9440,
  "injection_y": 1270,
  "injection_z": 8460,
  "product_id": 5,
  "specimen_name": "C57BL/6-145451",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 304585910,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "993",
  "injection_volume": 0.1168174896,
  "injection_x": 2840,
  "injection_y": 1770,
  "injection_z": 4460,
  "product_id": 45,
  "specimen_name": "Cux2-IRES-Cre-239314",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 518023283,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 179697864,
   "name": "Rorb-IRES2-Cre"
  },
  "gender": "F",
  "injection_structures": "394/533/894",
  "injection_volume": 0.0257289249,
  "injection_x": 8310,
  "injection_y": 550,
  "injection_z": 7080,
  "product_id": 5,
  "specimen_name": "Rorb-IRES2-Cre-D-131382",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISpm",
  "structure_id": 533,
  "structure_name": "posteromedial visual area",
  "primary_injection_structure": 394,
  "data_set_id": 301618122,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 298725927,
   "name": "Penk-IRES2-Cre-neo"
  },
  "gender": "F",
  "injection_structures": "993",
  "injection_volume": 0.00400693335,
  "injection_x": 2710,
  "injection_y": 2920,
  "injection_z": 3680,
  "product_id": 45,
  "specimen_name": "Penk-IRES2-Cre-neo-322765",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 603717939,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 179697864,
   "name": "Rorb-IRES2-Cre"
  },
  "gender": "F",
  "injection_structures": "402/409",
  "injection_volume": 0.0868859768,
  "injection_x": 8510,
  "injection_y": 1290,
  "injection_z": 9380,
  "product_id": 5,
  "specimen_name": "Rorb-IRES2-Cre-D_LURC-124653",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 402,
  "data_set_id": 298757426,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "353/378",
  "injection_volume": 0.29504909,
  "injection_x": 5870,
  "injection_y": 2920,
  "injection_z": 9660,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre_KL100-297",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSs",
  "structure_id": 378,
  "structure_name": "Supplemental somatosensory area",
  "primary_injection_structure": 353,
  "data_set_id": 286314623,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761781,
   "name": "Efr3a-Cre_NO108"
  },
  "gender": "F",
  "injection_structures": "723/746",
  "injection_volume": 0.006623203825,
  "injection_x": 3030,
  "injection_y": 3500,
  "injection_z": 7080,
  "product_id": 5,
  "specimen_name": "Efr3a-Cre_NO108-121498",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "ORBl",
  "structure_id": 723,
  "structure_name": "Orbital area, lateral part",
  "primary_injection_structure": 723,
  "data_set_id": 298230624,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "250/258",
  "injection_volume": 0.0646296084,
  "injection_x": 5060,
  "injection_y": 3380,
  "injection_z": 6150,
  "product_id": 5,
  "specimen_name": "378-1883",
  "strain": "C57BL/6J",
  "structure_abbrev": "LSr",
  "structure_id": 258,
  "structure_name": "Lateral septal nucleus, rostral (rostroventral) part",
  "primary_injection_structure": 250,
  "data_set_id": 272917631,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "385/879/894",
  "injection_volume": 0.1707563172,
  "injection_x": 9580,
  "injection_y": 1620,
  "injection_z": 3570,
  "product_id": 35,
  "specimen_name": "Cux2-IRES-Cre-246906",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 524666049,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "202/225",
  "injection_volume": 0.2775565996,
  "injection_x": 11580,
  "injection_y": 4820,
  "injection_z": 6780,
  "product_id": 5,
  "specimen_name": "378-1244",
  "strain": "C57BL/6J",
  "structure_abbrev": "MV",
  "structure_id": 202,
  "structure_name": "Medial vestibular nucleus",
  "primary_injection_structure": 202,
  "data_set_id": 121146455,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "39/48/886/985/993",
  "injection_volume": 0.0993856416,
  "injection_x": 5820,
  "injection_y": 1050,
  "injection_z": 4850,
  "product_id": 36,
  "specimen_name": "Rbp4-Cre_KL100-208502",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 39,
  "data_set_id": 496151571,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "385/879/894",
  "injection_volume": 0.1946487172,
  "injection_x": 9330,
  "injection_y": 1510,
  "injection_z": 3730,
  "product_id": 36,
  "specimen_name": "Emx1-IRES-Cre-212770",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 501115762,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 553502711,
   "name": "Chat-IRES-Cre-neo"
  },
  "gender": "M",
  "injection_structures": "672",
  "injection_volume": 0.0736350489,
  "injection_x": 6050,
  "injection_y": 3360,
  "injection_z": 8360,
  "product_id": 5,
  "specimen_name": "Chat-IRES-Cre-3190",
  "strain": "B6.129",
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 672,
  "data_set_id": 183009881,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "385/409/425",
  "injection_volume": 1.0830299872,
  "injection_x": 9450,
  "injection_y": 2180,
  "injection_z": 8700,
  "product_id": 5,
  "specimen_name": "C57BL/6-155459",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 309004492,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839285,
   "name": "Gpr26-Cre_KO250"
  },
  "gender": "M",
  "injection_structures": "119/961",
  "injection_volume": 0.0549915093,
  "injection_x": 4110,
  "injection_y": 5740,
  "injection_z": 9050,
  "product_id": 5,
  "specimen_name": "Gpr26-Cre_KO250-206290",
  "strain": null,
  "structure_abbrev": "PIR",
  "structure_id": 961,
  "structure_name": "Piriform area",
  "primary_injection_structure": 119,
  "data_set_id": 506117918,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "M",
  "injection_structures": "731/746",
  "injection_volume": 0.0560731892,
  "injection_x": 2970,
  "injection_y": 3540,
  "injection_z": 6370,
  "product_id": 5,
  "specimen_name": "Syt6-Cre_KI148-3660",
  "strain": "C57BL/6J",
  "structure_abbrev": "ORBm",
  "structure_id": 731,
  "structure_name": "Orbital area, medial part",
  "primary_injection_structure": 731,
  "data_set_id": 265820216,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837364,
   "name": "Kiss1-Cre"
  },
  "gender": "F",
  "injection_structures": "126/223",
  "injection_volume": 0.0561096969381164,
  "injection_x": 7160,
  "injection_y": 6910,
  "injection_z": 5980,
  "product_id": 5,
  "specimen_name": "Kiss1-Cre-3499",
  "strain": "C57BL/6J",
  "structure_abbrev": "ARH",
  "structure_id": 223,
  "structure_name": "Arcuate hypothalamic nucleus",
  "primary_injection_structure": 126,
  "data_set_id": 232310521,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837481,
   "name": "Scnn1a-Tg2-Cre"
  },
  "gender": "M",
  "injection_structures": "136/235/395/978/1107",
  "injection_volume": 0.0279516237,
  "injection_x": 12180,
  "injection_y": 7000,
  "injection_z": 6660,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg2-Cre-3490",
  "strain": "C57BL/6J",
  "structure_abbrev": "LRN",
  "structure_id": 235,
  "structure_name": "Lateral reticular nucleus",
  "primary_injection_structure": 136,
  "data_set_id": 263553934,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "533/879/894",
  "injection_volume": 0.00412520924375,
  "injection_x": 8250,
  "injection_y": 610,
  "injection_z": 4490,
  "product_id": 35,
  "specimen_name": "Tlx3-Cre_PL56-319681",
  "strain": "unknown",
  "structure_abbrev": "RSPagl",
  "structure_id": 894,
  "structure_name": "Retrosplenial area, lateral agranular part",
  "primary_injection_structure": 533,
  "data_set_id": 595884140,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838104,
   "name": "Pvalb-T2A-CreERT2"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.139805379,
  "injection_x": 8630,
  "injection_y": 1020,
  "injection_z": 8420,
  "product_id": 5,
  "specimen_name": "Pvalb-T2A-CreERT2-337655",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 637365535,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "F",
  "injection_structures": "993",
  "injection_volume": 0.1313743704,
  "injection_x": 4700,
  "injection_y": 1330,
  "injection_z": 6660,
  "product_id": 5,
  "specimen_name": "Tlx3-Cre_LURC-126302",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 294525944,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 562789797,
   "name": "Esr2-IRES2-Cre"
  },
  "gender": "M",
  "injection_structures": "795/872",
  "injection_volume": 0.0288555071,
  "injection_x": 9770,
  "injection_y": 3940,
  "injection_z": 5690,
  "product_id": 5,
  "specimen_name": "Esr2-IRES2-Cre-305318",
  "strain": null,
  "structure_abbrev": "DR",
  "structure_id": 872,
  "structure_name": "Dorsal nucleus raphe",
  "primary_injection_structure": 795,
  "data_set_id": 582609848,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "F",
  "injection_structures": "39/48/993",
  "injection_volume": 0.2743413176,
  "injection_x": 4450,
  "injection_y": 2600,
  "injection_z": 6580,
  "product_id": 5,
  "specimen_name": "Syt6-Cre-650",
  "strain": "C57BL/6J",
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 159330754,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838877,
   "name": "Gabrr3-Cre_KC112"
  },
  "gender": "M",
  "injection_structures": "194/262/470/685/797/1031",
  "injection_volume": 0.3068691836,
  "injection_x": 6660,
  "injection_y": 5150,
  "injection_z": 7310,
  "product_id": 5,
  "specimen_name": "Gabrr3-Cre-126",
  "strain": null,
  "structure_abbrev": "ZI",
  "structure_id": 797,
  "structure_name": "Zona incerta",
  "primary_injection_structure": 194,
  "data_set_id": 170263370,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 553502711,
   "name": "Chat-IRES-Cre-neo"
  },
  "gender": "M",
  "injection_structures": "226/298/342/523/596/689",
  "injection_volume": 0.015227378425,
  "injection_x": 5100,
  "injection_y": 6780,
  "injection_z": 4740,
  "product_id": 36,
  "specimen_name": "Chat-IRES-Cre-228350",
  "strain": null,
  "structure_abbrev": "NDB",
  "structure_id": 596,
  "structure_name": "Diagonal band nucleus",
  "primary_injection_structure": 226,
  "data_set_id": 504173156,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "337/361",
  "injection_volume": 0.1307809314,
  "injection_x": 6420,
  "injection_y": 1220,
  "injection_z": 7190,
  "product_id": 5,
  "specimen_name": "378-696",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-tr",
  "structure_id": 361,
  "structure_name": "Primary somatosensory area, trunk",
  "primary_injection_structure": 337,
  "data_set_id": 100141495,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "F",
  "injection_structures": "91/372",
  "injection_volume": 0.0447103342,
  "injection_x": 11490,
  "injection_y": 4050,
  "injection_z": 7290,
  "product_id": 5,
  "specimen_name": "Ntsr1-Cre_GN220-3716",
  "strain": "C57BL/6J",
  "structure_abbrev": "IP",
  "structure_id": 91,
  "structure_name": "Interposed nucleus",
  "primary_injection_structure": 91,
  "data_set_id": 264096952,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "258/266/310/452/564",
  "injection_volume": 0.3933468042,
  "injection_x": 5180,
  "injection_y": 4380,
  "injection_z": 5460,
  "product_id": 5,
  "specimen_name": "378-750",
  "strain": "C57BL/6J",
  "structure_abbrev": "MS",
  "structure_id": 564,
  "structure_name": "Medial septal nucleus",
  "primary_injection_structure": 258,
  "data_set_id": 100141597,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 309779993,
   "name": "Ctgf-T2A-dgCre"
  },
  "gender": "M",
  "injection_structures": "329/353/182305689",
  "injection_volume": 0.017415372975,
  "injection_x": 6060,
  "injection_y": 2460,
  "injection_z": 8360,
  "product_id": 5,
  "specimen_name": "Ctgf-2A-dgCre-241848",
  "strain": null,
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 522774776,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "385/894",
  "injection_volume": 0.115881178,
  "injection_x": 9470,
  "injection_y": 1660,
  "injection_z": 3050,
  "product_id": 35,
  "specimen_name": "Rbp4-Cre_KL100-326692",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 603479758,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838953,
   "name": "Oxt-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "390",
  "injection_volume": 0.006437300275,
  "injection_x": 6120,
  "injection_y": 6540,
  "injection_z": 7100,
  "product_id": 5,
  "specimen_name": "Oxt-IRES-Cre-1212",
  "strain": "B6.129",
  "structure_abbrev": "SO",
  "structure_id": 390,
  "structure_name": "Supraoptic nucleus",
  "primary_injection_structure": 390,
  "data_set_id": 147968866,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837777,
   "name": "Sim1-Cre_KJ18"
  },
  "gender": "M",
  "injection_structures": "329/353/182305689",
  "injection_volume": 0.1232772184,
  "injection_x": 5900,
  "injection_y": 2180,
  "injection_z": 8630,
  "product_id": 5,
  "specimen_name": "Sim1-Cre_KJ18-130754",
  "strain": null,
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 298760261,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837324,
   "name": "Slc18a2-Cre_OZ14"
  },
  "gender": "F",
  "injection_structures": "651/773/839",
  "injection_volume": 0.0206684164804687,
  "injection_x": 12650,
  "injection_y": 5400,
  "injection_z": 5980,
  "product_id": 5,
  "specimen_name": "Slc18a2-Cre_OZ14-3821",
  "strain": null,
  "structure_abbrev": "DMX",
  "structure_id": 839,
  "structure_name": "Dorsal motor nucleus of the vagus nerve",
  "primary_injection_structure": 651,
  "data_set_id": 287096578,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 310459452,
   "name": "Slc17a8-IRES2-Cre"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.00706817405,
  "injection_x": 8920,
  "injection_y": 1230,
  "injection_z": 8160,
  "product_id": 5,
  "specimen_name": "Slc17a8-IRES2-Cre-336352",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 623438145,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "59/149/186/362/483",
  "injection_volume": 0.1206430096375,
  "injection_x": 6410,
  "injection_y": 3720,
  "injection_z": 5690,
  "product_id": 5,
  "specimen_name": "378-1591",
  "strain": "C57BL/6J",
  "structure_abbrev": "MD",
  "structure_id": 362,
  "structure_name": "Mediodorsal nucleus of thalamus",
  "primary_injection_structure": 59,
  "data_set_id": 156931568,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838953,
   "name": "Oxt-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "38/63/88",
  "injection_volume": 0.015467804275,
  "injection_x": 6370,
  "injection_y": 5560,
  "injection_z": 5410,
  "product_id": 5,
  "specimen_name": "Oxt-IRES-Cre-1160",
  "strain": "B6.129",
  "structure_abbrev": "PVH",
  "structure_id": 38,
  "structure_name": "Paraventricular hypothalamic nucleus",
  "primary_injection_structure": 38,
  "data_set_id": 112951097,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839406,
   "name": "Nos1-CreERT2"
  },
  "gender": "F",
  "injection_structures": "589",
  "injection_volume": 0.00367129039375,
  "injection_x": 3850,
  "injection_y": 4730,
  "injection_z": 5840,
  "product_id": 5,
  "specimen_name": "Nos1-CreERT2-274955",
  "strain": "C57BL/6J",
  "structure_abbrev": "TT",
  "structure_id": 589,
  "structure_name": "Taenia tecta",
  "primary_injection_structure": 589,
  "data_set_id": 557972433,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "345/353/182305689",
  "injection_volume": 0.0444689455,
  "injection_x": 5180,
  "injection_y": 2240,
  "injection_z": 8860,
  "product_id": 5,
  "specimen_name": "378-935",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-n",
  "structure_id": 353,
  "structure_name": "Primary somatosensory area, nose",
  "primary_injection_structure": 345,
  "data_set_id": 112373124,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839285,
   "name": "Gpr26-Cre_KO250"
  },
  "gender": "F",
  "injection_structures": "985/993",
  "injection_volume": 0.391166216,
  "injection_x": 3430,
  "injection_y": 1690,
  "injection_z": 7360,
  "product_id": 5,
  "specimen_name": "Gpr26-Cre-35",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 181598954,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838715,
   "name": "Tac1-IRES2-Cre"
  },
  "gender": "F",
  "injection_structures": "329/353/182305689",
  "injection_volume": 0.1375164518,
  "injection_x": 5870,
  "injection_y": 2360,
  "injection_z": 8490,
  "product_id": 5,
  "specimen_name": "Tac1-IRES2-Cre-D-47",
  "strain": "B6.129",
  "structure_abbrev": "SSp-n",
  "structure_id": 353,
  "structure_name": "Primary somatosensory area, nose",
  "primary_injection_structure": 329,
  "data_set_id": 181371331,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "409/425",
  "injection_volume": 0.0437566374,
  "injection_x": 9780,
  "injection_y": 2000,
  "injection_z": 9230,
  "product_id": 5,
  "specimen_name": "Tlx3-Cre-125627",
  "strain": null,
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 409,
  "data_set_id": 297593942,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 180924870,
   "name": "Fezf1-T2A-dCre"
  },
  "gender": "F",
  "injection_structures": "390/403",
  "injection_volume": 0.0786781779,
  "injection_x": 6430,
  "injection_y": 6450,
  "injection_z": 7630,
  "product_id": 5,
  "specimen_name": "Fezf1-2A-dCre-D-124063",
  "strain": null,
  "structure_abbrev": "MEA",
  "structure_id": 403,
  "structure_name": "Medial amygdalar nucleus",
  "primary_injection_structure": 390,
  "data_set_id": 309386361,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "184/723/746",
  "injection_volume": 0.4227653752,
  "injection_x": 2570,
  "injection_y": 3810,
  "injection_z": 6810,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre-52",
  "strain": null,
  "structure_abbrev": "ORBl",
  "structure_id": 723,
  "structure_name": "Orbital area, lateral part",
  "primary_injection_structure": 184,
  "data_set_id": 156741826,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837788,
   "name": "Ins2-Cre_25"
  },
  "gender": "M",
  "injection_structures": "223/614/693/830/1004",
  "injection_volume": 0.0309443232,
  "injection_x": 7160,
  "injection_y": 6730,
  "injection_z": 6240,
  "product_id": 5,
  "specimen_name": "Ins2-Cre-139",
  "strain": "C57BL/6J",
  "structure_abbrev": "VMH",
  "structure_id": 693,
  "structure_name": "Ventromedial hypothalamic nucleus",
  "primary_injection_structure": 223,
  "data_set_id": 286319033,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838361,
   "name": "Etv1-CreERT2"
  },
  "gender": "F",
  "injection_structures": "329",
  "injection_volume": 0.1369404666,
  "injection_x": 6210,
  "injection_y": 1980,
  "injection_z": 8950,
  "product_id": 5,
  "specimen_name": "Etv1-CreERT2-3002",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 286772650,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "M",
  "injection_structures": "385/533/879/894",
  "injection_volume": 0.0058722512625,
  "injection_x": 8770,
  "injection_y": 830,
  "injection_z": 4000,
  "product_id": 36,
  "specimen_name": "Scnn1a-Tg3-Cre-195225",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 482640524,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838361,
   "name": "Etv1-CreERT2"
  },
  "gender": "F",
  "injection_structures": "385/394/533/312782546",
  "injection_volume": 0.1508570056,
  "injection_x": 7920,
  "injection_y": 480,
  "injection_z": 7500,
  "product_id": 5,
  "specimen_name": "Etv1-CreERT2-2944",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISpm",
  "structure_id": 533,
  "structure_name": "posteromedial visual area",
  "primary_injection_structure": 385,
  "data_set_id": 182896517,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "402/409/541/1011/1027/312782574",
  "injection_volume": 0.001155453840625,
  "injection_x": 7950,
  "injection_y": 2040,
  "injection_z": 1850,
  "product_id": 36,
  "specimen_name": "A930038C07Rik-Tg1-Cre-241722",
  "strain": null,
  "structure_abbrev": "VISal",
  "structure_id": 402,
  "structure_name": "Anterolateral visual area",
  "primary_injection_structure": 402,
  "data_set_id": 519487627,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838496,
   "name": "Grp-Cre_KH288"
  },
  "gender": "M",
  "injection_structures": "186/362/575/930",
  "injection_volume": 0.01389081155,
  "injection_x": 7180,
  "injection_y": 3510,
  "injection_z": 6390,
  "product_id": 5,
  "specimen_name": "Grp-Cre-63",
  "strain": "B6.FVB(ICR)",
  "structure_abbrev": "MD",
  "structure_id": 362,
  "structure_name": "Mediodorsal nucleus of thalamus",
  "primary_injection_structure": 186,
  "data_set_id": 183058837,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838715,
   "name": "Tac1-IRES2-Cre"
  },
  "gender": "M",
  "injection_structures": "186/362/483/575/726",
  "injection_volume": 0.0044093746375,
  "injection_x": 6460,
  "injection_y": 3160,
  "injection_z": 6130,
  "product_id": 5,
  "specimen_name": "Tac1-IRES2-Cre-209820",
  "strain": null,
  "structure_abbrev": "LH",
  "structure_id": 186,
  "structure_name": "Lateral habenula",
  "primary_injection_structure": 186,
  "data_set_id": 510125001,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838266,
   "name": "Erbb4-T2A-CreERT2"
  },
  "gender": "M",
  "injection_structures": "985/993",
  "injection_volume": 0.4344330592,
  "injection_x": 3470,
  "injection_y": 1950,
  "injection_z": 7410,
  "product_id": 5,
  "specimen_name": "Erbb4-2A-CreERT2-D-5677",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 127908879,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "952/961/966",
  "injection_volume": 0.22726837,
  "injection_x": 6950,
  "injection_y": 5620,
  "injection_z": 9600,
  "product_id": 5,
  "specimen_name": "378-1505",
  "strain": "C57BL/6J",
  "structure_abbrev": "PIR",
  "structure_id": 961,
  "structure_name": "Piriform area",
  "primary_injection_structure": 952,
  "data_set_id": 146857301,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.786978808,
  "injection_x": 8740,
  "injection_y": 1460,
  "injection_z": 7850,
  "product_id": 5,
  "specimen_name": "C57BL/6-155460",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 309113907,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.0316486933,
  "injection_x": 8770,
  "injection_y": 1340,
  "injection_z": 2990,
  "product_id": 36,
  "specimen_name": "Tlx3-Cre_PL56-226276",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 502140653,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838494,
   "name": "Prkcd-GluCla-CFP-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "423/463/726",
  "injection_volume": 0.01930587505,
  "injection_x": 6980,
  "injection_y": 2670,
  "injection_z": 7450,
  "product_id": 5,
  "specimen_name": "Prkcd-GluCla-CFP-IRES-Cre-46",
  "strain": null,
  "structure_abbrev": "CA3",
  "structure_id": 463,
  "structure_name": "Field CA3",
  "primary_injection_structure": 423,
  "data_set_id": 171274191,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.1487839724,
  "injection_x": 8770,
  "injection_y": 1510,
  "injection_z": 3370,
  "product_id": 35,
  "specimen_name": "Emx1-IRES-Cre-255366",
  "strain": "B6.129",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 536299435,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838266,
   "name": "Erbb4-T2A-CreERT2"
  },
  "gender": "F",
  "injection_structures": "536",
  "injection_volume": 0.0892765398,
  "injection_x": 6480,
  "injection_y": 5510,
  "injection_z": 8170,
  "product_id": 5,
  "specimen_name": "Erbb4-2A-CreERT2-D-5671",
  "strain": "C57BL/6J",
  "structure_abbrev": "CEA",
  "structure_id": 536,
  "structure_name": "Central amygdalar nucleus",
  "primary_injection_structure": 536,
  "data_set_id": 120281646,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.0474254977,
  "injection_x": 9190,
  "injection_y": 1140,
  "injection_z": 8370,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-3765",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 158255941,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "345/353/672",
  "injection_volume": 0.0807301607,
  "injection_x": 5640,
  "injection_y": 3750,
  "injection_z": 8990,
  "product_id": 5,
  "specimen_name": "378-1778",
  "strain": "C57BL/6J",
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 345,
  "data_set_id": 175072215,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839331,
   "name": "Nr5a1-Cre"
  },
  "gender": "M",
  "injection_structures": "385/409",
  "injection_volume": 0.0272920935,
  "injection_x": 8960,
  "injection_y": 1520,
  "injection_z": 9170,
  "product_id": 5,
  "specimen_name": "Nr5a1-Cre-2522",
  "strain": "B6.FVB",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 266177248,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838953,
   "name": "Oxt-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "38/63/347",
  "injection_volume": 0.0093178749125,
  "injection_x": 6430,
  "injection_y": 5780,
  "injection_z": 6030,
  "product_id": 5,
  "specimen_name": "Oxt-IRES-Cre-1192",
  "strain": "B6.129",
  "structure_abbrev": "PVH",
  "structure_id": 38,
  "structure_name": "Paraventricular hypothalamic nucleus",
  "primary_injection_structure": 38,
  "data_set_id": 146983504,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "918/926",
  "injection_volume": 0.02333017155,
  "injection_x": 9940,
  "injection_y": 3540,
  "injection_z": 10130,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-281",
  "strain": "C57BL/6J",
  "structure_abbrev": "ENTl",
  "structure_id": 918,
  "structure_name": "Entorhinal area, lateral part",
  "primary_injection_structure": 918,
  "data_set_id": 272779830,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838633,
   "name": "Trib2-F2A-CreERT2"
  },
  "gender": "F",
  "injection_structures": "39/993",
  "injection_volume": 0.1392685742,
  "injection_x": 4910,
  "injection_y": 1810,
  "injection_z": 5970,
  "product_id": 5,
  "specimen_name": "Trib2-2A-CreERT2-D-3285",
  "strain": "B6.129",
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 287599685,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182693192,
   "name": "Chrna2-Cre_OE25"
  },
  "gender": "F",
  "injection_structures": "353/182305689",
  "injection_volume": 0.0426892067,
  "injection_x": 5730,
  "injection_y": 2230,
  "injection_z": 8570,
  "product_id": 5,
  "specimen_name": "Chrna2-Cre_OE25-3868",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "SSp-n",
  "structure_id": 353,
  "structure_name": "Primary somatosensory area, nose",
  "primary_injection_structure": 353,
  "data_set_id": 267749821,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838361,
   "name": "Etv1-CreERT2"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.1190322406,
  "injection_x": 9680,
  "injection_y": 1220,
  "injection_z": 8450,
  "product_id": 5,
  "specimen_name": "Etv1-CreERT2-1177",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 156494530,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "329/378",
  "injection_volume": 0.1600087062,
  "injection_x": 7410,
  "injection_y": 1740,
  "injection_z": 1870,
  "product_id": 35,
  "specimen_name": "Cux2-IRES-Cre-300266",
  "strain": null,
  "structure_abbrev": "SSs",
  "structure_id": 378,
  "structure_name": "Supplemental somatosensory area",
  "primary_injection_structure": 329,
  "data_set_id": 577773267,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839285,
   "name": "Gpr26-Cre_KO250"
  },
  "gender": "M",
  "injection_structures": "91/217",
  "injection_volume": 0.015996666175,
  "injection_x": 11400,
  "injection_y": 3910,
  "injection_z": 7440,
  "product_id": 5,
  "specimen_name": "Gpr26-Cre_KO250-91",
  "strain": null,
  "structure_abbrev": "IP",
  "structure_id": 91,
  "structure_name": "Interposed nucleus",
  "primary_injection_structure": 91,
  "data_set_id": 265928489,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "993",
  "injection_volume": 0.01385575695,
  "injection_x": 4070,
  "injection_y": 2480,
  "injection_z": 7000,
  "product_id": 5,
  "specimen_name": "Ntsr1-Cre-3487",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 120584036,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "382",
  "injection_volume": 0.0936029752,
  "injection_x": 7460,
  "injection_y": 1400,
  "injection_z": 6970,
  "product_id": 5,
  "specimen_name": "378-1210",
  "strain": "C57BL/6J",
  "structure_abbrev": "CA1",
  "structure_id": 382,
  "structure_name": "Field CA1",
  "primary_injection_structure": 382,
  "data_set_id": 120436274,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838633,
   "name": "Trib2-F2A-CreERT2"
  },
  "gender": "M",
  "injection_structures": "329/353/182305689",
  "injection_volume": 0.2629120676,
  "injection_x": 5940,
  "injection_y": 2380,
  "injection_z": 8410,
  "product_id": 5,
  "specimen_name": "Trib2-2A-CreERT2-D-3235",
  "strain": "B6.129",
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 266172624,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "385/409/425/312782574/312782628",
  "injection_volume": 0.274146964,
  "injection_x": 9210,
  "injection_y": 2310,
  "injection_z": 2370,
  "product_id": 35,
  "specimen_name": "Cux2-IRES-Cre-309605",
  "strain": null,
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 385,
  "data_set_id": 587344810,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "385/394/417/533/312782546",
  "injection_volume": 0.141207955,
  "injection_x": 7900,
  "injection_y": 750,
  "injection_z": 3450,
  "product_id": 35,
  "specimen_name": "Emx1-IRES-Cre-363520",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 657042668,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838681,
   "name": "Slc6a4-Cre_ET33"
  },
  "gender": "M",
  "injection_structures": "83/222/230",
  "injection_volume": 0.02401052675,
  "injection_x": 11720,
  "injection_y": 6760,
  "injection_z": 5680,
  "product_id": 5,
  "specimen_name": "Slc6a4-Cre-Homo-282",
  "strain": "C57BL/6J",
  "structure_abbrev": "RO",
  "structure_id": 222,
  "structure_name": "Nucleus raphe obscurus",
  "primary_injection_structure": 83,
  "data_set_id": 141603895,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "F",
  "injection_structures": "385/425",
  "injection_volume": 0.0017362298625,
  "injection_x": 9870,
  "injection_y": 2170,
  "injection_z": 2900,
  "product_id": 35,
  "specimen_name": "Ntsr1-Cre_GN220-353478",
  "strain": null,
  "structure_abbrev": "VISpl",
  "structure_id": 425,
  "structure_name": "Posterolateral visual area",
  "primary_injection_structure": 385,
  "data_set_id": 649365810,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "39/48/985/993",
  "injection_volume": 0.3355186408,
  "injection_x": 5640,
  "injection_y": 1140,
  "injection_z": 5400,
  "product_id": 36,
  "specimen_name": "Cux2-IRES-Cre-207007",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 39,
  "data_set_id": 496150781,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182901722,
   "name": "Ppp1r17-Cre_NL146"
  },
  "gender": "F",
  "injection_structures": "111/131/952",
  "injection_volume": 0.02282163975,
  "injection_x": 6810,
  "injection_y": 5290,
  "injection_z": 9230,
  "product_id": 5,
  "specimen_name": "Ppp1r17-Cre_NL146-3942",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "LA",
  "structure_id": 131,
  "structure_name": "Lateral amygdalar nucleus",
  "primary_injection_structure": 111,
  "data_set_id": 287027773,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761781,
   "name": "Efr3a-Cre_NO108"
  },
  "gender": "F",
  "injection_structures": "39/993",
  "injection_volume": 0.0657039922,
  "injection_x": 4550,
  "injection_y": 1750,
  "injection_z": 6100,
  "product_id": 5,
  "specimen_name": "Efr3a-Cre_NO108-40",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 277908168,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 305451902,
   "name": "Dlg3-Cre_KG118"
  },
  "gender": "M",
  "injection_structures": "146/206/574/931/1093",
  "injection_volume": 0.0645768125520752,
  "injection_x": 9730,
  "injection_y": 6100,
  "injection_z": 6210,
  "product_id": 5,
  "specimen_name": "Dlg3-Cre_KG118-180917",
  "strain": "B6.FVB",
  "structure_abbrev": "TRN",
  "structure_id": 574,
  "structure_name": "Tegmental reticular nucleus",
  "primary_injection_structure": 146,
  "data_set_id": 478819500,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 179697864,
   "name": "Rorb-IRES2-Cre"
  },
  "gender": "F",
  "injection_structures": "294/302",
  "injection_volume": 0.0401958662,
  "injection_x": 8810,
  "injection_y": 1560,
  "injection_z": 6390,
  "product_id": 5,
  "specimen_name": "Rorb-IRES2-Cre-D-207238",
  "strain": null,
  "structure_abbrev": "SCs",
  "structure_id": 302,
  "structure_name": "Superior colliculus, sensory related",
  "primary_injection_structure": 294,
  "data_set_id": 508357710,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838074,
   "name": "Slc6a3-Cre"
  },
  "gender": "M",
  "injection_structures": "151/507",
  "injection_volume": 0.0289490089,
  "injection_x": 960,
  "injection_y": 3490,
  "injection_z": 6450,
  "product_id": 5,
  "specimen_name": "Slc6a3-Cre-4502",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOB",
  "structure_id": 507,
  "structure_name": "Main olfactory bulb",
  "primary_injection_structure": 151,
  "data_set_id": 166156241,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "M",
  "injection_structures": "993",
  "injection_volume": 0.1057052108,
  "injection_x": 3210,
  "injection_y": 3020,
  "injection_z": 7480,
  "product_id": 5,
  "specimen_name": "Syt6-Cre_KI148-3823",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 287633753,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838266,
   "name": "Erbb4-T2A-CreERT2"
  },
  "gender": "M",
  "injection_structures": "63/88/118/173/223/614/693/830",
  "injection_volume": 0.44969681798125,
  "injection_x": 6640,
  "injection_y": 6420,
  "injection_z": 6330,
  "product_id": 5,
  "specimen_name": "Erbb4-2A-CreERT2-D-5768",
  "strain": "C57BL/6J",
  "structure_abbrev": "VMH",
  "structure_id": 693,
  "structure_name": "Ventromedial hypothalamic nucleus",
  "primary_injection_structure": 63,
  "data_set_id": 277854208,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "502/541/312782574/312782628/484682470",
  "injection_volume": 0.3599235808,
  "injection_x": 9070,
  "injection_y": 2210,
  "injection_z": 9650,
  "product_id": 5,
  "specimen_name": "378-1406",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISpor",
  "structure_id": 312782628,
  "structure_name": "Postrhinal area",
  "primary_injection_structure": 502,
  "data_set_id": 127089669,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "F",
  "injection_structures": "369/985",
  "injection_volume": 0.1538548746,
  "injection_x": 4560,
  "injection_y": 1820,
  "injection_z": 7930,
  "product_id": 5,
  "specimen_name": "Syt6-Cre-3167",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOp",
  "structure_id": 985,
  "structure_name": "Primary motor area",
  "primary_injection_structure": 369,
  "data_set_id": 179641666,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837328,
   "name": "Syt17-Cre_NO14"
  },
  "gender": "F",
  "injection_structures": "926",
  "injection_volume": 0.103145147,
  "injection_x": 9880,
  "injection_y": 3720,
  "injection_z": 9220,
  "product_id": 5,
  "specimen_name": "Syt17-Cre_NO14-193",
  "strain": "C57BL/6J",
  "structure_abbrev": "ENTm",
  "structure_id": 926,
  "structure_name": "Entorhinal area, medial part, dorsal zone",
  "primary_injection_structure": 926,
  "data_set_id": 286484879,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838803,
   "name": "Grm2-Cre_MR90"
  },
  "gender": "M",
  "injection_structures": "895/918/922",
  "injection_volume": 0.5587466864,
  "injection_x": 8580,
  "injection_y": 5340,
  "injection_z": 10410,
  "product_id": 5,
  "specimen_name": "Grm2-Cre-108",
  "strain": null,
  "structure_abbrev": "ENTl",
  "structure_id": 918,
  "structure_name": "Entorhinal area, lateral part",
  "primary_injection_structure": 895,
  "data_set_id": 183225124,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839331,
   "name": "Nr5a1-Cre"
  },
  "gender": "M",
  "injection_structures": "369/985",
  "injection_volume": 0.1037905358,
  "injection_x": 4580,
  "injection_y": 2010,
  "injection_z": 7980,
  "product_id": 5,
  "specimen_name": "Nr5a1-Cre-2461",
  "strain": "B6.FVB",
  "structure_abbrev": "MOp",
  "structure_id": 985,
  "structure_name": "Primary motor area",
  "primary_injection_structure": 369,
  "data_set_id": 177893658,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761918,
   "name": "Oxtr-Cre_ON66"
  },
  "gender": "F",
  "injection_structures": "463/726",
  "injection_volume": 0.0645754634,
  "injection_x": 6490,
  "injection_y": 2600,
  "injection_z": 6360,
  "product_id": 5,
  "specimen_name": "Oxtr-Cre_ON66-124763",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "DG",
  "structure_id": 726,
  "structure_name": "Dentate gyrus",
  "primary_injection_structure": 463,
  "data_set_id": 301422682,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "1002/1011/1018/1027",
  "injection_volume": 0.1315973792,
  "injection_x": 8300,
  "injection_y": 3460,
  "injection_z": 10150,
  "product_id": 5,
  "specimen_name": "378-1074",
  "strain": "C57BL/6J",
  "structure_abbrev": "AUDv",
  "structure_id": 1018,
  "structure_name": "Ventral auditory area",
  "primary_injection_structure": 1002,
  "data_set_id": 112881858,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0524025845,
  "injection_x": 8250,
  "injection_y": 730,
  "injection_z": 8300,
  "product_id": 5,
  "specimen_name": "378-1924",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 277713580,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838502,
   "name": "Grik4-Cre"
  },
  "gender": "M",
  "injection_structures": "382/423/463",
  "injection_volume": 0.0487571266,
  "injection_x": 7500,
  "injection_y": 2670,
  "injection_z": 8830,
  "product_id": 5,
  "specimen_name": "Grik4-Cre-5065",
  "strain": "C57BL/6J",
  "structure_abbrev": "CA3",
  "structure_id": 463,
  "structure_name": "Field CA3",
  "primary_injection_structure": 382,
  "data_set_id": 179643824,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.2226931812,
  "injection_x": 8550,
  "injection_y": 1460,
  "injection_z": 3350,
  "product_id": 36,
  "specimen_name": "Emx1-IRES-Cre-191136",
  "strain": "B6.129",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 479983421,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "131/295/536",
  "injection_volume": 0.1665989612,
  "injection_x": 7360,
  "injection_y": 4990,
  "injection_z": 8640,
  "product_id": 5,
  "specimen_name": "378-950",
  "strain": "C57BL/6J",
  "structure_abbrev": "CEA",
  "structure_id": 536,
  "structure_name": "Central amygdalar nucleus",
  "primary_injection_structure": 131,
  "data_set_id": 112459547,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "M",
  "injection_structures": "345/1057",
  "injection_volume": 0.118740426,
  "injection_x": 4070,
  "injection_y": 3070,
  "injection_z": 8970,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg3-Cre-122367",
  "strain": "B6.C3H",
  "structure_abbrev": "SSp-m",
  "structure_id": 345,
  "structure_name": "Primary somatosensory area, mouth",
  "primary_injection_structure": 345,
  "data_set_id": 299447873,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "385/879/894",
  "injection_volume": 0.0640483165,
  "injection_x": 9710,
  "injection_y": 1710,
  "injection_z": 3870,
  "product_id": 35,
  "specimen_name": "Tlx3-Cre_PL56-273877",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 554641575,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 298341195,
   "name": "Htr3a-Cre_NO152"
  },
  "gender": "M",
  "injection_structures": "128/162/460/795/867/1052",
  "injection_volume": 0.0428554539,
  "injection_x": 10310,
  "injection_y": 3820,
  "injection_z": 6550,
  "product_id": 5,
  "specimen_name": "Htr3a-Cre_NO152-158483",
  "strain": null,
  "structure_abbrev": "PB",
  "structure_id": 867,
  "structure_name": "Parabrachial nucleus",
  "primary_injection_structure": 128,
  "data_set_id": 311845972,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "39/48/993",
  "injection_volume": 0.010451613025,
  "injection_x": 5350,
  "injection_y": 1830,
  "injection_z": 6370,
  "product_id": 5,
  "specimen_name": "Ntsr1-Cre-3666",
  "strain": "C57BL/6J",
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 181179335,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "726",
  "injection_volume": 0.03787178395,
  "injection_x": 8050,
  "injection_y": 2270,
  "injection_z": 7330,
  "product_id": 5,
  "specimen_name": "378-914",
  "strain": "C57BL/6J",
  "structure_abbrev": "DG",
  "structure_id": 726,
  "structure_name": "Dentate gyrus",
  "primary_injection_structure": 726,
  "data_set_id": 112167395,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838584,
   "name": "Gad2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "262/629/718/1022",
  "injection_volume": 0.1154459894,
  "injection_x": 6240,
  "injection_y": 4430,
  "injection_z": 7460,
  "product_id": 5,
  "specimen_name": "Gad2-IRES-Cre-1071",
  "strain": "C57BL/6J",
  "structure_abbrev": "RT",
  "structure_id": 262,
  "structure_name": "Reticular nucleus of the thalamus",
  "primary_injection_structure": 262,
  "data_set_id": 156252954,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838584,
   "name": "Gad2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "920/928",
  "injection_volume": 0.076363511,
  "injection_x": 11520,
  "injection_y": 2940,
  "injection_z": 6390,
  "product_id": 5,
  "specimen_name": "Gad2-IRES-Cre-3882",
  "strain": null,
  "structure_abbrev": "CUL",
  "structure_id": 928,
  "structure_name": "Culmen",
  "primary_injection_structure": 920,
  "data_set_id": 287809155,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 183941572,
   "name": "Plxnd1-Cre_OG1"
  },
  "gender": "F",
  "injection_structures": "329/353/182305689",
  "injection_volume": 0.093131703,
  "injection_x": 5920,
  "injection_y": 2210,
  "injection_z": 8600,
  "product_id": 5,
  "specimen_name": "Plxnd1-Cre-127306",
  "strain": null,
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 297668898,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 179697864,
   "name": "Rorb-IRES2-Cre"
  },
  "gender": "M",
  "injection_structures": "394/533",
  "injection_volume": 0.02041358865,
  "injection_x": 7970,
  "injection_y": 630,
  "injection_z": 7400,
  "product_id": 5,
  "specimen_name": "Rorb-IRES2-Cre-D-692",
  "strain": null,
  "structure_abbrev": "VISpm",
  "structure_id": 533,
  "structure_name": "posteromedial visual area",
  "primary_injection_structure": 394,
  "data_set_id": 287225041,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837858,
   "name": "Pvalb-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "374/381",
  "injection_volume": 0.3600297736,
  "injection_x": 8910,
  "injection_y": 5280,
  "injection_z": 7360,
  "product_id": 5,
  "specimen_name": "Pvalb-IRES-Cre-132807",
  "strain": "B6.129",
  "structure_abbrev": "SNr",
  "structure_id": 381,
  "structure_name": "Substantia nigra, reticular part",
  "primary_injection_structure": 374,
  "data_set_id": 299895444,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837328,
   "name": "Syt17-Cre_NO14"
  },
  "gender": "F",
  "injection_structures": "382/484682470",
  "injection_volume": 0.095978897,
  "injection_x": 8950,
  "injection_y": 5300,
  "injection_z": 9270,
  "product_id": 5,
  "specimen_name": "Syt17-Cre_NO14-200",
  "strain": "C57BL/6J",
  "structure_abbrev": "CA1",
  "structure_id": 382,
  "structure_name": "Field CA1",
  "primary_injection_structure": 382,
  "data_set_id": 286610216,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839406,
   "name": "Nos1-CreERT2"
  },
  "gender": "M",
  "injection_structures": "307/398/642/978/1048/1069/1093",
  "injection_volume": 0.013144057675,
  "injection_x": 10090,
  "injection_y": 6830,
  "injection_z": 6150,
  "product_id": 5,
  "specimen_name": "Nos1-CreERT2-285465",
  "strain": null,
  "structure_abbrev": "NTB",
  "structure_id": 642,
  "structure_name": "Nucleus of the trapezoid body",
  "primary_injection_structure": 307,
  "data_set_id": 574572418,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.2109193436,
  "injection_x": 8290,
  "injection_y": 1340,
  "injection_z": 3540,
  "product_id": 36,
  "specimen_name": "Emx1-IRES-Cre-212541",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 501004474,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "128/294",
  "injection_volume": 0.1789335548,
  "injection_x": 8780,
  "injection_y": 3420,
  "injection_z": 7190,
  "product_id": 5,
  "specimen_name": "378-1115",
  "strain": "C57BL/6J",
  "structure_abbrev": "MRN",
  "structure_id": 128,
  "structure_name": "Midbrain reticular nucleus",
  "primary_injection_structure": 128,
  "data_set_id": 113399428,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837315,
   "name": "Avp-IRES2-Cre"
  },
  "gender": "F",
  "injection_structures": "390/403",
  "injection_volume": 0.01817788525,
  "injection_x": 6120,
  "injection_y": 6740,
  "injection_z": 7180,
  "product_id": 5,
  "specimen_name": "Avp-IRES2-Cre-D-124978",
  "strain": "B6.129",
  "structure_abbrev": "SO",
  "structure_id": 390,
  "structure_name": "Supraoptic nucleus",
  "primary_injection_structure": 390,
  "data_set_id": 587294457,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "136/177/395/651/773/839/1098/1107",
  "injection_volume": 0.3969215404,
  "injection_x": 13000,
  "injection_y": 5840,
  "injection_z": 5690,
  "product_id": 5,
  "specimen_name": "378-1297",
  "strain": "C57BL/6J",
  "structure_abbrev": "XII",
  "structure_id": 773,
  "structure_name": "Hypoglossal nucleus",
  "primary_injection_structure": 136,
  "data_set_id": 127350480,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "385/409/312782574",
  "injection_volume": 0.5285815808,
  "injection_x": 9270,
  "injection_y": 2310,
  "injection_z": 9350,
  "product_id": 5,
  "specimen_name": "C57BL/6-152140",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 385,
  "data_set_id": 307295727,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.0700263508,
  "injection_x": 8330,
  "injection_y": 1400,
  "injection_z": 3400,
  "product_id": 35,
  "specimen_name": "Emx1-IRES-Cre-282583",
  "strain": "unknown",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 562061175,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "928/936",
  "injection_volume": 0.0541615277,
  "injection_x": 11680,
  "injection_y": 1630,
  "injection_z": 6710,
  "product_id": 5,
  "specimen_name": "378-948",
  "strain": "C57BL/6J",
  "structure_abbrev": "CUL",
  "structure_id": 928,
  "structure_name": "Culmen",
  "primary_injection_structure": 928,
  "data_set_id": 112424102,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "918",
  "injection_volume": 0.3379054896,
  "injection_x": 9530,
  "injection_y": 5390,
  "injection_z": 10100,
  "product_id": 5,
  "specimen_name": "378-1270",
  "strain": "C57BL/6J",
  "structure_abbrev": "ENTl",
  "structure_id": 918,
  "structure_name": "Entorhinal area, lateral part",
  "primary_injection_structure": 918,
  "data_set_id": 126116848,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "672",
  "injection_volume": 0.1059788072,
  "injection_x": 5170,
  "injection_y": 2980,
  "injection_z": 6960,
  "product_id": 5,
  "specimen_name": "378-962",
  "strain": "C57BL/6J",
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 672,
  "data_set_id": 112458831,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839285,
   "name": "Gpr26-Cre_KO250"
  },
  "gender": "M",
  "injection_structures": "1002/1011",
  "injection_volume": 0.0565299966,
  "injection_x": 7660,
  "injection_y": 2870,
  "injection_z": 9780,
  "product_id": 5,
  "specimen_name": "Gpr26-Cre-40",
  "strain": null,
  "structure_abbrev": "AUDp",
  "structure_id": 1002,
  "structure_name": "Primary auditory area",
  "primary_injection_structure": 1002,
  "data_set_id": 181600380,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "M",
  "injection_structures": "378",
  "injection_volume": 0.0323763776,
  "injection_x": 5970,
  "injection_y": 3500,
  "injection_z": 9970,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg3-Cre-4867",
  "strain": "B6.C3H",
  "structure_abbrev": "SSs",
  "structure_id": 378,
  "structure_name": "Supplemental somatosensory area",
  "primary_injection_structure": 378,
  "data_set_id": 257636467,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838942,
   "name": "Htr2a-Cre_KM207"
  },
  "gender": "F",
  "injection_structures": "39/972/993",
  "injection_volume": 0.2764553344,
  "injection_x": 3000,
  "injection_y": 2280,
  "injection_z": 6250,
  "product_id": 5,
  "specimen_name": "Htr2a-Cre-3440",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 39,
  "data_set_id": 183618139,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "F",
  "injection_structures": "48",
  "injection_volume": 0.0773285121,
  "injection_x": 5350,
  "injection_y": 2370,
  "injection_z": 6360,
  "product_id": 5,
  "specimen_name": "Syt6-Cre-590",
  "strain": "C57BL/6J",
  "structure_abbrev": "ACAv",
  "structure_id": 48,
  "structure_name": "Anterior cingulate area, ventral part",
  "primary_injection_structure": 48,
  "data_set_id": 126190033,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "F",
  "injection_structures": "409/425/541/312782628",
  "injection_volume": 0.0578966262,
  "injection_x": 9360,
  "injection_y": 2460,
  "injection_z": 1780,
  "product_id": 35,
  "specimen_name": "Tlx3-Cre_PL56-350986",
  "strain": null,
  "structure_abbrev": "VISpor",
  "structure_id": 312782628,
  "structure_name": "Postrhinal area",
  "primary_injection_structure": 409,
  "data_set_id": 646214807,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 180643392,
   "name": "Rasgrf2-T2A-dCre"
  },
  "gender": "F",
  "injection_structures": "39/48/993",
  "injection_volume": 0.180340678,
  "injection_x": 5250,
  "injection_y": 1900,
  "injection_z": 6160,
  "product_id": 5,
  "specimen_name": "Rasgrf2-2A-dCre-D-136965",
  "strain": null,
  "structure_abbrev": "ACAv",
  "structure_id": 48,
  "structure_name": "Anterior cingulate area, ventral part",
  "primary_injection_structure": 39,
  "data_set_id": 303783015,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "541/312782574/312782628",
  "injection_volume": 0.3671743656,
  "injection_x": 9240,
  "injection_y": 2820,
  "injection_z": 1270,
  "product_id": 36,
  "specimen_name": "Cux2-IRES-Cre-314799",
  "strain": null,
  "structure_abbrev": "VISpor",
  "structure_id": 312782628,
  "structure_name": "Postrhinal area",
  "primary_injection_structure": 541,
  "data_set_id": 592392441,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838266,
   "name": "Erbb4-T2A-CreERT2"
  },
  "gender": "M",
  "injection_structures": "329/353",
  "injection_volume": 0.130937898,
  "injection_x": 6100,
  "injection_y": 2180,
  "injection_z": 8860,
  "product_id": 5,
  "specimen_name": "Erbb4-2A-CreERT2-D-5678",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-n",
  "structure_id": 353,
  "structure_name": "Primary somatosensory area, nose",
  "primary_injection_structure": 329,
  "data_set_id": 127993375,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "345/353/378",
  "injection_volume": 0.5052837664,
  "injection_x": 5880,
  "injection_y": 3480,
  "injection_z": 9160,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-3805",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSs",
  "structure_id": 378,
  "structure_name": "Supplemental somatosensory area",
  "primary_injection_structure": 345,
  "data_set_id": 287951098,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 310459452,
   "name": "Slc17a8-IRES2-Cre"
  },
  "gender": "F",
  "injection_structures": "993",
  "injection_volume": 0.01094571555,
  "injection_x": 3330,
  "injection_y": 2340,
  "injection_z": 6940,
  "product_id": 5,
  "specimen_name": "Slc17a8-IRES2-Cre-292469",
  "strain": "unknown",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 569264117,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839331,
   "name": "Nr5a1-Cre"
  },
  "gender": "M",
  "injection_structures": "312782574/312782628",
  "injection_volume": 0.013264614825,
  "injection_x": 9360,
  "injection_y": 2250,
  "injection_z": 9600,
  "product_id": 5,
  "specimen_name": "Nr5a1-Cre-2544",
  "strain": "B6.FVB",
  "structure_abbrev": "VISli",
  "structure_id": 312782574,
  "structure_name": "Laterointermediate area",
  "primary_injection_structure": 312782574,
  "data_set_id": 286554724,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "M",
  "injection_structures": "1002/1011",
  "injection_volume": 0.03397580865,
  "injection_x": 7780,
  "injection_y": 2800,
  "injection_z": 9660,
  "product_id": 5,
  "specimen_name": "Syt6-Cre-587",
  "strain": "C57BL/6J",
  "structure_abbrev": "AUDp",
  "structure_id": 1002,
  "structure_name": "Primary auditory area",
  "primary_injection_structure": 1002,
  "data_set_id": 124060405,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837446,
   "name": "Adcyap1-2A-Cre"
  },
  "gender": "M",
  "injection_structures": "136/169/202/651/970/1048",
  "injection_volume": 0.0490859852,
  "injection_x": 11630,
  "injection_y": 5250,
  "injection_z": 6400,
  "product_id": 5,
  "specimen_name": "Adcyap1-2A-Cre-D-131314",
  "strain": "B6.129",
  "structure_abbrev": "MV",
  "structure_id": 202,
  "structure_name": "Medial vestibular nucleus",
  "primary_injection_structure": 136,
  "data_set_id": 558675974,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839090,
   "name": "Slc32a1-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "672/1022",
  "injection_volume": 0.2204553708,
  "injection_x": 6430,
  "injection_y": 4240,
  "injection_z": 8250,
  "product_id": 5,
  "specimen_name": "Slc32a1-IRES-Cre-2909",
  "strain": "B6.129",
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 672,
  "data_set_id": 264095536,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "M",
  "injection_structures": "337/369/985",
  "injection_volume": 0.15002526,
  "injection_x": 5520,
  "injection_y": 1590,
  "injection_z": 7520,
  "product_id": 5,
  "specimen_name": "Syt6-Cre-609",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOp",
  "structure_id": 985,
  "structure_name": "Primary motor area",
  "primary_injection_structure": 337,
  "data_set_id": 156786234,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "345/353",
  "injection_volume": 0.146688164,
  "injection_x": 5030,
  "injection_y": 2270,
  "injection_z": 8790,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-300",
  "strain": "B6.129",
  "structure_abbrev": "SSp-m",
  "structure_id": 345,
  "structure_name": "Primary somatosensory area, mouth",
  "primary_injection_structure": 345,
  "data_set_id": 278317945,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838622,
   "name": "Vip-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "952/961/966",
  "injection_volume": 0.0219160879,
  "injection_x": 6280,
  "injection_y": 6350,
  "injection_z": 9470,
  "product_id": 5,
  "specimen_name": "Vip-IRES-Cre-129391",
  "strain": "C57BL/6J",
  "structure_abbrev": "PIR",
  "structure_id": 961,
  "structure_name": "Piriform area",
  "primary_injection_structure": 952,
  "data_set_id": 299897573,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839135,
   "name": "Kcnc2-Cre"
  },
  "gender": "M",
  "injection_structures": "382/726",
  "injection_volume": 0.3196681404,
  "injection_x": 7550,
  "injection_y": 1610,
  "injection_z": 8090,
  "product_id": 5,
  "specimen_name": "Kcnc2-Cre-3469",
  "strain": "C57BL/6J",
  "structure_abbrev": "CA1",
  "structure_id": 382,
  "structure_name": "Field CA1",
  "primary_injection_structure": 382,
  "data_set_id": 182843806,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "385/409",
  "injection_volume": 0.0358146635,
  "injection_x": 9420,
  "injection_y": 1800,
  "injection_z": 8820,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-3957",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 292793967,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "382",
  "injection_volume": 0.012129723375,
  "injection_x": 7480,
  "injection_y": 1860,
  "injection_z": 8270,
  "product_id": 5,
  "specimen_name": "378-1213",
  "strain": "C57BL/6J",
  "structure_abbrev": "CA1",
  "structure_id": 382,
  "structure_name": "Field CA1",
  "primary_injection_structure": 382,
  "data_set_id": 120494729,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838266,
   "name": "Erbb4-T2A-CreERT2"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0785351322,
  "injection_x": 9950,
  "injection_y": 1580,
  "injection_z": 8630,
  "product_id": 5,
  "specimen_name": "Erbb4-2A-CreERT2-D-5751",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 182184486,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 301614880,
   "name": "Glt25d2-Cre_NF107"
  },
  "gender": "M",
  "injection_structures": "39/48/972/993",
  "injection_volume": 0.16882362,
  "injection_x": 4050,
  "injection_y": 2060,
  "injection_z": 5410,
  "product_id": 5,
  "specimen_name": "Glt25d2-Cre_NF107-200977",
  "strain": "unknown",
  "structure_abbrev": "ACAv",
  "structure_id": 48,
  "structure_name": "Anterior cingulate area, ventral part",
  "primary_injection_structure": 39,
  "data_set_id": 582814018,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 183238310,
   "name": "Ntrk1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "342/1022/1031",
  "injection_volume": 0.0553828184,
  "injection_x": 5880,
  "injection_y": 4950,
  "injection_z": 4140,
  "product_id": 35,
  "specimen_name": "Ntrk1-IRES-Cre-352538",
  "strain": null,
  "structure_abbrev": "GPe",
  "structure_id": 1022,
  "structure_name": "Globus pallidus, external segment",
  "primary_injection_structure": 342,
  "data_set_id": 647574144,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837777,
   "name": "Sim1-Cre_KJ18"
  },
  "gender": "M",
  "injection_structures": "378/677",
  "injection_volume": 0.0044497096,
  "injection_x": 6190,
  "injection_y": 4230,
  "injection_z": 10090,
  "product_id": 5,
  "specimen_name": "Sim1-Cre_KJ18-207258",
  "strain": null,
  "structure_abbrev": "VISC",
  "structure_id": 677,
  "structure_name": "Visceral area",
  "primary_injection_structure": 378,
  "data_set_id": 508373923,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761781,
   "name": "Efr3a-Cre_NO108"
  },
  "gender": "F",
  "injection_structures": "39/993",
  "injection_volume": 0.0411791492,
  "injection_x": 4830,
  "injection_y": 1660,
  "injection_z": 6170,
  "product_id": 5,
  "specimen_name": "Efr3a-Cre_NO108-24",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 267705378,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837315,
   "name": "Avp-IRES2-Cre"
  },
  "gender": "F",
  "injection_structures": "30/88/118/173/223/286/347/614/693",
  "injection_volume": 0.027186694346875,
  "injection_x": 6020,
  "injection_y": 6950,
  "injection_z": 5830,
  "product_id": 5,
  "specimen_name": "Avp-IRES2-Cre-D-3560",
  "strain": "B6.129",
  "structure_abbrev": "SCH",
  "structure_id": 286,
  "structure_name": "Suprachiasmatic nucleus",
  "primary_injection_structure": 30,
  "data_set_id": 287665706,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "88/693",
  "injection_volume": 0.090565573,
  "injection_x": 6380,
  "injection_y": 6860,
  "injection_z": 5910,
  "product_id": 5,
  "specimen_name": "378-1611",
  "strain": "C57BL/6J",
  "structure_abbrev": "VMH",
  "structure_id": 693,
  "structure_name": "Ventromedial hypothalamic nucleus",
  "primary_injection_structure": 88,
  "data_set_id": 157549402,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "345/378/1057",
  "injection_volume": 0.0611903817,
  "injection_x": 4370,
  "injection_y": 4060,
  "injection_z": 9020,
  "product_id": 5,
  "specimen_name": "Ntsr1-Cre-3578",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-m",
  "structure_id": 345,
  "structure_name": "Primary somatosensory area, mouth",
  "primary_injection_structure": 345,
  "data_set_id": 159750477,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182901722,
   "name": "Ppp1r17-Cre_NL146"
  },
  "gender": "F",
  "injection_structures": "149/186/362/483",
  "injection_volume": 0.0506361541,
  "injection_x": 6040,
  "injection_y": 3580,
  "injection_z": 6040,
  "product_id": 5,
  "specimen_name": "Ppp1r17-Cre_NL146-3899",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "MD",
  "structure_id": 362,
  "structure_name": "Mediodorsal nucleus of thalamus",
  "primary_injection_structure": 149,
  "data_set_id": 268399145,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "385/409",
  "injection_volume": 0.001193042353125,
  "injection_x": 9320,
  "injection_y": 1780,
  "injection_z": 2540,
  "product_id": 36,
  "specimen_name": "A930038C07Rik-Tg1-Cre-249277",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 527575484,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "385/879/894",
  "injection_volume": 0.203589316,
  "injection_x": 9060,
  "injection_y": 1370,
  "injection_z": 7560,
  "product_id": 5,
  "specimen_name": "378-1349",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 126862385,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "394/533",
  "injection_volume": 0.124456913,
  "injection_x": 7820,
  "injection_y": 700,
  "injection_z": 7560,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre-76",
  "strain": null,
  "structure_abbrev": "VISam",
  "structure_id": 394,
  "structure_name": "Anteromedial visual area",
  "primary_injection_structure": 394,
  "data_set_id": 159753308,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838361,
   "name": "Etv1-CreERT2"
  },
  "gender": "M",
  "injection_structures": "39/48/993",
  "injection_volume": 0.432210968,
  "injection_x": 4470,
  "injection_y": 1810,
  "injection_z": 6030,
  "product_id": 5,
  "specimen_name": "Etv1-CreERT2-2921",
  "strain": "C57BL/6J",
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 181858761,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838048,
   "name": "Slc17a6-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "294/531/634/795/1061",
  "injection_volume": 0.1328886272,
  "injection_x": 8040,
  "injection_y": 2660,
  "injection_z": 6160,
  "product_id": 5,
  "specimen_name": "Slc17a6-IRES-Cre-121217",
  "strain": "B6.129",
  "structure_abbrev": "NPC",
  "structure_id": 634,
  "structure_name": "Nucleus of the posterior commissure",
  "primary_injection_structure": 294,
  "data_set_id": 294175704,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "985/993",
  "injection_volume": 0.1709493576,
  "injection_x": 3320,
  "injection_y": 2110,
  "injection_z": 7590,
  "product_id": 5,
  "specimen_name": "C57BL6J-314641",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 585025284,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "918/312782628",
  "injection_volume": 0.1560871284,
  "injection_x": 9720,
  "injection_y": 3760,
  "injection_z": 9880,
  "product_id": 5,
  "specimen_name": "C57BL6J_OLD-343580",
  "strain": "C57BL/6J",
  "structure_abbrev": "ENTl",
  "structure_id": 918,
  "structure_name": "Entorhinal area, lateral part",
  "primary_injection_structure": 918,
  "data_set_id": 642967852,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "F",
  "injection_structures": "402/409/312782574",
  "injection_volume": 0.0126789166,
  "injection_x": 8600,
  "injection_y": 2060,
  "injection_z": 9260,
  "product_id": 5,
  "specimen_name": "Syt6-Cre-639",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 402,
  "data_set_id": 159224478,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838494,
   "name": "Prkcd-GluCla-CFP-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "15/127/362/1113/1120",
  "injection_volume": 0.0519244474,
  "injection_x": 6110,
  "injection_y": 4350,
  "injection_z": 5420,
  "product_id": 35,
  "specimen_name": "Prkcd-GluCla-CFP-IRES-Cre-294973",
  "strain": null,
  "structure_abbrev": "AM",
  "structure_id": 127,
  "structure_name": "Anteromedial nucleus",
  "primary_injection_structure": 15,
  "data_set_id": 573035760,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837281,
   "name": "Calb2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "867/920",
  "injection_volume": 0.1235631824,
  "injection_x": 10580,
  "injection_y": 4200,
  "injection_z": 6900,
  "product_id": 5,
  "specimen_name": "Calb2-IRES-Cre-221",
  "strain": "C57BL/6J",
  "structure_abbrev": "PB",
  "structure_id": 867,
  "structure_name": "Parabrachial nucleus",
  "primary_injection_structure": 867,
  "data_set_id": 183284388,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "136/203/978/1048",
  "injection_volume": 0.048658715,
  "injection_x": 11870,
  "injection_y": 6200,
  "injection_z": 6360,
  "product_id": 5,
  "specimen_name": "378-1315",
  "strain": "C57BL/6J",
  "structure_abbrev": "GRN",
  "structure_id": 1048,
  "structure_name": "Gigantocellular reticular nucleus",
  "primary_injection_structure": 136,
  "data_set_id": 126843905,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837481,
   "name": "Scnn1a-Tg2-Cre"
  },
  "gender": "M",
  "injection_structures": "926",
  "injection_volume": 0.005949531,
  "injection_x": 9630,
  "injection_y": 4420,
  "injection_z": 9040,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg2-Cre-122200",
  "strain": "C57BL/6J",
  "structure_abbrev": "ENTm",
  "structure_id": 926,
  "structure_name": "Entorhinal area, medial part, dorsal zone",
  "primary_injection_structure": 926,
  "data_set_id": 302014692,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838877,
   "name": "Gabrr3-Cre_KC112"
  },
  "gender": "M",
  "injection_structures": "574/931",
  "injection_volume": 0.0835544374,
  "injection_x": 9210,
  "injection_y": 6560,
  "injection_z": 6110,
  "product_id": 5,
  "specimen_name": "Gabrr3-Cre_KC112-3470",
  "strain": "C57BL/6J",
  "structure_abbrev": "PG",
  "structure_id": 931,
  "structure_name": "Pontine gray",
  "primary_injection_structure": 574,
  "data_set_id": 586448498,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.1220100294,
  "injection_x": 8590,
  "injection_y": 1140,
  "injection_z": 2770,
  "product_id": 36,
  "specimen_name": "Cux2-IRES-Cre-230158",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 506890448,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839331,
   "name": "Nr5a1-Cre"
  },
  "gender": "F",
  "injection_structures": "345",
  "injection_volume": 0.0826879508,
  "injection_x": 4980,
  "injection_y": 2180,
  "injection_z": 8700,
  "product_id": 5,
  "specimen_name": "Nr5a1-Cre-2452",
  "strain": "B6.FVB",
  "structure_abbrev": "SSp-m",
  "structure_id": 345,
  "structure_name": "Primary somatosensory area, mouth",
  "primary_injection_structure": 345,
  "data_set_id": 177782493,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "F",
  "injection_structures": "394/894",
  "injection_volume": 0.02111251975,
  "injection_x": 7830,
  "injection_y": 1110,
  "injection_z": 4450,
  "product_id": 35,
  "specimen_name": "Ntsr1-Cre_GN220-307263",
  "strain": null,
  "structure_abbrev": "RSPagl",
  "structure_id": 894,
  "structure_name": "Retrosplenial area, lateral agranular part",
  "primary_injection_structure": 394,
  "data_set_id": 585021827,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 179697864,
   "name": "Rorb-IRES2-Cre"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.0418577551,
  "injection_x": 9150,
  "injection_y": 1230,
  "injection_z": 8610,
  "product_id": 5,
  "specimen_name": "Rorb-IRES2-Cre-359147",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 656842599,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838622,
   "name": "Vip-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "353/182305689",
  "injection_volume": 0.0775763639,
  "injection_x": 5540,
  "injection_y": 2030,
  "injection_z": 9010,
  "product_id": 5,
  "specimen_name": "Vip-IRES-Cre-180",
  "strain": null,
  "structure_abbrev": "SSp-n",
  "structure_id": 353,
  "structure_name": "Primary somatosensory area, nose",
  "primary_injection_structure": 353,
  "data_set_id": 180075597,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838622,
   "name": "Vip-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "30/286",
  "injection_volume": 0.02282766093125,
  "injection_x": 5760,
  "injection_y": 6980,
  "injection_z": 5900,
  "product_id": 5,
  "specimen_name": "Vip-IRES-Cre-122673",
  "strain": "C57BL/6J",
  "structure_abbrev": "SCH",
  "structure_id": 286,
  "structure_name": "Suprachiasmatic nucleus",
  "primary_injection_structure": 30,
  "data_set_id": 293431163,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839331,
   "name": "Nr5a1-Cre"
  },
  "gender": "F",
  "injection_structures": "329",
  "injection_volume": 0.03786097945,
  "injection_x": 6970,
  "injection_y": 1200,
  "injection_z": 8470,
  "product_id": 5,
  "specimen_name": "Nr5a1-Cre-146",
  "strain": "B6.FVB",
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 115956702,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "151/184/507/723/993",
  "injection_volume": 0.2319124528,
  "injection_x": 2290,
  "injection_y": 3270,
  "injection_z": 7440,
  "product_id": 5,
  "specimen_name": "BL/6-3382",
  "strain": "C57BL/6J",
  "structure_abbrev": "ORBl",
  "structure_id": 723,
  "structure_name": "Orbital area, lateral part",
  "primary_injection_structure": 151,
  "data_set_id": 170721670,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182901722,
   "name": "Ppp1r17-Cre_NL146"
  },
  "gender": "M",
  "injection_structures": "294/302",
  "injection_volume": 0.032858674079834,
  "injection_x": 9420,
  "injection_y": 1880,
  "injection_z": 5950,
  "product_id": 5,
  "specimen_name": "Ppp1r17-Cre_NL146-3927",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "SCm",
  "structure_id": 294,
  "structure_name": "Superior colliculus, motor related",
  "primary_injection_structure": 294,
  "data_set_id": 278513131,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182693192,
   "name": "Chrna2-Cre_OE25"
  },
  "gender": "F",
  "injection_structures": "44/48/814/972",
  "injection_volume": 0.1740825352,
  "injection_x": 3740,
  "injection_y": 3660,
  "injection_z": 6450,
  "product_id": 5,
  "specimen_name": "Chrna2-Cre_OE25-131332",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "ILA",
  "structure_id": 44,
  "structure_name": "Infralimbic area",
  "primary_injection_structure": 44,
  "data_set_id": 298272589,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838828,
   "name": "Gal-Cre_KI87"
  },
  "gender": "F",
  "injection_structures": "463/726",
  "injection_volume": 0.02497290635,
  "injection_x": 8540,
  "injection_y": 3040,
  "injection_z": 8310,
  "product_id": 5,
  "specimen_name": "Gal-Cre_KI87-3474",
  "strain": "C57BL/6J",
  "structure_abbrev": "DG",
  "structure_id": 726,
  "structure_name": "Dentate gyrus",
  "primary_injection_structure": 463,
  "data_set_id": 266173339,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 309483400,
   "name": "Foxp2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "173/614/693",
  "injection_volume": 0.0464796654,
  "injection_x": 6710,
  "injection_y": 7010,
  "injection_z": 4750,
  "product_id": 45,
  "specimen_name": "Foxp2-IRES-Cre-260827",
  "strain": null,
  "structure_abbrev": "TU",
  "structure_id": 614,
  "structure_name": "Tuberal nucleus",
  "primary_injection_structure": 173,
  "data_set_id": 551756337,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 459995486,
   "name": "Gnb4-IRES2-Cre"
  },
  "gender": "M",
  "injection_structures": "111/583/672/952/1057",
  "injection_volume": 0.141621466,
  "injection_x": 5330,
  "injection_y": 5260,
  "injection_z": 9220,
  "product_id": 5,
  "specimen_name": "Gnb4-IRES2-Cre-neo-193745",
  "strain": null,
  "structure_abbrev": "CLA",
  "structure_id": 583,
  "structure_name": "Claustrum",
  "primary_injection_structure": 111,
  "data_set_id": 485903475,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "385/394/533",
  "injection_volume": 0.0082437385625,
  "injection_x": 7970,
  "injection_y": 990,
  "injection_z": 3810,
  "product_id": 35,
  "specimen_name": "A930038C07Rik-Tg1-Cre-360391.0",
  "strain": null,
  "structure_abbrev": "VISpm",
  "structure_id": 533,
  "structure_name": "posteromedial visual area",
  "primary_injection_structure": 385,
  "data_set_id": 656958144,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838502,
   "name": "Grik4-Cre"
  },
  "gender": "F",
  "injection_structures": "463/726",
  "injection_volume": 0.2000629232,
  "injection_x": 7850,
  "injection_y": 2410,
  "injection_z": 7810,
  "product_id": 5,
  "specimen_name": "Grik4-Cre-4958",
  "strain": "C57BL/6J",
  "structure_abbrev": "DG",
  "structure_id": 726,
  "structure_name": "Dentate gyrus",
  "primary_injection_structure": 463,
  "data_set_id": 114047146,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837834,
   "name": "Crh-IRES-Cre_BL"
  },
  "gender": "F",
  "injection_structures": "985/993",
  "injection_volume": 0.0493045546,
  "injection_x": 4060,
  "injection_y": 1830,
  "injection_z": 7350,
  "product_id": 5,
  "specimen_name": "Crh-IRES-Cre-279",
  "strain": "B6.129",
  "structure_abbrev": "MOp",
  "structure_id": 985,
  "structure_name": "Primary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 183103805,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "920",
  "injection_volume": 0.1819163416,
  "injection_x": 11000,
  "injection_y": 2850,
  "injection_z": 6100,
  "product_id": 5,
  "specimen_name": "378-1196",
  "strain": "C57BL/6J",
  "structure_abbrev": "CENT",
  "structure_id": 920,
  "structure_name": "Central lobule",
  "primary_injection_structure": 920,
  "data_set_id": 119848249,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "F",
  "injection_structures": "345/353/378",
  "injection_volume": 0.1226451184,
  "injection_x": 5400,
  "injection_y": 2970,
  "injection_z": 9300,
  "product_id": 5,
  "specimen_name": "Syt6-Cre_KI148-132909",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-m",
  "structure_id": 345,
  "structure_name": "Primary somatosensory area, mouth",
  "primary_injection_structure": 345,
  "data_set_id": 299829181,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837481,
   "name": "Scnn1a-Tg2-Cre"
  },
  "gender": "M",
  "injection_structures": "725/733/1020",
  "injection_volume": 0.0117668453,
  "injection_x": 7240,
  "injection_y": 4440,
  "injection_z": 7200,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg2-Cre-3373",
  "strain": "C57BL/6J",
  "structure_abbrev": "VPM",
  "structure_id": 733,
  "structure_name": "Ventral posteromedial nucleus of the thalamus",
  "primary_injection_structure": 725,
  "data_set_id": 156202979,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837979,
   "name": "Drd1a-Cre_EY262"
  },
  "gender": "F",
  "injection_structures": "672",
  "injection_volume": 0.0533445213,
  "injection_x": 4360,
  "injection_y": 3120,
  "injection_z": 6940,
  "product_id": 5,
  "specimen_name": "Drd1a-Cre-4885",
  "strain": "C57BL/6J",
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 672,
  "data_set_id": 159223001,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "879/886/894",
  "injection_volume": 0.1263186778,
  "injection_x": 9090,
  "injection_y": 1040,
  "injection_z": 4450,
  "product_id": 35,
  "specimen_name": "Emx1-IRES-Cre-283752",
  "strain": null,
  "structure_abbrev": "RSPd",
  "structure_id": 879,
  "structure_name": "Retrosplenial area, dorsal part",
  "primary_injection_structure": 879,
  "data_set_id": 562521707,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "886",
  "injection_volume": 0.1572012512,
  "injection_x": 7220,
  "injection_y": 1110,
  "injection_z": 5960,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre-77",
  "strain": null,
  "structure_abbrev": "RSPv",
  "structure_id": 886,
  "structure_name": "Retrosplenial area, ventral part",
  "primary_injection_structure": 886,
  "data_set_id": 159832064,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 459995486,
   "name": "Gnb4-IRES2-Cre"
  },
  "gender": "M",
  "injection_structures": "104/119/345/583/672",
  "injection_volume": 0.0141904686,
  "injection_x": 4800,
  "injection_y": 4740,
  "injection_z": 8830,
  "product_id": 5,
  "specimen_name": "Gnb4-IRES2-Cre-231701",
  "strain": null,
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 104,
  "data_set_id": 514505957,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 179697864,
   "name": "Rorb-IRES2-Cre"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.0431056038,
  "injection_x": 9500,
  "injection_y": 1150,
  "injection_z": 8270,
  "product_id": 5,
  "specimen_name": "Rorb-IRES2-Cre-337571",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 623855077,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 309779993,
   "name": "Ctgf-T2A-dgCre"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.0228879147,
  "injection_x": 8340,
  "injection_y": 1460,
  "injection_z": 8100,
  "product_id": 5,
  "specimen_name": "Ctgf-T2A-dgCre-335953",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 627869431,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838502,
   "name": "Grik4-Cre"
  },
  "gender": "F",
  "injection_structures": "155/218/463/726",
  "injection_volume": 0.0409381574,
  "injection_x": 7140,
  "injection_y": 2450,
  "injection_z": 7310,
  "product_id": 5,
  "specimen_name": "Grik4-Cre-5108",
  "strain": "C57BL/6J",
  "structure_abbrev": "LP",
  "structure_id": 218,
  "structure_name": "Lateral posterior nucleus of the thalamus",
  "primary_injection_structure": 155,
  "data_set_id": 183175010,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "394/879/894",
  "injection_volume": 0.0309630804,
  "injection_x": 7790,
  "injection_y": 1130,
  "injection_z": 4410,
  "product_id": 36,
  "specimen_name": "Ntsr1-Cre_GN220-275958",
  "strain": "unknown",
  "structure_abbrev": "RSPagl",
  "structure_id": 894,
  "structure_name": "Retrosplenial area, lateral agranular part",
  "primary_injection_structure": 394,
  "data_set_id": 557827228,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "385/409",
  "injection_volume": 0.792197112,
  "injection_x": 9460,
  "injection_y": 2030,
  "injection_z": 8460,
  "product_id": 5,
  "specimen_name": "C57BL/6-152141",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 307296433,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 287226763,
   "name": "Calb1-T2A-dgCre"
  },
  "gender": "F",
  "injection_structures": "329/353/182305689",
  "injection_volume": 0.02066662465,
  "injection_x": 5970,
  "injection_y": 2100,
  "injection_z": 9030,
  "product_id": 5,
  "specimen_name": "Calb1-2A-dgCre-189010",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 485022109,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838502,
   "name": "Grik4-Cre"
  },
  "gender": "M",
  "injection_structures": "423/463",
  "injection_volume": 0.1143456846,
  "injection_x": 7900,
  "injection_y": 3760,
  "injection_z": 8730,
  "product_id": 5,
  "specimen_name": "Grik4-Cre-5062",
  "strain": "C57BL/6J",
  "structure_abbrev": "CA3",
  "structure_id": 463,
  "structure_name": "Field CA3",
  "primary_injection_structure": 423,
  "data_set_id": 178382220,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "385/879/894",
  "injection_volume": 0.1878396576,
  "injection_x": 9180,
  "injection_y": 1260,
  "injection_z": 3930,
  "product_id": 36,
  "specimen_name": "Emx1-IRES-Cre-265435",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 546103149,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838074,
   "name": "Slc6a3-Cre"
  },
  "gender": "F",
  "injection_structures": "58/246/749",
  "injection_volume": 0.0603639281,
  "injection_x": 8590,
  "injection_y": 4750,
  "injection_z": 6150,
  "product_id": 5,
  "specimen_name": "Slc6a3-Cre-4472",
  "strain": "C57BL/6J",
  "structure_abbrev": "VTA",
  "structure_id": 749,
  "structure_name": "Ventral tegmental area",
  "primary_injection_structure": 58,
  "data_set_id": 160540751,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838496,
   "name": "Grp-Cre_KH288"
  },
  "gender": "F",
  "injection_structures": "382/423/463",
  "injection_volume": 0.03548329365,
  "injection_x": 7360,
  "injection_y": 2690,
  "injection_z": 8550,
  "product_id": 5,
  "specimen_name": "Grp-Cre-58",
  "strain": "B6.FVB(ICR)",
  "structure_abbrev": "CA3",
  "structure_id": 463,
  "structure_name": "Field CA3",
  "primary_injection_structure": 382,
  "data_set_id": 182615063,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 517749378,
   "name": "Rorb-IRES2-Cre-neo"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.01650363365,
  "injection_x": 8510,
  "injection_y": 1020,
  "injection_z": 8320,
  "product_id": 5,
  "specimen_name": "Rorb-IRES2-Cre-neo-347764",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 648048922,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "337/369",
  "injection_volume": 0.0520767051,
  "injection_x": 6270,
  "injection_y": 1350,
  "injection_z": 7450,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-3794",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-ll",
  "structure_id": 337,
  "structure_name": "Primary somatosensory area, lower limb",
  "primary_injection_structure": 337,
  "data_set_id": 166323186,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 460133878,
   "name": "Ntng2-IRES2-Cre"
  },
  "gender": "M",
  "injection_structures": "104/111/119",
  "injection_volume": 0.0590923781,
  "injection_x": 4790,
  "injection_y": 4970,
  "injection_z": 9280,
  "product_id": 5,
  "specimen_name": "Ntng2-IRES2-Cre-191956",
  "strain": null,
  "structure_abbrev": "AIp",
  "structure_id": 111,
  "structure_name": "Agranular insular area, posterior part",
  "primary_injection_structure": 104,
  "data_set_id": 485847695,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "39/48/993",
  "injection_volume": 0.0227802274,
  "injection_x": 4870,
  "injection_y": 1390,
  "injection_z": 6220,
  "product_id": 5,
  "specimen_name": "Tlx3-Cre_LURC-3607",
  "strain": null,
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 293432575,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 266649644,
   "name": "Drd3-Cre_KI196"
  },
  "gender": "M",
  "injection_structures": "312782628",
  "injection_volume": 0.019194342475,
  "injection_x": 9600,
  "injection_y": 2950,
  "injection_z": 9780,
  "product_id": 5,
  "specimen_name": "Drd3-Cre_KI196_LURC-132026",
  "strain": null,
  "structure_abbrev": "VISpor",
  "structure_id": 312782628,
  "structure_name": "Postrhinal area",
  "primary_injection_structure": 312782628,
  "data_set_id": 527577056,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "329/353/378",
  "injection_volume": 0.302667414,
  "injection_x": 6530,
  "injection_y": 3220,
  "injection_z": 9270,
  "product_id": 5,
  "specimen_name": "378-965",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSs",
  "structure_id": 378,
  "structure_name": "Supplemental somatosensory area",
  "primary_injection_structure": 329,
  "data_set_id": 112514915,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "879/886/894/1037",
  "injection_volume": 0.0891643592,
  "injection_x": 8350,
  "injection_y": 1540,
  "injection_z": 6970,
  "product_id": 5,
  "specimen_name": "378-967",
  "strain": "C57BL/6J",
  "structure_abbrev": "RSPv",
  "structure_id": 886,
  "structure_name": "Retrosplenial area, ventral part",
  "primary_injection_structure": 879,
  "data_set_id": 112595376,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838584,
   "name": "Gad2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "382/726/312782546",
  "injection_volume": 0.1904295036,
  "injection_x": 7760,
  "injection_y": 2170,
  "injection_z": 7940,
  "product_id": 5,
  "specimen_name": "Gad2-IRES-Cre_LURC-123563",
  "strain": "C57BL/6J",
  "structure_abbrev": "CA1",
  "structure_id": 382,
  "structure_name": "Field CA1",
  "primary_injection_structure": 382,
  "data_set_id": 305380411,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "337/369",
  "injection_volume": 0.1895632424,
  "injection_x": 5830,
  "injection_y": 1520,
  "injection_z": 7640,
  "product_id": 5,
  "specimen_name": "378-928",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-ul",
  "structure_id": 369,
  "structure_name": "Primary somatosensory area, upper limb",
  "primary_injection_structure": 337,
  "data_set_id": 112229814,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "48/250/258",
  "injection_volume": 0.01526120435,
  "injection_x": 4880,
  "injection_y": 3370,
  "injection_z": 5910,
  "product_id": 5,
  "specimen_name": "378-1209",
  "strain": "C57BL/6J",
  "structure_abbrev": "LSc",
  "structure_id": 250,
  "structure_name": "Lateral septal nucleus, caudal (caudodorsal) part",
  "primary_injection_structure": 48,
  "data_set_id": 120436988,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "541/312782574/312782628",
  "injection_volume": 0.2123242324,
  "injection_x": 9050,
  "injection_y": 2230,
  "injection_z": 1630,
  "product_id": 36,
  "specimen_name": "Rbp4-Cre_KL100-296539",
  "strain": "unknown",
  "structure_abbrev": "VISpor",
  "structure_id": 312782628,
  "structure_name": "Postrhinal area",
  "primary_injection_structure": 541,
  "data_set_id": 576340889,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182693192,
   "name": "Chrna2-Cre_OE25"
  },
  "gender": "M",
  "injection_structures": "993",
  "injection_volume": 0.1100907696,
  "injection_x": 3150,
  "injection_y": 2660,
  "injection_z": 7110,
  "product_id": 5,
  "specimen_name": "Chrna2-Cre_OE25-3909",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 292210312,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "F",
  "injection_structures": "329/417",
  "injection_volume": 0.04005350405,
  "injection_x": 7490,
  "injection_y": 1300,
  "injection_z": 2700,
  "product_id": 35,
  "specimen_name": "Tlx3-Cre_PL56-244761",
  "strain": "B6.FVB",
  "structure_abbrev": "VISrl",
  "structure_id": 417,
  "structure_name": "Rostrolateral visual area",
  "primary_injection_structure": 329,
  "data_set_id": 522635991,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "M",
  "injection_structures": "385/394/533/312782546",
  "injection_volume": 0.02356249035,
  "injection_x": 7800,
  "injection_y": 840,
  "injection_z": 3560,
  "product_id": 35,
  "specimen_name": "Scnn1a-Tg3-Cre-341422",
  "strain": null,
  "structure_abbrev": "VISpm",
  "structure_id": 533,
  "structure_name": "posteromedial visual area",
  "primary_injection_structure": 385,
  "data_set_id": 636803957,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838048,
   "name": "Slc17a6-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "15/127/149/362/1113",
  "injection_volume": 0.0648569978,
  "injection_x": 5970,
  "injection_y": 4330,
  "injection_z": 6130,
  "product_id": 5,
  "specimen_name": "Slc17a6-IRES-Cre_LURC-121396",
  "strain": "C57BL/6J",
  "structure_abbrev": "PT",
  "structure_id": 15,
  "structure_name": "Parataenial nucleus",
  "primary_injection_structure": 15,
  "data_set_id": 305125123,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "329/361/417/312782546",
  "injection_volume": 0.1304121966,
  "injection_x": 7240,
  "injection_y": 1020,
  "injection_z": 3370,
  "product_id": 36,
  "specimen_name": "Rbp4-Cre_KL100-360339",
  "strain": null,
  "structure_abbrev": "VISa",
  "structure_id": 312782546,
  "structure_name": "Anterior area",
  "primary_injection_structure": 329,
  "data_set_id": 656959718,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "F",
  "injection_structures": "39/993",
  "injection_volume": 0.01421069825,
  "injection_x": 5280,
  "injection_y": 1510,
  "injection_z": 6130,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg3-Cre-4890",
  "strain": "B6.C3H",
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 267101850,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "44/972",
  "injection_volume": 0.0730102205,
  "injection_x": 3280,
  "injection_y": 3290,
  "injection_z": 6300,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-122237",
  "strain": "C57BL/6J",
  "structure_abbrev": "PL",
  "structure_id": 972,
  "structure_name": "Prelimbic area",
  "primary_injection_structure": 44,
  "data_set_id": 294396492,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "385/425",
  "injection_volume": 0.0197889538,
  "injection_x": 10020,
  "injection_y": 2190,
  "injection_z": 8900,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre_KL100-285",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISpl",
  "structure_id": 425,
  "structure_name": "Posterolateral visual area",
  "primary_injection_structure": 385,
  "data_set_id": 286417464,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "F",
  "injection_structures": "394/879/894",
  "injection_volume": 0.0641102819,
  "injection_x": 7980,
  "injection_y": 1080,
  "injection_z": 4430,
  "product_id": 36,
  "specimen_name": "Ntsr1-Cre_GN220-271817",
  "strain": "unknown",
  "structure_abbrev": "RSPagl",
  "structure_id": 894,
  "structure_name": "Retrosplenial area, lateral agranular part",
  "primary_injection_structure": 394,
  "data_set_id": 553746532,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838927,
   "name": "Pcdh9-Cre_NP276"
  },
  "gender": "F",
  "injection_structures": "843/926",
  "injection_volume": 0.006545665,
  "injection_x": 9820,
  "injection_y": 3770,
  "injection_z": 8580,
  "product_id": 5,
  "specimen_name": "Pcdh9-Cre-126532",
  "strain": "C57BL/6J",
  "structure_abbrev": "PAR",
  "structure_id": 843,
  "structure_name": "Parasubiculum",
  "primary_injection_structure": 843,
  "data_set_id": 308721177,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "M",
  "injection_structures": "39/48/972",
  "injection_volume": 0.0909820632,
  "injection_x": 4230,
  "injection_y": 2730,
  "injection_z": 6580,
  "product_id": 5,
  "specimen_name": "Syt6-Cre-600",
  "strain": "C57BL/6J",
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 156785529,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "44/48/589/814/972/993",
  "injection_volume": 0.210919206514844,
  "injection_x": 4050,
  "injection_y": 3670,
  "injection_z": 6180,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-3796",
  "strain": "C57BL/6J",
  "structure_abbrev": "PL",
  "structure_id": 972,
  "structure_name": "Prelimbic area",
  "primary_injection_structure": 44,
  "data_set_id": 287494320,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "1002/1011/1027",
  "injection_volume": 0.3339246904,
  "injection_x": 8030,
  "injection_y": 2370,
  "injection_z": 1780,
  "product_id": 35,
  "specimen_name": "Emx1-IRES-Cre-283754",
  "strain": null,
  "structure_abbrev": "AUDp",
  "structure_id": 1002,
  "structure_name": "Primary auditory area",
  "primary_injection_structure": 1002,
  "data_set_id": 562671482,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "M",
  "injection_structures": "39/972/993",
  "injection_volume": 0.2339653372,
  "injection_x": 3260,
  "injection_y": 2220,
  "injection_z": 6350,
  "product_id": 5,
  "specimen_name": "Syt6-Cre-3153",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 39,
  "data_set_id": 177459319,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "1002/1011",
  "injection_volume": 0.0987757582,
  "injection_x": 7670,
  "injection_y": 2820,
  "injection_z": 9520,
  "product_id": 5,
  "specimen_name": "Ntsr1-Cre_GN220-3735",
  "strain": "C57BL/6J",
  "structure_abbrev": "AUDp",
  "structure_id": 1002,
  "structure_name": "Primary auditory area",
  "primary_injection_structure": 1002,
  "data_set_id": 266963362,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "337/369",
  "injection_volume": 0.3683622432,
  "injection_x": 6160,
  "injection_y": 1310,
  "injection_z": 7690,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre_KL100-128971",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-ll",
  "structure_id": 337,
  "structure_name": "Primary somatosensory area, lower limb",
  "primary_injection_structure": 337,
  "data_set_id": 297652799,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837834,
   "name": "Crh-IRES-Cre_BL"
  },
  "gender": "F",
  "injection_structures": "15/127/149/362/1113",
  "injection_volume": 0.075330174715332,
  "injection_x": 5880,
  "injection_y": 4470,
  "injection_z": 6180,
  "product_id": 5,
  "specimen_name": "Crh-IRES-Cre-228",
  "strain": "B6.129",
  "structure_abbrev": "PT",
  "structure_id": 15,
  "structure_name": "Parataenial nucleus",
  "primary_injection_structure": 15,
  "data_set_id": 167373923,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "353/378",
  "injection_volume": 0.3059070392,
  "injection_x": 5560,
  "injection_y": 3190,
  "injection_z": 9590,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre-135",
  "strain": null,
  "structure_abbrev": "SSs",
  "structure_id": 378,
  "structure_name": "Supplemental somatosensory area",
  "primary_injection_structure": 353,
  "data_set_id": 168163498,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 266649644,
   "name": "Drd3-Cre_KI196"
  },
  "gender": "M",
  "injection_structures": "918/922/312782628",
  "injection_volume": 0.1377935664,
  "injection_x": 9490,
  "injection_y": 3400,
  "injection_z": 9990,
  "product_id": 5,
  "specimen_name": "Drd3-Cre_KI196_LURC-130780",
  "strain": null,
  "structure_abbrev": "ENTl",
  "structure_id": 918,
  "structure_name": "Entorhinal area, lateral part",
  "primary_injection_structure": 918,
  "data_set_id": 518207061,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "463",
  "injection_volume": 0.0477132796,
  "injection_x": 6900,
  "injection_y": 2480,
  "injection_z": 7980,
  "product_id": 5,
  "specimen_name": "378-1056",
  "strain": "C57BL/6J",
  "structure_abbrev": "CA3",
  "structure_id": 463,
  "structure_name": "Field CA3",
  "primary_injection_structure": 463,
  "data_set_id": 112745073,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "409/425/312782574/312782628",
  "injection_volume": 0.1308292258,
  "injection_x": 9510,
  "injection_y": 2300,
  "injection_z": 9450,
  "product_id": 5,
  "specimen_name": "378-1162",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 409,
  "data_set_id": 114250546,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.01131337235,
  "injection_x": 9100,
  "injection_y": 1210,
  "injection_z": 3400,
  "product_id": 36,
  "specimen_name": "Rbp4-Cre_KL100-238777",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 517072832,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "329/385/417",
  "injection_volume": 0.0463737323,
  "injection_x": 7930,
  "injection_y": 1350,
  "injection_z": 2970,
  "product_id": 36,
  "specimen_name": "Tlx3-Cre_PL56-241021",
  "strain": null,
  "structure_abbrev": "VISrl",
  "structure_id": 417,
  "structure_name": "Rostrolateral visual area",
  "primary_injection_structure": 329,
  "data_set_id": 518741580,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839331,
   "name": "Nr5a1-Cre"
  },
  "gender": "F",
  "injection_structures": "385/402/409",
  "injection_volume": 0.0299148479,
  "injection_x": 8720,
  "injection_y": 1410,
  "injection_z": 9160,
  "product_id": 5,
  "specimen_name": "Nr5a1-Cre-2539",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 385,
  "data_set_id": 277849965,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "329",
  "injection_volume": 0.0706896344,
  "injection_x": 7150,
  "injection_y": 2320,
  "injection_z": 2490,
  "product_id": 35,
  "specimen_name": "Ntsr1-Cre_GN220-275953",
  "strain": "unknown",
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 557826519,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "647",
  "injection_volume": 0.224047404,
  "injection_x": 8130,
  "injection_y": 6670,
  "injection_z": 8390,
  "product_id": 5,
  "specimen_name": "378-1161",
  "strain": "C57BL/6J",
  "structure_abbrev": "COAp",
  "structure_id": 647,
  "structure_name": "Cortical amygdalar area, posterior part",
  "primary_injection_structure": 647,
  "data_set_id": 114249084,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "184/723/731/746/972/993",
  "injection_volume": 0.1155556808,
  "injection_x": 2930,
  "injection_y": 2920,
  "injection_z": 6610,
  "product_id": 5,
  "specimen_name": "378-939",
  "strain": "C57BL/6J",
  "structure_abbrev": "ORBvl",
  "structure_id": 746,
  "structure_name": "Orbital area, ventrolateral part",
  "primary_injection_structure": 184,
  "data_set_id": 112423392,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838681,
   "name": "Slc6a4-Cre_ET33"
  },
  "gender": "M",
  "injection_structures": "115/591/795/872",
  "injection_volume": 0.051669261525,
  "injection_x": 9410,
  "injection_y": 3830,
  "injection_z": 5680,
  "product_id": 36,
  "specimen_name": "Slc6a4-Cre_ET33-353229",
  "strain": null,
  "structure_abbrev": "DR",
  "structure_id": 872,
  "structure_name": "Dorsal nucleus raphe",
  "primary_injection_structure": 115,
  "data_set_id": 649181539,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "F",
  "injection_structures": "7/385/534/612/867/1049",
  "injection_volume": 0.0300445558,
  "injection_x": 9950,
  "injection_y": 4650,
  "injection_z": 7920,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg3-Cre-4762",
  "strain": "B6.C3H",
  "structure_abbrev": "NLL",
  "structure_id": 612,
  "structure_name": "Nucleus of the lateral lemniscus",
  "primary_injection_structure": 7,
  "data_set_id": 156313344,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "M",
  "injection_structures": "369",
  "injection_volume": 0.1220225734,
  "injection_x": 5340,
  "injection_y": 2160,
  "injection_z": 8070,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg3-Cre-4833",
  "strain": "B6.C3H",
  "structure_abbrev": "SSp-ul",
  "structure_id": 369,
  "structure_name": "Primary somatosensory area, upper limb",
  "primary_injection_structure": 369,
  "data_set_id": 181860173,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "184/723/746",
  "injection_volume": 0.316557444,
  "injection_x": 2790,
  "injection_y": 2980,
  "injection_z": 7050,
  "product_id": 5,
  "specimen_name": "378-929",
  "strain": "C57BL/6J",
  "structure_abbrev": "ORBl",
  "structure_id": 723,
  "structure_name": "Orbital area, lateral part",
  "primary_injection_structure": 184,
  "data_set_id": 112306316,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "104/345/672/985/1057",
  "injection_volume": 0.03207388345,
  "injection_x": 4150,
  "injection_y": 4030,
  "injection_z": 8750,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-3951",
  "strain": null,
  "structure_abbrev": "GU",
  "structure_id": 1057,
  "structure_name": "Gustatory areas",
  "primary_injection_structure": 104,
  "data_set_id": 292375491,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "F",
  "injection_structures": "184/972/993",
  "injection_volume": 0.0100271364375,
  "injection_x": 3250,
  "injection_y": 2730,
  "injection_z": 4470,
  "product_id": 45,
  "specimen_name": "Ntsr1-Cre_GN220-239692",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 184,
  "data_set_id": 518218546,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "394/533/312782546",
  "injection_volume": 0.0004968152109375,
  "injection_x": 7690,
  "injection_y": 1050,
  "injection_z": 3570,
  "product_id": 36,
  "specimen_name": "A930038C07Rik-Tg1-Cre-255585",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISa",
  "structure_id": 312782546,
  "structure_name": "Anterior area",
  "primary_injection_structure": 394,
  "data_set_id": 535696750,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "F",
  "injection_structures": "337/369",
  "injection_volume": 0.1295145754,
  "injection_x": 5890,
  "injection_y": 1560,
  "injection_z": 7780,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg3-Cre-4786",
  "strain": "B6.C3H",
  "structure_abbrev": "SSp-ul",
  "structure_id": 369,
  "structure_name": "Primary somatosensory area, upper limb",
  "primary_injection_structure": 337,
  "data_set_id": 166326029,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "402/409",
  "injection_volume": 0.0812447636,
  "injection_x": 8390,
  "injection_y": 1670,
  "injection_z": 2080,
  "product_id": 35,
  "specimen_name": "Ntsr1-Cre_GN220-262653",
  "strain": null,
  "structure_abbrev": "VISal",
  "structure_id": 402,
  "structure_name": "Anterolateral visual area",
  "primary_injection_structure": 402,
  "data_set_id": 544488964,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "4/128/294/533/580",
  "injection_volume": 0.3442473832,
  "injection_x": 9330,
  "injection_y": 3170,
  "injection_z": 7540,
  "product_id": 5,
  "specimen_name": "378-1180",
  "strain": "C57BL/6J",
  "structure_abbrev": "MRN",
  "structure_id": 128,
  "structure_name": "Midbrain reticular nucleus",
  "primary_injection_structure": 4,
  "data_set_id": 114474520,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.0480481358,
  "injection_x": 9290,
  "injection_y": 1120,
  "injection_z": 8270,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre-48",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 121510421,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839331,
   "name": "Nr5a1-Cre"
  },
  "gender": "M",
  "injection_structures": "329/361/417/312782546",
  "injection_volume": 0.0922317788,
  "injection_x": 7180,
  "injection_y": 1070,
  "injection_z": 8340,
  "product_id": 5,
  "specimen_name": "Nr5a1-Cre-2457",
  "strain": "B6.FVB",
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 176881134,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761781,
   "name": "Efr3a-Cre_NO108"
  },
  "gender": "F",
  "injection_structures": "337/361/312782546",
  "injection_volume": 0.1388951942,
  "injection_x": 7000,
  "injection_y": 1130,
  "injection_z": 7380,
  "product_id": 5,
  "specimen_name": "Efr3a-Cre_NO108-38",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "SSp-tr",
  "structure_id": 361,
  "structure_name": "Primary somatosensory area, trunk",
  "primary_injection_structure": 337,
  "data_set_id": 277907462,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837481,
   "name": "Scnn1a-Tg2-Cre"
  },
  "gender": "M",
  "injection_structures": "797",
  "injection_volume": 0.020512885925,
  "injection_x": 7790,
  "injection_y": 4920,
  "injection_z": 7440,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg2-Cre-214165",
  "strain": null,
  "structure_abbrev": "ZI",
  "structure_id": 797,
  "structure_name": "Zona incerta",
  "primary_injection_structure": 797,
  "data_set_id": 511549277,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "385/409",
  "injection_volume": 0.0716512055,
  "injection_x": 8590,
  "injection_y": 1900,
  "injection_z": 2500,
  "product_id": 35,
  "specimen_name": "Emx1-IRES-Cre-283751",
  "strain": null,
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 385,
  "data_set_id": 562520963,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838074,
   "name": "Slc6a3-Cre"
  },
  "gender": "F",
  "injection_structures": "651/773/839",
  "injection_volume": 0.00738427795,
  "injection_x": 12850,
  "injection_y": 5580,
  "injection_z": 6060,
  "product_id": 5,
  "specimen_name": "Slc6a3-Cre-4520",
  "strain": "C57BL/6J",
  "structure_abbrev": "DMX",
  "structure_id": 839,
  "structure_name": "Dorsal motor nucleus of the vagus nerve",
  "primary_injection_structure": 651,
  "data_set_id": 171021122,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "385/425/894",
  "injection_volume": 0.1270577938,
  "injection_x": 9600,
  "injection_y": 2140,
  "injection_z": 2900,
  "product_id": 35,
  "specimen_name": "Emx1-IRES-Cre-315581",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 593016678,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "184/993",
  "injection_volume": 0.054459335,
  "injection_x": 2330,
  "injection_y": 2390,
  "injection_z": 4400,
  "product_id": 45,
  "specimen_name": "Tlx3-Cre_PL56-257666",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 184,
  "data_set_id": 540140513,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "337/361/382/894/985/312782546",
  "injection_volume": 0.4138070384,
  "injection_x": 6880,
  "injection_y": 1110,
  "injection_z": 6950,
  "product_id": 5,
  "specimen_name": "378-1342",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-tr",
  "structure_id": 361,
  "structure_name": "Primary somatosensory area, trunk",
  "primary_injection_structure": 337,
  "data_set_id": 126852363,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "329/182305689",
  "injection_volume": 0.019783506225,
  "injection_x": 6700,
  "injection_y": 1500,
  "injection_z": 8170,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre-188",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 183171679,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "1002/1011",
  "injection_volume": 0.2381089536,
  "injection_x": 7650,
  "injection_y": 3130,
  "injection_z": 9870,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-203",
  "strain": "C57BL/6J",
  "structure_abbrev": "AUDp",
  "structure_id": 1002,
  "structure_name": "Primary auditory area",
  "primary_injection_structure": 1002,
  "data_set_id": 184159706,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182693192,
   "name": "Chrna2-Cre_OE25"
  },
  "gender": "M",
  "injection_structures": "886",
  "injection_volume": 0.010428632025,
  "injection_x": 7160,
  "injection_y": 1170,
  "injection_z": 6130,
  "product_id": 5,
  "specimen_name": "Chrna2-Cre_OE25-3893",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "RSPv",
  "structure_id": 886,
  "structure_name": "Retrosplenial area, ventral part",
  "primary_injection_structure": 886,
  "data_set_id": 292124058,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0766292282,
  "injection_x": 8360,
  "injection_y": 1460,
  "injection_z": 2920,
  "product_id": 35,
  "specimen_name": "Tlx3-Cre_PL56-252235",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 529692491,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 183238310,
   "name": "Ntrk1-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "1022/1031",
  "injection_volume": 0.0391539498,
  "injection_x": 5860,
  "injection_y": 4990,
  "injection_z": 7310,
  "product_id": 5,
  "specimen_name": "Ntrk1-IRES-Cre-3532",
  "strain": "B6.129",
  "structure_abbrev": "GPe",
  "structure_id": 1022,
  "structure_name": "Globus pallidus, external segment",
  "primary_injection_structure": 1022,
  "data_set_id": 265943460,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182693192,
   "name": "Chrna2-Cre_OE25"
  },
  "gender": "M",
  "injection_structures": "985/993",
  "injection_volume": 0.2239528536,
  "injection_x": 4240,
  "injection_y": 1700,
  "injection_z": 7150,
  "product_id": 5,
  "specimen_name": "Chrna2-Cre_OE25-130464",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "MOp",
  "structure_id": 985,
  "structure_name": "Primary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 298273313,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "731/746/972",
  "injection_volume": 0.1997273712,
  "injection_x": 2630,
  "injection_y": 3690,
  "injection_z": 5280,
  "product_id": 35,
  "specimen_name": "Emx1-IRES-Cre-187654",
  "strain": null,
  "structure_abbrev": "ORBm",
  "structure_id": 731,
  "structure_name": "Orbital area, medial part",
  "primary_injection_structure": 731,
  "data_set_id": 478582494,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837563,
   "name": "Slc17a8-iCre"
  },
  "gender": "F",
  "injection_structures": "184/993",
  "injection_volume": 0.0460450158,
  "injection_x": 2930,
  "injection_y": 2340,
  "injection_z": 4450,
  "product_id": 45,
  "specimen_name": "Slc17a8-iCre-350810",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 184,
  "data_set_id": 647807416,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "394/533/312782546",
  "injection_volume": 0.048282444,
  "injection_x": 7720,
  "injection_y": 870,
  "injection_z": 3970,
  "product_id": 36,
  "specimen_name": "Tlx3-Cre_PL56-241024",
  "strain": null,
  "structure_abbrev": "VISam",
  "structure_id": 394,
  "structure_name": "Anteromedial visual area",
  "primary_injection_structure": 394,
  "data_set_id": 518619451,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 183238310,
   "name": "Ntrk1-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "56",
  "injection_volume": 0.0746188562,
  "injection_x": 3760,
  "injection_y": 5350,
  "injection_z": 6460,
  "product_id": 5,
  "specimen_name": "Ntrk1-IRES-Cre_LURC-125683",
  "strain": null,
  "structure_abbrev": "ACB",
  "structure_id": 56,
  "structure_name": "Nucleus accumbens",
  "primary_injection_structure": 56,
  "data_set_id": 309740347,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "329/361/369/182305689",
  "injection_volume": 0.1759204272,
  "injection_x": 6920,
  "injection_y": 1660,
  "injection_z": 8020,
  "product_id": 5,
  "specimen_name": "378-850",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 100142655,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 298725927,
   "name": "Penk-IRES2-Cre-neo"
  },
  "gender": "M",
  "injection_structures": "502/918/926/484682470",
  "injection_volume": 0.1543759994,
  "injection_x": 9450,
  "injection_y": 5180,
  "injection_z": 9740,
  "product_id": 5,
  "specimen_name": "Penk-IRES2-Cre-neo-145872",
  "strain": null,
  "structure_abbrev": "ENTl",
  "structure_id": 918,
  "structure_name": "Entorhinal area, lateral part",
  "primary_injection_structure": 502,
  "data_set_id": 305973112,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "385/417/312782546",
  "injection_volume": 0.0998405478,
  "injection_x": 7740,
  "injection_y": 1230,
  "injection_z": 3240,
  "product_id": 35,
  "specimen_name": "Cux2-IRES-Cre-287722",
  "strain": null,
  "structure_abbrev": "VISrl",
  "structure_id": 417,
  "structure_name": "Rostrolateral visual area",
  "primary_injection_structure": 385,
  "data_set_id": 566454054,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "985",
  "injection_volume": 0.0299198361,
  "injection_x": 4850,
  "injection_y": 2000,
  "injection_z": 7650,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-3936",
  "strain": null,
  "structure_abbrev": "MOp",
  "structure_id": 985,
  "structure_name": "Primary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 287461719,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "44/731/972",
  "injection_volume": 0.1376534656,
  "injection_x": 3350,
  "injection_y": 3320,
  "injection_z": 6400,
  "product_id": 5,
  "specimen_name": "378-1622",
  "strain": "C57BL/6J",
  "structure_abbrev": "PL",
  "structure_id": 972,
  "structure_name": "Prelimbic area",
  "primary_injection_structure": 44,
  "data_set_id": 157711748,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838048,
   "name": "Slc17a6-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "96/101",
  "injection_volume": 0.0918047144,
  "injection_x": 11180,
  "injection_y": 5080,
  "injection_z": 8090,
  "product_id": 5,
  "specimen_name": "Slc17a6-IRES-Cre-3627",
  "strain": "B6.129",
  "structure_abbrev": "DCO",
  "structure_id": 96,
  "structure_name": "Dorsal cochlear nucleus",
  "primary_injection_structure": 96,
  "data_set_id": 286556914,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839513,
   "name": "Drd2-Cre_ER44"
  },
  "gender": "F",
  "injection_structures": "672",
  "injection_volume": 0.101497277,
  "injection_x": 6690,
  "injection_y": 2740,
  "injection_z": 8700,
  "product_id": 5,
  "specimen_name": "Drd2-Cre-2033",
  "strain": "C57BL/6J",
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 672,
  "data_set_id": 160540013,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "F",
  "injection_structures": "385/879/894",
  "injection_volume": 0.0317113692,
  "injection_x": 9810,
  "injection_y": 1910,
  "injection_z": 3310,
  "product_id": 36,
  "specimen_name": "Ntsr1-Cre_GN220-235604",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 514332492,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 310459452,
   "name": "Slc17a8-IRES2-Cre"
  },
  "gender": "M",
  "injection_structures": "15/149",
  "injection_volume": 0.0269189291,
  "injection_x": 5800,
  "injection_y": 4670,
  "injection_z": 6030,
  "product_id": 5,
  "specimen_name": "Slc17a8-IRES2-Cre-257838",
  "strain": null,
  "structure_abbrev": "PT",
  "structure_id": 15,
  "structure_name": "Parataenial nucleus",
  "primary_injection_structure": 15,
  "data_set_id": 545428296,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837281,
   "name": "Calb2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "59/149/362/599",
  "injection_volume": 0.049983396925,
  "injection_x": 6370,
  "injection_y": 3830,
  "injection_z": 5690,
  "product_id": 5,
  "specimen_name": "Calb2-IRES-Cre-227",
  "strain": "C57BL/6J",
  "structure_abbrev": "PVT",
  "structure_id": 149,
  "structure_name": "Paraventricular nucleus of the thalamus",
  "primary_injection_structure": 59,
  "data_set_id": 184157585,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "385/409",
  "injection_volume": 0.00107554969375,
  "injection_x": 8870,
  "injection_y": 2030,
  "injection_z": 2460,
  "product_id": 35,
  "specimen_name": "Ntsr1-Cre_GN220-204693",
  "strain": null,
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 385,
  "data_set_id": 495875751,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182901722,
   "name": "Ppp1r17-Cre_NL146"
  },
  "gender": "F",
  "injection_structures": "294/302",
  "injection_volume": 0.0356931773274658,
  "injection_x": 8600,
  "injection_y": 1580,
  "injection_z": 6100,
  "product_id": 5,
  "specimen_name": "Ppp1r17-Cre_NL146-3952",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "SCs",
  "structure_id": 302,
  "structure_name": "Superior colliculus, sensory related",
  "primary_injection_structure": 294,
  "data_set_id": 287459601,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838331,
   "name": "Cdhr1-Cre_KG66"
  },
  "gender": "M",
  "injection_structures": "151/159",
  "injection_volume": 0.0113838074,
  "injection_x": 2210,
  "injection_y": 3880,
  "injection_z": 7040,
  "product_id": 5,
  "specimen_name": "Cdhr1-Cre-154",
  "strain": "C57BL/6J",
  "structure_abbrev": "AOB",
  "structure_id": 151,
  "structure_name": "Accessory olfactory bulb",
  "primary_injection_structure": 151,
  "data_set_id": 182686228,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "394/533/879/894",
  "injection_volume": 0.00162560470625,
  "injection_x": 7860,
  "injection_y": 710,
  "injection_z": 4600,
  "product_id": 36,
  "specimen_name": "A930038C07Rik-Tg1-Cre-299922",
  "strain": "C57BL/6J",
  "structure_abbrev": "RSPagl",
  "structure_id": 894,
  "structure_name": "Retrosplenial area, lateral agranular part",
  "primary_injection_structure": 394,
  "data_set_id": 577298618,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "345",
  "injection_volume": 0.1780094736,
  "injection_x": 4470,
  "injection_y": 2700,
  "injection_z": 9140,
  "product_id": 5,
  "specimen_name": "378-1079",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-m",
  "structure_id": 345,
  "structure_name": "Primary somatosensory area, mouth",
  "primary_injection_structure": 345,
  "data_set_id": 112936582,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839468,
   "name": "Gabra6-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "928",
  "injection_volume": 0.0732877712,
  "injection_x": 11820,
  "injection_y": 2200,
  "injection_z": 6180,
  "product_id": 5,
  "specimen_name": "Gabra6-IRES-Cre-182",
  "strain": "B6.129",
  "structure_abbrev": "CUL",
  "structure_id": 928,
  "structure_name": "Culmen",
  "primary_injection_structure": 928,
  "data_set_id": 181128406,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "F",
  "injection_structures": "329/417",
  "injection_volume": 0.1630097896,
  "injection_x": 7810,
  "injection_y": 1760,
  "injection_z": 8760,
  "product_id": 5,
  "specimen_name": "Ntsr1-Cre-3619",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 166533924,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761781,
   "name": "Efr3a-Cre_NO108"
  },
  "gender": "F",
  "injection_structures": "409/312782574",
  "injection_volume": 0.0055655700625,
  "injection_x": 8730,
  "injection_y": 1840,
  "injection_z": 9210,
  "product_id": 5,
  "specimen_name": "Efr3a-Cre_NO108-125130",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 409,
  "data_set_id": 309564818,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837797,
   "name": "Th-Cre_FI172"
  },
  "gender": "M",
  "injection_structures": "128/147/162/460/616/795/867/898/1052",
  "injection_volume": 0.0839578936,
  "injection_x": 10420,
  "injection_y": 4130,
  "injection_z": 6530,
  "product_id": 5,
  "specimen_name": "Th-Cre_FI172-131671",
  "strain": "B6.FVB",
  "structure_abbrev": "PB",
  "structure_id": 867,
  "structure_name": "Parabrachial nucleus",
  "primary_injection_structure": 128,
  "data_set_id": 301016175,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837777,
   "name": "Sim1-Cre_KJ18"
  },
  "gender": "F",
  "injection_structures": "993",
  "injection_volume": 0.1086211812,
  "injection_x": 3170,
  "injection_y": 2060,
  "injection_z": 7690,
  "product_id": 5,
  "specimen_name": "Sim1-Cre_KJ18-125418",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 297946935,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "385/409",
  "injection_volume": 0.0043676829875,
  "injection_x": 9080,
  "injection_y": 1740,
  "injection_z": 2410,
  "product_id": 36,
  "specimen_name": "A930038C07Rik-Tg1-Cre-190112",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 479203646,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 183238310,
   "name": "Ntrk1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "672",
  "injection_volume": 0.0168787115,
  "injection_x": 6030,
  "injection_y": 3250,
  "injection_z": 8390,
  "product_id": 5,
  "specimen_name": "Ntrk1-IRES-Cre-3573",
  "strain": "B6.129",
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 672,
  "data_set_id": 287995180,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837324,
   "name": "Slc18a2-Cre_OZ14"
  },
  "gender": "M",
  "injection_structures": "250/258/266/310/564/581",
  "injection_volume": 0.166972189414844,
  "injection_x": 5060,
  "injection_y": 4450,
  "injection_z": 6200,
  "product_id": 5,
  "specimen_name": "Slc18a2-Cre_OZ14-122350",
  "strain": null,
  "structure_abbrev": "LSr",
  "structure_id": 258,
  "structure_name": "Lateral septal nucleus, rostral (rostroventral) part",
  "primary_injection_structure": 250,
  "data_set_id": 301672755,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839285,
   "name": "Gpr26-Cre_KO250"
  },
  "gender": "F",
  "injection_structures": "879/886",
  "injection_volume": 0.0640251297,
  "injection_x": 7000,
  "injection_y": 900,
  "injection_z": 6000,
  "product_id": 5,
  "specimen_name": "Gpr26-Cre_KO250-94",
  "strain": null,
  "structure_abbrev": "RSPv",
  "structure_id": 886,
  "structure_name": "Retrosplenial area, ventral part",
  "primary_injection_structure": 879,
  "data_set_id": 267658040,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761781,
   "name": "Efr3a-Cre_NO108"
  },
  "gender": "F",
  "injection_structures": "39/993",
  "injection_volume": 0.4741441096,
  "injection_x": 4280,
  "injection_y": 2630,
  "injection_z": 6630,
  "product_id": 5,
  "specimen_name": "Efr3a-Cre_NO108-79",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 39,
  "data_set_id": 287995889,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "319/647/780",
  "injection_volume": 0.0866657904,
  "injection_x": 7390,
  "injection_y": 6150,
  "injection_z": 8270,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre_KL100-137547",
  "strain": "C57BL/6J",
  "structure_abbrev": "PA",
  "structure_id": 780,
  "structure_name": "Posterior amygdalar nucleus",
  "primary_injection_structure": 319,
  "data_set_id": 304721447,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761781,
   "name": "Efr3a-Cre_NO108"
  },
  "gender": "F",
  "injection_structures": "39/993",
  "injection_volume": 0.0807576938,
  "injection_x": 3290,
  "injection_y": 2420,
  "injection_z": 6520,
  "product_id": 5,
  "specimen_name": "Efr3a-Cre_NO108-50",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 39,
  "data_set_id": 278181912,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "378/1002/1011/1018",
  "injection_volume": 0.3036722472,
  "injection_x": 7720,
  "injection_y": 2650,
  "injection_z": 9460,
  "product_id": 5,
  "specimen_name": "378-1628",
  "strain": "C57BL/6J",
  "structure_abbrev": "AUDd",
  "structure_id": 1011,
  "structure_name": "Dorsal auditory area",
  "primary_injection_structure": 378,
  "data_set_id": 158314278,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182693192,
   "name": "Chrna2-Cre_OE25"
  },
  "gender": "F",
  "injection_structures": "541/312782628",
  "injection_volume": 0.0651812504,
  "injection_x": 9180,
  "injection_y": 2860,
  "injection_z": 9900,
  "product_id": 5,
  "specimen_name": "Chrna2-Cre_OE25-3907",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "TEa",
  "structure_id": 541,
  "structure_name": "Temporal association areas",
  "primary_injection_structure": 541,
  "data_set_id": 292208876,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839044,
   "name": "Lepr-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "194/830/946",
  "injection_volume": 0.03417774255,
  "injection_x": 7150,
  "injection_y": 5870,
  "injection_z": 6040,
  "product_id": 5,
  "specimen_name": "Lepr-IRES-Cre-Homo-340",
  "strain": "B6.129",
  "structure_abbrev": "DMH",
  "structure_id": 830,
  "structure_name": "Dorsomedial nucleus of the hypothalamus",
  "primary_injection_structure": 194,
  "data_set_id": 160317628,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "385/879/894",
  "injection_volume": 0.2377390624,
  "injection_x": 9490,
  "injection_y": 1440,
  "injection_z": 3990,
  "product_id": 35,
  "specimen_name": "Cux2-IRES-Cre-280582",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 560725737,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 256980970,
   "name": "Lypd6-Cre_KL156"
  },
  "gender": "M",
  "injection_structures": "128/867/1052",
  "injection_volume": 0.0746379613,
  "injection_x": 9960,
  "injection_y": 4320,
  "injection_z": 6680,
  "product_id": 5,
  "specimen_name": "Lypd6-Cre_KL156-126613",
  "strain": null,
  "structure_abbrev": "PPN",
  "structure_id": 1052,
  "structure_name": "Pedunculopontine nucleus",
  "primary_injection_structure": 128,
  "data_set_id": 298079222,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 256980970,
   "name": "Lypd6-Cre_KL156"
  },
  "gender": "F",
  "injection_structures": "797",
  "injection_volume": 0.0069725891375,
  "injection_x": 6840,
  "injection_y": 5530,
  "injection_z": 6580,
  "product_id": 5,
  "specimen_name": "Lypd6-Cre_KL156-132209",
  "strain": null,
  "structure_abbrev": "ZI",
  "structure_id": 797,
  "structure_name": "Zona incerta",
  "primary_injection_structure": 797,
  "data_set_id": 299760587,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "39/48/972/993",
  "injection_volume": 0.170980404,
  "injection_x": 3750,
  "injection_y": 2140,
  "injection_z": 6220,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-3795",
  "strain": "C57BL/6J",
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 166323896,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761781,
   "name": "Efr3a-Cre_NO108"
  },
  "gender": "M",
  "injection_structures": "385/312782546",
  "injection_volume": 0.0119634382,
  "injection_x": 7980,
  "injection_y": 1300,
  "injection_z": 8120,
  "product_id": 5,
  "specimen_name": "Efr3a-Cre_NO108-73",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 286773358,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 287226763,
   "name": "Calb1-T2A-dgCre"
  },
  "gender": "F",
  "injection_structures": "985/993",
  "injection_volume": 0.2399521964,
  "injection_x": 4170,
  "injection_y": 1620,
  "injection_z": 7210,
  "product_id": 5,
  "specimen_name": "Calb1-2A-dgCre-175741",
  "strain": null,
  "structure_abbrev": "MOp",
  "structure_id": 985,
  "structure_name": "Primary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 477836675,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "146/534/1052/1093/549009215",
  "injection_volume": 0.1208461912,
  "injection_x": 9790,
  "injection_y": 5290,
  "injection_z": 6640,
  "product_id": 5,
  "specimen_name": "378-1861",
  "strain": "C57BL/6J",
  "structure_abbrev": "PRNr",
  "structure_id": 146,
  "structure_name": "Pontine reticular nucleus",
  "primary_injection_structure": 146,
  "data_set_id": 272700063,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838584,
   "name": "Gad2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "1/126/223/830/1004/1126",
  "injection_volume": 0.145855206875,
  "injection_x": 7620,
  "injection_y": 6700,
  "injection_z": 6140,
  "product_id": 5,
  "specimen_name": "Gad2-IRES-Cre-1152",
  "strain": "C57BL/6J",
  "structure_abbrev": "PVp",
  "structure_id": 126,
  "structure_name": "Periventricular hypothalamic nucleus, posterior part",
  "primary_injection_structure": 1,
  "data_set_id": 168362462,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837777,
   "name": "Sim1-Cre_KJ18"
  },
  "gender": "M",
  "injection_structures": "337/369/985",
  "injection_volume": 0.0774431721,
  "injection_x": 5240,
  "injection_y": 1710,
  "injection_z": 7610,
  "product_id": 5,
  "specimen_name": "Sim1-Cre_KJ18-123149",
  "strain": null,
  "structure_abbrev": "SSp-ll",
  "structure_id": 337,
  "structure_name": "Primary somatosensory area, lower limb",
  "primary_injection_structure": 337,
  "data_set_id": 297714071,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "985/993",
  "injection_volume": 0.2200984156,
  "injection_x": 3560,
  "injection_y": 1910,
  "injection_z": 7190,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre-44",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 120814821,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839425,
   "name": "Pdzk1ip1-Cre_KD31"
  },
  "gender": "M",
  "injection_structures": "4",
  "injection_volume": 0.0640784711,
  "injection_x": 10670,
  "injection_y": 2020,
  "injection_z": 6240,
  "product_id": 5,
  "specimen_name": "Pdzk1ip1-Cre_KD31-252",
  "strain": "C57BL/6J",
  "structure_abbrev": "IC",
  "structure_id": 4,
  "structure_name": "Inferior colliculus",
  "primary_injection_structure": 4,
  "data_set_id": 266966337,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839090,
   "name": "Slc32a1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "262",
  "injection_volume": 0.027656874,
  "injection_x": 5930,
  "injection_y": 4380,
  "injection_z": 7170,
  "product_id": 5,
  "specimen_name": "Slc32a1-IRES-Cre-2853",
  "strain": "B6.129",
  "structure_abbrev": "RT",
  "structure_id": 262,
  "structure_name": "Reticular nucleus of the thalamus",
  "primary_injection_structure": 262,
  "data_set_id": 171064488,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838496,
   "name": "Grp-Cre_KH288"
  },
  "gender": "F",
  "injection_structures": "39/44/731/972",
  "injection_volume": 0.0651092083772949,
  "injection_x": 2720,
  "injection_y": 3180,
  "injection_z": 5850,
  "product_id": 5,
  "specimen_name": "Grp-Cre_KH288-3905",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "ORBm",
  "structure_id": 731,
  "structure_name": "Orbital area, medial part",
  "primary_injection_structure": 39,
  "data_set_id": 292477301,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761781,
   "name": "Efr3a-Cre_NO108"
  },
  "gender": "M",
  "injection_structures": "394/312782546",
  "injection_volume": 0.0556067729,
  "injection_x": 7310,
  "injection_y": 1050,
  "injection_z": 7300,
  "product_id": 5,
  "specimen_name": "Efr3a-Cre_NO108-33",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "VISa",
  "structure_id": 312782546,
  "structure_name": "Anterior area",
  "primary_injection_structure": 394,
  "data_set_id": 267761438,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "385/409/425",
  "injection_volume": 0.202187328,
  "injection_x": 9620,
  "injection_y": 1720,
  "injection_z": 9200,
  "product_id": 5,
  "specimen_name": "378-1407",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 127138787,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "385/879/894",
  "injection_volume": 0.0320947011,
  "injection_x": 9270,
  "injection_y": 1380,
  "injection_z": 3880,
  "product_id": 35,
  "specimen_name": "Ntsr1-Cre_GN220-261405",
  "strain": null,
  "structure_abbrev": "RSPd",
  "structure_id": 879,
  "structure_name": "Retrosplenial area, dorsal part",
  "primary_injection_structure": 385,
  "data_set_id": 543680289,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "723/746",
  "injection_volume": 0.1098276102,
  "injection_x": 2690,
  "injection_y": 3940,
  "injection_z": 6590,
  "product_id": 5,
  "specimen_name": "378-1641",
  "strain": "C57BL/6J",
  "structure_abbrev": "ORBvl",
  "structure_id": 746,
  "structure_name": "Orbital area, ventrolateral part",
  "primary_injection_structure": 723,
  "data_set_id": 158435116,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838502,
   "name": "Grik4-Cre"
  },
  "gender": "M",
  "injection_structures": "15/149/186/362",
  "injection_volume": 0.0890037764,
  "injection_x": 6280,
  "injection_y": 3470,
  "injection_z": 5940,
  "product_id": 5,
  "specimen_name": "Grik4-Cre-5026",
  "strain": "C57BL/6J",
  "structure_abbrev": "MD",
  "structure_id": 362,
  "structure_name": "Mediodorsal nucleus of thalamus",
  "primary_injection_structure": 15,
  "data_set_id": 168301446,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0456806567,
  "injection_x": 9350,
  "injection_y": 1910,
  "injection_z": 2970,
  "product_id": 35,
  "specimen_name": "Ntsr1-Cre_GN220-257917",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 539738599,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839406,
   "name": "Nos1-CreERT2"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.02429098805,
  "injection_x": 9780,
  "injection_y": 1090,
  "injection_z": 8490,
  "product_id": 5,
  "specimen_name": "Nos1-CreERT2-120349",
  "strain": "B6.129",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 293116943,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "F",
  "injection_structures": "104/345/1057",
  "injection_volume": 0.186192748,
  "injection_x": 4180,
  "injection_y": 3000,
  "injection_z": 9090,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg3-Cre-4911",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-m",
  "structure_id": 345,
  "structure_name": "Primary somatosensory area, mouth",
  "primary_injection_structure": 104,
  "data_set_id": 277852088,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838942,
   "name": "Htr2a-Cre_KM207"
  },
  "gender": "F",
  "injection_structures": "985/993",
  "injection_volume": 0.1920817248,
  "injection_x": 3290,
  "injection_y": 1930,
  "injection_z": 7550,
  "product_id": 5,
  "specimen_name": "Htr2a-Cre_KM207-3449",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 258915564,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 305451902,
   "name": "Dlg3-Cre_KG118"
  },
  "gender": "F",
  "injection_structures": "463",
  "injection_volume": 0.215645766,
  "injection_x": 7260,
  "injection_y": 3020,
  "injection_z": 8440,
  "product_id": 5,
  "specimen_name": "Dlg3-Cre_KG118-180001",
  "strain": "B6.FVB",
  "structure_abbrev": "CA3",
  "structure_id": 463,
  "structure_name": "Field CA3",
  "primary_injection_structure": 463,
  "data_set_id": 477926293,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "385/879/894",
  "injection_volume": 0.1613077256,
  "injection_x": 9570,
  "injection_y": 1140,
  "injection_z": 4280,
  "product_id": 36,
  "specimen_name": "Cux2-IRES-Cre-239319",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 517077687,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "329/378",
  "injection_volume": 0.01130260215,
  "injection_x": 7570,
  "injection_y": 1690,
  "injection_z": 2430,
  "product_id": 35,
  "specimen_name": "A930038C07Rik-Tg1-Cre-347487",
  "strain": null,
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 643749624,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "F",
  "injection_structures": "353/378",
  "injection_volume": 0.0761803735,
  "injection_x": 5820,
  "injection_y": 2520,
  "injection_z": 8930,
  "product_id": 5,
  "specimen_name": "Syt6-Cre-2819",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-n",
  "structure_id": 353,
  "structure_name": "Primary somatosensory area, nose",
  "primary_injection_structure": 353,
  "data_set_id": 168096467,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "345",
  "injection_volume": 0.014307381375,
  "injection_x": 5040,
  "injection_y": 2340,
  "injection_z": 9060,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre_KL100-137106",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-m",
  "structure_id": 345,
  "structure_name": "Primary somatosensory area, mouth",
  "primary_injection_structure": 345,
  "data_set_id": 303785454,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "1002/1011/1018/1027",
  "injection_volume": 0.0983124044,
  "injection_x": 8170,
  "injection_y": 3200,
  "injection_z": 10390,
  "product_id": 5,
  "specimen_name": "378-1191",
  "strain": "C57BL/6J",
  "structure_abbrev": "AUDp",
  "structure_id": 1002,
  "structure_name": "Primary auditory area",
  "primary_injection_structure": 1002,
  "data_set_id": 116903230,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "394/879/894/312782546",
  "injection_volume": 0.163840565,
  "injection_x": 7590,
  "injection_y": 580,
  "injection_z": 4230,
  "product_id": 35,
  "specimen_name": "Rbp4-Cre_KL100-351510",
  "strain": null,
  "structure_abbrev": "RSPagl",
  "structure_id": 894,
  "structure_name": "Retrosplenial area, lateral agranular part",
  "primary_injection_structure": 394,
  "data_set_id": 646525156,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838803,
   "name": "Grm2-Cre_MR90"
  },
  "gender": "F",
  "injection_structures": "59/149/362/609/930",
  "injection_volume": 0.10227162645,
  "injection_x": 6880,
  "injection_y": 3900,
  "injection_z": 5670,
  "product_id": 5,
  "specimen_name": "Grm2-Cre-94",
  "strain": null,
  "structure_abbrev": "IMD",
  "structure_id": 59,
  "structure_name": "Intermediodorsal nucleus of the thalamus",
  "primary_injection_structure": 59,
  "data_set_id": 182185289,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "F",
  "injection_structures": "369/985",
  "injection_volume": 0.0418772816,
  "injection_x": 4950,
  "injection_y": 2160,
  "injection_z": 7760,
  "product_id": 5,
  "specimen_name": "Tlx3-Cre-137308",
  "strain": null,
  "structure_abbrev": "SSp-ul",
  "structure_id": 369,
  "structure_name": "Primary somatosensory area, upper limb",
  "primary_injection_structure": 369,
  "data_set_id": 303615412,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838108,
   "name": "Agrp-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "126/223/491/980/1004/1126",
  "injection_volume": 0.00525109085703125,
  "injection_x": 7670,
  "injection_y": 6970,
  "injection_z": 5720,
  "product_id": 5,
  "specimen_name": "Agrp-IRES-Cre-498",
  "strain": "C57BL/6J",
  "structure_abbrev": "PVp",
  "structure_id": 126,
  "structure_name": "Periventricular hypothalamic nucleus, posterior part",
  "primary_injection_structure": 126,
  "data_set_id": 167117360,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "382/385/417/312782546",
  "injection_volume": 0.3529908648,
  "injection_x": 8060,
  "injection_y": 1340,
  "injection_z": 3410,
  "product_id": 35,
  "specimen_name": "Cux2-IRES-Cre-346555",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 382,
  "data_set_id": 643483018,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 298725927,
   "name": "Penk-IRES2-Cre-neo"
  },
  "gender": "M",
  "injection_structures": "993",
  "injection_volume": 0.015275209775,
  "injection_x": 2590,
  "injection_y": 2360,
  "injection_z": 3770,
  "product_id": 45,
  "specimen_name": "Penk-IRES2-Cre-neo-343077",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 639647390,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839406,
   "name": "Nos1-CreERT2"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.03184096195,
  "injection_x": 9640,
  "injection_y": 1070,
  "injection_z": 8430,
  "product_id": 5,
  "specimen_name": "Nos1-CreERT2-182",
  "strain": "B6.129",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 267931685,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838584,
   "name": "Gad2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "672",
  "injection_volume": 0.1502256896,
  "injection_x": 6180,
  "injection_y": 2820,
  "injection_z": 8500,
  "product_id": 5,
  "specimen_name": "Gad2-IRES-Cre-1096",
  "strain": "C57BL/6J",
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 672,
  "data_set_id": 157911832,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 179697864,
   "name": "Rorb-IRES2-Cre"
  },
  "gender": "M",
  "injection_structures": "329/353/378",
  "injection_volume": 0.0671106352,
  "injection_x": 6220,
  "injection_y": 3350,
  "injection_z": 9940,
  "product_id": 5,
  "specimen_name": "Rorb-IRES2-Cre-D_LURC-124657",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSs",
  "structure_id": 378,
  "structure_name": "Supplemental somatosensory area",
  "primary_injection_structure": 329,
  "data_set_id": 301580339,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "394/312782546",
  "injection_volume": 0.1075468758,
  "injection_x": 7490,
  "injection_y": 690,
  "injection_z": 3900,
  "product_id": 35,
  "specimen_name": "Cux2-IRES-Cre-287721",
  "strain": null,
  "structure_abbrev": "VISam",
  "structure_id": 394,
  "structure_name": "Anteromedial visual area",
  "primary_injection_structure": 394,
  "data_set_id": 566244185,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "879/886/894",
  "injection_volume": 0.00281467361875,
  "injection_x": 8470,
  "injection_y": 920,
  "injection_z": 4450,
  "product_id": 36,
  "specimen_name": "Ntsr1-Cre_GN220-322524",
  "strain": null,
  "structure_abbrev": "RSPagl",
  "structure_id": 894,
  "structure_name": "Retrosplenial area, lateral agranular part",
  "primary_injection_structure": 879,
  "data_set_id": 599136295,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838803,
   "name": "Grm2-Cre_MR90"
  },
  "gender": "F",
  "injection_structures": "385/463/726",
  "injection_volume": 0.1076171908,
  "injection_x": 8550,
  "injection_y": 3520,
  "injection_z": 8460,
  "product_id": 5,
  "specimen_name": "Grm2-Cre_MR90-120",
  "strain": null,
  "structure_abbrev": "DG",
  "structure_id": 726,
  "structure_name": "Dentate gyrus",
  "primary_injection_structure": 385,
  "data_set_id": 264872385,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182901722,
   "name": "Ppp1r17-Cre_NL146"
  },
  "gender": "M",
  "injection_structures": "725/733/1020",
  "injection_volume": 0.265126456,
  "injection_x": 7690,
  "injection_y": 4140,
  "injection_z": 7290,
  "product_id": 5,
  "specimen_name": "Ppp1r17-Cre_NL146-3884",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "VPM",
  "structure_id": 733,
  "structure_name": "Ventral posteromedial nucleus of the thalamus",
  "primary_injection_structure": 725,
  "data_set_id": 268399868,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "385/409/312782574",
  "injection_volume": 0.1300420212,
  "injection_x": 8920,
  "injection_y": 2180,
  "injection_z": 2310,
  "product_id": 35,
  "specimen_name": "Emx1-IRES-Cre-191133",
  "strain": "B6.129",
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 385,
  "data_set_id": 479673887,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838259,
   "name": "Cort-T2A-Cre"
  },
  "gender": "M",
  "injection_structures": "1002/1011/1018",
  "injection_volume": 0.3412455648,
  "injection_x": 7290,
  "injection_y": 3060,
  "injection_z": 10100,
  "product_id": 5,
  "specimen_name": "Cort-T2A-Cre-106",
  "strain": null,
  "structure_abbrev": "AUDp",
  "structure_id": 1002,
  "structure_name": "Primary auditory area",
  "primary_injection_structure": 1002,
  "data_set_id": 165975810,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837281,
   "name": "Calb2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "39/48/993",
  "injection_volume": 0.0188190975987659,
  "injection_x": 5460,
  "injection_y": 1180,
  "injection_z": 5980,
  "product_id": 5,
  "specimen_name": "Calb2-IRES-Cre-179",
  "strain": "C57BL/6J",
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 175719341,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0685708254,
  "injection_x": 9290,
  "injection_y": 1600,
  "injection_z": 3020,
  "product_id": 36,
  "specimen_name": "Rbp4-Cre_KL100-227917",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 503323656,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "1002",
  "injection_volume": 0.0969045854,
  "injection_x": 7760,
  "injection_y": 3170,
  "injection_z": 9630,
  "product_id": 5,
  "specimen_name": "378-1211",
  "strain": "C57BL/6J",
  "structure_abbrev": "AUDp",
  "structure_id": 1002,
  "structure_name": "Primary auditory area",
  "primary_injection_structure": 1002,
  "data_set_id": 120491896,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837481,
   "name": "Scnn1a-Tg2-Cre"
  },
  "gender": "M",
  "injection_structures": "136/661/852",
  "injection_volume": 0.1756295632,
  "injection_x": 10830,
  "injection_y": 6570,
  "injection_z": 7240,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg2-Cre-3453",
  "strain": "C57BL/6J",
  "structure_abbrev": "VII",
  "structure_id": 661,
  "structure_name": "Facial motor nucleus",
  "primary_injection_structure": 136,
  "data_set_id": 177905562,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "382/918/922",
  "injection_volume": 0.354961684,
  "injection_x": 8980,
  "injection_y": 4730,
  "injection_z": 9730,
  "product_id": 5,
  "specimen_name": "378-1175",
  "strain": "C57BL/6J",
  "structure_abbrev": "ENTl",
  "structure_id": 918,
  "structure_name": "Entorhinal area, lateral part",
  "primary_injection_structure": 382,
  "data_set_id": 114472145,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839406,
   "name": "Nos1-CreERT2"
  },
  "gender": "M",
  "injection_structures": "1007",
  "injection_volume": 0.0731994438,
  "injection_x": 10870,
  "injection_y": 2610,
  "injection_z": 7880,
  "product_id": 5,
  "specimen_name": "Nos1-CreERT2-126105",
  "strain": "B6.129",
  "structure_abbrev": "SIM",
  "structure_id": 1007,
  "structure_name": "Simple lobule",
  "primary_injection_structure": 1007,
  "data_set_id": 300622004,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838828,
   "name": "Gal-Cre_KI87"
  },
  "gender": "F",
  "injection_structures": "64/127/255/1113",
  "injection_volume": 0.184191588,
  "injection_x": 5850,
  "injection_y": 4300,
  "injection_z": 6470,
  "product_id": 5,
  "specimen_name": "Gal-Cre-1298",
  "strain": "C57BL/6J",
  "structure_abbrev": "AV",
  "structure_id": 255,
  "structure_name": "Anteroventral nucleus of thalamus",
  "primary_injection_structure": 64,
  "data_set_id": 175818392,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.2334350788,
  "injection_x": 9060,
  "injection_y": 1290,
  "injection_z": 3480,
  "product_id": 36,
  "specimen_name": "Cux2-IRES-Cre-214772",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 501116471,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "329/385/417/312782546",
  "injection_volume": 0.1763110356,
  "injection_x": 7940,
  "injection_y": 1010,
  "injection_z": 2900,
  "product_id": 36,
  "specimen_name": "Cux2-IRES-Cre-239321",
  "strain": null,
  "structure_abbrev": "VISrl",
  "structure_id": 417,
  "structure_name": "Rostrolateral visual area",
  "primary_injection_structure": 329,
  "data_set_id": 517325325,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "39/48",
  "injection_volume": 0.1230279064,
  "injection_x": 4290,
  "injection_y": 2330,
  "injection_z": 6120,
  "product_id": 5,
  "specimen_name": "378-949",
  "strain": "C57BL/6J",
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 112458114,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "541/312782574/312782628",
  "injection_volume": 0.181428576,
  "injection_x": 9100,
  "injection_y": 2240,
  "injection_z": 9660,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre_KL100-129286",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISpor",
  "structure_id": 312782628,
  "structure_name": "Postrhinal area",
  "primary_injection_structure": 541,
  "data_set_id": 298178204,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 301467016,
   "name": "Npr3-IRES2-Cre"
  },
  "gender": "F",
  "injection_structures": "502/879/886",
  "injection_volume": 0.05660676,
  "injection_x": 7260,
  "injection_y": 1330,
  "injection_z": 6080,
  "product_id": 5,
  "specimen_name": "Npr3-IRES2-Cre-299665",
  "strain": null,
  "structure_abbrev": "RSPv",
  "structure_id": 886,
  "structure_name": "Retrosplenial area, ventral part",
  "primary_injection_structure": 502,
  "data_set_id": 586042591,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838502,
   "name": "Grik4-Cre"
  },
  "gender": "M",
  "injection_structures": "502/918/926",
  "injection_volume": 0.0476933709,
  "injection_x": 9510,
  "injection_y": 5400,
  "injection_z": 8960,
  "product_id": 5,
  "specimen_name": "Grik4-Cre-285275",
  "strain": "unknown",
  "structure_abbrev": "ENTm",
  "structure_id": 926,
  "structure_name": "Entorhinal area, medial part, dorsal zone",
  "primary_injection_structure": 502,
  "data_set_id": 585775993,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837858,
   "name": "Pvalb-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "936/1007/1017",
  "injection_volume": 0.2250135076,
  "injection_x": 11640,
  "injection_y": 2350,
  "injection_z": 7860,
  "product_id": 5,
  "specimen_name": "Pvalb-IRES-Cre-5035",
  "strain": "B6.129",
  "structure_abbrev": "AN",
  "structure_id": 1017,
  "structure_name": "Ansiform lobule",
  "primary_injection_structure": 936,
  "data_set_id": 167201465,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "369/985",
  "injection_volume": 0.0965105274,
  "injection_x": 5080,
  "injection_y": 2200,
  "injection_z": 7980,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre_KL100-294",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-ul",
  "structure_id": 369,
  "structure_name": "Primary somatosensory area, upper limb",
  "primary_injection_structure": 369,
  "data_set_id": 286312782,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "394/312782546",
  "injection_volume": 0.0627471803,
  "injection_x": 7450,
  "injection_y": 1250,
  "injection_z": 7410,
  "product_id": 5,
  "specimen_name": "Ntsr1-Cre-3507",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISam",
  "structure_id": 394,
  "structure_name": "Anteromedial visual area",
  "primary_injection_structure": 394,
  "data_set_id": 156671933,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 183238310,
   "name": "Ntrk1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "101",
  "injection_volume": 0.02883530685,
  "injection_x": 10450,
  "injection_y": 5040,
  "injection_z": 8290,
  "product_id": 5,
  "specimen_name": "Ntrk1-IRES-Cre-3525",
  "strain": "B6.129",
  "structure_abbrev": "VCO",
  "structure_id": 101,
  "structure_name": "Ventral cochlear nucleus",
  "primary_injection_structure": 101,
  "data_set_id": 265135682,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839494,
   "name": "Hdc-Cre_IM1"
  },
  "gender": "M",
  "injection_structures": "194/210/525",
  "injection_volume": 0.00533816535,
  "injection_x": 8070,
  "injection_y": 5990,
  "injection_z": 6570,
  "product_id": 5,
  "specimen_name": "Hdc-Cre-90",
  "strain": null,
  "structure_abbrev": "LM",
  "structure_id": 210,
  "structure_name": "Lateral mammillary nucleus",
  "primary_injection_structure": 194,
  "data_set_id": 157952778,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838633,
   "name": "Trib2-F2A-CreERT2"
  },
  "gender": "F",
  "injection_structures": "1002",
  "injection_volume": 0.011713409575,
  "injection_x": 7830,
  "injection_y": 2690,
  "injection_z": 10080,
  "product_id": 5,
  "specimen_name": "Trib2-2A-CreERT2-D-3220",
  "strain": "B6.129",
  "structure_abbrev": "AUDp",
  "structure_id": 1002,
  "structure_name": "Primary auditory area",
  "primary_injection_structure": 1002,
  "data_set_id": 182686935,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "409/425/312782574/312782628",
  "injection_volume": 0.2522034312,
  "injection_x": 9330,
  "injection_y": 2530,
  "injection_z": 2250,
  "product_id": 36,
  "specimen_name": "Cux2-IRES-Cre-270245",
  "strain": null,
  "structure_abbrev": "VISpor",
  "structure_id": 312782628,
  "structure_name": "Postrhinal area",
  "primary_injection_structure": 409,
  "data_set_id": 552544756,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "385/843/484682508",
  "injection_volume": 0.02514612135,
  "injection_x": 9860,
  "injection_y": 2770,
  "injection_z": 8370,
  "product_id": 5,
  "specimen_name": "378-1441",
  "strain": "C57BL/6J",
  "structure_abbrev": "PAR",
  "structure_id": 843,
  "structure_name": "Parasubiculum",
  "primary_injection_structure": 385,
  "data_set_id": 127711098,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 305451902,
   "name": "Dlg3-Cre_KG118"
  },
  "gender": "M",
  "injection_structures": "574",
  "injection_volume": 0.03144232735,
  "injection_x": 9450,
  "injection_y": 6070,
  "injection_z": 6300,
  "product_id": 5,
  "specimen_name": "Dlg3-Cre_KG118-180916",
  "strain": "B6.FVB",
  "structure_abbrev": "TRN",
  "structure_id": 574,
  "structure_name": "Tegmental reticular nucleus",
  "primary_injection_structure": 574,
  "data_set_id": 478818791,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838828,
   "name": "Gal-Cre_KI87"
  },
  "gender": "M",
  "injection_structures": "127/362/599/907/1113",
  "injection_volume": 0.01408530725,
  "injection_x": 6350,
  "injection_y": 4220,
  "injection_z": 6330,
  "product_id": 5,
  "specimen_name": "Gal-Cre-1287",
  "strain": "C57BL/6J",
  "structure_abbrev": "IAD",
  "structure_id": 1113,
  "structure_name": "Interanterodorsal nucleus of the thalamus",
  "primary_injection_structure": 127,
  "data_set_id": 168095041,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839406,
   "name": "Nos1-CreERT2"
  },
  "gender": "M",
  "injection_structures": "329/182305689",
  "injection_volume": 0.0387887136,
  "injection_x": 6350,
  "injection_y": 1690,
  "injection_z": 8770,
  "product_id": 5,
  "specimen_name": "Nos1-CreERT2-116",
  "strain": "B6.129",
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 182043055,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 301613169,
   "name": "Sepw1-Cre_NP39"
  },
  "gender": "F",
  "injection_structures": "39/48",
  "injection_volume": 0.00399318976875,
  "injection_x": 4960,
  "injection_y": 2360,
  "injection_z": 5900,
  "product_id": 5,
  "specimen_name": "Sepw1-Cre_NP39-174466",
  "strain": null,
  "structure_abbrev": "ACAv",
  "structure_id": 48,
  "structure_name": "Anterior cingulate area, ventral part",
  "primary_injection_structure": 39,
  "data_set_id": 478780588,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837481,
   "name": "Scnn1a-Tg2-Cre"
  },
  "gender": "F",
  "injection_structures": "170",
  "injection_volume": 0.0270298504,
  "injection_x": 7790,
  "injection_y": 3260,
  "injection_z": 8060,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg2-Cre-338214",
  "strain": null,
  "structure_abbrev": "LGd",
  "structure_id": 170,
  "structure_name": "Dorsal part of the lateral geniculate complex",
  "primary_injection_structure": 170,
  "data_set_id": 642177206,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "672/1022",
  "injection_volume": 0.140029309,
  "injection_x": 6320,
  "injection_y": 3970,
  "injection_z": 8250,
  "product_id": 5,
  "specimen_name": "378-931",
  "strain": "C57BL/6J",
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 672,
  "data_set_id": 112307754,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "39/44/972",
  "injection_volume": 0.260149548015308,
  "injection_x": 3520,
  "injection_y": 3400,
  "injection_z": 6290,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-301",
  "strain": "B6.129",
  "structure_abbrev": "PL",
  "structure_id": 972,
  "structure_name": "Prelimbic area",
  "primary_injection_structure": 39,
  "data_set_id": 278433737,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838681,
   "name": "Slc6a4-Cre_ET33"
  },
  "gender": "M",
  "injection_structures": "83/222",
  "injection_volume": 0.0206274908,
  "injection_x": 12540,
  "injection_y": 6560,
  "injection_z": 5690,
  "product_id": 5,
  "specimen_name": "Slc6a4-Cre-Homo-281",
  "strain": "C57BL/6J",
  "structure_abbrev": "RO",
  "structure_id": 222,
  "structure_name": "Nucleus raphe obscurus",
  "primary_injection_structure": 83,
  "data_set_id": 133286781,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.5185301128,
  "injection_x": 9060,
  "injection_y": 1450,
  "injection_z": 7760,
  "product_id": 5,
  "specimen_name": "C57BL/6-154701",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 307593747,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837777,
   "name": "Sim1-Cre_KJ18"
  },
  "gender": "M",
  "injection_structures": "993",
  "injection_volume": 0.00764711885,
  "injection_x": 3180,
  "injection_y": 2460,
  "injection_z": 7500,
  "product_id": 5,
  "specimen_name": "Sim1-Cre_KJ18-318904",
  "strain": "unknown",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 596798450,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838496,
   "name": "Grp-Cre_KH288"
  },
  "gender": "F",
  "injection_structures": "39/44/972/993",
  "injection_volume": 0.1217812876,
  "injection_x": 3150,
  "injection_y": 2360,
  "injection_z": 6010,
  "product_id": 5,
  "specimen_name": "Grp-Cre_KH288-130",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "PL",
  "structure_id": 972,
  "structure_name": "Prelimbic area",
  "primary_injection_structure": 39,
  "data_set_id": 266582782,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "12/100/146/679/607344830",
  "injection_volume": 0.1226759743125,
  "injection_x": 8880,
  "injection_y": 5500,
  "injection_z": 5690,
  "product_id": 5,
  "specimen_name": "378-1791",
  "strain": "C57BL/6J",
  "structure_abbrev": "IPN",
  "structure_id": 100,
  "structure_name": "Interpeduncular nucleus",
  "primary_injection_structure": 12,
  "data_set_id": 175158844,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 309779993,
   "name": "Ctgf-T2A-dgCre"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0231274757,
  "injection_x": 8470,
  "injection_y": 1510,
  "injection_z": 8070,
  "product_id": 5,
  "specimen_name": "Ctgf-T2A-dgCre-335948",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 626118851,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839090,
   "name": "Slc32a1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "262/718",
  "injection_volume": 0.1330624302,
  "injection_x": 6660,
  "injection_y": 3810,
  "injection_z": 8110,
  "product_id": 5,
  "specimen_name": "Slc32a1-IRES-Cre-2854",
  "strain": "B6.129",
  "structure_abbrev": "RT",
  "structure_id": 262,
  "structure_name": "Reticular nucleus of the thalamus",
  "primary_injection_structure": 262,
  "data_set_id": 171065200,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0614102398,
  "injection_x": 8750,
  "injection_y": 1310,
  "injection_z": 2810,
  "product_id": 35,
  "specimen_name": "Scnn1a-Tg3-Cre-371018",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 666909423,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838584,
   "name": "Gad2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "985/993",
  "injection_volume": 0.2033732456,
  "injection_x": 3800,
  "injection_y": 1780,
  "injection_z": 7240,
  "product_id": 5,
  "specimen_name": "Gad2-IRES-Cre-1142",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 167440619,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "F",
  "injection_structures": "985",
  "injection_volume": 0.020452824175,
  "injection_x": 4870,
  "injection_y": 2440,
  "injection_z": 7520,
  "product_id": 5,
  "specimen_name": "Ntsr1-Cre-3574",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOp",
  "structure_id": 985,
  "structure_name": "Primary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 159651060,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837779,
   "name": "Pcp2-Cre_GN135"
  },
  "gender": "M",
  "injection_structures": "1007/1017",
  "injection_volume": 0.2569607824,
  "injection_x": 11200,
  "injection_y": 2820,
  "injection_z": 7840,
  "product_id": 5,
  "specimen_name": "Pcp2-Cre (NH)-3625",
  "strain": "C57BL/6J",
  "structure_abbrev": "SIM",
  "structure_id": 1007,
  "structure_name": "Simple lobule",
  "primary_injection_structure": 1007,
  "data_set_id": 168299316,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837328,
   "name": "Syt17-Cre_NO14"
  },
  "gender": "F",
  "injection_structures": "952/961/966",
  "injection_volume": 0.1191577688,
  "injection_x": 7300,
  "injection_y": 5950,
  "injection_z": 9560,
  "product_id": 5,
  "specimen_name": "Syt17-Cre_NO14-131",
  "strain": null,
  "structure_abbrev": "PIR",
  "structure_id": 961,
  "structure_name": "Piriform area",
  "primary_injection_structure": 952,
  "data_set_id": 184259031,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 301613169,
   "name": "Sepw1-Cre_NP39"
  },
  "gender": "M",
  "injection_structures": "312782546",
  "injection_volume": 0.015373173025,
  "injection_x": 7230,
  "injection_y": 560,
  "injection_z": 7640,
  "product_id": 5,
  "specimen_name": "Sepw1-Cre_NP39-184055",
  "strain": null,
  "structure_abbrev": "VISa",
  "structure_id": 312782546,
  "structure_name": "Anterior area",
  "primary_injection_structure": 312782546,
  "data_set_id": 478782005,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "378/1002/1011",
  "injection_volume": 0.3707403112,
  "injection_x": 7530,
  "injection_y": 2740,
  "injection_z": 9430,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-105",
  "strain": null,
  "structure_abbrev": "AUDd",
  "structure_id": 1011,
  "structure_name": "Dorsal auditory area",
  "primary_injection_structure": 378,
  "data_set_id": 176430283,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838496,
   "name": "Grp-Cre_KH288"
  },
  "gender": "M",
  "injection_structures": "104/119",
  "injection_volume": 0.16103948,
  "injection_x": 3850,
  "injection_y": 4600,
  "injection_z": 8580,
  "product_id": 5,
  "specimen_name": "Grp-Cre-76",
  "strain": "B6.FVB(ICR)",
  "structure_abbrev": "AId",
  "structure_id": 104,
  "structure_name": "Agranular insular area, dorsal part",
  "primary_injection_structure": 104,
  "data_set_id": 184168899,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "39/48",
  "injection_volume": 0.3920265776,
  "injection_x": 4400,
  "injection_y": 2310,
  "injection_z": 6200,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre-83",
  "strain": null,
  "structure_abbrev": "ACAv",
  "structure_id": 48,
  "structure_name": "Anterior cingulate area, ventral part",
  "primary_injection_structure": 39,
  "data_set_id": 161458737,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "329",
  "injection_volume": 0.1789803008,
  "injection_x": 6800,
  "injection_y": 1170,
  "injection_z": 2420,
  "product_id": 36,
  "specimen_name": "Cux2-IRES-Cre-280584",
  "strain": null,
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 560736273,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "39/993",
  "injection_volume": 0.3059074508,
  "injection_x": 4380,
  "injection_y": 2540,
  "injection_z": 6710,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre-130",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 39,
  "data_set_id": 168162771,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182693192,
   "name": "Chrna2-Cre_OE25"
  },
  "gender": "F",
  "injection_structures": "39/48/972",
  "injection_volume": 0.1122949072,
  "injection_x": 3870,
  "injection_y": 2540,
  "injection_z": 6310,
  "product_id": 5,
  "specimen_name": "Chrna2-Cre_OE25-3896",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 292125472,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "583/952/961",
  "injection_volume": 0.2898555996,
  "injection_x": 4520,
  "injection_y": 5590,
  "injection_z": 8660,
  "product_id": 5,
  "specimen_name": "378-1318",
  "strain": "C57BL/6J",
  "structure_abbrev": "PIR",
  "structure_id": 961,
  "structure_name": "Piriform area",
  "primary_injection_structure": 583,
  "data_set_id": 127907465,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "345/369/182305689",
  "injection_volume": 0.030884602,
  "injection_x": 4830,
  "injection_y": 2260,
  "injection_z": 8590,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-125535",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-m",
  "structure_id": 345,
  "structure_name": "Primary somatosensory area, mouth",
  "primary_injection_structure": 345,
  "data_set_id": 298796577,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838710,
   "name": "Sst-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0676115671,
  "injection_x": 9550,
  "injection_y": 1040,
  "injection_z": 8260,
  "product_id": 5,
  "specimen_name": "Sst-IRES-Cre-230",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 288261928,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838665,
   "name": "Vipr2-Cre_KE2"
  },
  "gender": "M",
  "injection_structures": "189/362/366/599/685/741/907",
  "injection_volume": 0.088655504,
  "injection_x": 7170,
  "injection_y": 4780,
  "injection_z": 6180,
  "product_id": 5,
  "specimen_name": "Vipr2-Cre_KE2-3327",
  "strain": null,
  "structure_abbrev": "SMT",
  "structure_id": 366,
  "structure_name": "Submedial nucleus of the thalamus",
  "primary_injection_structure": 189,
  "data_set_id": 268163228,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838927,
   "name": "Pcdh9-Cre_NP276"
  },
  "gender": "F",
  "injection_structures": "926",
  "injection_volume": 0.0248089744,
  "injection_x": 9890,
  "injection_y": 3290,
  "injection_z": 9190,
  "product_id": 5,
  "specimen_name": "Pcdh9-Cre-124673",
  "strain": "C57BL/6J",
  "structure_abbrev": "ENTm",
  "structure_id": 926,
  "structure_name": "Entorhinal area, medial part, dorsal zone",
  "primary_injection_structure": 926,
  "data_set_id": 308642254,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "385/879/894",
  "injection_volume": 0.1299575452,
  "injection_x": 9790,
  "injection_y": 1150,
  "injection_z": 7230,
  "product_id": 5,
  "specimen_name": "378-1764",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 174361040,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 460114142,
   "name": "Tacr1-T2A-Cre"
  },
  "gender": "M",
  "injection_structures": "4/616/795/867",
  "injection_volume": 0.02858131045,
  "injection_x": 10330,
  "injection_y": 1740,
  "injection_z": 6620,
  "product_id": 5,
  "specimen_name": "Tacr1-2A-Cre-281299",
  "strain": null,
  "structure_abbrev": "PB",
  "structure_id": 867,
  "structure_name": "Parabrachial nucleus",
  "primary_injection_structure": 4,
  "data_set_id": 565721603,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "1002/1011",
  "injection_volume": 0.0734511372,
  "injection_x": 7620,
  "injection_y": 2970,
  "injection_z": 9780,
  "product_id": 5,
  "specimen_name": "378-905",
  "strain": "C57BL/6J",
  "structure_abbrev": "AUDp",
  "structure_id": 1002,
  "structure_name": "Primary auditory area",
  "primary_injection_structure": 1002,
  "data_set_id": 100149109,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838942,
   "name": "Htr2a-Cre_KM207"
  },
  "gender": "F",
  "injection_structures": "184/723/993",
  "injection_volume": 0.1903711152,
  "injection_x": 2060,
  "injection_y": 2540,
  "injection_z": 7280,
  "product_id": 5,
  "specimen_name": "Htr2a-Cre_KM207-120416",
  "strain": null,
  "structure_abbrev": "FRP",
  "structure_id": 184,
  "structure_name": "Frontal pole, cerebral cortex",
  "primary_injection_structure": 184,
  "data_set_id": 293433996,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "402/409/1027/312782574",
  "injection_volume": 0.0187084009,
  "injection_x": 8500,
  "injection_y": 2150,
  "injection_z": 2180,
  "product_id": 36,
  "specimen_name": "Ntsr1-Cre_GN220-238036",
  "strain": null,
  "structure_abbrev": "VISal",
  "structure_id": 402,
  "structure_name": "Anterolateral visual area",
  "primary_injection_structure": 402,
  "data_set_id": 516274127,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "395/429/437/1098",
  "injection_volume": 0.2266317816,
  "injection_x": 12550,
  "injection_y": 6710,
  "injection_z": 7450,
  "product_id": 5,
  "specimen_name": "378-1547",
  "strain": "C57BL/6J",
  "structure_abbrev": "SPVC",
  "structure_id": 429,
  "structure_name": "Spinal nucleus of the trigeminal, caudal part",
  "primary_injection_structure": 395,
  "data_set_id": 147215105,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838710,
   "name": "Sst-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "1039",
  "injection_volume": 0.001334277503125,
  "injection_x": 13020,
  "injection_y": 5120,
  "injection_z": 6050,
  "product_id": 5,
  "specimen_name": "Sst-IRES-Cre-123835",
  "strain": null,
  "structure_abbrev": "GR",
  "structure_id": 1039,
  "structure_name": "Gracile nucleus",
  "primary_injection_structure": 1039,
  "data_set_id": 300843120,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.2042788832,
  "injection_x": 8800,
  "injection_y": 910,
  "injection_z": 8070,
  "product_id": 5,
  "specimen_name": "378-602",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 100141219,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837777,
   "name": "Sim1-Cre_KJ18"
  },
  "gender": "M",
  "injection_structures": "378/677",
  "injection_volume": 0.03615781295,
  "injection_x": 6110,
  "injection_y": 3630,
  "injection_z": 9990,
  "product_id": 5,
  "specimen_name": "Sim1-Cre_KJ18-213425",
  "strain": null,
  "structure_abbrev": "SSs",
  "structure_id": 378,
  "structure_name": "Supplemental somatosensory area",
  "primary_injection_structure": 378,
  "data_set_id": 520728084,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839406,
   "name": "Nos1-CreERT2"
  },
  "gender": "F",
  "injection_structures": "1002/1011",
  "injection_volume": 0.018233445125,
  "injection_x": 7560,
  "injection_y": 2710,
  "injection_z": 10280,
  "product_id": 5,
  "specimen_name": "Nos1-CreERT2-181",
  "strain": "B6.129",
  "structure_abbrev": "AUDp",
  "structure_id": 1002,
  "structure_name": "Primary auditory area",
  "primary_injection_structure": 1002,
  "data_set_id": 267930268,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "329/385/394/417/533/312782546",
  "injection_volume": 0.02916804135,
  "injection_x": 8030,
  "injection_y": 1120,
  "injection_z": 3280,
  "product_id": 36,
  "specimen_name": "A930038C07Rik-Tg1-Cre-350976",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 329,
  "data_set_id": 646527844,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182901722,
   "name": "Ppp1r17-Cre_NL146"
  },
  "gender": "M",
  "injection_structures": "463",
  "injection_volume": 0.017673845525,
  "injection_x": 7480,
  "injection_y": 3000,
  "injection_z": 8450,
  "product_id": 5,
  "specimen_name": "Ppp1r17-Cre_NL146-3906",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "CA3",
  "structure_id": 463,
  "structure_name": "Field CA3",
  "primary_injection_structure": 463,
  "data_set_id": 278318653,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838622,
   "name": "Vip-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0236976446,
  "injection_x": 9980,
  "injection_y": 1200,
  "injection_z": 8360,
  "product_id": 5,
  "specimen_name": "Vip-IRES-Cre-192",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 182293273,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837315,
   "name": "Avp-IRES2-Cre"
  },
  "gender": "F",
  "injection_structures": "38/63/88/181/797/830/1077",
  "injection_volume": 0.00975637727268066,
  "injection_x": 6450,
  "injection_y": 5650,
  "injection_z": 5890,
  "product_id": 5,
  "specimen_name": "Avp-IRES2-Cre-D-3565",
  "strain": "B6.129",
  "structure_abbrev": "PVH",
  "structure_id": 38,
  "structure_name": "Paraventricular hypothalamic nucleus",
  "primary_injection_structure": 38,
  "data_set_id": 287044088,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "F",
  "injection_structures": "337/985",
  "injection_volume": 0.0201353593,
  "injection_x": 5740,
  "injection_y": 1200,
  "injection_z": 7120,
  "product_id": 5,
  "specimen_name": "Tlx3-Cre_PL56-303737",
  "strain": null,
  "structure_abbrev": "SSp-ll",
  "structure_id": 337,
  "structure_name": "Primary somatosensory area, lower limb",
  "primary_injection_structure": 337,
  "data_set_id": 584902900,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837777,
   "name": "Sim1-Cre_KJ18"
  },
  "gender": "M",
  "injection_structures": "337/985",
  "injection_volume": 0.01860453805,
  "injection_x": 5690,
  "injection_y": 1500,
  "injection_z": 7270,
  "product_id": 5,
  "specimen_name": "Sim1-Cre_KJ18-324771",
  "strain": null,
  "structure_abbrev": "MOp",
  "structure_id": 985,
  "structure_name": "Primary motor area",
  "primary_injection_structure": 337,
  "data_set_id": 606250170,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "385/409",
  "injection_volume": 0.1484338086,
  "injection_x": 8550,
  "injection_y": 1860,
  "injection_z": 2590,
  "product_id": 35,
  "specimen_name": "Cux2-IRES-Cre-247055",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 526501457,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837777,
   "name": "Sim1-Cre_KJ18"
  },
  "gender": "F",
  "injection_structures": "385/417",
  "injection_volume": 0.002334406703125,
  "injection_x": 8230,
  "injection_y": 1290,
  "injection_z": 2920,
  "product_id": 35,
  "specimen_name": "Sim1-Cre_KJ18-324773",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 602166541,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "337/369/985",
  "injection_volume": 0.0292742464,
  "injection_x": 4910,
  "injection_y": 1710,
  "injection_z": 7250,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-3803",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOp",
  "structure_id": 985,
  "structure_name": "Primary motor area",
  "primary_injection_structure": 337,
  "data_set_id": 166461193,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838584,
   "name": "Gad2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "262/718",
  "injection_volume": 0.0119503846,
  "injection_x": 6580,
  "injection_y": 3510,
  "injection_z": 7750,
  "product_id": 5,
  "specimen_name": "Gad2-IRES-Cre-1075",
  "strain": "C57BL/6J",
  "structure_abbrev": "RT",
  "structure_id": 262,
  "structure_name": "Reticular nucleus of the thalamus",
  "primary_injection_structure": 262,
  "data_set_id": 156254369,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "96/101",
  "injection_volume": 0.0582355935,
  "injection_x": 10630,
  "injection_y": 5320,
  "injection_z": 8280,
  "product_id": 5,
  "specimen_name": "378-1870",
  "strain": "C57BL/6J",
  "structure_abbrev": "VCO",
  "structure_id": 101,
  "structure_name": "Ventral cochlear nucleus",
  "primary_injection_structure": 96,
  "data_set_id": 272825299,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838633,
   "name": "Trib2-F2A-CreERT2"
  },
  "gender": "M",
  "injection_structures": "886",
  "injection_volume": 0.0525725165,
  "injection_x": 6750,
  "injection_y": 1220,
  "injection_z": 6100,
  "product_id": 5,
  "specimen_name": "Trib2-2A-CreERT2-D-123723",
  "strain": "B6.129",
  "structure_abbrev": "RSPv",
  "structure_id": 886,
  "structure_name": "Retrosplenial area, ventral part",
  "primary_injection_structure": 886,
  "data_set_id": 308721884,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "329/385/417/312782546",
  "injection_volume": 0.2791398484,
  "injection_x": 7920,
  "injection_y": 1290,
  "injection_z": 3100,
  "product_id": 35,
  "specimen_name": "Rbp4-Cre_KL100-247184",
  "strain": null,
  "structure_abbrev": "VISrl",
  "structure_id": 417,
  "structure_name": "Rostrolateral visual area",
  "primary_injection_structure": 329,
  "data_set_id": 524874308,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.1705708816,
  "injection_x": 8650,
  "injection_y": 1450,
  "injection_z": 7860,
  "product_id": 5,
  "specimen_name": "Ntsr1-Cre-3672",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 181258571,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "44/48/972",
  "injection_volume": 0.2579104416,
  "injection_x": 3510,
  "injection_y": 3430,
  "injection_z": 6350,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-3952",
  "strain": null,
  "structure_abbrev": "PL",
  "structure_id": 972,
  "structure_name": "Prelimbic area",
  "primary_injection_structure": 44,
  "data_set_id": 292374777,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "409/541/312782574/312782628",
  "injection_volume": 0.0117286792,
  "injection_x": 8810,
  "injection_y": 2020,
  "injection_z": 2030,
  "product_id": 36,
  "specimen_name": "A930038C07Rik-Tg1-Cre-355049",
  "strain": null,
  "structure_abbrev": "VISli",
  "structure_id": 312782574,
  "structure_name": "Laterointermediate area",
  "primary_injection_structure": 409,
  "data_set_id": 650408717,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "385/409/425",
  "injection_volume": 0.1896128696,
  "injection_x": 9370,
  "injection_y": 2200,
  "injection_z": 2580,
  "product_id": 35,
  "specimen_name": "Cux2-IRES-Cre-314804",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 591623061,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "541/312782574/312782628",
  "injection_volume": 0.0888389698,
  "injection_x": 9080,
  "injection_y": 2600,
  "injection_z": 1710,
  "product_id": 35,
  "specimen_name": "Rbp4-Cre_KL100-351924",
  "strain": null,
  "structure_abbrev": "VISpor",
  "structure_id": 312782628,
  "structure_name": "Postrhinal area",
  "primary_injection_structure": 541,
  "data_set_id": 648050335,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838502,
   "name": "Grik4-Cre"
  },
  "gender": "F",
  "injection_structures": "502/918/926",
  "injection_volume": 0.1826923252,
  "injection_x": 9620,
  "injection_y": 3860,
  "injection_z": 9080,
  "product_id": 5,
  "specimen_name": "Grik4-Cre-5128",
  "strain": "C57BL/6J",
  "structure_abbrev": "ENTm",
  "structure_id": 926,
  "structure_name": "Entorhinal area, medial part, dorsal zone",
  "primary_injection_structure": 502,
  "data_set_id": 267999034,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 517749378,
   "name": "Rorb-IRES2-Cre-neo"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.0094927847,
  "injection_x": 9490,
  "injection_y": 1260,
  "injection_z": 8170,
  "product_id": 5,
  "specimen_name": "Rorb-IRES2-Cre-neo-330781",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 616677276,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837858,
   "name": "Pvalb-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "39/48",
  "injection_volume": 0.1943668888,
  "injection_x": 5100,
  "injection_y": 1750,
  "injection_z": 6010,
  "product_id": 5,
  "specimen_name": "Pvalb-IRES-Cre-5045",
  "strain": "B6.129",
  "structure_abbrev": "ACAv",
  "structure_id": 48,
  "structure_name": "Anterior cingulate area, ventral part",
  "primary_injection_structure": 39,
  "data_set_id": 168401109,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "337/361/312782546",
  "injection_volume": 0.0784095942,
  "injection_x": 6570,
  "injection_y": 1120,
  "injection_z": 7090,
  "product_id": 5,
  "specimen_name": "Tlx3-Cre_LURC-130341",
  "strain": null,
  "structure_abbrev": "SSp-tr",
  "structure_id": 361,
  "structure_name": "Primary somatosensory area, trunk",
  "primary_injection_structure": 337,
  "data_set_id": 298182842,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "385/394/533/894/312782546",
  "injection_volume": 0.3533669888,
  "injection_x": 7680,
  "injection_y": 730,
  "injection_z": 3630,
  "product_id": 36,
  "specimen_name": "Rbp4-Cre_KL100-319618",
  "strain": null,
  "structure_abbrev": "VISa",
  "structure_id": 312782546,
  "structure_name": "Anterior area",
  "primary_injection_structure": 385,
  "data_set_id": 597007143,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838584,
   "name": "Gad2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "329/182305689",
  "injection_volume": 0.2480662044,
  "injection_x": 5940,
  "injection_y": 1890,
  "injection_z": 8570,
  "product_id": 5,
  "specimen_name": "Gad2-IRES-Cre-1143",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 167441329,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838828,
   "name": "Gal-Cre_KI87"
  },
  "gender": "M",
  "injection_structures": "15/149/362",
  "injection_volume": 0.038807930175,
  "injection_x": 5750,
  "injection_y": 4160,
  "injection_z": 5850,
  "product_id": 5,
  "specimen_name": "Gal-Cre_KI87-3469",
  "strain": "C57BL/6J",
  "structure_abbrev": "PVT",
  "structure_id": 149,
  "structure_name": "Paraventricular nucleus of the thalamus",
  "primary_injection_structure": 15,
  "data_set_id": 204908781,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "985/993",
  "injection_volume": 0.0658625366,
  "injection_x": 3370,
  "injection_y": 2170,
  "injection_z": 7580,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre_KL100-330694",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 606785720,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "337/361/894/312782546",
  "injection_volume": 0.1691241468,
  "injection_x": 6950,
  "injection_y": 1260,
  "injection_z": 6960,
  "product_id": 5,
  "specimen_name": "Ntsr1-Cre_GN220-141917",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-tr",
  "structure_id": 361,
  "structure_name": "Primary somatosensory area, trunk",
  "primary_injection_structure": 337,
  "data_set_id": 304641170,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "378/1002/1011",
  "injection_volume": 0.121590462,
  "injection_x": 7680,
  "injection_y": 2700,
  "injection_z": 9710,
  "product_id": 5,
  "specimen_name": "378-1345",
  "strain": "C57BL/6J",
  "structure_abbrev": "AUDd",
  "structure_id": 1011,
  "structure_name": "Dorsal auditory area",
  "primary_injection_structure": 378,
  "data_set_id": 139519496,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "385/894",
  "injection_volume": 0.0060511122875,
  "injection_x": 9790,
  "injection_y": 1380,
  "injection_z": 8140,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre_KL100-277",
  "strain": "B6.129",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 272930013,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "F",
  "injection_structures": "329/417/312782546",
  "injection_volume": 0.1445271758,
  "injection_x": 7540,
  "injection_y": 1330,
  "injection_z": 8270,
  "product_id": 5,
  "specimen_name": "Tlx3-Cre-137310",
  "strain": null,
  "structure_abbrev": "VISrl",
  "structure_id": 417,
  "structure_name": "Rostrolateral visual area",
  "primary_injection_structure": 329,
  "data_set_id": 303616127,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "918/926/312782628",
  "injection_volume": 0.1545385912,
  "injection_x": 9860,
  "injection_y": 3900,
  "injection_z": 9760,
  "product_id": 5,
  "specimen_name": "C57BL6J_OLD-343588",
  "strain": "C57BL/6J",
  "structure_abbrev": "ENTl",
  "structure_id": 918,
  "structure_name": "Entorhinal area, lateral part",
  "primary_injection_structure": 918,
  "data_set_id": 640081415,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837481,
   "name": "Scnn1a-Tg2-Cre"
  },
  "gender": "M",
  "injection_structures": "170/218/733",
  "injection_volume": 0.03866763215,
  "injection_x": 7380,
  "injection_y": 3400,
  "injection_z": 3270,
  "product_id": 36,
  "specimen_name": "Scnn1a-Tg2-Cre-190463",
  "strain": "B6.C3H",
  "structure_abbrev": "LGd",
  "structure_id": 170,
  "structure_name": "Dorsal part of the lateral geniculate complex",
  "primary_injection_structure": 170,
  "data_set_id": 479671695,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 301614880,
   "name": "Glt25d2-Cre_NF107"
  },
  "gender": "M",
  "injection_structures": "993",
  "injection_volume": 0.014262017175,
  "injection_x": 3280,
  "injection_y": 2280,
  "injection_z": 4460,
  "product_id": 45,
  "specimen_name": "Glt25d2-Cre_NF107-336501",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 614737677,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837446,
   "name": "Adcyap1-2A-Cre"
  },
  "gender": "M",
  "injection_structures": "525/946",
  "injection_volume": 0.2673089013,
  "injection_x": 7900,
  "injection_y": 5460,
  "injection_z": 5690,
  "product_id": 5,
  "specimen_name": "Adcyap1-2A-Cre-D-127770",
  "strain": "B6.129",
  "structure_abbrev": "PH",
  "structure_id": 946,
  "structure_name": "Posterior hypothalamic nucleus",
  "primary_injection_structure": 525,
  "data_set_id": 551738231,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "F",
  "injection_structures": "985",
  "injection_volume": 0.1753412668,
  "injection_x": 5080,
  "injection_y": 2450,
  "injection_z": 7190,
  "product_id": 5,
  "specimen_name": "Syt6-Cre-2824",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOp",
  "structure_id": 985,
  "structure_name": "Primary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 168229113,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839406,
   "name": "Nos1-CreERT2"
  },
  "gender": "M",
  "injection_structures": "463",
  "injection_volume": 0.0343943838,
  "injection_x": 7580,
  "injection_y": 2970,
  "injection_z": 8330,
  "product_id": 5,
  "specimen_name": "Nos1-CreERT2-126101",
  "strain": "B6.129",
  "structure_abbrev": "CA3",
  "structure_id": 463,
  "structure_name": "Field CA3",
  "primary_injection_structure": 463,
  "data_set_id": 301326316,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838259,
   "name": "Cort-T2A-Cre"
  },
  "gender": "F",
  "injection_structures": "39/48/993",
  "injection_volume": 0.412037668,
  "injection_x": 4860,
  "injection_y": 1940,
  "injection_z": 5980,
  "product_id": 5,
  "specimen_name": "Cort-T2A-Cre-99",
  "strain": null,
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 165644051,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "M",
  "injection_structures": "429",
  "injection_volume": 0.0162502424,
  "injection_x": 13030,
  "injection_y": 6450,
  "injection_z": 7480,
  "product_id": 5,
  "specimen_name": "Syt6-Cre_KI148-3642",
  "strain": "C57BL/6J",
  "structure_abbrev": "SPVC",
  "structure_id": 429,
  "structure_name": "Spinal nucleus of the trigeminal, caudal part",
  "primary_injection_structure": 429,
  "data_set_id": 264096244,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 180643392,
   "name": "Rasgrf2-T2A-dCre"
  },
  "gender": "M",
  "injection_structures": "147/460/795/867/898/920",
  "injection_volume": 0.1502277868,
  "injection_x": 10470,
  "injection_y": 3510,
  "injection_z": 6590,
  "product_id": 5,
  "specimen_name": "Rasgrf2-2A-dCre-D-160232",
  "strain": "C57BL/6J",
  "structure_abbrev": "PB",
  "structure_id": 867,
  "structure_name": "Parabrachial nucleus",
  "primary_injection_structure": 147,
  "data_set_id": 313141786,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "385/402/409",
  "injection_volume": 0.1753837008,
  "injection_x": 8460,
  "injection_y": 1690,
  "injection_z": 2330,
  "product_id": 36,
  "specimen_name": "Rbp4-Cre_KL100-195023",
  "strain": null,
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 385,
  "data_set_id": 482580380,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761918,
   "name": "Oxtr-Cre_ON66"
  },
  "gender": "M",
  "injection_structures": "104/119/583/672/952",
  "injection_volume": 0.355313896,
  "injection_x": 4190,
  "injection_y": 5310,
  "injection_z": 8230,
  "product_id": 5,
  "specimen_name": "Oxtr-Cre_ON66-3865",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "AIv",
  "structure_id": 119,
  "structure_name": "Agranular insular area, ventral part",
  "primary_injection_structure": 104,
  "data_set_id": 267762859,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "993",
  "injection_volume": 0.2433296096,
  "injection_x": 4570,
  "injection_y": 1320,
  "injection_z": 6510,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-289",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 272822110,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "F",
  "injection_structures": "985/993",
  "injection_volume": 0.0368752734,
  "injection_x": 3700,
  "injection_y": 2090,
  "injection_z": 7620,
  "product_id": 5,
  "specimen_name": "Tlx3-Cre_PL56-313541",
  "strain": null,
  "structure_abbrev": "MOp",
  "structure_id": 985,
  "structure_name": "Primary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 591612976,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "56/258/266",
  "injection_volume": 0.083919017,
  "injection_x": 4240,
  "injection_y": 4920,
  "injection_z": 6340,
  "product_id": 5,
  "specimen_name": "378-1292",
  "strain": "C57BL/6J",
  "structure_abbrev": "ACB",
  "structure_id": 56,
  "structure_name": "Nucleus accumbens",
  "primary_injection_structure": 56,
  "data_set_id": 127255254,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837446,
   "name": "Adcyap1-2A-Cre"
  },
  "gender": "M",
  "injection_structures": "952/961/966",
  "injection_volume": 0.2195188044,
  "injection_x": 6210,
  "injection_y": 6170,
  "injection_z": 9060,
  "product_id": 5,
  "specimen_name": "Adcyap1-2A-Cre-D-157",
  "strain": "B6.129.FVB",
  "structure_abbrev": "PIR",
  "structure_id": 961,
  "structure_name": "Piriform area",
  "primary_injection_structure": 952,
  "data_set_id": 287225747,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "4",
  "injection_volume": 0.3545705856,
  "injection_x": 10510,
  "injection_y": 2080,
  "injection_z": 7300,
  "product_id": 5,
  "specimen_name": "378-1114",
  "strain": "C57BL/6J",
  "structure_abbrev": "IC",
  "structure_id": 4,
  "structure_name": "Inferior colliculus",
  "primary_injection_structure": 4,
  "data_set_id": 113368085,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838584,
   "name": "Gad2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "258/266/310/452/564/581",
  "injection_volume": 0.2100414547,
  "injection_x": 5150,
  "injection_y": 4670,
  "injection_z": 5590,
  "product_id": 5,
  "specimen_name": "Gad2-IRES-Cre_LURC-122965",
  "strain": "C57BL/6J",
  "structure_abbrev": "MS",
  "structure_id": 564,
  "structure_name": "Medial septal nucleus",
  "primary_injection_structure": 258,
  "data_set_id": 303580293,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "39/48/993",
  "injection_volume": 0.226570414,
  "injection_x": 4750,
  "injection_y": 1890,
  "injection_z": 5510,
  "product_id": 35,
  "specimen_name": "Rbp4-Cre_KL100-211661",
  "strain": null,
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 496576666,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "886",
  "injection_volume": 0.0827087758,
  "injection_x": 7370,
  "injection_y": 1300,
  "injection_z": 5910,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-208",
  "strain": "C57BL/6J",
  "structure_abbrev": "RSPv",
  "structure_id": 886,
  "structure_name": "Retrosplenial area, ventral part",
  "primary_injection_structure": 886,
  "data_set_id": 184168193,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182901722,
   "name": "Ppp1r17-Cre_NL146"
  },
  "gender": "M",
  "injection_structures": "1007",
  "injection_volume": 0.0236490709,
  "injection_x": 10890,
  "injection_y": 1900,
  "injection_z": 7870,
  "product_id": 5,
  "specimen_name": "Ppp1r17-Cre_NL146-3880",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "SIM",
  "structure_id": 1007,
  "structure_name": "Simple lobule",
  "primary_injection_structure": 1007,
  "data_set_id": 268165349,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "F",
  "injection_structures": "394/533/879/894",
  "injection_volume": 0.0994217056,
  "injection_x": 8230,
  "injection_y": 1110,
  "injection_z": 4060,
  "product_id": 36,
  "specimen_name": "Ntsr1-Cre_GN220-275954",
  "strain": "unknown",
  "structure_abbrev": "VISpm",
  "structure_id": 533,
  "structure_name": "posteromedial visual area",
  "primary_injection_structure": 394,
  "data_set_id": 557342452,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0813999221,
  "injection_x": 8580,
  "injection_y": 980,
  "injection_z": 8610,
  "product_id": 5,
  "specimen_name": "C57BL/6-144799",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 304564721,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "385/402/409/425/541/312782574/312782628",
  "injection_volume": 0.3998250256,
  "injection_x": 8510,
  "injection_y": 1960,
  "injection_z": 1870,
  "product_id": 36,
  "specimen_name": "Rbp4-Cre_KL100-368449",
  "strain": null,
  "structure_abbrev": "VISpor",
  "structure_id": 312782628,
  "structure_name": "Postrhinal area",
  "primary_injection_structure": 385,
  "data_set_id": 661987195,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "F",
  "injection_structures": "48",
  "injection_volume": 0.136174087,
  "injection_x": 4650,
  "injection_y": 2860,
  "injection_z": 6280,
  "product_id": 5,
  "specimen_name": "Syt6-Cre-589",
  "strain": "C57BL/6J",
  "structure_abbrev": "ACAv",
  "structure_id": 48,
  "structure_name": "Anterior cingulate area, ventral part",
  "primary_injection_structure": 48,
  "data_set_id": 125801033,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838225,
   "name": "Cart-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "961",
  "injection_volume": 0.049097118,
  "injection_x": 6970,
  "injection_y": 5820,
  "injection_z": 9650,
  "product_id": 5,
  "specimen_name": "Cart-Tg1-Cre-4440",
  "strain": "C57BL/6J",
  "structure_abbrev": "PIR",
  "structure_id": 961,
  "structure_name": "Piriform area",
  "primary_injection_structure": 961,
  "data_set_id": 171485060,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837324,
   "name": "Slc18a2-Cre_OZ14"
  },
  "gender": "F",
  "injection_structures": "329/353/378",
  "injection_volume": 0.01667311975,
  "injection_x": 5930,
  "injection_y": 2940,
  "injection_z": 9720,
  "product_id": 5,
  "specimen_name": "Slc18a2-Cre_OZ14-3814",
  "strain": null,
  "structure_abbrev": "SSs",
  "structure_id": 378,
  "structure_name": "Supplemental somatosensory area",
  "primary_injection_structure": 329,
  "data_set_id": 287039061,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "194/292/342/403",
  "injection_volume": 0.6084683584,
  "injection_x": 6760,
  "injection_y": 5940,
  "injection_z": 7430,
  "product_id": 5,
  "specimen_name": "378-1255",
  "strain": "C57BL/6J",
  "structure_abbrev": "MEA",
  "structure_id": 403,
  "structure_name": "Medial amygdalar nucleus",
  "primary_injection_structure": 194,
  "data_set_id": 125361005,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182846862,
   "name": "Chrnb4-Cre_OL57"
  },
  "gender": "M",
  "injection_structures": "39/993",
  "injection_volume": 0.114944004,
  "injection_x": 3960,
  "injection_y": 1970,
  "injection_z": 6320,
  "product_id": 5,
  "specimen_name": "Chrnb4-Cre_OL57-125948",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 39,
  "data_set_id": 298000163,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "385/879/894",
  "injection_volume": 0.2356820816,
  "injection_x": 9770,
  "injection_y": 1060,
  "injection_z": 7320,
  "product_id": 5,
  "specimen_name": "378-1867",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 272782668,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "F",
  "injection_structures": "723",
  "injection_volume": 0.0969581326,
  "injection_x": 2910,
  "injection_y": 3730,
  "injection_z": 7170,
  "product_id": 5,
  "specimen_name": "Syt6-Cre-591",
  "strain": "C57BL/6J",
  "structure_abbrev": "ORBl",
  "structure_id": 723,
  "structure_name": "Orbital area, lateral part",
  "primary_injection_structure": 723,
  "data_set_id": 126353451,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 460133878,
   "name": "Ntng2-IRES2-Cre"
  },
  "gender": "M",
  "injection_structures": "104/111/119/583/952/961/1057",
  "injection_volume": 0.3486978376,
  "injection_x": 4600,
  "injection_y": 5450,
  "injection_z": 8820,
  "product_id": 5,
  "specimen_name": "Ntng2-IRES2-Cre-229424",
  "strain": null,
  "structure_abbrev": "AIv",
  "structure_id": 119,
  "structure_name": "Agranular insular area, ventral part",
  "primary_injection_structure": 104,
  "data_set_id": 513773998,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 179697864,
   "name": "Rorb-IRES2-Cre"
  },
  "gender": "F",
  "injection_structures": "337/369/985",
  "injection_volume": 0.0625209718,
  "injection_x": 5370,
  "injection_y": 1730,
  "injection_z": 7580,
  "product_id": 5,
  "specimen_name": "Rorb-IRES2-Cre-D_LURC-124112",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-ll",
  "structure_id": 337,
  "structure_name": "Primary somatosensory area, lower limb",
  "primary_injection_structure": 337,
  "data_set_id": 293700454,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837797,
   "name": "Th-Cre_FI172"
  },
  "gender": "F",
  "injection_structures": "223/614/693/830/10671",
  "injection_volume": 0.0991203506234375,
  "injection_x": 7000,
  "injection_y": 7050,
  "injection_z": 6080,
  "product_id": 5,
  "specimen_name": "Th-Cre_FI172-6219",
  "strain": "B6.FVB",
  "structure_abbrev": "ARH",
  "structure_id": 223,
  "structure_name": "Arcuate hypothalamic nucleus",
  "primary_injection_structure": 223,
  "data_set_id": 286726777,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "385/409",
  "injection_volume": 0.0192506349,
  "injection_x": 8940,
  "injection_y": 1660,
  "injection_z": 2480,
  "product_id": 35,
  "specimen_name": "A930038C07Rik-Tg1-Cre-242325",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 520019334,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "985/993",
  "injection_volume": 0.3371651976,
  "injection_x": 3450,
  "injection_y": 1960,
  "injection_z": 7850,
  "product_id": 5,
  "specimen_name": "C57BL6J-314640",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOp",
  "structure_id": 985,
  "structure_name": "Primary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 584903636,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.078968988,
  "injection_x": 8940,
  "injection_y": 1060,
  "injection_z": 8650,
  "product_id": 5,
  "specimen_name": "C57BL/6-144797",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 304762965,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 298725927,
   "name": "Penk-IRES2-Cre-neo"
  },
  "gender": "F",
  "injection_structures": "329/672",
  "injection_volume": 0.0078262708125,
  "injection_x": 6580,
  "injection_y": 2680,
  "injection_z": 8420,
  "product_id": 5,
  "specimen_name": "Penk-IRES2-Cre-neo-168879",
  "strain": null,
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 329,
  "data_set_id": 477924853,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 301467016,
   "name": "Npr3-IRES2-Cre"
  },
  "gender": "F",
  "injection_structures": "337/361",
  "injection_volume": 0.0630971873,
  "injection_x": 6570,
  "injection_y": 1270,
  "injection_z": 7420,
  "product_id": 5,
  "specimen_name": "Npr3-IRES2-Cre-301566",
  "strain": null,
  "structure_abbrev": "SSp-tr",
  "structure_id": 361,
  "structure_name": "Primary somatosensory area, trunk",
  "primary_injection_structure": 337,
  "data_set_id": 586052938,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839331,
   "name": "Nr5a1-Cre"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.012721349375,
  "injection_x": 9440,
  "injection_y": 1210,
  "injection_z": 8360,
  "product_id": 5,
  "specimen_name": "Nr5a1-Cre-2354",
  "strain": "B6.FVB",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 157767683,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 256980970,
   "name": "Lypd6-Cre_KL156"
  },
  "gender": "F",
  "injection_structures": "128/162/898",
  "injection_volume": 0.01096187085,
  "injection_x": 10290,
  "injection_y": 4120,
  "injection_z": 6300,
  "product_id": 5,
  "specimen_name": "Lypd6-Cre_KL156-132208",
  "strain": null,
  "structure_abbrev": "LDT",
  "structure_id": 162,
  "structure_name": "Laterodorsal tegmental nucleus",
  "primary_injection_structure": 128,
  "data_set_id": 539010002,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839494,
   "name": "Hdc-Cre_IM1"
  },
  "gender": "M",
  "injection_structures": "1/210/491/525/1004",
  "injection_volume": 0.02839045055,
  "injection_x": 7790,
  "injection_y": 6580,
  "injection_z": 6390,
  "product_id": 5,
  "specimen_name": "Hdc-Cre_IM1-204106",
  "strain": null,
  "structure_abbrev": "LM",
  "structure_id": 210,
  "structure_name": "Lateral mammillary nucleus",
  "primary_injection_structure": 1,
  "data_set_id": 520619072,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "39/48/993",
  "injection_volume": 0.123935749,
  "injection_x": 5170,
  "injection_y": 1510,
  "injection_z": 5990,
  "product_id": 5,
  "specimen_name": "378-1344",
  "strain": "C57BL/6J",
  "structure_abbrev": "ACAv",
  "structure_id": 48,
  "structure_name": "Anterior cingulate area, ventral part",
  "primary_injection_structure": 39,
  "data_set_id": 139520203,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "100/128/146/231/238/574/591/679/757",
  "injection_volume": 0.4448152233,
  "injection_x": 8990,
  "injection_y": 5140,
  "injection_z": 5560,
  "product_id": 5,
  "specimen_name": "378-1588",
  "strain": "C57BL/6J",
  "structure_abbrev": "CS",
  "structure_id": 679,
  "structure_name": "Superior central nucleus raphe",
  "primary_injection_structure": 100,
  "data_set_id": 156929391,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837777,
   "name": "Sim1-Cre_KJ18"
  },
  "gender": "M",
  "injection_structures": "993",
  "injection_volume": 0.020150692625,
  "injection_x": 2920,
  "injection_y": 3130,
  "injection_z": 3700,
  "product_id": 45,
  "specimen_name": "Sim1-Cre_KJ18-230436",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 508273699,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837315,
   "name": "Avp-IRES2-Cre"
  },
  "gender": "F",
  "injection_structures": "194/614/576073704",
  "injection_volume": 0.0072736831125,
  "injection_x": 6930,
  "injection_y": 6730,
  "injection_z": 6950,
  "product_id": 5,
  "specimen_name": "Avp-IRES2-Cre-D-3537",
  "strain": "B6.129",
  "structure_abbrev": "TU",
  "structure_id": 614,
  "structure_name": "Tuberal nucleus",
  "primary_injection_structure": 194,
  "data_set_id": 267397226,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182901722,
   "name": "Ppp1r17-Cre_NL146"
  },
  "gender": "M",
  "injection_structures": "178/463",
  "injection_volume": 0.0509285616,
  "injection_x": 7650,
  "injection_y": 3550,
  "injection_z": 8490,
  "product_id": 5,
  "specimen_name": "Ppp1r17-Cre_NL146-3909",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "LGv",
  "structure_id": 178,
  "structure_name": "Ventral part of the lateral geniculate complex",
  "primary_injection_structure": 178,
  "data_set_id": 278398949,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 301467016,
   "name": "Npr3-IRES2-Cre"
  },
  "gender": "M",
  "injection_structures": "385/425/879/894/1037/484682508",
  "injection_volume": 0.2162888224,
  "injection_x": 9680,
  "injection_y": 2100,
  "injection_z": 8190,
  "product_id": 5,
  "specimen_name": "Npr3-IRES2-Cre-296841",
  "strain": "unknown",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 574950390,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "385/425/533/879/894",
  "injection_volume": 0.7989729888,
  "injection_x": 9410,
  "injection_y": 640,
  "injection_z": 4270,
  "product_id": 36,
  "specimen_name": "Emx1-IRES-Cre-305952",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 584511827,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "329/417",
  "injection_volume": 0.090038333,
  "injection_x": 7470,
  "injection_y": 1490,
  "injection_z": 8370,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-126172",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 294435580,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "39/48/993",
  "injection_volume": 0.03717759115,
  "injection_x": 4300,
  "injection_y": 2680,
  "injection_z": 6320,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-125536",
  "strain": "C57BL/6J",
  "structure_abbrev": "ACAv",
  "structure_id": 48,
  "structure_name": "Anterior cingulate area, ventral part",
  "primary_injection_structure": 39,
  "data_set_id": 298797288,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "F",
  "injection_structures": "402/409/312782574",
  "injection_volume": 0.01812990445,
  "injection_x": 8420,
  "injection_y": 1740,
  "injection_z": 2030,
  "product_id": 35,
  "specimen_name": "Scnn1a-Tg3-Cre-326397",
  "strain": null,
  "structure_abbrev": "VISal",
  "structure_id": 402,
  "structure_name": "Anterolateral visual area",
  "primary_injection_structure": 402,
  "data_set_id": 603208501,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "329/353/182305689",
  "injection_volume": 0.0318756858,
  "injection_x": 5780,
  "injection_y": 1970,
  "injection_z": 8750,
  "product_id": 5,
  "specimen_name": "Tlx3-Cre_LURC-3599",
  "strain": null,
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 288170549,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "319/403/619",
  "injection_volume": 0.1041345648,
  "injection_x": 6280,
  "injection_y": 6610,
  "injection_z": 7780,
  "product_id": 5,
  "specimen_name": "378-1612",
  "strain": "C57BL/6J",
  "structure_abbrev": "MEA",
  "structure_id": 403,
  "structure_name": "Medial amygdalar nucleus",
  "primary_injection_structure": 319,
  "data_set_id": 157550122,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "672/985",
  "injection_volume": 0.2208879232,
  "injection_x": 4490,
  "injection_y": 3910,
  "injection_z": 7880,
  "product_id": 5,
  "specimen_name": "378-1651",
  "strain": "C57BL/6J",
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 672,
  "data_set_id": 158916311,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838048,
   "name": "Slc17a6-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "629/685",
  "injection_volume": 0.0075174036,
  "injection_x": 6730,
  "injection_y": 4580,
  "injection_z": 6470,
  "product_id": 5,
  "specimen_name": "Slc17a6-IRES-Cre_LURC-125782",
  "strain": "C57BL/6J",
  "structure_abbrev": "VAL",
  "structure_id": 629,
  "structure_name": "Ventral anterior-lateral complex of the thalamus",
  "primary_injection_structure": 629,
  "data_set_id": 310193233,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837328,
   "name": "Syt17-Cre_NO14"
  },
  "gender": "F",
  "injection_structures": "895/918/922",
  "injection_volume": 0.181149276,
  "injection_x": 8040,
  "injection_y": 4630,
  "injection_z": 10150,
  "product_id": 5,
  "specimen_name": "Syt17-Cre_NO14_LURC-120284",
  "strain": "C57BL/6J",
  "structure_abbrev": "ENTl",
  "structure_id": 918,
  "structure_name": "Entorhinal area, lateral part",
  "primary_injection_structure": 895,
  "data_set_id": 293009265,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "M",
  "injection_structures": "345/353/378",
  "injection_volume": 0.0716514407,
  "injection_x": 5260,
  "injection_y": 3060,
  "injection_z": 9630,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg3-Cre-4869",
  "strain": "B6.C3H",
  "structure_abbrev": "SSp-m",
  "structure_id": 345,
  "structure_name": "Primary somatosensory area, mouth",
  "primary_injection_structure": 345,
  "data_set_id": 257666283,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "952/961",
  "injection_volume": 0.1386554568,
  "injection_x": 4540,
  "injection_y": 5540,
  "injection_z": 8590,
  "product_id": 5,
  "specimen_name": "378-930",
  "strain": "C57BL/6J",
  "structure_abbrev": "PIR",
  "structure_id": 961,
  "structure_name": "Piriform area",
  "primary_injection_structure": 952,
  "data_set_id": 112307046,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838496,
   "name": "Grp-Cre_KH288"
  },
  "gender": "M",
  "injection_structures": "463",
  "injection_volume": 0.0172975929,
  "injection_x": 8050,
  "injection_y": 4050,
  "injection_z": 8800,
  "product_id": 5,
  "specimen_name": "Grp-Cre-57",
  "strain": "B6.FVB(ICR)",
  "structure_abbrev": "CA3",
  "structure_id": 463,
  "structure_name": "Field CA3",
  "primary_injection_structure": 463,
  "data_set_id": 182615771,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "F",
  "injection_structures": "385/425/879/894/484682508",
  "injection_volume": 0.0477063559,
  "injection_x": 10050,
  "injection_y": 1880,
  "injection_z": 3010,
  "product_id": 36,
  "specimen_name": "Tlx3-Cre_PL56-313538",
  "strain": null,
  "structure_abbrev": "VISpl",
  "structure_id": 425,
  "structure_name": "Posterolateral visual area",
  "primary_injection_structure": 385,
  "data_set_id": 591224520,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "385/409",
  "injection_volume": 0.1691691092,
  "injection_x": 9210,
  "injection_y": 2310,
  "injection_z": 2370,
  "product_id": 36,
  "specimen_name": "Emx1-IRES-Cre-194296",
  "strain": "B6.129",
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 385,
  "data_set_id": 479980810,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "409/312782574",
  "injection_volume": 0.0904887998,
  "injection_x": 8680,
  "injection_y": 2180,
  "injection_z": 2170,
  "product_id": 35,
  "specimen_name": "Cux2-IRES-Cre-203183",
  "strain": null,
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 409,
  "data_set_id": 495344543,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "39/48",
  "injection_volume": 0.1195470346,
  "injection_x": 4220,
  "injection_y": 2200,
  "injection_z": 5440,
  "product_id": 35,
  "specimen_name": "Cux2-IRES-Cre-293773",
  "strain": null,
  "structure_abbrev": "ACAv",
  "structure_id": 48,
  "structure_name": "Anterior cingulate area, ventral part",
  "primary_injection_structure": 39,
  "data_set_id": 571823196,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "M",
  "injection_structures": "329/385/417",
  "injection_volume": 0.0494818072,
  "injection_x": 7740,
  "injection_y": 1190,
  "injection_z": 8540,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg3-Cre-4858",
  "strain": "B6.C3H",
  "structure_abbrev": "VISrl",
  "structure_id": 417,
  "structure_name": "Rostrolateral visual area",
  "primary_injection_structure": 329,
  "data_set_id": 184073678,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 266649644,
   "name": "Drd3-Cre_KI196"
  },
  "gender": "M",
  "injection_structures": "394/533",
  "injection_volume": 0.0804796139,
  "injection_x": 7930,
  "injection_y": 990,
  "injection_z": 7300,
  "product_id": 5,
  "specimen_name": "Drd3-Cre_KI196_LURC-128047",
  "strain": null,
  "structure_abbrev": "VISam",
  "structure_id": 394,
  "structure_name": "Anteromedial visual area",
  "primary_injection_structure": 394,
  "data_set_id": 297670312,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "402/541/1027/312782574",
  "injection_volume": 0.0670449605,
  "injection_x": 8250,
  "injection_y": 1920,
  "injection_z": 1740,
  "product_id": 35,
  "specimen_name": "Cux2-IRES-Cre-274795",
  "strain": "C57BL/6J",
  "structure_abbrev": "AUDpo",
  "structure_id": 1027,
  "structure_name": "Posterior auditory area",
  "primary_injection_structure": 402,
  "data_set_id": 556922099,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "M",
  "injection_structures": "1002/1011",
  "injection_volume": 0.0265108326,
  "injection_x": 7690,
  "injection_y": 2560,
  "injection_z": 10050,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg3-Cre-4778",
  "strain": "B6.C3H",
  "structure_abbrev": "AUDp",
  "structure_id": 1002,
  "structure_name": "Primary auditory area",
  "primary_injection_structure": 1002,
  "data_set_id": 166268376,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837777,
   "name": "Sim1-Cre_KJ18"
  },
  "gender": "M",
  "injection_structures": "985/993",
  "injection_volume": 0.0767151203,
  "injection_x": 4510,
  "injection_y": 2040,
  "injection_z": 7020,
  "product_id": 5,
  "specimen_name": "Sim1-Cre_KJ18-126134",
  "strain": null,
  "structure_abbrev": "MOp",
  "structure_id": 985,
  "structure_name": "Primary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 297711339,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "895/918/922/312782628",
  "injection_volume": 0.31699472,
  "injection_x": 9300,
  "injection_y": 3440,
  "injection_z": 9820,
  "product_id": 5,
  "specimen_name": "378-1860",
  "strain": "C57BL/6J",
  "structure_abbrev": "ENTl",
  "structure_id": 918,
  "structure_name": "Entorhinal area, lateral part",
  "primary_injection_structure": 895,
  "data_set_id": 272414403,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "394/533",
  "injection_volume": 0.12548312,
  "injection_x": 7900,
  "injection_y": 580,
  "injection_z": 7380,
  "product_id": 5,
  "specimen_name": "378-757",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISam",
  "structure_id": 394,
  "structure_name": "Anteromedial visual area",
  "primary_injection_structure": 394,
  "data_set_id": 100141599,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "879/886/894",
  "injection_volume": 0.1015541758,
  "injection_x": 8260,
  "injection_y": 1040,
  "injection_z": 6940,
  "product_id": 5,
  "specimen_name": "378-1889",
  "strain": "C57BL/6J",
  "structure_abbrev": "RSPd",
  "structure_id": 879,
  "structure_name": "Retrosplenial area, dorsal part",
  "primary_injection_structure": 879,
  "data_set_id": 272916915,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "385/879/894",
  "injection_volume": 0.0440423613,
  "injection_x": 9660,
  "injection_y": 1370,
  "injection_z": 3550,
  "product_id": 36,
  "specimen_name": "Rbp4-Cre_KL100-233300",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 510417255,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839425,
   "name": "Pdzk1ip1-Cre_KD31"
  },
  "gender": "M",
  "injection_structures": "414/609/795/946",
  "injection_volume": 0.0205911034,
  "injection_x": 7690,
  "injection_y": 4770,
  "injection_z": 5980,
  "product_id": 5,
  "specimen_name": "Pdzk1ip1-Cre-130",
  "strain": null,
  "structure_abbrev": "SPFm",
  "structure_id": 414,
  "structure_name": "Subparafascicular nucleus, magnocellular part",
  "primary_injection_structure": 414,
  "data_set_id": 159258618,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838496,
   "name": "Grp-Cre_KH288"
  },
  "gender": "F",
  "injection_structures": "39/48/993",
  "injection_volume": 0.0973490938,
  "injection_x": 5010,
  "injection_y": 2050,
  "injection_z": 6210,
  "product_id": 5,
  "specimen_name": "Grp-Cre_KH288-103",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "ACAv",
  "structure_id": 48,
  "structure_name": "Anterior cingulate area, ventral part",
  "primary_injection_structure": 39,
  "data_set_id": 265288825,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "155/218/1020",
  "injection_volume": 0.02230432715,
  "injection_x": 6790,
  "injection_y": 3300,
  "injection_z": 6960,
  "product_id": 5,
  "specimen_name": "378-1894",
  "strain": "C57BL/6J",
  "structure_abbrev": "LD",
  "structure_id": 155,
  "structure_name": "Lateral dorsal nucleus of thalamus",
  "primary_injection_structure": 155,
  "data_set_id": 272967913,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761781,
   "name": "Efr3a-Cre_NO108"
  },
  "gender": "M",
  "injection_structures": "879/886",
  "injection_volume": 0.1484401198,
  "injection_x": 7700,
  "injection_y": 830,
  "injection_z": 6240,
  "product_id": 5,
  "specimen_name": "Efr3a-Cre_NO108-44",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "RSPd",
  "structure_id": 879,
  "structure_name": "Retrosplenial area, dorsal part",
  "primary_injection_structure": 879,
  "data_set_id": 278179794,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "104/111/345/353/378/583/677/1057",
  "injection_volume": 1.1352898592,
  "injection_x": 5520,
  "injection_y": 4470,
  "injection_z": 10080,
  "product_id": 5,
  "specimen_name": "378-1827",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISC",
  "structure_id": 677,
  "structure_name": "Visceral area",
  "primary_injection_structure": 104,
  "data_set_id": 180436360,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "672/993",
  "injection_volume": 0.2629579512,
  "injection_x": 4650,
  "injection_y": 2970,
  "injection_z": 6680,
  "product_id": 5,
  "specimen_name": "378-1254",
  "strain": "C57BL/6J",
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 672,
  "data_set_id": 124059700,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "385/409/425/843/879/894/484682508",
  "injection_volume": 0.3186711472,
  "injection_x": 9800,
  "injection_y": 2260,
  "injection_z": 2930,
  "product_id": 35,
  "specimen_name": "Rbp4-Cre_KL100-315836",
  "strain": null,
  "structure_abbrev": "VISpl",
  "structure_id": 425,
  "structure_name": "Posterolateral visual area",
  "primary_injection_structure": 385,
  "data_set_id": 593277684,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761781,
   "name": "Efr3a-Cre_NO108"
  },
  "gender": "F",
  "injection_structures": "394/894",
  "injection_volume": 0.0434574091,
  "injection_x": 7680,
  "injection_y": 930,
  "injection_z": 7060,
  "product_id": 5,
  "specimen_name": "Efr3a-Cre_NO108-125132",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "VISam",
  "structure_id": 394,
  "structure_name": "Anteromedial visual area",
  "primary_injection_structure": 394,
  "data_set_id": 309515141,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "337",
  "injection_volume": 0.083456947,
  "injection_x": 5930,
  "injection_y": 1450,
  "injection_z": 7400,
  "product_id": 5,
  "specimen_name": "Ntsr1-Cre_GN220-3740",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-ll",
  "structure_id": 337,
  "structure_name": "Primary somatosensory area, lower limb",
  "primary_injection_structure": 337,
  "data_set_id": 266964788,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "184/723/746/993",
  "injection_volume": 0.018359740175,
  "injection_x": 2610,
  "injection_y": 3220,
  "injection_z": 6850,
  "product_id": 5,
  "specimen_name": "Tlx3-Cre_LURC-3608",
  "strain": null,
  "structure_abbrev": "ORBvl",
  "structure_id": 746,
  "structure_name": "Orbital area, ventrolateral part",
  "primary_injection_structure": 184,
  "data_set_id": 293433283,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837324,
   "name": "Slc18a2-Cre_OZ14"
  },
  "gender": "F",
  "injection_structures": "672",
  "injection_volume": 0.0483414449,
  "injection_x": 5610,
  "injection_y": 3670,
  "injection_z": 7220,
  "product_id": 5,
  "specimen_name": "Slc18a2-Cre_OZ14-121810",
  "strain": null,
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 672,
  "data_set_id": 301620241,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837328,
   "name": "Syt17-Cre_NO14"
  },
  "gender": "M",
  "injection_structures": "159/507/589/746",
  "injection_volume": 0.2121674912,
  "injection_x": 3000,
  "injection_y": 4290,
  "injection_z": 6510,
  "product_id": 5,
  "specimen_name": "Syt17-Cre-85",
  "strain": null,
  "structure_abbrev": "AON",
  "structure_id": 159,
  "structure_name": "Anterior olfactory nucleus",
  "primary_injection_structure": 159,
  "data_set_id": 170949055,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 460133878,
   "name": "Ntng2-IRES2-Cre"
  },
  "gender": "M",
  "injection_structures": "918/926",
  "injection_volume": 0.221604558,
  "injection_x": 9680,
  "injection_y": 4980,
  "injection_z": 9280,
  "product_id": 5,
  "specimen_name": "Ntng2-IRES2-Cre-272873",
  "strain": "C57BL/6J",
  "structure_abbrev": "ENTm",
  "structure_id": 926,
  "structure_name": "Entorhinal area, medial part, dorsal zone",
  "primary_injection_structure": 918,
  "data_set_id": 557199437,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "879",
  "injection_volume": 0.0305850601,
  "injection_x": 8070,
  "injection_y": 580,
  "injection_z": 6500,
  "product_id": 5,
  "specimen_name": "378-947",
  "strain": "C57BL/6J",
  "structure_abbrev": "RSPd",
  "structure_id": 879,
  "structure_name": "Retrosplenial area, dorsal part",
  "primary_injection_structure": 879,
  "data_set_id": 112424813,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "F",
  "injection_structures": "39/48",
  "injection_volume": 0.0151330277,
  "injection_x": 4160,
  "injection_y": 2270,
  "injection_z": 5960,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg3-Cre-4917",
  "strain": "B6.C3H",
  "structure_abbrev": "ACAv",
  "structure_id": 48,
  "structure_name": "Anterior cingulate area, ventral part",
  "primary_injection_structure": 39,
  "data_set_id": 278502708,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182693192,
   "name": "Chrna2-Cre_OE25"
  },
  "gender": "M",
  "injection_structures": "39/972/993",
  "injection_volume": 0.0737654428,
  "injection_x": 2930,
  "injection_y": 2340,
  "injection_z": 6360,
  "product_id": 5,
  "specimen_name": "Chrna2-Cre_OE25-3902",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 39,
  "data_set_id": 292172846,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "F",
  "injection_structures": "104/993",
  "injection_volume": 0.0933809366,
  "injection_x": 2540,
  "injection_y": 2930,
  "injection_z": 3600,
  "product_id": 45,
  "specimen_name": "Tlx3-Cre_PL56;Sst-IRES-FlpO;Ai65F-239014",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 104,
  "data_set_id": 517313256,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837328,
   "name": "Syt17-Cre_NO14"
  },
  "gender": "M",
  "injection_structures": "159",
  "injection_volume": 0.0285012469,
  "injection_x": 2390,
  "injection_y": 5110,
  "injection_z": 6960,
  "product_id": 5,
  "specimen_name": "Syt17-Cre-44",
  "strain": null,
  "structure_abbrev": "AON",
  "structure_id": 159,
  "structure_name": "Anterior olfactory nucleus",
  "primary_injection_structure": 159,
  "data_set_id": 159994059,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "39/44/731/972",
  "injection_volume": 0.391319397658643,
  "injection_x": 3480,
  "injection_y": 3440,
  "injection_z": 6280,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre_KL100-222",
  "strain": null,
  "structure_abbrev": "PL",
  "structure_id": 972,
  "structure_name": "Prelimbic area",
  "primary_injection_structure": 39,
  "data_set_id": 263106036,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839285,
   "name": "Gpr26-Cre_KO250"
  },
  "gender": "F",
  "injection_structures": "255",
  "injection_volume": 0.0430990574,
  "injection_x": 5940,
  "injection_y": 3470,
  "injection_z": 6790,
  "product_id": 5,
  "specimen_name": "Gpr26-Cre_KO250-117",
  "strain": null,
  "structure_abbrev": "AV",
  "structure_id": 255,
  "structure_name": "Anteroventral nucleus of thalamus",
  "primary_injection_structure": 255,
  "data_set_id": 286553311,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.014149243675,
  "injection_x": 9530,
  "injection_y": 1310,
  "injection_z": 8640,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-198",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 183471884,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 183238310,
   "name": "Ntrk1-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "12/100",
  "injection_volume": 0.0165353544125,
  "injection_x": 8780,
  "injection_y": 5130,
  "injection_z": 5620,
  "product_id": 5,
  "specimen_name": "Ntrk1-IRES-Cre-3520",
  "strain": "B6.129",
  "structure_abbrev": "IPN",
  "structure_id": 100,
  "structure_name": "Interpeduncular nucleus",
  "primary_injection_structure": 12,
  "data_set_id": 184212995,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "402/1002/1011/1027",
  "injection_volume": 0.2100451248,
  "injection_x": 8150,
  "injection_y": 2510,
  "injection_z": 9640,
  "product_id": 5,
  "specimen_name": "378-1185",
  "strain": "C57BL/6J",
  "structure_abbrev": "AUDpo",
  "structure_id": 1027,
  "structure_name": "Posterior auditory area",
  "primary_injection_structure": 402,
  "data_set_id": 115958825,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837858,
   "name": "Pvalb-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "985/993",
  "injection_volume": 0.445154024,
  "injection_x": 3760,
  "injection_y": 1770,
  "injection_z": 7200,
  "product_id": 5,
  "specimen_name": "Pvalb-IRES-Cre-4995",
  "strain": "B6.129",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 158138440,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "385/533/879/886/894",
  "injection_volume": 0.1690642296,
  "injection_x": 9040,
  "injection_y": 510,
  "injection_z": 4260,
  "product_id": 35,
  "specimen_name": "Cux2-IRES-Cre-203184",
  "strain": null,
  "structure_abbrev": "VISpm",
  "structure_id": 533,
  "structure_name": "posteromedial visual area",
  "primary_injection_structure": 385,
  "data_set_id": 495345251,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 179697864,
   "name": "Rorb-IRES2-Cre"
  },
  "gender": "F",
  "injection_structures": "337",
  "injection_volume": 0.0946172164,
  "injection_x": 6150,
  "injection_y": 1170,
  "injection_z": 7490,
  "product_id": 5,
  "specimen_name": "Rorb-IRES2-Cre-D-695",
  "strain": null,
  "structure_abbrev": "SSp-ll",
  "structure_id": 337,
  "structure_name": "Primary somatosensory area, lower limb",
  "primary_injection_structure": 337,
  "data_set_id": 287880102,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "329/385/402/409/312782574",
  "injection_volume": 0.0010688902875,
  "injection_x": 8480,
  "injection_y": 1930,
  "injection_z": 2050,
  "product_id": 36,
  "specimen_name": "A930038C07Rik-Tg1-Cre-255584",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISal",
  "structure_id": 402,
  "structure_name": "Anterolateral visual area",
  "primary_injection_structure": 329,
  "data_set_id": 534175823,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 517749378,
   "name": "Rorb-IRES2-Cre-neo"
  },
  "gender": "M",
  "injection_structures": "993",
  "injection_volume": 0.0226243192,
  "injection_x": 2820,
  "injection_y": 2210,
  "injection_z": 4390,
  "product_id": 45,
  "specimen_name": "Rorb-IRES2-Cre-neo-347765",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 647576768,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "329/378",
  "injection_volume": 0.1835244824,
  "injection_x": 7510,
  "injection_y": 2180,
  "injection_z": 9070,
  "product_id": 5,
  "specimen_name": "378-672",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 100141473,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "502/541/895/918/926/312782628",
  "injection_volume": 0.3551519608,
  "injection_x": 9280,
  "injection_y": 2990,
  "injection_z": 9580,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-120538",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISpor",
  "structure_id": 312782628,
  "structure_name": "Postrhinal area",
  "primary_injection_structure": 502,
  "data_set_id": 293470216,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "385/409",
  "injection_volume": 0.0464237368,
  "injection_x": 8810,
  "injection_y": 1600,
  "injection_z": 9330,
  "product_id": 5,
  "specimen_name": "Tlx3-Cre-125626",
  "strain": null,
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 385,
  "data_set_id": 297225422,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838803,
   "name": "Grm2-Cre_MR90"
  },
  "gender": "F",
  "injection_structures": "184/993",
  "injection_volume": 0.0990099292,
  "injection_x": 2330,
  "injection_y": 2500,
  "injection_z": 7260,
  "product_id": 5,
  "specimen_name": "Grm2-Cre-100",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 184,
  "data_set_id": 182795499,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838048,
   "name": "Slc17a6-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "926",
  "injection_volume": 0.1330526694,
  "injection_x": 9900,
  "injection_y": 4250,
  "injection_z": 9360,
  "product_id": 5,
  "specimen_name": "Slc17a6-IRES-Cre-246452",
  "strain": null,
  "structure_abbrev": "ENTm",
  "structure_id": 926,
  "structure_name": "Entorhinal area, medial part, dorsal zone",
  "primary_injection_structure": 926,
  "data_set_id": 527393818,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837446,
   "name": "Adcyap1-2A-Cre"
  },
  "gender": "F",
  "injection_structures": "292/403/619",
  "injection_volume": 0.159680171,
  "injection_x": 6350,
  "injection_y": 6580,
  "injection_z": 7350,
  "product_id": 5,
  "specimen_name": "Adcyap1-2A-Cre-D-149",
  "strain": "B6.129",
  "structure_abbrev": "MEA",
  "structure_id": 403,
  "structure_name": "Medial amygdalar nucleus",
  "primary_injection_structure": 292,
  "data_set_id": 272819994,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.0288728531,
  "injection_x": 9450,
  "injection_y": 1870,
  "injection_z": 8320,
  "product_id": 5,
  "specimen_name": "Syt6-Cre-588",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 125435783,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838496,
   "name": "Grp-Cre_KH288"
  },
  "gender": "M",
  "injection_structures": "184/993",
  "injection_volume": 0.0564355785,
  "injection_x": 2090,
  "injection_y": 2360,
  "injection_z": 6750,
  "product_id": 5,
  "specimen_name": "Grp-Cre_KH288-3898",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 184,
  "data_set_id": 292034715,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839425,
   "name": "Pdzk1ip1-Cre_KD31"
  },
  "gender": "M",
  "injection_structures": "136/307/661/978/1048/1093",
  "injection_volume": 0.0297670198,
  "injection_x": 10680,
  "injection_y": 6220,
  "injection_z": 6370,
  "product_id": 5,
  "specimen_name": "Pdzk1ip1-Cre-215",
  "strain": "C57BL/6J",
  "structure_abbrev": "GRN",
  "structure_id": 1048,
  "structure_name": "Gigantocellular reticular nucleus",
  "primary_injection_structure": 136,
  "data_set_id": 182030590,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839044,
   "name": "Lepr-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "126/223/10671",
  "injection_volume": 0.00220172649375,
  "injection_x": 7380,
  "injection_y": 7070,
  "injection_z": 5850,
  "product_id": 5,
  "specimen_name": "Lepr-IRES-Cre-Homo-376",
  "strain": "B6.129",
  "structure_abbrev": "ARH",
  "structure_id": 223,
  "structure_name": "Arcuate hypothalamic nucleus",
  "primary_injection_structure": 126,
  "data_set_id": 171482142,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838803,
   "name": "Grm2-Cre_MR90"
  },
  "gender": "M",
  "injection_structures": "149/186/483/726",
  "injection_volume": 0.110144914323648,
  "injection_x": 6830,
  "injection_y": 2620,
  "injection_z": 6010,
  "product_id": 5,
  "specimen_name": "Grm2-Cre_MR90-127350",
  "strain": null,
  "structure_abbrev": "MH",
  "structure_id": 483,
  "structure_name": "Medial habenula",
  "primary_injection_structure": 149,
  "data_set_id": 300236056,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 301614880,
   "name": "Glt25d2-Cre_NF107"
  },
  "gender": "M",
  "injection_structures": "918/926",
  "injection_volume": 0.0330965404,
  "injection_x": 9700,
  "injection_y": 3710,
  "injection_z": 1320,
  "product_id": 5,
  "specimen_name": "Glt25d2-Cre_NF107-200976",
  "strain": null,
  "structure_abbrev": "ENTl",
  "structure_id": 918,
  "structure_name": "Entorhinal area, lateral part",
  "primary_injection_structure": 918,
  "data_set_id": 506426038,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 301614880,
   "name": "Glt25d2-Cre_NF107"
  },
  "gender": "M",
  "injection_structures": "993",
  "injection_volume": 0.0254336705,
  "injection_x": 2750,
  "injection_y": 2950,
  "injection_z": 3990,
  "product_id": 45,
  "specimen_name": "Glt25d2-Cre_NF107-331547",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 616857128,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 553502711,
   "name": "Chat-IRES-Cre-neo"
  },
  "gender": "M",
  "injection_structures": "672",
  "injection_volume": 0.005570075,
  "injection_x": 4330,
  "injection_y": 3280,
  "injection_z": 6840,
  "product_id": 5,
  "specimen_name": "Chat-IRES-Cre-3269",
  "strain": "B6.129",
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 672,
  "data_set_id": 292959343,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 503823716,
   "name": "Gnb4-IRES2-CreERT2"
  },
  "gender": "F",
  "injection_structures": "378",
  "injection_volume": 0.014138368125,
  "injection_x": 5960,
  "injection_y": 3980,
  "injection_z": 9630,
  "product_id": 5,
  "specimen_name": "Gnb4-IRES2-CreERT2-270132",
  "strain": null,
  "structure_abbrev": "SSs",
  "structure_id": 378,
  "structure_name": "Supplemental somatosensory area",
  "primary_injection_structure": 378,
  "data_set_id": 553446684,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "F",
  "injection_structures": "385/394/533/894",
  "injection_volume": 0.1173730516,
  "injection_x": 8500,
  "injection_y": 1020,
  "injection_z": 4000,
  "product_id": 36,
  "specimen_name": "Tlx3-Cre_PL56-313271",
  "strain": null,
  "structure_abbrev": "VISpm",
  "structure_id": 533,
  "structure_name": "posteromedial visual area",
  "primary_injection_structure": 385,
  "data_set_id": 590987294,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "385/533",
  "injection_volume": 0.0890302168,
  "injection_x": 8590,
  "injection_y": 990,
  "injection_z": 4120,
  "product_id": 35,
  "specimen_name": "Rbp4-Cre_KL100-199043",
  "strain": null,
  "structure_abbrev": "VISpm",
  "structure_id": 533,
  "structure_name": "posteromedial visual area",
  "primary_injection_structure": 385,
  "data_set_id": 485237081,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "918/922",
  "injection_volume": 0.0553523257,
  "injection_x": 8810,
  "injection_y": 4660,
  "injection_z": 9890,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre_KL100-288",
  "strain": "C57BL/6J",
  "structure_abbrev": "ENTl",
  "structure_id": 918,
  "structure_name": "Entorhinal area, lateral part",
  "primary_injection_structure": 918,
  "data_set_id": 286483411,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.1942056592,
  "injection_x": 9440,
  "injection_y": 1970,
  "injection_z": 2940,
  "product_id": 36,
  "specimen_name": "Emx1-IRES-Cre-191138",
  "strain": "B6.129",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 480069939,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837446,
   "name": "Adcyap1-2A-Cre"
  },
  "gender": "F",
  "injection_structures": "795/872",
  "injection_volume": 0.5535362028,
  "injection_x": 9380,
  "injection_y": 3360,
  "injection_z": 5690,
  "product_id": 5,
  "specimen_name": "Adcyap1-2A-Cre-D-166",
  "strain": "B6.129",
  "structure_abbrev": "PAG",
  "structure_id": 795,
  "structure_name": "Periaqueductal gray",
  "primary_injection_structure": 795,
  "data_set_id": 287247978,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "M",
  "injection_structures": "104/583/985",
  "injection_volume": 0.0837560234,
  "injection_x": 3600,
  "injection_y": 4110,
  "injection_z": 8240,
  "product_id": 5,
  "specimen_name": "Syt6-Cre_KI148-3654",
  "strain": "C57BL/6J",
  "structure_abbrev": "AId",
  "structure_id": 104,
  "structure_name": "Agranular insular area, dorsal part",
  "primary_injection_structure": 104,
  "data_set_id": 264320859,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "F",
  "injection_structures": "385/533",
  "injection_volume": 0.0382413346,
  "injection_x": 9270,
  "injection_y": 720,
  "injection_z": 7360,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg3-Cre-4907",
  "strain": "B6.C3H",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 268040381,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838803,
   "name": "Grm2-Cre_MR90"
  },
  "gender": "M",
  "injection_structures": "128/294/795",
  "injection_volume": 0.142921044,
  "injection_x": 9970,
  "injection_y": 3060,
  "injection_z": 6480,
  "product_id": 5,
  "specimen_name": "Grm2-Cre_MR90-3864",
  "strain": null,
  "structure_abbrev": "PAG",
  "structure_id": 795,
  "structure_name": "Periaqueductal gray",
  "primary_injection_structure": 128,
  "data_set_id": 292624169,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "879/886",
  "injection_volume": 0.1056040062,
  "injection_x": 7500,
  "injection_y": 880,
  "injection_z": 5040,
  "product_id": 35,
  "specimen_name": "Emx1-IRES-Cre-314296",
  "strain": null,
  "structure_abbrev": "RSPv",
  "structure_id": 886,
  "structure_name": "Retrosplenial area, ventral part",
  "primary_injection_structure": 879,
  "data_set_id": 591535205,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 301467016,
   "name": "Npr3-IRES2-Cre"
  },
  "gender": "F",
  "injection_structures": "184/723/746",
  "injection_volume": 0.1740864356,
  "injection_x": 2820,
  "injection_y": 2990,
  "injection_z": 7090,
  "product_id": 5,
  "specimen_name": "Npr3-IRES2-Cre-299663",
  "strain": null,
  "structure_abbrev": "ORBl",
  "structure_id": 723,
  "structure_name": "Orbital area, lateral part",
  "primary_injection_structure": 184,
  "data_set_id": 585935640,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838048,
   "name": "Slc17a6-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "64/127/255/362/1113",
  "injection_volume": 0.0665478457,
  "injection_x": 6080,
  "injection_y": 3910,
  "injection_z": 5020,
  "product_id": 35,
  "specimen_name": "Slc17a6-IRES-Cre-235565",
  "strain": null,
  "structure_abbrev": "AM",
  "structure_id": 127,
  "structure_name": "Anteromedial nucleus",
  "primary_injection_structure": 64,
  "data_set_id": 514333422,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 301613169,
   "name": "Sepw1-Cre_NP39"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.0183987944,
  "injection_x": 8600,
  "injection_y": 850,
  "injection_z": 8200,
  "product_id": 5,
  "specimen_name": "Sepw1-Cre_NP39-210484",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 511234217,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839494,
   "name": "Hdc-Cre_IM1"
  },
  "gender": "M",
  "injection_structures": "1/210/491/525/1004",
  "injection_volume": 0.02272414445,
  "injection_x": 7810,
  "injection_y": 6560,
  "injection_z": 6460,
  "product_id": 5,
  "specimen_name": "Hdc-Cre_IM1-204105",
  "strain": null,
  "structure_abbrev": "PMv",
  "structure_id": 1004,
  "structure_name": "Ventral premammillary nucleus",
  "primary_injection_structure": 1,
  "data_set_id": 520342605,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "1025",
  "injection_volume": 0.1711338916,
  "injection_x": 12770,
  "injection_y": 4380,
  "injection_z": 8470,
  "product_id": 5,
  "specimen_name": "378-1264",
  "strain": "C57BL/6J",
  "structure_abbrev": "PRM",
  "structure_id": 1025,
  "structure_name": "Paramedian lobule",
  "primary_injection_structure": 1025,
  "data_set_id": 125801739,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "385/409",
  "injection_volume": 0.0047845511,
  "injection_x": 9610,
  "injection_y": 1910,
  "injection_z": 2620,
  "product_id": 36,
  "specimen_name": "A930038C07Rik-Tg1-Cre-212451",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 496152279,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838715,
   "name": "Tac1-IRES2-Cre"
  },
  "gender": "F",
  "injection_structures": "39/993",
  "injection_volume": 0.102250407,
  "injection_x": 5110,
  "injection_y": 1460,
  "injection_z": 6150,
  "product_id": 5,
  "specimen_name": "Tac1-IRES2-Cre-D-3861",
  "strain": "B6.129",
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 286646877,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "385/402/409/417",
  "injection_volume": 0.9087897504,
  "injection_x": 8740,
  "injection_y": 1590,
  "injection_z": 8140,
  "product_id": 5,
  "specimen_name": "C57BL/6-155461",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 309372716,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "394/894",
  "injection_volume": 0.0006962138203125,
  "injection_x": 7690,
  "injection_y": 920,
  "injection_z": 4300,
  "product_id": 35,
  "specimen_name": "A930038C07Rik-Tg1-Cre-280209",
  "strain": null,
  "structure_abbrev": "VISam",
  "structure_id": 394,
  "structure_name": "Anteromedial visual area",
  "primary_injection_structure": 394,
  "data_set_id": 560724955,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839285,
   "name": "Gpr26-Cre_KO250"
  },
  "gender": "F",
  "injection_structures": "337",
  "injection_volume": 0.1837564288,
  "injection_x": 6320,
  "injection_y": 1290,
  "injection_z": 7580,
  "product_id": 5,
  "specimen_name": "Gpr26-Cre_KO250-95",
  "strain": null,
  "structure_abbrev": "SSp-ll",
  "structure_id": 337,
  "structure_name": "Primary somatosensory area, lower limb",
  "primary_injection_structure": 337,
  "data_set_id": 267658747,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "294/795/872",
  "injection_volume": 0.081270134728125,
  "injection_x": 9260,
  "injection_y": 2640,
  "injection_z": 5870,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre_KL100-121998",
  "strain": "C57BL/6J",
  "structure_abbrev": "PAG",
  "structure_id": 795,
  "structure_name": "Periaqueductal gray",
  "primary_injection_structure": 294,
  "data_set_id": 301541824,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "409/312782574",
  "injection_volume": 0.0411706477,
  "injection_x": 9230,
  "injection_y": 2300,
  "injection_z": 9560,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-3804",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 409,
  "data_set_id": 166461899,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "385/409",
  "injection_volume": 0.010716791225,
  "injection_x": 9090,
  "injection_y": 1750,
  "injection_z": 2370,
  "product_id": 35,
  "specimen_name": "A930038C07Rik-Tg1-Cre-270159",
  "strain": null,
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 385,
  "data_set_id": 552430870,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "329",
  "injection_volume": 0.0420059115,
  "injection_x": 6780,
  "injection_y": 2110,
  "injection_z": 8440,
  "product_id": 5,
  "specimen_name": "Ntsr1-Cre-3555",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 159321097,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.07211526,
  "injection_x": 9100,
  "injection_y": 1240,
  "injection_z": 3580,
  "product_id": 35,
  "specimen_name": "Cux2-IRES-Cre-196733",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 483013787,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "F",
  "injection_structures": "39/48",
  "injection_volume": 0.0710485251,
  "injection_x": 5180,
  "injection_y": 2210,
  "injection_z": 6490,
  "product_id": 5,
  "specimen_name": "Syt6-Cre-3155",
  "strain": "C57BL/6J",
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 177783204,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "345/353/182305689",
  "injection_volume": 0.1438892742,
  "injection_x": 5130,
  "injection_y": 2230,
  "injection_z": 9070,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre_KL100-216",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-n",
  "structure_id": 353,
  "structure_name": "Primary somatosensory area, nose",
  "primary_injection_structure": 345,
  "data_set_id": 249327301,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "985",
  "injection_volume": 0.1751966776,
  "injection_x": 3920,
  "injection_y": 2280,
  "injection_z": 8020,
  "product_id": 5,
  "specimen_name": "378-795",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOp",
  "structure_id": 985,
  "structure_name": "Primary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 100141780,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 460114142,
   "name": "Tacr1-T2A-Cre"
  },
  "gender": "M",
  "injection_structures": "15/258/266/310/452/564/581",
  "injection_volume": 0.1282029189,
  "injection_x": 5260,
  "injection_y": 4480,
  "injection_z": 5690,
  "product_id": 5,
  "specimen_name": "Tacr1-2A-Cre-271338",
  "strain": null,
  "structure_abbrev": "MS",
  "structure_id": 564,
  "structure_name": "Medial septal nucleus",
  "primary_injection_structure": 15,
  "data_set_id": 554021622,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "135/136/203/437/445/852/978",
  "injection_volume": 0.3389808632,
  "injection_x": 12040,
  "injection_y": 6690,
  "injection_z": 7250,
  "product_id": 5,
  "specimen_name": "378-1277",
  "strain": "C57BL/6J",
  "structure_abbrev": "PARN",
  "structure_id": 852,
  "structure_name": "Parvicellular reticular nucleus",
  "primary_injection_structure": 135,
  "data_set_id": 126189322,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "127/155/255/262/629",
  "injection_volume": 0.1622704188,
  "injection_x": 5840,
  "injection_y": 4010,
  "injection_z": 6970,
  "product_id": 5,
  "specimen_name": "378-825",
  "strain": "C57BL/6J",
  "structure_abbrev": "AV",
  "structure_id": 255,
  "structure_name": "Anteroventral nucleus of thalamus",
  "primary_injection_structure": 127,
  "data_set_id": 100142569,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 530349719,
   "name": "Pdyn-T2A-CreERT2"
  },
  "gender": "M",
  "injection_structures": "295/403/536/1105",
  "injection_volume": 0.0043756601875,
  "injection_x": 7170,
  "injection_y": 5440,
  "injection_z": 8530,
  "product_id": 5,
  "specimen_name": "Pdyn-T2A-CreERT2-255282",
  "strain": null,
  "structure_abbrev": "CEA",
  "structure_id": 536,
  "structure_name": "Central amygdalar nucleus",
  "primary_injection_structure": 295,
  "data_set_id": 543875354,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "M",
  "injection_structures": "886",
  "injection_volume": 0.0612544933,
  "injection_x": 8400,
  "injection_y": 1520,
  "injection_z": 7030,
  "product_id": 5,
  "specimen_name": "Syt6-Cre-2830",
  "strain": "C57BL/6J",
  "structure_abbrev": "RSPv",
  "structure_id": 886,
  "structure_name": "Retrosplenial area, ventral part",
  "primary_injection_structure": 886,
  "data_set_id": 168363168,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838361,
   "name": "Etv1-CreERT2"
  },
  "gender": "M",
  "injection_structures": "39/48/993",
  "injection_volume": 0.177611476,
  "injection_x": 5210,
  "injection_y": 1400,
  "injection_z": 6060,
  "product_id": 5,
  "specimen_name": "Etv1-CreERT2-1151",
  "strain": "C57BL/6J",
  "structure_abbrev": "ACAv",
  "structure_id": 48,
  "structure_name": "Anterior cingulate area, ventral part",
  "primary_injection_structure": 39,
  "data_set_id": 114475228,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "F",
  "injection_structures": "385/1037",
  "injection_volume": 0.108834439,
  "injection_x": 9230,
  "injection_y": 2040,
  "injection_z": 7770,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg3-Cre-4722",
  "strain": "B6.C3H",
  "structure_abbrev": "POST",
  "structure_id": 1037,
  "structure_name": "Postsubiculum",
  "primary_injection_structure": 385,
  "data_set_id": 113934579,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837777,
   "name": "Sim1-Cre_KJ18"
  },
  "gender": "M",
  "injection_structures": "39/972/993",
  "injection_volume": 0.0545462071,
  "injection_x": 3070,
  "injection_y": 1850,
  "injection_z": 6500,
  "product_id": 5,
  "specimen_name": "Sim1-Cre_KJ18-120885",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 39,
  "data_set_id": 293827470,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182693192,
   "name": "Chrna2-Cre_OE25"
  },
  "gender": "M",
  "injection_structures": "329/382",
  "injection_volume": 0.09163539,
  "injection_x": 7250,
  "injection_y": 1420,
  "injection_z": 8510,
  "product_id": 5,
  "specimen_name": "Chrna2-Cre_OE25-121326",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 293889922,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "184/993",
  "injection_volume": 0.1561897932,
  "injection_x": 2290,
  "injection_y": 2450,
  "injection_z": 7160,
  "product_id": 5,
  "specimen_name": "378-557",
  "strain": "C57BL/6J",
  "structure_abbrev": "FRP",
  "structure_id": 184,
  "structure_name": "Frontal pole, cerebral cortex",
  "primary_injection_structure": 184,
  "data_set_id": 100140756,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182693192,
   "name": "Chrna2-Cre_OE25"
  },
  "gender": "F",
  "injection_structures": "409/425/312782574/312782628",
  "injection_volume": 0.086415028,
  "injection_x": 9210,
  "injection_y": 2250,
  "injection_z": 9380,
  "product_id": 5,
  "specimen_name": "Chrna2-Cre_OE25-132668",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "VISli",
  "structure_id": 312782574,
  "structure_name": "Laterointermediate area",
  "primary_injection_structure": 409,
  "data_set_id": 299782273,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "F",
  "injection_structures": "378/1011",
  "injection_volume": 0.0679390782,
  "injection_x": 7210,
  "injection_y": 2470,
  "injection_z": 10000,
  "product_id": 5,
  "specimen_name": "Tlx3-Cre_LURC-3603",
  "strain": null,
  "structure_abbrev": "SSs",
  "structure_id": 378,
  "structure_name": "Supplemental somatosensory area",
  "primary_injection_structure": 378,
  "data_set_id": 288171256,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182901722,
   "name": "Ppp1r17-Cre_NL146"
  },
  "gender": "M",
  "injection_structures": "414/525/795/946",
  "injection_volume": 0.36349323325,
  "injection_x": 8020,
  "injection_y": 4890,
  "injection_z": 5690,
  "product_id": 5,
  "specimen_name": "Ppp1r17-Cre_NL146-3900",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "PH",
  "structure_id": 946,
  "structure_name": "Posterior hypothalamic nucleus",
  "primary_injection_structure": 414,
  "data_set_id": 268204599,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 553502711,
   "name": "Chat-IRES-Cre-neo"
  },
  "gender": "M",
  "injection_structures": "867",
  "injection_volume": 0.01727097365,
  "injection_x": 10430,
  "injection_y": 4180,
  "injection_z": 7430,
  "product_id": 5,
  "specimen_name": "Chat-IRES-Cre-3210",
  "strain": "B6.129",
  "structure_abbrev": "PB",
  "structure_id": 867,
  "structure_name": "Parabrachial nucleus",
  "primary_injection_structure": 867,
  "data_set_id": 264565965,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "409/425/312782628",
  "injection_volume": 0.011844736925,
  "injection_x": 9720,
  "injection_y": 2520,
  "injection_z": 9630,
  "product_id": 5,
  "specimen_name": "Tlx3-Cre-125622",
  "strain": null,
  "structure_abbrev": "VISpor",
  "structure_id": 312782628,
  "structure_name": "Postrhinal area",
  "primary_injection_structure": 409,
  "data_set_id": 294516943,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837324,
   "name": "Slc18a2-Cre_OZ14"
  },
  "gender": "F",
  "injection_structures": "222",
  "injection_volume": 0.002951246878125,
  "injection_x": 12810,
  "injection_y": 6490,
  "injection_z": 5710,
  "product_id": 5,
  "specimen_name": "Slc18a2-Cre_OZ14-3969",
  "strain": null,
  "structure_abbrev": "RO",
  "structure_id": 222,
  "structure_name": "Nucleus raphe obscurus",
  "primary_injection_structure": 222,
  "data_set_id": 292794673,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "128/422/741/879/886/930",
  "injection_volume": 0.03537505265,
  "injection_x": 7600,
  "injection_y": 4360,
  "injection_z": 6490,
  "product_id": 5,
  "specimen_name": "BL/6-3825",
  "strain": "C57BL/6J",
  "structure_abbrev": "PF",
  "structure_id": 930,
  "structure_name": "Parafascicular nucleus",
  "primary_injection_structure": 128,
  "data_set_id": 266693274,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 309779993,
   "name": "Ctgf-T2A-dgCre"
  },
  "gender": "M",
  "injection_structures": "378/1002/1011/1018",
  "injection_volume": 0.00718751845,
  "injection_x": 7480,
  "injection_y": 3010,
  "injection_z": 9510,
  "product_id": 5,
  "specimen_name": "Ctgf-2A-dgCre-neo-179275",
  "strain": null,
  "structure_abbrev": "AUDd",
  "structure_id": 1011,
  "structure_name": "Dorsal auditory area",
  "primary_injection_structure": 378,
  "data_set_id": 478097779,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182901722,
   "name": "Ppp1r17-Cre_NL146"
  },
  "gender": "M",
  "injection_structures": "337/369/382/423/463",
  "injection_volume": 0.0506591939,
  "injection_x": 6620,
  "injection_y": 2170,
  "injection_z": 7470,
  "product_id": 5,
  "specimen_name": "Ppp1r17-Cre_NL146-3907",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "CA3",
  "structure_id": 463,
  "structure_name": "Field CA3",
  "primary_injection_structure": 337,
  "data_set_id": 278319402,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839004,
   "name": "Cux2-CreERT2"
  },
  "gender": "F",
  "injection_structures": "425/926",
  "injection_volume": 0.4027659272,
  "injection_x": 9810,
  "injection_y": 4100,
  "injection_z": 9420,
  "product_id": 5,
  "specimen_name": "Cux2-CreERT2-151",
  "strain": "C57BL/6J",
  "structure_abbrev": "ENTm",
  "structure_id": 926,
  "structure_name": "Entorhinal area, medial part, dorsal zone",
  "primary_injection_structure": 425,
  "data_set_id": 181859467,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838048,
   "name": "Slc17a6-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "186/362",
  "injection_volume": 0.0979560568,
  "injection_x": 6640,
  "injection_y": 3450,
  "injection_z": 5300,
  "product_id": 35,
  "specimen_name": "Slc17a6-IRES-Cre-272450",
  "strain": "C57BL/6J",
  "structure_abbrev": "MD",
  "structure_id": 362,
  "structure_name": "Mediodorsal nucleus of thalamus",
  "primary_injection_structure": 186,
  "data_set_id": 553747363,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.02765839055,
  "injection_x": 8770,
  "injection_y": 1330,
  "injection_z": 2640,
  "product_id": 35,
  "specimen_name": "A930038C07Rik-Tg1-Cre-249403",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 526924964,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "329/385/394/417/533/312782546",
  "injection_volume": 0.9064811056,
  "injection_x": 7160,
  "injection_y": 1400,
  "injection_z": 3330,
  "product_id": 36,
  "specimen_name": "Ntsr1-Cre_GN220-323304",
  "strain": null,
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 601268292,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "F",
  "injection_structures": "353/378",
  "injection_volume": 0.0208919781,
  "injection_x": 5840,
  "injection_y": 3150,
  "injection_z": 9270,
  "product_id": 5,
  "specimen_name": "Ntsr1-Cre_GN220-3796",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSs",
  "structure_id": 378,
  "structure_name": "Supplemental somatosensory area",
  "primary_injection_structure": 353,
  "data_set_id": 287037604,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 298725927,
   "name": "Penk-IRES2-Cre-neo"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.0427155687,
  "injection_x": 8220,
  "injection_y": 650,
  "injection_z": 8150,
  "product_id": 5,
  "specimen_name": "Penk-IRES2-Cre-neo-363614",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 671033786,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "394/894/312782546",
  "injection_volume": 0.1215971946,
  "injection_x": 7540,
  "injection_y": 600,
  "injection_z": 4130,
  "product_id": 36,
  "specimen_name": "Tlx3-Cre_PL56-312675",
  "strain": null,
  "structure_abbrev": "VISam",
  "structure_id": 394,
  "structure_name": "Anteromedial visual area",
  "primary_injection_structure": 394,
  "data_set_id": 590548119,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839044,
   "name": "Lepr-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "1/126/210/525/1004",
  "injection_volume": 0.0549251682,
  "injection_x": 7680,
  "injection_y": 6680,
  "injection_z": 6120,
  "product_id": 5,
  "specimen_name": "Lepr-IRES-Cre-Homo-362",
  "strain": "B6.129",
  "structure_abbrev": "PMv",
  "structure_id": 1004,
  "structure_name": "Ventral premammillary nucleus",
  "primary_injection_structure": 1,
  "data_set_id": 167656152,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "F",
  "injection_structures": "337/985",
  "injection_volume": 0.00662463095,
  "injection_x": 5290,
  "injection_y": 1470,
  "injection_z": 7180,
  "product_id": 5,
  "specimen_name": "Scnn1a-Tg3-Cre-4816",
  "strain": "B6.C3H",
  "structure_abbrev": "MOp",
  "structure_id": 985,
  "structure_name": "Primary motor area",
  "primary_injection_structure": 337,
  "data_set_id": 179640955,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "39/48/972",
  "injection_volume": 0.3339583632,
  "injection_x": 3850,
  "injection_y": 2470,
  "injection_z": 5990,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-126690",
  "strain": "C57BL/6J",
  "structure_abbrev": "ACAv",
  "structure_id": 48,
  "structure_name": "Anterior cingulate area, ventral part",
  "primary_injection_structure": 39,
  "data_set_id": 294434161,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "394/894",
  "injection_volume": 0.1370264518,
  "injection_x": 7870,
  "injection_y": 540,
  "injection_z": 4300,
  "product_id": 35,
  "specimen_name": "Emx1-IRES-Cre-283756",
  "strain": null,
  "structure_abbrev": "RSPagl",
  "structure_id": 894,
  "structure_name": "Retrosplenial area, lateral agranular part",
  "primary_injection_structure": 394,
  "data_set_id": 562674923,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838048,
   "name": "Slc17a6-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "91/846",
  "injection_volume": 0.0068924282,
  "injection_x": 11260,
  "injection_y": 4010,
  "injection_z": 8100,
  "product_id": 5,
  "specimen_name": "Slc17a6-IRES-Cre-3628",
  "strain": "B6.129",
  "structure_abbrev": "DN",
  "structure_id": 846,
  "structure_name": "Dentate nucleus",
  "primary_injection_structure": 91,
  "data_set_id": 286608092,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837777,
   "name": "Sim1-Cre_KJ18"
  },
  "gender": "F",
  "injection_structures": "993",
  "injection_volume": 0.0366118445,
  "injection_x": 4200,
  "injection_y": 1950,
  "injection_z": 6660,
  "product_id": 5,
  "specimen_name": "Sim1-Cre_KJ18-125423",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 297947641,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838361,
   "name": "Etv1-CreERT2"
  },
  "gender": "F",
  "injection_structures": "993",
  "injection_volume": 0.256714332,
  "injection_x": 3840,
  "injection_y": 1590,
  "injection_z": 7140,
  "product_id": 5,
  "specimen_name": "Etv1-CreERT2-1163",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 156492394,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838074,
   "name": "Slc6a3-Cre"
  },
  "gender": "M",
  "injection_structures": "1/126/194/980/1004/1126",
  "injection_volume": 0.110425959,
  "injection_x": 7630,
  "injection_y": 6680,
  "injection_z": 6210,
  "product_id": 5,
  "specimen_name": "Slc6a3-Cre-4579",
  "strain": "C57BL/6J",
  "structure_abbrev": "PMv",
  "structure_id": 1004,
  "structure_name": "Ventral premammillary nucleus",
  "primary_injection_structure": 1,
  "data_set_id": 263780018,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839331,
   "name": "Nr5a1-Cre"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.0619028025,
  "injection_x": 8070,
  "injection_y": 710,
  "injection_z": 8160,
  "product_id": 5,
  "specimen_name": "Nr5a1-Cre-2439",
  "strain": "B6.FVB",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 176901480,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837979,
   "name": "Drd1a-Cre_EY262"
  },
  "gender": "F",
  "injection_structures": "56/342/351",
  "injection_volume": 0.2496510604,
  "injection_x": 4450,
  "injection_y": 5600,
  "injection_z": 6090,
  "product_id": 5,
  "specimen_name": "Drd1a-Cre-4926",
  "strain": "C57BL/6J",
  "structure_abbrev": "ACB",
  "structure_id": 56,
  "structure_name": "Nucleus accumbens",
  "primary_injection_structure": 56,
  "data_set_id": 168614604,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838803,
   "name": "Grm2-Cre_MR90"
  },
  "gender": "M",
  "injection_structures": "726",
  "injection_volume": 0.0760011168,
  "injection_x": 7600,
  "injection_y": 2330,
  "injection_z": 7530,
  "product_id": 5,
  "specimen_name": "Grm2-Cre_MR90-118",
  "strain": null,
  "structure_abbrev": "DG",
  "structure_id": 726,
  "structure_name": "Dentate gyrus",
  "primary_injection_structure": 726,
  "data_set_id": 264709761,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "723/731/746/972",
  "injection_volume": 0.5264251496,
  "injection_x": 2710,
  "injection_y": 3990,
  "injection_z": 6400,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre-129",
  "strain": null,
  "structure_abbrev": "ORBvl",
  "structure_id": 746,
  "structure_name": "Orbital area, ventrolateral part",
  "primary_injection_structure": 723,
  "data_set_id": 167902586,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838953,
   "name": "Oxt-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "38/63",
  "injection_volume": 0.0100202458125,
  "injection_x": 6450,
  "injection_y": 5610,
  "injection_z": 5870,
  "product_id": 5,
  "specimen_name": "Oxt-IRES-Cre-1193",
  "strain": "B6.129",
  "structure_abbrev": "PVHd",
  "structure_id": 63,
  "structure_name": "Paraventricular hypothalamic nucleus, descending division",
  "primary_injection_structure": 38,
  "data_set_id": 147051682,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182693192,
   "name": "Chrna2-Cre_OE25"
  },
  "gender": "M",
  "injection_structures": "382/484682470",
  "injection_volume": 0.0591708173,
  "injection_x": 8680,
  "injection_y": 3210,
  "injection_z": 9730,
  "product_id": 5,
  "specimen_name": "Chrna2-Cre_OE25-124322",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "CA1",
  "structure_id": 382,
  "structure_name": "Field CA1",
  "primary_injection_structure": 382,
  "data_set_id": 297894255,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839285,
   "name": "Gpr26-Cre_KO250"
  },
  "gender": "F",
  "injection_structures": "39/993",
  "injection_volume": 0.113622964,
  "injection_x": 4280,
  "injection_y": 1960,
  "injection_z": 6040,
  "product_id": 5,
  "specimen_name": "Gpr26-Cre_KO250-122",
  "strain": null,
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 287248684,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 553502711,
   "name": "Chat-IRES-Cre-neo"
  },
  "gender": "M",
  "injection_structures": "128/1052",
  "injection_volume": 0.0397111141,
  "injection_x": 9550,
  "injection_y": 4400,
  "injection_z": 6910,
  "product_id": 5,
  "specimen_name": "Chat-IRES-Cre-3211",
  "strain": "B6.129",
  "structure_abbrev": "PPN",
  "structure_id": 1052,
  "structure_name": "Pedunculopontine nucleus",
  "primary_injection_structure": 128,
  "data_set_id": 264566672,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "F",
  "injection_structures": "39/48/879/886/894/993",
  "injection_volume": 0.2197360312,
  "injection_x": 7280,
  "injection_y": 1290,
  "injection_z": 6290,
  "product_id": 5,
  "specimen_name": "Syt6-Cre-2837",
  "strain": "C57BL/6J",
  "structure_abbrev": "RSPv",
  "structure_id": 886,
  "structure_name": "Retrosplenial area, ventral part",
  "primary_injection_structure": 39,
  "data_set_id": 176497015,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "M",
  "injection_structures": "502/918/926/312782574/312782628",
  "injection_volume": 0.00693162085,
  "injection_x": 9420,
  "injection_y": 2930,
  "injection_z": 9490,
  "product_id": 5,
  "specimen_name": "Syt6-Cre_KI148-3669",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISpor",
  "structure_id": 312782628,
  "structure_name": "Postrhinal area",
  "primary_injection_structure": 502,
  "data_set_id": 267031577,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838633,
   "name": "Trib2-F2A-CreERT2"
  },
  "gender": "M",
  "injection_structures": "886",
  "injection_volume": 0.0299341882,
  "injection_x": 7060,
  "injection_y": 1140,
  "injection_z": 5340,
  "product_id": 5,
  "specimen_name": "Trib2-2A-CreERT2-D-3289",
  "strain": "B6.129",
  "structure_abbrev": "RSPv",
  "structure_id": 886,
  "structure_name": "Retrosplenial area, ventral part",
  "primary_injection_structure": 886,
  "data_set_id": 287601100,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "402/409",
  "injection_volume": 0.0419703081,
  "injection_x": 8430,
  "injection_y": 1680,
  "injection_z": 2130,
  "product_id": 35,
  "specimen_name": "Rbp4-Cre_KL100-247182",
  "strain": null,
  "structure_abbrev": "VISal",
  "structure_id": 402,
  "structure_name": "Anterolateral visual area",
  "primary_injection_structure": 402,
  "data_set_id": 524666904,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0658819896,
  "injection_x": 8700,
  "injection_y": 930,
  "injection_z": 8220,
  "product_id": 5,
  "specimen_name": "Tlx3-Cre-123685",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 296052133,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838584,
   "name": "Gad2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "382/463/726",
  "injection_volume": 0.1158464468,
  "injection_x": 6980,
  "injection_y": 2670,
  "injection_z": 6310,
  "product_id": 5,
  "specimen_name": "Gad2-IRES-Cre_LURC-123565",
  "strain": "C57BL/6J",
  "structure_abbrev": "DG",
  "structure_id": 726,
  "structure_name": "Dentate gyrus",
  "primary_injection_structure": 382,
  "data_set_id": 305400137,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 287226763,
   "name": "Calb1-T2A-dgCre"
  },
  "gender": "M",
  "injection_structures": "39/993",
  "injection_volume": 0.1001133308,
  "injection_x": 5540,
  "injection_y": 1430,
  "injection_z": 5960,
  "product_id": 5,
  "specimen_name": "Calb1-2A-dgCre-243345",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 39,
  "data_set_id": 524268045,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "385/894",
  "injection_volume": 0.071878492,
  "injection_x": 10180,
  "injection_y": 1380,
  "injection_z": 3620,
  "product_id": 36,
  "specimen_name": "Cux2-IRES-Cre-287719",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 566732238,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761781,
   "name": "Efr3a-Cre_NO108"
  },
  "gender": "M",
  "injection_structures": "39/48",
  "injection_volume": 0.0263726673,
  "injection_x": 4540,
  "injection_y": 2400,
  "injection_z": 6130,
  "product_id": 5,
  "specimen_name": "Efr3a-Cre_NO108-29",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "ACAv",
  "structure_id": 48,
  "structure_name": "Anterior cingulate area, ventral part",
  "primary_injection_structure": 39,
  "data_set_id": 267749107,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "385/879/894",
  "injection_volume": 0.02662797935,
  "injection_x": 9720,
  "injection_y": 1630,
  "injection_z": 3930,
  "product_id": 35,
  "specimen_name": "Rbp4-Cre_KL100-245456",
  "strain": null,
  "structure_abbrev": "RSPd",
  "structure_id": 879,
  "structure_name": "Retrosplenial area, dorsal part",
  "primary_injection_structure": 385,
  "data_set_id": 523718075,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "879/886",
  "injection_volume": 0.093930697,
  "injection_x": 7360,
  "injection_y": 640,
  "injection_z": 6080,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-3799",
  "strain": "C57BL/6J",
  "structure_abbrev": "RSPv",
  "structure_id": 886,
  "structure_name": "Retrosplenial area, ventral part",
  "primary_injection_structure": 879,
  "data_set_id": 166325321,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837797,
   "name": "Th-Cre_FI172"
  },
  "gender": "F",
  "injection_structures": "591/795/872",
  "injection_volume": 0.127068849425,
  "injection_x": 9720,
  "injection_y": 3220,
  "injection_z": 5680,
  "product_id": 5,
  "specimen_name": "Th-Cre_FI172-6212",
  "strain": "B6.FVB",
  "structure_abbrev": "PAG",
  "structure_id": 795,
  "structure_name": "Periaqueductal gray",
  "primary_injection_structure": 591,
  "data_set_id": 272699357,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182693192,
   "name": "Chrna2-Cre_OE25"
  },
  "gender": "M",
  "injection_structures": "104/119/583",
  "injection_volume": 0.0257606916,
  "injection_x": 3580,
  "injection_y": 4400,
  "injection_z": 8240,
  "product_id": 5,
  "specimen_name": "Chrna2-Cre_OE25-3908",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "AId",
  "structure_id": 104,
  "structure_name": "Agranular insular area, dorsal part",
  "primary_injection_structure": 104,
  "data_set_id": 292209592,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "409/502/541/895/918/926/312782574/312782628",
  "injection_volume": 0.103147646,
  "injection_x": 9240,
  "injection_y": 2410,
  "injection_z": 9610,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-3830",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISpor",
  "structure_id": 312782628,
  "structure_name": "Postrhinal area",
  "primary_injection_structure": 409,
  "data_set_id": 176429574,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "985/993",
  "injection_volume": 0.4247628504,
  "injection_x": 3200,
  "injection_y": 2470,
  "injection_z": 8050,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre-113",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 166082128,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "M",
  "injection_structures": "184/723/993",
  "injection_volume": 0.0172036795,
  "injection_x": 2790,
  "injection_y": 3000,
  "injection_z": 7190,
  "product_id": 5,
  "specimen_name": "Syt6-Cre-3162",
  "strain": "C57BL/6J",
  "structure_abbrev": "FRP",
  "structure_id": 184,
  "structure_name": "Frontal pole, cerebral cortex",
  "primary_injection_structure": 184,
  "data_set_id": 177890246,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837788,
   "name": "Ins2-Cre_25"
  },
  "gender": "M",
  "injection_structures": "88/118/693/830",
  "injection_volume": 0.03741639755,
  "injection_x": 6760,
  "injection_y": 6000,
  "injection_z": 5930,
  "product_id": 5,
  "specimen_name": "Ins2-Cre-120",
  "strain": "C57BL/6J",
  "structure_abbrev": "DMH",
  "structure_id": 830,
  "structure_name": "Dorsomedial nucleus of the hypothalamus",
  "primary_injection_structure": 88,
  "data_set_id": 265813096,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "385/417",
  "injection_volume": 0.0087452364125,
  "injection_x": 7810,
  "injection_y": 1200,
  "injection_z": 3040,
  "product_id": 35,
  "specimen_name": "A930038C07Rik-Tg1-Cre-299925",
  "strain": null,
  "structure_abbrev": "VISrl",
  "structure_id": 417,
  "structure_name": "Rostrolateral visual area",
  "primary_injection_structure": 385,
  "data_set_id": 576660608,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837516,
   "name": "Scnn1a-Tg3-Cre"
  },
  "gender": "M",
  "injection_structures": "385/533",
  "injection_volume": 0.019914108375,
  "injection_x": 8300,
  "injection_y": 940,
  "injection_z": 3790,
  "product_id": 35,
  "specimen_name": "Scnn1a-Tg3-Cre-227541",
  "strain": null,
  "structure_abbrev": "VISpm",
  "structure_id": 533,
  "structure_name": "posteromedial visual area",
  "primary_injection_structure": 385,
  "data_set_id": 503813164,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837281,
   "name": "Calb2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0146920718,
  "injection_x": 9560,
  "injection_y": 1130,
  "injection_z": 8480,
  "product_id": 5,
  "specimen_name": "Calb2-IRES-Cre-173",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 168515938,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "184/993",
  "injection_volume": 0.2369332672,
  "injection_x": 2230,
  "injection_y": 2520,
  "injection_z": 7370,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre_KL100-223",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 184,
  "data_set_id": 263242463,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182693192,
   "name": "Chrna2-Cre_OE25"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.0517806912,
  "injection_x": 8600,
  "injection_y": 1320,
  "injection_z": 8280,
  "product_id": 5,
  "specimen_name": "Chrna2-Cre_OE25-3898",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 292126180,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "385/425/879/894/1037/484682508",
  "injection_volume": 0.597899176,
  "injection_x": 9720,
  "injection_y": 2100,
  "injection_z": 3400,
  "product_id": 36,
  "specimen_name": "Emx1-IRES-Cre-323879",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 601476074,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "39/44/731/972/993",
  "injection_volume": 0.1551617921875,
  "injection_x": 2970,
  "injection_y": 3120,
  "injection_z": 5650,
  "product_id": 36,
  "specimen_name": "Cux2-IRES-Cre-203191",
  "strain": "C57BL/6J",
  "structure_abbrev": "ILA",
  "structure_id": 44,
  "structure_name": "Infralimbic area",
  "primary_injection_structure": 39,
  "data_set_id": 496097913,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "337/985",
  "injection_volume": 0.2351111336,
  "injection_x": 5590,
  "injection_y": 1390,
  "injection_z": 7460,
  "product_id": 5,
  "specimen_name": "378-671",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOp",
  "structure_id": 985,
  "structure_name": "Primary motor area",
  "primary_injection_structure": 337,
  "data_set_id": 100141273,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "111/378/677",
  "injection_volume": 0.4499517904,
  "injection_x": 6040,
  "injection_y": 4400,
  "injection_z": 10240,
  "product_id": 5,
  "specimen_name": "378-1763",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISC",
  "structure_id": 677,
  "structure_name": "Visceral area",
  "primary_injection_structure": 111,
  "data_set_id": 174361746,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "59/189/362/599/1120",
  "injection_volume": 0.029631217984375,
  "injection_x": 6590,
  "injection_y": 4440,
  "injection_z": 5690,
  "product_id": 5,
  "specimen_name": "378-1532",
  "strain": "C57BL/6J",
  "structure_abbrev": "CM",
  "structure_id": 599,
  "structure_name": "Central medial nucleus of the thalamus",
  "primary_injection_structure": 59,
  "data_set_id": 147135107,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838715,
   "name": "Tac1-IRES2-Cre"
  },
  "gender": "M",
  "injection_structures": "985/993",
  "injection_volume": 0.4558051344,
  "injection_x": 3350,
  "injection_y": 1960,
  "injection_z": 7200,
  "product_id": 5,
  "specimen_name": "Tac1-IRES2-Cre-D-46",
  "strain": "B6.129",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 181259279,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761781,
   "name": "Efr3a-Cre_NO108"
  },
  "gender": "M",
  "injection_structures": "38/63/118/181/797",
  "injection_volume": 0.079534334075,
  "injection_x": 6570,
  "injection_y": 5640,
  "injection_z": 5670,
  "product_id": 5,
  "specimen_name": "Efr3a-Cre_NO108-132091",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "PVHd",
  "structure_id": 63,
  "structure_name": "Paraventricular hypothalamic nucleus, descending division",
  "primary_injection_structure": 38,
  "data_set_id": 299759881,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839285,
   "name": "Gpr26-Cre_KO250"
  },
  "gender": "M",
  "injection_structures": "394/312782546",
  "injection_volume": 0.08457743,
  "injection_x": 7540,
  "injection_y": 1240,
  "injection_z": 7460,
  "product_id": 5,
  "specimen_name": "Gpr26-Cre_KO250-106",
  "strain": null,
  "structure_abbrev": "VISam",
  "structure_id": 394,
  "structure_name": "Anteromedial visual area",
  "primary_injection_structure": 394,
  "data_set_id": 267657327,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761781,
   "name": "Efr3a-Cre_NO108"
  },
  "gender": "M",
  "injection_structures": "329",
  "injection_volume": 0.0900056892,
  "injection_x": 6850,
  "injection_y": 1610,
  "injection_z": 8600,
  "product_id": 5,
  "specimen_name": "Efr3a-Cre_NO108-64",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 286648997,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "402/541/1002/1027",
  "injection_volume": 0.165472608,
  "injection_x": 8530,
  "injection_y": 2160,
  "injection_z": 1590,
  "product_id": 35,
  "specimen_name": "Cux2-IRES-Cre-287715",
  "strain": null,
  "structure_abbrev": "AUDpo",
  "structure_id": 1027,
  "structure_name": "Posterior auditory area",
  "primary_injection_structure": 402,
  "data_set_id": 566730846,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 298725927,
   "name": "Penk-IRES2-Cre-neo"
  },
  "gender": "M",
  "injection_structures": "993",
  "injection_volume": 0.0364393939,
  "injection_x": 3310,
  "injection_y": 2800,
  "injection_z": 4060,
  "product_id": 45,
  "specimen_name": "Penk-IRES2-Cre-neo;Slc17a6-IRES2-FlpO-365241",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 671463542,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "329",
  "injection_volume": 0.0198454263,
  "injection_x": 6270,
  "injection_y": 1990,
  "injection_z": 8870,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-3852",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 182467026,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "184/993",
  "injection_volume": 0.1254951446,
  "injection_x": 2130,
  "injection_y": 2480,
  "injection_z": 7190,
  "product_id": 5,
  "specimen_name": "378-1623",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 184,
  "data_set_id": 157710335,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838633,
   "name": "Trib2-F2A-CreERT2"
  },
  "gender": "F",
  "injection_structures": "39/48",
  "injection_volume": 0.174568364281067,
  "injection_x": 5120,
  "injection_y": 1770,
  "injection_z": 6020,
  "product_id": 5,
  "specimen_name": "Trib2-2A-CreERT2-D-3287",
  "strain": "B6.129",
  "structure_abbrev": "ACAv",
  "structure_id": 48,
  "structure_name": "Anterior cingulate area, ventral part",
  "primary_injection_structure": 39,
  "data_set_id": 287600392,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "39/985/993",
  "injection_volume": 0.5682902792,
  "injection_x": 5070,
  "injection_y": 2170,
  "injection_z": 6610,
  "product_id": 5,
  "specimen_name": "378-1824",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 39,
  "data_set_id": 180916954,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "879/886",
  "injection_volume": 0.0817127234,
  "injection_x": 8280,
  "injection_y": 690,
  "injection_z": 6000,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-146",
  "strain": null,
  "structure_abbrev": "RSPd",
  "structure_id": 879,
  "structure_name": "Retrosplenial area, dorsal part",
  "primary_injection_structure": 879,
  "data_set_id": 182467736,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "382/726",
  "injection_volume": 0.0262364522,
  "injection_x": 6980,
  "injection_y": 2580,
  "injection_z": 6310,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre-163",
  "strain": null,
  "structure_abbrev": "DG",
  "structure_id": 726,
  "structure_name": "Dentate gyrus",
  "primary_injection_structure": 382,
  "data_set_id": 182295393,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "409/312782574",
  "injection_volume": 0.1061046098,
  "injection_x": 9080,
  "injection_y": 2270,
  "injection_z": 9460,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-3797",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 409,
  "data_set_id": 287495026,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "250/258/266/310/672",
  "injection_volume": 0.3858661408,
  "injection_x": 5090,
  "injection_y": 4370,
  "injection_z": 6290,
  "product_id": 5,
  "specimen_name": "378-673",
  "strain": "C57BL/6J",
  "structure_abbrev": "LSr",
  "structure_id": 258,
  "structure_name": "Lateral septal nucleus, rostral (rostroventral) part",
  "primary_injection_structure": 250,
  "data_set_id": 100141435,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838266,
   "name": "Erbb4-T2A-CreERT2"
  },
  "gender": "M",
  "injection_structures": "39/48",
  "injection_volume": 0.1414893032,
  "injection_x": 5080,
  "injection_y": 1710,
  "injection_z": 5520,
  "product_id": 5,
  "specimen_name": "Erbb4-2A-CreERT2-D-5681",
  "strain": "C57BL/6J",
  "structure_abbrev": "ACAd",
  "structure_id": 39,
  "structure_name": "Anterior cingulate area, dorsal part",
  "primary_injection_structure": 39,
  "data_set_id": 156742543,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "928/989",
  "injection_volume": 0.1853504184,
  "injection_x": 11650,
  "injection_y": 2870,
  "injection_z": 6580,
  "product_id": 5,
  "specimen_name": "378-1440",
  "strain": "C57BL/6J",
  "structure_abbrev": "FN",
  "structure_id": 989,
  "structure_name": "Fastigial nucleus",
  "primary_injection_structure": 928,
  "data_set_id": 127650431,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "429/437",
  "injection_volume": 0.1596212142,
  "injection_x": 12350,
  "injection_y": 5470,
  "injection_z": 7640,
  "product_id": 5,
  "specimen_name": "378-1865",
  "strain": "C57BL/6J",
  "structure_abbrev": "SPVI",
  "structure_id": 437,
  "structure_name": "Spinal nucleus of the trigeminal, interpolar part",
  "primary_injection_structure": 429,
  "data_set_id": 272738620,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "M",
  "injection_structures": "993",
  "injection_volume": 0.02206646645,
  "injection_x": 3920,
  "injection_y": 2760,
  "injection_z": 7290,
  "product_id": 5,
  "specimen_name": "Syt6-Cre_KI148-303005",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 584513749,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 180643392,
   "name": "Rasgrf2-T2A-dCre"
  },
  "gender": "F",
  "injection_structures": "337/361",
  "injection_volume": 0.0927333526,
  "injection_x": 6460,
  "injection_y": 1210,
  "injection_z": 7200,
  "product_id": 5,
  "specimen_name": "Rasgrf2-2A-dCre-D-135266",
  "strain": null,
  "structure_abbrev": "SSp-tr",
  "structure_id": 361,
  "structure_name": "Primary somatosensory area, trunk",
  "primary_injection_structure": 337,
  "data_set_id": 303713313,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 266649644,
   "name": "Drd3-Cre_KI196"
  },
  "gender": "M",
  "injection_structures": "39/48",
  "injection_volume": 0.002298976640625,
  "injection_x": 4630,
  "injection_y": 2600,
  "injection_z": 6110,
  "product_id": 5,
  "specimen_name": "Drd3-Cre_KI196_LURC-128044",
  "strain": null,
  "structure_abbrev": "ACAv",
  "structure_id": 48,
  "structure_name": "Anterior cingulate area, ventral part",
  "primary_injection_structure": 39,
  "data_set_id": 516848906,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838828,
   "name": "Gal-Cre_KI87"
  },
  "gender": "F",
  "injection_structures": "15/64/127/255/1113",
  "injection_volume": 0.0959051716,
  "injection_x": 6090,
  "injection_y": 4300,
  "injection_z": 6130,
  "product_id": 5,
  "specimen_name": "Gal-Cre_KI87-3473",
  "strain": "C57BL/6J",
  "structure_abbrev": "AM",
  "structure_id": 127,
  "structure_name": "Anteromedial nucleus",
  "primary_injection_structure": 15,
  "data_set_id": 266174045,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 301613169,
   "name": "Sepw1-Cre_NP39"
  },
  "gender": "F",
  "injection_structures": "159/723",
  "injection_volume": 0.0773083241,
  "injection_x": 2600,
  "injection_y": 4050,
  "injection_z": 6940,
  "product_id": 5,
  "specimen_name": "Sepw1-Cre_NP39-237402",
  "strain": null,
  "structure_abbrev": "ORBl",
  "structure_id": 723,
  "structure_name": "Orbital area, lateral part",
  "primary_injection_structure": 159,
  "data_set_id": 521600217,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837797,
   "name": "Th-Cre_FI172"
  },
  "gender": "F",
  "injection_structures": "329/353/378/672",
  "injection_volume": 0.02304169875,
  "injection_x": 6630,
  "injection_y": 3620,
  "injection_z": 9150,
  "product_id": 5,
  "specimen_name": "Th-Cre_FI172-6211",
  "strain": "B6.FVB",
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 329,
  "data_set_id": 272697238,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838074,
   "name": "Slc6a3-Cre"
  },
  "gender": "F",
  "injection_structures": "159/507",
  "injection_volume": 0.0051773516375,
  "injection_x": 2430,
  "injection_y": 4560,
  "injection_z": 6140,
  "product_id": 5,
  "specimen_name": "Slc6a3-Cre-4546",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOB",
  "structure_id": 507,
  "structure_name": "Main olfactory bulb",
  "primary_injection_structure": 159,
  "data_set_id": 182042349,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.017167989125,
  "injection_x": 7960,
  "injection_y": 1060,
  "injection_z": 7970,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre-189",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 183173527,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "F",
  "injection_structures": "394/533",
  "injection_volume": 0.0374068205,
  "injection_x": 7800,
  "injection_y": 600,
  "injection_z": 7270,
  "product_id": 5,
  "specimen_name": "Tlx3-Cre-126196",
  "strain": null,
  "structure_abbrev": "VISam",
  "structure_id": 394,
  "structure_name": "Anteromedial visual area",
  "primary_injection_structure": 394,
  "data_set_id": 297233422,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "394/533/312782546",
  "injection_volume": 0.21717731,
  "injection_x": 7980,
  "injection_y": 690,
  "injection_z": 3670,
  "product_id": 36,
  "specimen_name": "Cux2-IRES-Cre-239322",
  "strain": null,
  "structure_abbrev": "VISam",
  "structure_id": 394,
  "structure_name": "Anteromedial visual area",
  "primary_injection_structure": 394,
  "data_set_id": 517326050,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 309483400,
   "name": "Foxp2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "149/186/362/483",
  "injection_volume": 0.084566748,
  "injection_x": 6280,
  "injection_y": 3470,
  "injection_z": 5390,
  "product_id": 5,
  "specimen_name": "Foxp2-IRES-Cre-276059",
  "strain": null,
  "structure_abbrev": "MD",
  "structure_id": 362,
  "structure_name": "Mediodorsal nucleus of thalamus",
  "primary_injection_structure": 149,
  "data_set_id": 558697990,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838877,
   "name": "Gabrr3-Cre_KC112"
  },
  "gender": "M",
  "injection_structures": "470/797",
  "injection_volume": 0.0879617032,
  "injection_x": 7430,
  "injection_y": 4960,
  "injection_z": 7710,
  "product_id": 5,
  "specimen_name": "Gabrr3-Cre-127",
  "strain": null,
  "structure_abbrev": "ZI",
  "structure_id": 797,
  "structure_name": "Zona incerta",
  "primary_injection_structure": 470,
  "data_set_id": 170858675,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837324,
   "name": "Slc18a2-Cre_OZ14"
  },
  "gender": "M",
  "injection_structures": "345",
  "injection_volume": 0.012586577325,
  "injection_x": 4810,
  "injection_y": 2240,
  "injection_z": 8710,
  "product_id": 5,
  "specimen_name": "Slc18a2-Cre_OZ14-121804",
  "strain": null,
  "structure_abbrev": "SSp-m",
  "structure_id": 345,
  "structure_name": "Primary somatosensory area, mouth",
  "primary_injection_structure": 345,
  "data_set_id": 306959448,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "993",
  "injection_volume": 0.1318701328,
  "injection_x": 2840,
  "injection_y": 1950,
  "injection_z": 3820,
  "product_id": 45,
  "specimen_name": "Cux2-IRES-Cre-223004",
  "strain": "C57BL/6J",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 504105304,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 309779993,
   "name": "Ctgf-T2A-dgCre"
  },
  "gender": "M",
  "injection_structures": "385/425/1037/484682508",
  "injection_volume": 0.0030646266,
  "injection_x": 9680,
  "injection_y": 2170,
  "injection_z": 8470,
  "product_id": 5,
  "specimen_name": "Ctgf-T2A-dgCre-325053",
  "strain": null,
  "structure_abbrev": "VISpl",
  "structure_id": 425,
  "structure_name": "Posterolateral visual area",
  "primary_injection_structure": 385,
  "data_set_id": 605660419,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "345/353",
  "injection_volume": 0.1196805792,
  "injection_x": 5370,
  "injection_y": 2870,
  "injection_z": 9240,
  "product_id": 5,
  "specimen_name": "378-913",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-n",
  "structure_id": 353,
  "structure_name": "Primary somatosensory area, nose",
  "primary_injection_structure": 345,
  "data_set_id": 112162251,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839159,
   "name": "Cck-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "128/795/872",
  "injection_volume": 0.014496876625,
  "injection_x": 9370,
  "injection_y": 3180,
  "injection_z": 6030,
  "product_id": 5,
  "specimen_name": "Cck-IRES-Cre-121992",
  "strain": null,
  "structure_abbrev": "PAG",
  "structure_id": 795,
  "structure_name": "Periaqueductal gray",
  "primary_injection_structure": 128,
  "data_set_id": 299856390,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "385/409/425",
  "injection_volume": 0.0641696699,
  "injection_x": 9630,
  "injection_y": 1850,
  "injection_z": 9170,
  "product_id": 5,
  "specimen_name": "Ntsr1-Cre_GN220-125495",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 385,
  "data_set_id": 297629988,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "385/394/417/533/312782546",
  "injection_volume": 0.105423941,
  "injection_x": 7880,
  "injection_y": 1100,
  "injection_z": 3600,
  "product_id": 35,
  "specimen_name": "Rbp4-Cre_KL100-359552",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 656632388,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838710,
   "name": "Sst-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "1002/1011/1018",
  "injection_volume": 0.1861256964,
  "injection_x": 7490,
  "injection_y": 3070,
  "injection_z": 9810,
  "product_id": 5,
  "specimen_name": "Sst-IRES-Cre-180",
  "strain": null,
  "structure_abbrev": "AUDp",
  "structure_id": 1002,
  "structure_name": "Primary auditory area",
  "primary_injection_structure": 1002,
  "data_set_id": 182886547,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 301467016,
   "name": "Npr3-IRES2-Cre"
  },
  "gender": "M",
  "injection_structures": "985",
  "injection_volume": 0.120511972,
  "injection_x": 4970,
  "injection_y": 2000,
  "injection_z": 7270,
  "product_id": 5,
  "specimen_name": "Npr3-IRES2-Cre-213450",
  "strain": null,
  "structure_abbrev": "MOp",
  "structure_id": 985,
  "structure_name": "Primary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 512130198,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838302,
   "name": "Gng7-Cre_KH71"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.2408309036,
  "injection_x": 9660,
  "injection_y": 1260,
  "injection_z": 8520,
  "product_id": 5,
  "specimen_name": "Gng7-Cre-969",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 157955639,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "385/894",
  "injection_volume": 0.0439855409,
  "injection_x": 9930,
  "injection_y": 1630,
  "injection_z": 2970,
  "product_id": 36,
  "specimen_name": "Emx1-IRES-Cre-301200",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 580150240,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "F",
  "injection_structures": "39/48",
  "injection_volume": 0.0525710906,
  "injection_x": 5020,
  "injection_y": 1990,
  "injection_z": 6080,
  "product_id": 5,
  "specimen_name": "A930038C07Rik-Tg1-Cre-3775",
  "strain": "C57BL/6J",
  "structure_abbrev": "ACAv",
  "structure_id": 48,
  "structure_name": "Anterior cingulate area, ventral part",
  "primary_injection_structure": 39,
  "data_set_id": 159319654,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "541/312782574/312782628",
  "injection_volume": 0.04006049635,
  "injection_x": 9090,
  "injection_y": 2380,
  "injection_z": 1630,
  "product_id": 36,
  "specimen_name": "Tlx3-Cre_PL56-319680",
  "strain": "unknown",
  "structure_abbrev": "VISli",
  "structure_id": 312782574,
  "structure_name": "Laterointermediate area",
  "primary_injection_structure": 541,
  "data_set_id": 595858299,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838803,
   "name": "Grm2-Cre_MR90"
  },
  "gender": "M",
  "injection_structures": "920/928",
  "injection_volume": 0.0817994289,
  "injection_x": 11570,
  "injection_y": 2010,
  "injection_z": 6410,
  "product_id": 5,
  "specimen_name": "Grm2-Cre_MR90-3858",
  "strain": null,
  "structure_abbrev": "CUL",
  "structure_id": 928,
  "structure_name": "Culmen",
  "primary_injection_structure": 920,
  "data_set_id": 292531359,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839406,
   "name": "Nos1-CreERT2"
  },
  "gender": "M",
  "injection_structures": "382",
  "injection_volume": 0.010430739025,
  "injection_x": 7740,
  "injection_y": 1760,
  "injection_z": 8250,
  "product_id": 5,
  "specimen_name": "Nos1-CreERT2-126100",
  "strain": "B6.129",
  "structure_abbrev": "CA1",
  "structure_id": 382,
  "structure_name": "Field CA1",
  "primary_injection_structure": 382,
  "data_set_id": 301325601,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 309779993,
   "name": "Ctgf-T2A-dgCre"
  },
  "gender": "M",
  "injection_structures": "385/1037",
  "injection_volume": 0.011411248625,
  "injection_x": 9730,
  "injection_y": 1570,
  "injection_z": 8080,
  "product_id": 5,
  "specimen_name": "Ctgf-2A-dgCre-neo-179276",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 477475894,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "88/133/272/347/515/523/576073699",
  "injection_volume": 0.2162678112,
  "injection_x": 5260,
  "injection_y": 6660,
  "injection_z": 5920,
  "product_id": 5,
  "specimen_name": "378-1178",
  "strain": "C57BL/6J",
  "structure_abbrev": "MPN",
  "structure_id": 515,
  "structure_name": "Medial preoptic nucleus",
  "primary_injection_structure": 88,
  "data_set_id": 114472860,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 298725927,
   "name": "Penk-IRES2-Cre-neo"
  },
  "gender": "F",
  "injection_structures": "993",
  "injection_volume": 0.02983928745,
  "injection_x": 3230,
  "injection_y": 2670,
  "injection_z": 4230,
  "product_id": 45,
  "specimen_name": "Penk-IRES2-Cre-neo-343078",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 639813139,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "394/533/879/894/312782546",
  "injection_volume": 0.4500445768,
  "injection_x": 8100,
  "injection_y": 450,
  "injection_z": 4160,
  "product_id": 35,
  "specimen_name": "Cux2-IRES-Cre-270244",
  "strain": null,
  "structure_abbrev": "VISam",
  "structure_id": 394,
  "structure_name": "Anteromedial visual area",
  "primary_injection_structure": 394,
  "data_set_id": 552431726,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "533/879/886/894",
  "injection_volume": 0.0112121702,
  "injection_x": 8600,
  "injection_y": 1070,
  "injection_z": 4190,
  "product_id": 36,
  "specimen_name": "Ntsr1-Cre_GN220-204696",
  "strain": null,
  "structure_abbrev": "VISpm",
  "structure_id": 533,
  "structure_name": "posteromedial visual area",
  "primary_injection_structure": 533,
  "data_set_id": 496097187,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "337/369",
  "injection_volume": 0.083695822,
  "injection_x": 6220,
  "injection_y": 1520,
  "injection_z": 7700,
  "product_id": 5,
  "specimen_name": "Ntsr1-Cre-3519",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-ll",
  "structure_id": 337,
  "structure_name": "Primary somatosensory area, lower limb",
  "primary_injection_structure": 337,
  "data_set_id": 157768393,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839331,
   "name": "Nr5a1-Cre"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0219538277,
  "injection_x": 9130,
  "injection_y": 760,
  "injection_z": 7650,
  "product_id": 5,
  "specimen_name": "Nr5a1-Cre-2447",
  "strain": "B6.FVB",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 177781745,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838681,
   "name": "Slc6a4-Cre_ET33"
  },
  "gender": "M",
  "injection_structures": "115/591/795/872",
  "injection_volume": 0.1096707612,
  "injection_x": 9550,
  "injection_y": 4110,
  "injection_z": 5690,
  "product_id": 5,
  "specimen_name": "Slc6a4-Cre-Homo-279",
  "strain": "C57BL/6J",
  "structure_abbrev": "DR",
  "structure_id": 872,
  "structure_name": "Dorsal nucleus raphe",
  "primary_injection_structure": 115,
  "data_set_id": 128055110,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "44/731/972",
  "injection_volume": 0.2076688404,
  "injection_x": 2610,
  "injection_y": 3000,
  "injection_z": 5690,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-120542",
  "strain": "C57BL/6J",
  "structure_abbrev": "ORBm",
  "structure_id": 731,
  "structure_name": "Orbital area, medial part",
  "primary_injection_structure": 44,
  "data_set_id": 293471629,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 183238310,
   "name": "Ntrk1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "194/226/342/351/1022",
  "injection_volume": 0.02909315955,
  "injection_x": 5700,
  "injection_y": 5780,
  "injection_z": 7260,
  "product_id": 5,
  "specimen_name": "Ntrk1-IRES-Cre_LURC-125688",
  "strain": null,
  "structure_abbrev": "SI",
  "structure_id": 342,
  "structure_name": "Substantia innominata",
  "primary_injection_structure": 194,
  "data_set_id": 299404532,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "44/731/972",
  "injection_volume": 0.0206488548,
  "injection_x": 3130,
  "injection_y": 3450,
  "injection_z": 5950,
  "product_id": 5,
  "specimen_name": "Ntsr1-Cre-3565",
  "strain": "C57BL/6J",
  "structure_abbrev": "ILA",
  "structure_id": 44,
  "structure_name": "Infralimbic area",
  "primary_injection_structure": 44,
  "data_set_id": 159511623,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 301021674,
   "name": "Oxtr-T2A-Cre"
  },
  "gender": "F",
  "injection_structures": "223/614/693/830/1004",
  "injection_volume": 0.1511702136,
  "injection_x": 7080,
  "injection_y": 7150,
  "injection_z": 4790,
  "product_id": 45,
  "specimen_name": "Oxtr-2A-Cre;Ai14-222369",
  "strain": null,
  "structure_abbrev": "VMH",
  "structure_id": 693,
  "structure_name": "Ventromedial hypothalamic nucleus",
  "primary_injection_structure": 223,
  "data_set_id": 503036583,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838942,
   "name": "Htr2a-Cre_KM207"
  },
  "gender": "M",
  "injection_structures": "369/985",
  "injection_volume": 0.2388479324,
  "injection_x": 5050,
  "injection_y": 2130,
  "injection_z": 7780,
  "product_id": 5,
  "specimen_name": "Htr2a-Cre_KM207-3473",
  "strain": null,
  "structure_abbrev": "MOp",
  "structure_id": 985,
  "structure_name": "Primary motor area",
  "primary_injection_structure": 369,
  "data_set_id": 277957908,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837777,
   "name": "Sim1-Cre_KJ18"
  },
  "gender": "M",
  "injection_structures": "184/723/993",
  "injection_volume": 0.0086368723,
  "injection_x": 2690,
  "injection_y": 2550,
  "injection_z": 4320,
  "product_id": 45,
  "specimen_name": "Sim1-Cre_KJ18-239625",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 184,
  "data_set_id": 518222038,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 298725927,
   "name": "Penk-IRES2-Cre-neo"
  },
  "gender": "F",
  "injection_structures": "993",
  "injection_volume": 0.015842673875,
  "injection_x": 2590,
  "injection_y": 1930,
  "injection_z": 4420,
  "product_id": 45,
  "specimen_name": "Penk-IRES2-Cre-neo-321018",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 603211567,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "541/895/922/1018",
  "injection_volume": 1.075152512,
  "injection_x": 7860,
  "injection_y": 3740,
  "injection_z": 10390,
  "product_id": 5,
  "specimen_name": "378-1825",
  "strain": "C57BL/6J",
  "structure_abbrev": "ECT",
  "structure_id": 895,
  "structure_name": "Ectorhinal area",
  "primary_injection_structure": 541,
  "data_set_id": 180435652,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "985/993",
  "injection_volume": 0.0807070523,
  "injection_x": 3510,
  "injection_y": 2210,
  "injection_z": 7700,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre_KL100-319625",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 985,
  "data_set_id": 598604288,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "F",
  "injection_structures": "382/541/1002/1027/312782628",
  "injection_volume": 0.0730580984,
  "injection_x": 8340,
  "injection_y": 2290,
  "injection_z": 1810,
  "product_id": 36,
  "specimen_name": "Rbp4-Cre_KL100-345688",
  "strain": null,
  "structure_abbrev": "AUDpo",
  "structure_id": 1027,
  "structure_name": "Posterior auditory area",
  "primary_injection_structure": 382,
  "data_set_id": 642809043,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "402/541/1002/1027/312782574",
  "injection_volume": 0.3971654232,
  "injection_x": 8470,
  "injection_y": 2080,
  "injection_z": 1810,
  "product_id": 35,
  "specimen_name": "Emx1-IRES-Cre-281287",
  "strain": null,
  "structure_abbrev": "AUDpo",
  "structure_id": 1027,
  "structure_name": "Posterior auditory area",
  "primary_injection_structure": 402,
  "data_set_id": 554421791,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "382/423/463",
  "injection_volume": 0.1998477936,
  "injection_x": 7610,
  "injection_y": 3110,
  "injection_z": 8700,
  "product_id": 5,
  "specimen_name": "378-1183",
  "strain": "C57BL/6J",
  "structure_abbrev": "CA3",
  "structure_id": 463,
  "structure_name": "Field CA3",
  "primary_injection_structure": 382,
  "data_set_id": 114429338,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761781,
   "name": "Efr3a-Cre_NO108"
  },
  "gender": "M",
  "injection_structures": "345/353/378",
  "injection_volume": 0.016750320475,
  "injection_x": 5670,
  "injection_y": 3150,
  "injection_z": 9460,
  "product_id": 5,
  "specimen_name": "Efr3a-Cre_NO108-126001",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "SSp-n",
  "structure_id": 353,
  "structure_name": "Primary somatosensory area, nose",
  "primary_injection_structure": 345,
  "data_set_id": 301178953,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177836119,
   "name": "Emx1-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "385",
  "injection_volume": 0.1637749736,
  "injection_x": 9040,
  "injection_y": 1450,
  "injection_z": 3560,
  "product_id": 35,
  "specimen_name": "Emx1-IRES-Cre-194295",
  "strain": "B6.129",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 479756361,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839331,
   "name": "Nr5a1-Cre"
  },
  "gender": "M",
  "injection_structures": "378",
  "injection_volume": 0.0519777202,
  "injection_x": 5580,
  "injection_y": 3490,
  "injection_z": 9860,
  "product_id": 5,
  "specimen_name": "Nr5a1-Cre-2524",
  "strain": "B6.FVB",
  "structure_abbrev": "SSs",
  "structure_id": 378,
  "structure_name": "Supplemental somatosensory area",
  "primary_injection_structure": 378,
  "data_set_id": 266175461,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 180924870,
   "name": "Fezf1-T2A-dCre"
  },
  "gender": "F",
  "injection_structures": "118/223/614/693/830",
  "injection_volume": 0.458548456724527,
  "injection_x": 6690,
  "injection_y": 6350,
  "injection_z": 6460,
  "product_id": 5,
  "specimen_name": "Fezf1-2A-DD-Cre-D-669",
  "strain": "B6.129",
  "structure_abbrev": "VMH",
  "structure_id": 693,
  "structure_name": "Ventromedial hypothalamic nucleus",
  "primary_injection_structure": 118,
  "data_set_id": 267540168,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838494,
   "name": "Prkcd-GluCla-CFP-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "59/149/362/930",
  "injection_volume": 0.2018957584,
  "injection_x": 7400,
  "injection_y": 4160,
  "injection_z": 6040,
  "product_id": 5,
  "specimen_name": "Prkcd-GluCla-CFP-IRES-Cre-127",
  "strain": "C57BL/6J",
  "structure_abbrev": "MD",
  "structure_id": 362,
  "structure_name": "Mediodorsal nucleus of thalamus",
  "primary_injection_structure": 59,
  "data_set_id": 267607635,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 180643392,
   "name": "Rasgrf2-T2A-dCre"
  },
  "gender": "F",
  "injection_structures": "329/353/182305689",
  "injection_volume": 0.0432994576,
  "injection_x": 5940,
  "injection_y": 1960,
  "injection_z": 8840,
  "product_id": 5,
  "specimen_name": "Rasgrf2-2A-dCre-D-130402",
  "strain": null,
  "structure_abbrev": "SSp-bfd",
  "structure_id": 329,
  "structure_name": "Primary somatosensory area, barrel field",
  "primary_injection_structure": 329,
  "data_set_id": 298718778,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.02616217065,
  "injection_x": 8550,
  "injection_y": 1450,
  "injection_z": 3370,
  "product_id": 36,
  "specimen_name": "Ntsr1-Cre_GN220-235600",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 514313871,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "470/1031",
  "injection_volume": 0.013761068125,
  "injection_x": 7180,
  "injection_y": 5400,
  "injection_z": 7310,
  "product_id": 5,
  "specimen_name": "378-1513",
  "strain": "C57BL/6J",
  "structure_abbrev": "STN",
  "structure_id": 470,
  "structure_name": "Subthalamic nucleus",
  "primary_injection_structure": 470,
  "data_set_id": 146986331,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "M",
  "injection_structures": "329/417",
  "injection_volume": 0.0990182886,
  "injection_x": 7520,
  "injection_y": 1520,
  "injection_z": 8280,
  "product_id": 5,
  "specimen_name": "Syt6-Cre_KI148-131916",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISrl",
  "structure_id": 417,
  "structure_name": "Rostrolateral visual area",
  "primary_injection_structure": 329,
  "data_set_id": 299820770,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 553502711,
   "name": "Chat-IRES-Cre-neo"
  },
  "gender": "F",
  "injection_structures": "186/382/483/726",
  "injection_volume": 0.0080654275625,
  "injection_x": 7380,
  "injection_y": 2970,
  "injection_z": 5970,
  "product_id": 5,
  "specimen_name": "Chat-IRES-Cre-3235",
  "strain": "B6.129",
  "structure_abbrev": "MH",
  "structure_id": 483,
  "structure_name": "Medial habenula",
  "primary_injection_structure": 186,
  "data_set_id": 268321927,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838942,
   "name": "Htr2a-Cre_KM207"
  },
  "gender": "F",
  "injection_structures": "186/218/575/930",
  "injection_volume": 0.0079404285625,
  "injection_x": 7280,
  "injection_y": 3200,
  "injection_z": 6400,
  "product_id": 5,
  "specimen_name": "Htr2a-Cre_KM207-129169",
  "strain": "unknown",
  "structure_abbrev": "LH",
  "structure_id": 186,
  "structure_name": "Lateral habenula",
  "primary_injection_structure": 186,
  "data_set_id": 551351756,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838927,
   "name": "Pcdh9-Cre_NP276"
  },
  "gender": "F",
  "injection_structures": "918/926",
  "injection_volume": 0.00344958928125,
  "injection_x": 10120,
  "injection_y": 3510,
  "injection_z": 9900,
  "product_id": 5,
  "specimen_name": "Pcdh9-Cre-80",
  "strain": null,
  "structure_abbrev": "ENTm",
  "structure_id": 926,
  "structure_name": "Entorhinal area, medial part, dorsal zone",
  "primary_injection_structure": 918,
  "data_set_id": 267211671,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 266649644,
   "name": "Drd3-Cre_KI196"
  },
  "gender": "F",
  "injection_structures": "961/966",
  "injection_volume": 0.0499866591,
  "injection_x": 6310,
  "injection_y": 5880,
  "injection_z": 9580,
  "product_id": 5,
  "specimen_name": "Drd3-Cre_KI196_LURC-132829",
  "strain": null,
  "structure_abbrev": "PIR",
  "structure_id": 961,
  "structure_name": "Piriform area",
  "primary_injection_structure": 961,
  "data_set_id": 300109663,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 460133878,
   "name": "Ntng2-IRES2-Cre"
  },
  "gender": "F",
  "injection_structures": "385/409/425/926/312782574/312782628",
  "injection_volume": 0.0444596355,
  "injection_x": 9400,
  "injection_y": 2490,
  "injection_z": 9090,
  "product_id": 5,
  "specimen_name": "Ntng2-IRES2-Cre-272877",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 385,
  "data_set_id": 557200148,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "394/533/879/886/894",
  "injection_volume": 0.2233704788,
  "injection_x": 8110,
  "injection_y": 530,
  "injection_z": 4270,
  "product_id": 36,
  "specimen_name": "Cux2-IRES-Cre-296270",
  "strain": "unknown",
  "structure_abbrev": "RSPagl",
  "structure_id": 894,
  "structure_name": "Retrosplenial area, lateral agranular part",
  "primary_injection_structure": 394,
  "data_set_id": 576036240,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "104/119/583/985",
  "injection_volume": 0.3841124112,
  "injection_x": 3660,
  "injection_y": 4850,
  "injection_z": 8700,
  "product_id": 5,
  "specimen_name": "Rbp4-Cre_KL100-128404",
  "strain": "C57BL/6J",
  "structure_abbrev": "AId",
  "structure_id": 104,
  "structure_name": "Agranular insular area, dorsal part",
  "primary_injection_structure": 104,
  "data_set_id": 296048512,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "M",
  "injection_structures": "402/409/541/312782574",
  "injection_volume": 0.02333161215,
  "injection_x": 8540,
  "injection_y": 1980,
  "injection_z": 1800,
  "product_id": 35,
  "specimen_name": "Tlx3-Cre_PL56-273880",
  "strain": null,
  "structure_abbrev": "VISli",
  "structure_id": 312782574,
  "structure_name": "Laterointermediate area",
  "primary_injection_structure": 402,
  "data_set_id": 554420321,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "F",
  "injection_structures": "385/533/894",
  "injection_volume": 0.01806967855,
  "injection_x": 9080,
  "injection_y": 1090,
  "injection_z": 7310,
  "product_id": 5,
  "specimen_name": "Syt6-Cre-2820",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 168097187,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839376,
   "name": "Otof-Cre"
  },
  "gender": "M",
  "injection_structures": "918",
  "injection_volume": 0.234974404,
  "injection_x": 9000,
  "injection_y": 5070,
  "injection_z": 9860,
  "product_id": 5,
  "specimen_name": "Otof-Cre-2811",
  "strain": "C57BL/6J",
  "structure_abbrev": "ENTl",
  "structure_id": 918,
  "structure_name": "Entorhinal area, lateral part",
  "primary_injection_structure": 918,
  "data_set_id": 167212932,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182901722,
   "name": "Ppp1r17-Cre_NL146"
  },
  "gender": "M",
  "injection_structures": "294/302/795",
  "injection_volume": 0.03932642245,
  "injection_x": 9200,
  "injection_y": 1900,
  "injection_z": 6060,
  "product_id": 5,
  "specimen_name": "Ppp1r17-Cre_NL146-123198",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "SCm",
  "structure_id": 294,
  "structure_name": "Superior colliculus, motor related",
  "primary_injection_structure": 294,
  "data_set_id": 307691605,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 301021674,
   "name": "Oxtr-T2A-Cre"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0011576905375,
  "injection_x": 9400,
  "injection_y": 1510,
  "injection_z": 8810,
  "product_id": 5,
  "specimen_name": "Oxtr-T2A-Cre;Ai14-318593",
  "strain": "unknown",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 596571282,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "385/843",
  "injection_volume": 0.0499916571,
  "injection_x": 9880,
  "injection_y": 3420,
  "injection_z": 8560,
  "product_id": 5,
  "specimen_name": "378-1203",
  "strain": "C57BL/6J",
  "structure_abbrev": "PAR",
  "structure_id": 843,
  "structure_name": "Parasubiculum",
  "primary_injection_structure": 385,
  "data_set_id": 120280939,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 286615058,
   "name": "Drd3-Cre_KI198"
  },
  "gender": "M",
  "injection_structures": "382",
  "injection_volume": 0.062584907,
  "injection_x": 6980,
  "injection_y": 1650,
  "injection_z": 7220,
  "product_id": 5,
  "specimen_name": "Drd3-Cre_KI198-131454",
  "strain": null,
  "structure_abbrev": "CA1",
  "structure_id": 382,
  "structure_name": "Field CA1",
  "primary_injection_structure": 382,
  "data_set_id": 301130077,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839216,
   "name": "Syt6-Cre_KI148"
  },
  "gender": "M",
  "injection_structures": "329/353/182305689",
  "injection_volume": 0.087009888,
  "injection_x": 5940,
  "injection_y": 2250,
  "injection_z": 8580,
  "product_id": 5,
  "specimen_name": "Syt6-Cre-586",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-n",
  "structure_id": 353,
  "structure_name": "Primary somatosensory area, nose",
  "primary_injection_structure": 329,
  "data_set_id": 123664417,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 183238310,
   "name": "Ntrk1-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "672",
  "injection_volume": 0.1177077216,
  "injection_x": 4920,
  "injection_y": 4570,
  "injection_z": 7010,
  "product_id": 5,
  "specimen_name": "Ntrk1-IRES-Cre-3572",
  "strain": "B6.129",
  "structure_abbrev": "CP",
  "structure_id": 672,
  "structure_name": "Caudoputamen",
  "primary_injection_structure": 672,
  "data_set_id": 287994474,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837858,
   "name": "Pvalb-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "612/616/867/1052",
  "injection_volume": 0.050191043,
  "injection_x": 9750,
  "injection_y": 3960,
  "injection_z": 6950,
  "product_id": 5,
  "specimen_name": "Pvalb-IRES-Cre-120335",
  "strain": "B6.129",
  "structure_abbrev": "PPN",
  "structure_id": 1052,
  "structure_name": "Pedunculopontine nucleus",
  "primary_injection_structure": 612,
  "data_set_id": 293252166,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838435,
   "name": "Rbp4-Cre_KL100"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0710987697,
  "injection_x": 8690,
  "injection_y": 1440,
  "injection_z": 3090,
  "product_id": 36,
  "specimen_name": "Rbp4-Cre_KL100-227918",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 503069254,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "345",
  "injection_volume": 0.19785759,
  "injection_x": 4280,
  "injection_y": 3130,
  "injection_z": 9140,
  "product_id": 5,
  "specimen_name": "378-912",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-m",
  "structure_id": 345,
  "structure_name": "Primary somatosensory area, mouth",
  "primary_injection_structure": 345,
  "data_set_id": 100149969,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839406,
   "name": "Nos1-CreERT2"
  },
  "gender": "F",
  "injection_structures": "993",
  "injection_volume": 0.03721190585,
  "injection_x": 3610,
  "injection_y": 1620,
  "injection_z": 7240,
  "product_id": 5,
  "specimen_name": "Nos1-CreERT2-178",
  "strain": "B6.129",
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 267813224,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "F",
  "injection_structures": "409/425/312782574/312782628",
  "injection_volume": 0.0271573337,
  "injection_x": 9310,
  "injection_y": 1910,
  "injection_z": 2160,
  "product_id": 36,
  "specimen_name": "Tlx3-Cre_PL56-295779",
  "strain": null,
  "structure_abbrev": "VISl",
  "structure_id": 409,
  "structure_name": "Lateral visual area",
  "primary_injection_structure": 409,
  "data_set_id": 572770444,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 553502711,
   "name": "Chat-IRES-Cre-neo"
  },
  "gender": "F",
  "injection_structures": "194/298/342/351/1022",
  "injection_volume": 0.1036776496,
  "injection_x": 5790,
  "injection_y": 5990,
  "injection_z": 3900,
  "product_id": 35,
  "specimen_name": "Chat-IRES-Cre-228352",
  "strain": null,
  "structure_abbrev": "SI",
  "structure_id": 342,
  "structure_name": "Substantia innominata",
  "primary_injection_structure": 194,
  "data_set_id": 504521113,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "155/170/218/361/463/718/733/1020",
  "injection_volume": 0.099815352,
  "injection_x": 7070,
  "injection_y": 4360,
  "injection_z": 7650,
  "product_id": 5,
  "specimen_name": "378-1637",
  "strain": "C57BL/6J",
  "structure_abbrev": "VPM",
  "structure_id": 733,
  "structure_name": "Ventral posteromedial nucleus of the thalamus",
  "primary_injection_structure": 155,
  "data_set_id": 158375425,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839022,
   "name": "Cux2-IRES-Cre"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.22368353,
  "injection_x": 8870,
  "injection_y": 860,
  "injection_z": 7920,
  "product_id": 5,
  "specimen_name": "Cux2-IRES-Cre-235",
  "strain": null,
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 263780729,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 265180449,
   "name": "Tlx3-Cre_PL56"
  },
  "gender": "F",
  "injection_structures": "993",
  "injection_volume": 0.0488038334,
  "injection_x": 4350,
  "injection_y": 1330,
  "injection_z": 6550,
  "product_id": 5,
  "specimen_name": "Tlx3-Cre_LURC-128446",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 993,
  "data_set_id": 297892130,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 182761781,
   "name": "Efr3a-Cre_NO108"
  },
  "gender": "F",
  "injection_structures": "39/48/993",
  "injection_volume": 0.02779847175,
  "injection_x": 4680,
  "injection_y": 2530,
  "injection_z": 6160,
  "product_id": 5,
  "specimen_name": "Efr3a-Cre_NO108-126003",
  "strain": "FVB.CD1(ICR)",
  "structure_abbrev": "ACAv",
  "structure_id": 48,
  "structure_name": "Anterior cingulate area, ventral part",
  "primary_injection_structure": 39,
  "data_set_id": 301179679,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838048,
   "name": "Slc17a6-IRES-Cre"
  },
  "gender": "F",
  "injection_structures": "226/342/985/993",
  "injection_volume": 0.0746314198,
  "injection_x": 4530,
  "injection_y": 1520,
  "injection_z": 6950,
  "product_id": 5,
  "specimen_name": "Slc17a6-IRES-Cre_LURC-123366",
  "strain": "C57BL/6J",
  "structure_abbrev": "SI",
  "structure_id": 342,
  "structure_name": "Substantia innominata",
  "primary_injection_structure": 226,
  "data_set_id": 302739608,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 179697864,
   "name": "Rorb-IRES2-Cre"
  },
  "gender": "M",
  "injection_structures": "184/993",
  "injection_volume": 0.02144558745,
  "injection_x": 2550,
  "injection_y": 2400,
  "injection_z": 4450,
  "product_id": 45,
  "specimen_name": "Rorb-IRES2-Cre-324490",
  "strain": null,
  "structure_abbrev": "MOs",
  "structure_id": 993,
  "structure_name": "Secondary motor area",
  "primary_injection_structure": 184,
  "data_set_id": 603891334,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177838542,
   "name": "A930038C07Rik-Tg1-Cre"
  },
  "gender": "M",
  "injection_structures": "4",
  "injection_volume": 0.0638111663,
  "injection_x": 10280,
  "injection_y": 1910,
  "injection_z": 4230,
  "product_id": 35,
  "specimen_name": "A930038C07Rik-Tg1-Cre-306721",
  "strain": null,
  "structure_abbrev": "IC",
  "structure_id": 4,
  "structure_name": "Inferior colliculus",
  "primary_injection_structure": 4,
  "data_set_id": 583320505,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "731/746/972",
  "injection_volume": 0.282773346165625,
  "injection_x": 2350,
  "injection_y": 3590,
  "injection_z": 6020,
  "product_id": 5,
  "specimen_name": "378-1347",
  "strain": "C57BL/6J",
  "structure_abbrev": "ORBm",
  "structure_id": 731,
  "structure_name": "Orbital area, medial part",
  "primary_injection_structure": 731,
  "data_set_id": 126860974,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "1039",
  "injection_volume": 0.004262452425,
  "injection_x": 13090,
  "injection_y": 5100,
  "injection_z": 6030,
  "product_id": 5,
  "specimen_name": "378-1182",
  "strain": "C57BL/6J",
  "structure_abbrev": "GR",
  "structure_id": 1039,
  "structure_name": "Gracile nucleus",
  "primary_injection_structure": 1039,
  "data_set_id": 114400640,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": null,
   "name": null
  },
  "gender": "M",
  "injection_structures": "337/369",
  "injection_volume": 0.4286064888,
  "injection_x": 6350,
  "injection_y": 1950,
  "injection_z": 7740,
  "product_id": 5,
  "specimen_name": "378-1060",
  "strain": "C57BL/6J",
  "structure_abbrev": "SSp-ll",
  "structure_id": 337,
  "structure_name": "Primary somatosensory area, lower limb",
  "primary_injection_structure": 337,
  "data_set_id": 112791318,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177837713,
   "name": "Ntsr1-Cre_GN220"
  },
  "gender": "M",
  "injection_structures": "385",
  "injection_volume": 0.0315516733,
  "injection_x": 9340,
  "injection_y": 1850,
  "injection_z": 8290,
  "product_id": 5,
  "specimen_name": "Ntsr1-Cre-3502",
  "strain": "C57BL/6J",
  "structure_abbrev": "VISp",
  "structure_id": 385,
  "structure_name": "Primary visual area",
  "primary_injection_structure": 385,
  "data_set_id": 156545918,
  "storage_directory": "asdf"
 },
 {
  "transgenic_line": {
   "transgenic_line_id": 177839406,
   "name": "Nos1-CreERT2"
  },
  "gender": "M",
  "injection_structures": "879/886",
  "injection_volume": 0.006806284975,
  "injection_x": 7420,
  "injection_y": 850,
  "injection_z": 6220,
  "product_id": 5,
  "specimen_name": "Nos1-CreERT2-122404",
  "strain": "B6.129",
  "structure_abbrev": "RSPv",
  "structure_id": 886,
  "structure_name": "Retrosplenial area, ventral part",
  "primary_injection_structure": 879,
  "data_set_id": 308027576,
  "storage_directory": "asdf"
 }
]